In [1]:
import numpy as np
from scipy.stats import truncnorm
from transitions import Machine

# simulation/calculation step size, make sure smaller than any relevant values
# all frequency values will be rounded to step size
STEP_SIZE = 5   # in GHz

# Note: cooling is positive in GHz and heating is negative in GHz
# Laser parameters
NUM_OF_LASERS = 8   # number of lasers and/or MRMs
LASER_FREQ_SPACING = 400  # in GHz
LASER_FREQ_UNCERTAINTY = 10  # in GHz, per stdev
# MRM parameters
MRM_FSR = 1800  # in GHz
MRM_FREQ_SPACING = -200  # in GHz, negative freq is longer wavelength
MRM_FREQ_UNCERTAINTY = 50  # in GHz, per stdev
# CRR parameters
CRR_FSR = 1800  # in GHz
CRR_FREQ_SPACING = -200  # in GHz, negative freq is longer wavelength
CRR_FREQ_UNCERTAINTY = 50  # in GHz , per stdev

# MRM, CRR, and laser local uncertainty/variations will be sampled from
# truncated Gaussian distribution (`truncnorm`). The value `TRUNCATION` below
# refers to how many stdev should we truncate the Gaussian.
# `corners` in the code below will evaluate at this `TRUNCATION` * stdev value
TRUNCATION = 5  # * stdev

# will track the resonances -5*FSR over, ..., 0*FSR over, ... 4*FSR over
# for both MRMs and CRRs.
WHICH_RESONANCES = list(range(-5, 5, 1))
BARREL_ROLL_SEQ = [0, 5, 1, 6, 2, 7, 3, 4]


class Lasers(object):

    """Lasers
    Initialize the lasers. Flags:
    - variations (bool): whether to enable laser wavelength uncertainty
    - corners (bool): if True, only evaluate the corners of the wavelength
        uncertainties. Won't do anything if `variations` is `False`.
    - reverse (bool): if True, go in decreasing frequency.
        (higher frequnency first in the list).
    - offest (int): specify the offset from 0. Otherwise, the lowest frequency
        will always be 0 GHz.

    The object has two important properties:
    - `laser_lines` (list): gives us the frequency of the lasers.
    - `brightness` (list): will say `bright` if that laser is uncaptured.
        Capture a laser line with the function `capture_laser_line`.
        That specific laser brightness will change to the MRM/CRR ring idx.
    """

    def __init__(self,
                 variations=False,
                 corners=False,
                 reverse=False,
                 offset=0) -> None:
        self._delta = None

        # start by having all the laser lines 'bright'
        # will be replaced by the MRM number that captures that laser
        # Note: need to considerq upstream vs. downstream
        self._brightness = ['bright'] * NUM_OF_LASERS

        # If reverse, the first laser line will have the highest frequency
        self._reverse = reverse

        # offset specifies the additive frequency offset from 0
        self._offset = offset

        self.populate_laser_lines(variations, corners)

    def populate_laser_lines(self, variations, corners):
        # the first laser line is always at 0 GHz
        laser_lines = np.arange(
            NUM_OF_LASERS, dtype=int) * LASER_FREQ_SPACING

        if self._reverse:
            laser_lines = np.flip(laser_lines)

        laser_lines += self._offset

        if variations:
            if corners:  # only evaluate the corners
                delta = np.random.randint(2, size=laser_lines.size)
                delta = 2*delta - 1  # convert to [-1, +1] random numbers
                delta *= LASER_FREQ_UNCERTAINTY * TRUNCATION
            else:
                delta = truncnorm.rvs(a=-1*TRUNCATION, b=TRUNCATION,
                                      scale=LASER_FREQ_UNCERTAINTY,
                                      size=laser_lines.size)
            # round to the nearest STEP_SIZE
            delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE
            delta = delta.astype(int)

            self._delta = delta
            laser_lines += delta

        self._laser_lines = laser_lines

    def capture_laser_line(self, which_laser, which_ring, type='mrm'):
        if type == 'mrm':
            assert self._brightness[which_laser] == 'bright', \
                f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be bright to be captured"
        else:  # 'crr'
            assert self._brightness[which_laser] == which_ring, \
                f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be numbered {which_ring} to be captured"
        # which_ring will modulate this laser
        self._brightness[which_laser] = which_ring

    @property
    def laser_lines(self):
        return self._laser_lines

    @property
    def brightness(self):
        return self._brightness

    def __str__(self) -> str:
        return f"Lines: {self.laser_lines}, Brightness: {self.brightness}"

    def __repr__(self) -> str:
        return self.__str__()


class RingFSM(object):
    """RingFSM
    Base class for all rings: MRMs and CRRs.
    However, they will be modified below, use MRM_FSR and CRR_FSR instead.
    """

    states = ['idle', 'cold', 'search', 'lock']
    INITIAL_COOLDOWN = 200  # in GHz
    # Using this default value because it should clear laser uncertainty and
    # the ring resonance uncertainty at ± 100 GHz.
    DEFAULT_FSR = 1800  # in GHz

    def __init__(self,
                 resonances,
                 lasers,
                 initial_loc=0,
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz
                 step_size=STEP_SIZE,
                 heating_threshold=-1*DEFAULT_FSR,
                 cooling_threshold=INITIAL_COOLDOWN,
                 ring_fsr=DEFAULT_FSR,
                 ) -> None:
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self.loc = initial_loc  # location of the resonance

        self.locked_laser_idx = None

        self._initial_loc = initial_loc
        self._this_ring = this_ring
        self._previous_ring = previous_ring
        self._initial_resonances = resonances
        self._lasers = lasers

        self._cool_down_by = cool_down_by
        self._step_size = step_size
        self._ring_fsr = abs(ring_fsr)

        assert heating_threshold >= -1*self._ring_fsr and heating_threshold <= 0
        self._heating_threshold = heating_threshold
        assert cooling_threshold <= self._ring_fsr and cooling_threshold >= 0
        self._cooling_threshold = cooling_threshold

        self._found_all_lines = False

        self.machine = Machine(
            model=self, states=RingFSM.states, initial='idle')

        # 'start' starts the search algorithm
        self.machine.add_transition(trigger='start',
                                    source='idle',
                                    dest='cold',
                                    after='cool_step')

        # once the ring has cooled, start searching
        self.machine.add_transition(trigger='keep_searching',
                                    source='cold',
                                    dest='search',
                                    after='heat_step')

        # 'reset' puts the ring back to idle from any state
        self.machine.add_transition(trigger='reset',
                                    source='*',
                                    dest='idle',
                                    after='reset_resonances')

        # will 'keep_searching' until we find all the laser lines
        self.machine.add_transition(
            trigger='keep_searching',
            source='search',
            dest='search',
            after='heat_step')

        # lock ring once the laser lines have been found
        self.machine.add_transition(
            trigger='lock_to_line',
            source='search',
            dest='lock',
            after='lock_to_laser',
            conditions=['found_all_lines'])

    def reset_resonances(self):
        # reset everything
        self.loc = self._initial_loc
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self._found_all_lines = False

    def cool(self, amount):
        self.loc += amount

    def heat(self, amount):
        self.loc -= amount

    def cool_step(self):
        # cool the resonances by the cool down by amount
        self.cool(self._cool_down_by)

    def heat_step(self):
        # check if there's a laser signal to be found
        found = self._check_laser_signal()
        # found is tuple (bool, idx | None)
        # if found: (True, laser idx), and if not found: (False, None)
        if found[0]:
            self.laser_locs_seen.append(self.loc)
            self.laser_brightness_seen.append(
                self._lasers.brightness[found[1]])
            self.laser_indices_seen.append(found[1])

        # check if we have found all lasers
        self._check_exit_search_condition()

        # move the resonance location by specified step_size
        self.heat(self._step_size)

    def lock_to_laser(self):

        # Correct the range of numbers to between [-1*FSR, cooling_threshold]
        candidate_loc = self.laser_locs_seen[-1]
        # bring any number beyond 0 and -1*fsr to this range
        candidate_loc = candidate_loc % (-1*self._ring_fsr)
        # bring any number between -1*fsr and the cooling limit to a positive value (cold)
        cooling_limit = self._cooling_threshold - self._ring_fsr
        if candidate_loc > (-1*self._ring_fsr) and candidate_loc <= cooling_limit:
            candidate_loc += self._ring_fsr

        # jump to the laser signal we want to capture
        self.loc = candidate_loc
        # check and capture that laser signal
        self._check_laser_signal(capture=True)

    def _capture_line(self, idx, ring_number):
        return self._lasers.capture_laser_line(idx, ring_number, type='mrm')

    def _check_laser_signal(self, capture=False):
        # check if there's any laser signal at this location
        # and, also decide to capture.
        found_laser = False
        idx = None
        for r in self.resonances:
            if (r in self._lasers.laser_lines):
                # get the index
                idx = np.where(self._lasers.laser_lines == r)
                assert len(idx) == 1
                idx = idx[0][0]
                if capture:
                    self._capture_line(idx, self._this_ring)
                    self.locked_laser_idx = idx
                found_laser = True
        # if there's a laser there, found_laser will return that laser's index
        return (found_laser, idx)

    def _check_exit_search_condition(self):
        # This is the crux of the search algorithm.
        # In this case, we consider the simple algo.
        # if we have seen one dim and one bright, we have found all lines
        if self._previous_ring is None:  # the first ring in the bank
            if len(self.laser_locs_seen) >= 2:
                self._found_all_lines = True
        else:  # all the other rings
            if len(self.laser_locs_seen) >= 2:
                # NB: this line is the only significant change in the simplified algorithm
                if self.laser_brightness_seen[-2] != 'bright' and self.laser_brightness_seen[-1] == 'bright':
                    self._found_all_lines = True

    @property
    def found_all_lines(self):
        return self._found_all_lines

    @property
    def resonances(self):
        return self.loc + self._initial_resonances

    @property
    def within_threshold(self):
        # if the number is beyond the cooling and heating threshold, then it's outside of threshold
        # otherwise, it's within threshold
        return self.loc >= self._heating_threshold and self.loc <= self._cooling_threshold


class MRM_FSM(RingFSM):

    INITIAL_COOLDOWN = 200  # in GHz

    def __init__(self,
                 resonances,  # a list of this ring's resonances
                 lasers,  # laser object
                 initial_loc=0,  # initial heating/cooling in GHz
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz before searching
                 step_size=STEP_SIZE,
                 heating_threshold=-1*MRM_FSR,
                 cooling_threshold=INITIAL_COOLDOWN
                 ) -> None:

        super().__init__(resonances, lasers, initial_loc, this_ring,
                         previous_ring, cool_down_by, step_size, heating_threshold, cooling_threshold, MRM_FSR)


class CRR_FSM(RingFSM):

    INITIAL_COOLDOWN = 200  # in GHz

    def __init__(self,
                 resonances,  # a list of this ring's resonances
                 lasers,  # laser object
                 initial_loc=0,  # initial heating/cooling in GHz
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz before searching
                 step_size=STEP_SIZE,
                 heating_threshold=-1*CRR_FSR,
                 cooling_threshold=INITIAL_COOLDOWN
                 ) -> None:

        super().__init__(resonances, lasers, initial_loc, this_ring,
                         previous_ring, cool_down_by, step_size, heating_threshold, cooling_threshold, CRR_FSR)

    def _check_exit_search_condition(self):
        # This is the crux of the search algorithm.
        if len(self.laser_locs_seen) >= 1:
            # CRR needs to capture laser used by an MRM of the same idx
            if self.laser_brightness_seen[-1] == self._this_ring:
                self._found_all_lines = True

    def _capture_line(self, idx, ring_number):
        return self._lasers.capture_laser_line(idx, ring_number, type='crr')

In [2]:
from collections import deque
import copy


def populate_resonances(freq_spacing,
                        freq_uncertainty,
                        fsr,
                        global_variations=False,
                        variations=False,
                        corners=False):
    """populate_resonances (base function)
    USE THE FUNCTIONS `populate_mrm_resonances` and `populate_crr_resonances`
    BELOW INSTEAD.

    Initialize the resonances of the rings. Flags:
    - freq_spacing (int): frequency spacing from one ring to the other.
    - freq_uncertainty (int): local resonance frequency uncertainty will be
        sampled from a uniform dist between [-1*freq_uncertainty, freq_uncertainty].
    - fsr (int): the FSR of each ring.
    - global_variations (bool): whether to enable global resonance frequency
        uncertainty with a uniform distribution of [-1*MRM_FSR, MRM_FSR]
    - variations (bool): whether to enable local resonance frequency uncertainty
    - corners (bool): if True, only evaluate the corners of the frequency
        uncertainties. Won't do anything if `variations` is `False`.

    Returns:
    - lines (np.ndarray): A numpy 2D-array, organized where the first dim
        corresponds to which ring in the array and the second dim
        corresponds to which resonance frequency of that ring.

        For example:
        lines[3] will return the resonances of the 4th ring in the bank.
        lines[3][0] will return the first resonance (lowest value in freq)
            of the 4th ring in the bank.
    """

    lines = np.arange(NUM_OF_LASERS, dtype=np.int64) * freq_spacing
    lines = np.vstack([lines] * len(WHICH_RESONANCES)).T

    fsr_shifts = np.array(WHICH_RESONANCES) * fsr
    lines += fsr_shifts

    if variations:  # assume we keep the FSR to be the same
        if corners:  # only evaluate the corners
            delta = np.random.randint(2, size=NUM_OF_LASERS)
            delta = 2*delta - 1  # convert to [-1, +1] random numbers
            delta *= freq_uncertainty * TRUNCATION
        else:
            delta = truncnorm.rvs(a=-1*TRUNCATION, b=TRUNCATION,
                                  scale=freq_uncertainty, size=NUM_OF_LASERS)
        # round to the nearest STEP_SIZE
        delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE

        delta = delta.astype(int)
        delta = delta[:, np.newaxis]
        lines += delta

    if global_variations:  # the global variation can be as much as ±1 FSR
        global_delta = np.random.uniform(-.5*fsr, .5*fsr)
        global_delta = np.round(global_delta/STEP_SIZE, decimals=0) * STEP_SIZE
        global_delta = global_delta.astype(int)
        lines += global_delta

    return lines


def populate_mrm_resonances(global_variations=False,
                            variations=False,
                            corners=False):
    return populate_resonances(MRM_FREQ_SPACING, MRM_FREQ_UNCERTAINTY, MRM_FSR,
                               global_variations, variations, corners)


def populate_crr_resonances(global_variations=False,
                            variations=False,
                            corners=False):
    return populate_resonances(CRR_FREQ_SPACING, CRR_FREQ_UNCERTAINTY, CRR_FSR,
                               global_variations, variations, corners)


# HELPER FUNCTIONS
def check_barrel_roll(list1, list2):
    # check for barrel roll in sequence or reverse sequence
    assert len(list1) == len(list2)

    def check_equivalent(list1, list2):
        d1 = deque(list1)
        d2 = deque(list2)

        equivalent = d1 == d2

        for _ in range(len(list1)):
            d1.rotate(1)
            if d1 == d2:
                equivalent = True

        return equivalent

    list1_ = copy.deepcopy(list1)
    equivalent = check_equivalent(list1_, list2)
    list1_.reverse()
    equivalent = equivalent or check_equivalent(list1_, list2)
    return equivalent


def calculate_maxima(amt_heat_applied):
    colder = [n for n in amt_heat_applied if n >= 0]
    hotter = [n for n in amt_heat_applied if n <= 0]

    max_cool = max(colder) if len(colder) > 0 else 0
    max_heat = min(hotter) if len(hotter) > 0 else 0
    max_diff = max_cool - max_heat
    return max_cool, max_heat, max_diff

In [3]:
# Try out the algorithm with a single try
lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING, variations=True, corners=False)
all_mrm_resonances_ = populate_mrm_resonances(variations=False, corners=False)
all_mrm_resonances = all_mrm_resonances_
all_mrm_resonances[2] = all_mrm_resonances_[6]
all_mrm_resonances[6] = all_mrm_resonances_[2]

all_mrms = []

# Lock all the MRMs
for idx in range(NUM_OF_LASERS):
    previous_ring = None if idx == 0 else (idx-1)

    mrm = MRM_FSM(resonances=all_mrm_resonances[idx],
                  lasers=lasers,
                  this_ring=idx,
                  previous_ring=previous_ring)
    mrm.start()

    while not mrm.found_all_lines:
        mrm.keep_searching()

    mrm.lock_to_line()
    all_mrms.append(mrm)

    # print(f"Locked to laser # {mrm.locked_laser_idx}")
    # print(lasers.brightness)

all_crr_resonances = populate_crr_resonances(variations=True, corners=False)
all_crrs = []

# Now lock all the CRRs
for idx in range(NUM_OF_LASERS):
    import copy
    lasers_ = copy.deepcopy(lasers)
    previous_ring = None if idx == 0 else (idx-1)
    crr = CRR_FSM(resonances=all_crr_resonances[idx],
                  lasers=lasers_,
                  this_ring=idx,
                  previous_ring=previous_ring)
    crr.start()

    while not crr.found_all_lines:
        crr.keep_searching()

    crr.lock_to_line()
    all_crrs.append(crr)


mrm_lasers_locked_to = [int(m.locked_laser_idx) for m in all_mrms]
crr_lasers_locked_to = [int(m.locked_laser_idx) for m in all_crrs]

# check for barrel roll in sequence or reverse sequence
print("Barrel roll check?", check_barrel_roll(mrm_lasers_locked_to, BARREL_ROLL_SEQ))

print(f"MRM and CRR lasers: {mrm_lasers_locked_to} & {crr_lasers_locked_to}")

amt_heat_applied = [m.loc for m in all_mrms]
print(f"Amount of heat applied for MRM: {amt_heat_applied}")
amt_heat_applied = [m.loc for m in all_crrs]
print(f"Amount of heat applied for CRR: {amt_heat_applied}")

Barrel roll check? True
MRM and CRR lasers: [4, 0, 5, 1, 6, 2, 7, 3] & [4, 0, 5, 1, 6, 2, 7, 3]
Amount of heat applied for MRM: [-410, -380, -1420, -395, -420, -395, -395, -390]
Amount of heat applied for CRR: [-445, -325, -400, -355, -430, -395, -330, -385]


In [4]:
# Run Monte Carlo and get distribution

import pandas as pd
import copy
from tqdm.notebook import tqdm

eval_corners = False
global_variations = True  # main difficulty
local_variations = True
laser_variations = True

cycling_threshold_mrm = 4/9  # ratio of FSR
cycling_threshold_crr = 6/9  # ratio of FSR
cooling_threshold_mrm = 2/9  # ratio of FSR
cooling_threshold_crr = 2/9  # ratio of FSR


# CYCLING: uncomment one of the two below
# Option A: with cycling:
cycling_freqs = [-k * 100 for k in range(18)]
# Option B: without cycling
# cycling_freqs = [0]

num_trials = 1000
num_trials = 2001

trial_results_mrm = []
trial_results_crr = []

failed_trials = []

for _ in tqdm(range(num_trials)):
    lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING,
                    variations=laser_variations, corners=eval_corners)
    all_mrm_resonances = populate_mrm_resonances(global_variations=global_variations,
                                                 variations=local_variations,
                                                 corners=eval_corners)
    all_crr_resonances = populate_crr_resonances(global_variations=global_variations,
                                                 variations=local_variations,
                                                 corners=eval_corners)

    for ctry, cfreq in enumerate(cycling_freqs):
        # Get all the MRMs to lock properly first
        lasers_ = copy.deepcopy(lasers)
        all_mrms = []
        for idx in range(NUM_OF_LASERS):
            previous_ring = None if idx == 0 else (idx-1)
            mrm = MRM_FSM(resonances=all_mrm_resonances[idx],
                            lasers=lasers_,
                            heating_threshold=-cycling_threshold_mrm * MRM_FSR,
                            cooling_threshold=cooling_threshold_mrm * MRM_FSR,
                            cool_down_by=cooling_threshold_mrm * MRM_FSR,
                            initial_loc=cfreq,
                            this_ring=idx,
                            previous_ring=previous_ring)
                            
            # Run the search algorithm for each MRM
            mrm.start()
            while not mrm.found_all_lines:
                mrm.keep_searching()
            mrm.lock_to_line()
            all_mrms.append(mrm)

        # Save all the MRM information here
        mrm_lasers_locked_to = [int(m.locked_laser_idx) for m in all_mrms]
        amt_heat_applied_mrm = [m.loc for m in all_mrms]
        max_cool_mrm, max_heat_mrm, max_diff_mrm = \
            calculate_maxima(amt_heat_applied_mrm)

        # Lock all the CRRs to the corresponding MRMs
        all_crrs = []
        for idx in range(NUM_OF_LASERS):
            lasers__ = copy.deepcopy(lasers_)
            previous_ring = None if idx == 0 else (idx-1)
            crr = CRR_FSM(resonances=all_crr_resonances[idx],
                            lasers=lasers__,
                            this_ring=idx,
                            previous_ring=idx-1,
                            heating_threshold=-cycling_threshold_crr * MRM_FSR,
                            cooling_threshold=cooling_threshold_crr * MRM_FSR,
                            cool_down_by=cooling_threshold_crr * MRM_FSR)
            crr.start()
            while not crr.found_all_lines:
                crr.keep_searching()
            crr.lock_to_line()
            all_crrs.append(crr)

        # Save all the CRR information here
        crr_lasers_locked_to = [int(m.locked_laser_idx) for m in all_crrs]
        amt_heat_applied_crr = [m.loc for m in all_crrs]
        max_cool_crr, max_heat_crr, max_diff_crr = calculate_maxima(
            amt_heat_applied_crr)

        # Check if cycling is needed
        mrm_cycling_condition = bool(np.array([m.within_threshold for m in all_mrms]).all())
        crr_cycling_condition = bool(np.array([m.within_threshold for m in all_crrs]).all())
        # if the above are true, then we're within threshold--we don't cycle.
        cycling_condition = not(mrm_cycling_condition and crr_cycling_condition)

        # Previous cycling condition below
        # mrm_cycling_condition = abs(max_heat_mrm) > (MRM_FSR * cycling_threshold_mrm)
        # crr_cycling_condition = abs(max_heat_crr) > (MRM_FSR * cycling_threshold_crr)
        # cycling_condition = mrm_cycling_condition or crr_cycling_condition

        # if `cycling_condition == True`: cycle, otherwise break the loop
        if not cycling_condition:
            break

    mrm_result = {'rings_are_locked_to': mrm_lasers_locked_to,
                  # check if barrel roll equivalent
                  'barrel_roll': check_barrel_roll(mrm_lasers_locked_to, BARREL_ROLL_SEQ),
                  'max_cool': max_cool_mrm,
                  'max_heat': max_heat_mrm,
                  'max_diff': max_diff_mrm}

    mrm_numbers = list(range(NUM_OF_LASERS))
    # (-) value is heat and (+) value is cool
    mrm_heat_result = dict(zip(mrm_numbers, amt_heat_applied_mrm))
    mrm_result.update(mrm_heat_result)

    # check if we were able to lock within the threshold
    mrm_threshold = [m.within_threshold for m in all_mrms]
    crr_threshold = [m.within_threshold for m in all_crrs]
    successful_lock = mrm_threshold + crr_threshold
    successful_lock = bool(np.array(successful_lock).all())

    overall_result = {'cycling_trial': ctry, 'successful_lock': successful_lock}
    mrm_result.update(overall_result)

    crr_result = {'rings_are_locked_to': crr_lasers_locked_to,
                  'mrm_crr_equivalent': mrm_lasers_locked_to == crr_lasers_locked_to,
                  'max_cool': max_cool_crr,
                  'max_heat': max_heat_crr,
                  'max_diff': max_diff_crr}

    crr_numbers = list(range(NUM_OF_LASERS))
    # (-) value is heat and (+) value is cool
    heat_result = dict(zip(crr_numbers, amt_heat_applied_crr))
    crr_result.update(heat_result)

    trial_results_mrm.append(mrm_result)
    trial_results_crr.append(crr_result)

    if not successful_lock:  # these are the failed results
       laser_numbers = [f"laser_{idx}" for idx in range(NUM_OF_LASERS)]
       middle_idx = WHICH_RESONANCES.index(0)
       mrm_numbers = [f"mrm_{idx}" for idx in range(NUM_OF_LASERS)]
       mrm_resonance = [int(r[middle_idx]) for r in all_mrm_resonances]
       crr_numbers = [f"crr_{idx}" for idx in range(NUM_OF_LASERS)]
       crr_resonance = [int(r[middle_idx]) for r in all_crr_resonances]

       failed_result = dict(zip(laser_numbers, list(lasers.laser_lines)))
       M = dict(zip(mrm_numbers, mrm_resonance))
       failed_result.update(M)
       C = dict(zip(crr_numbers, crr_resonance))
       failed_result.update(C)
       failed_result.update({'rings_are_locked_to': mrm_lasers_locked_to})

       failed_trials.append(failed_result)
       

trial_results_mrm = pd.DataFrame(trial_results_mrm)
trial_results_crr = pd.DataFrame(trial_results_crr)

# output failed trials to an excel file
failed_trials = pd.DataFrame(failed_trials)
failed_trials.to_excel("failed_trials.xlsx")

trial_results_mrm.to_csv('trial_results_mrm_simple_algo.csv')
trial_results_crr.to_csv('trial_results_crr_simple_algo.csv')

  0%|          | 0/2001 [00:00<?, ?it/s]

In [5]:
trial_results_mrm.describe()

,max_cool,max_heat,max_diff,0,1,2,3,4,5,6,7,cycling_trial
count,2001.000000,2001.000000,2001.00000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000
mean,91.644178,-248.675662,340.31984,-30.037481,-54.035482,-73.610695,-97.918541,-115.202399,-142.373813,-166.154423,-201.284358,1.010995
std,93.813487,188.591345,135.64742,194.449829,211.251318,215.216795,216.642223,212.414681,211.809554,203.315129,194.795276,2.409124
min,-0.000000,-800.000000,70.00000,-780.000000,-765.000000,-745.000000,-795.000000,-795.000000,-800.000000,-795.000000,-800.000000,0.000000
25%,0.000000,-340.000000,260.00000,-140.000000,-170.000000,-190.000000,-210.000000,-220.000000,-240.000000,-255.000000,-300.000000,0.000000
50%,70.000000,-200.000000,305.00000,30.000000,-20.000000,-45.000000,-70.000000,-85.000000,-110.000000,-125.000000,-160.000000,0.000000
75%,165.000000,-110.000000,375.00000,120.000000,105.000000,85.000000,60.000000,30.000000,0.000000,-35.000000,-60.000000,1.000000
max,400.000000,0.000000,800.00000,305.000000,350.000000,355.000000,400.000000,350.000000,325.000000,350.000000,245.000000,17.000000


In [6]:
trial_results_crr.describe()

,max_cool,max_heat,max_diff,0,1,2,3,4,5,6,7
count,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000
mean,117.536232,-407.781109,525.317341,-175.852074,-193.590705,-214.880060,-238.533233,-258.315842,-281.676662,-308.050975,-342.531234
std,140.126467,393.147236,298.981959,416.622924,414.385585,417.093947,416.309182,415.027303,414.145687,412.866049,415.334416
min,-0.000000,-1200.000000,85.000000,-1200.000000,-1195.000000,-1185.000000,-1195.000000,-1200.000000,-1200.000000,-1200.000000,-1200.000000
25%,0.000000,-745.000000,305.000000,-530.000000,-550.000000,-575.000000,-595.000000,-615.000000,-635.000000,-655.000000,-705.000000
50%,0.000000,-295.000000,380.000000,-75.000000,-100.000000,-110.000000,-130.000000,-145.000000,-180.000000,-205.000000,-245.000000
75%,255.000000,-30.000000,745.000000,200.000000,175.000000,155.000000,115.000000,95.000000,65.000000,45.000000,20.000000
max,400.000000,0.000000,1600.000000,400.000000,400.000000,400.000000,400.000000,400.000000,395.000000,400.000000,385.000000


In [7]:
# Now plot the results of the Monte Carlo runs
import plotly.express as px

# Plot MRM
# Heat and cool amount
fig = px.histogram(np.array(trial_results_mrm[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", title="MRM heating & cooling", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-MRM_FSR, MRM_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"mrm_heating_{MRM_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot the maximum differences
fig = px.histogram(trial_results_mrm, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", title="MRM max diff between rings", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*MRM_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"mrm_diff_{MRM_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot CRR
# Heat and cool amount
fig = px.histogram(np.array(trial_results_crr[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", title="CRR heating & cooling", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-CRR_FSR, CRR_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"crr_heating_{CRR_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot the maximum differences
fig = px.histogram(trial_results_crr, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", title="CRR max diff between rings", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*CRR_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"crr_diff_{CRR_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

In [8]:
# check the results: barrel-roll and meeting our threshold requirements
print("All in barrel-roll sequence? ", trial_results_mrm['barrel_roll'].all())
print("All locked within the heating and cooling limits?", trial_results_mrm['successful_lock'].all())
percent_success = np.count_nonzero(trial_results_mrm['successful_lock'])/len(trial_results_mrm['successful_lock'])
print(f"Percentage of successful lock: {percent_success * 100}%")

# display all the trial results in a markdown table
from IPython.display import display, Markdown
display(Markdown("# MRM results"))
display(Markdown(trial_results_mrm.to_markdown()))

display(Markdown("# CRR results"))
display(Markdown(trial_results_crr.to_markdown()))

All in barrel-roll sequence?  True
All locked within the heating and cooling limits? True
Percentage of successful lock: 100.0%


# MRM results

|      | rings_are_locked_to      | barrel_roll   |   max_cool |   max_heat |   max_diff |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |   cycling_trial | successful_lock   |
|-----:|:-------------------------|:--------------|-----------:|-----------:|-----------:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|----------------:|:------------------|
|    0 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -210 |        210 |  -70 |  -65 |  -10 |  -65 |   -5 |  -20 | -205 | -210 |               0 | True              |
|    1 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         50 |       -135 |        185 |  -85 | -135 |  -35 |  -50 |   -0 |   50 |   -0 |  -55 |               0 | True              |
|    2 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -705 |        705 | -480 | -440 | -535 | -520 | -430 | -510 | -705 | -685 |               6 | True              |
|    3 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -60 |        260 |  200 |   95 |  110 |  100 |  195 |  155 |  200 |  -60 |               0 | True              |
|    4 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        225 |        -20 |        245 |  140 |  155 |  195 |  175 |  225 |  -20 |   50 |   35 |               0 | True              |
|    5 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         85 |       -185 |        270 |   50 |   35 |   20 |  -50 |   -0 |   85 | -135 | -185 |               0 | True              |
|    6 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -365 |        365 | -255 | -285 | -245 | -205 | -250 | -220 | -365 | -270 |               1 | True              |
|    7 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        275 |        -30 |        305 |  175 |  275 |  250 |   70 |   45 |  -30 |   40 |   50 |               0 | True              |
|    8 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        125 |       -125 |        250 |  125 |   70 |   55 | -110 |  -85 | -125 |  -70 | -125 |               0 | True              |
|    9 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -155 | -220 | -135 | -195 | -160 | -230 | -105 | -305 |               0 | True              |
|   10 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        270 |        -85 |        355 |  190 |  140 |  140 |  130 |  185 |  270 |  180 |  -85 |               0 | True              |
|   11 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        280 |          0 |        280 |  195 |  280 |  195 |  260 |   85 |   50 |   25 |   60 |               0 | True              |
|   12 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -465 |        465 | -280 | -255 | -205 | -235 | -465 | -450 | -415 | -420 |               3 | True              |
|   13 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -205 |        320 |  115 |  -20 | -135 | -110 | -140 | -205 |  -15 | -110 |               0 | True              |
|   14 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -425 |        425 | -170 | -155 | -110 | -165 | -370 | -425 | -415 | -415 |               2 | True              |
|   15 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |        -55 |        285 |  205 |  190 |  230 |   40 |   10 |   10 |  -50 |  -55 |               0 | True              |
|   16 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         95 |       -205 |        300 |   95 | -130 |  -85 | -165 | -100 | -205 | -110 | -135 |               0 | True              |
|   17 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -240 |        300 |   60 |  -95 |  -55 |   25 |  -30 |    5 |  -95 | -240 |               0 | True              |
|   18 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |       -190 |        385 |  195 |  -45 |   -5 |  -80 | -110 | -190 | -140 |  -90 |               0 | True              |
|   19 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -160 |        180 |   20 | -150 | -160 |  -85 | -125 | -140 |  -55 |  -75 |               0 | True              |
|   20 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -480 |        480 | -155 | -290 | -270 | -245 | -150 | -300 | -480 | -445 |               2 | True              |
|   21 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        210 |       -235 |        445 |    5 |  210 | -135 | -235 |  -35 | -170 |  -90 |  -90 |               1 | True              |
|   22 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |        -20 |        220 |  170 |  200 |  100 |  100 |  155 |   45 |  -20 |  -15 |               0 | True              |
|   23 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -465 |        465 | -225 | -275 | -255 | -195 | -240 | -410 | -465 | -395 |               3 | True              |
|   24 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        255 |        -45 |        300 |  195 |  235 |  255 |  190 |  245 |   70 |   60 |  -45 |               0 | True              |
|   25 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |        -80 |        265 |   35 |  -20 |   60 |  115 |  185 |   85 |   10 |  -80 |               0 | True              |
|   26 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |       -130 |        290 |  130 |   80 |   95 |   15 |   15 |  150 |  160 | -130 |               0 | True              |
|   27 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -460 |        460 | -235 | -245 | -370 | -420 | -425 | -425 | -385 | -460 |               3 | True              |
|   28 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        235 |        -90 |        325 |  130 |  145 |  125 |  225 |   90 |  235 |  -90 |   -5 |               0 | True              |
|   29 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         70 |        -90 |        160 |  -30 |   55 |    5 |   65 |   -0 |   70 |   35 |  -90 |               0 | True              |
|   30 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -365 |        365 | -125 | -270 | -320 | -315 | -290 | -365 | -220 | -240 |               2 | True              |
|   31 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        165 |        -95 |        260 |  125 |  165 |  115 |  130 |  125 |   40 |  -65 |  -95 |               0 | True              |
|   32 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -510 |        510 | -310 | -250 | -210 | -295 | -290 | -290 | -475 | -510 |               4 | True              |
|   33 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |       -170 |        370 |   35 |   60 |  115 |   45 |  200 |   65 | -130 | -170 |               0 | True              |
|   34 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -515 |        515 | -245 | -365 | -515 | -400 | -490 | -430 | -505 | -485 |               3 | True              |
|   35 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |       -130 |        290 |   60 |   50 |   90 |  115 |   55 |   -0 |  160 | -130 |               0 | True              |
|   36 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |        -65 |        265 |  180 |  200 |  130 |  -65 |  -15 |   10 |  -35 |  -40 |               0 | True              |
|   37 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -155 |        295 |   50 |  140 | -145 | -155 | -110 |  -95 | -135 |  -60 |               0 | True              |
|   38 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -495 |        495 | -375 | -435 | -420 | -465 | -470 | -490 | -430 | -495 |               2 | True              |
|   39 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |       -120 |        250 |  130 | -115 |  -90 | -110 |  -60 | -105 | -120 |  -45 |               0 | True              |
|   40 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        165 |        -95 |        260 |  165 |  125 |  120 |   85 |  110 |  -75 |  -60 |  -95 |               0 | True              |
|   41 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |          0 |        165 |  165 |   60 |   40 |   20 |   95 |   95 |   50 |   95 |               0 | True              |
|   42 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -225 |        360 |  135 |  -10 |   20 | -225 | -175 |  -95 | -150 | -150 |               0 | True              |
|   43 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -175 |        285 |   55 |   95 |  110 |   30 |   45 | -175 | -115 | -165 |               0 | True              |
|   44 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |        -90 |        110 |  -80 |   20 |  -50 |  -90 |  -75 |   -5 |  -10 |  -55 |               0 | True              |
|   45 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -525 |        525 | -215 | -330 | -525 | -510 | -515 | -510 | -430 | -445 |               3 | True              |
|   46 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |        -95 |        335 |  205 |  240 |  190 |  155 |   80 |  160 |  150 |  -95 |               0 | True              |
|   47 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        105 |       -145 |        250 |  105 |  -75 | -125 | -140 | -100 | -145 | -130 | -135 |               0 | True              |
|   48 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -605 |        605 | -390 | -360 | -535 | -605 | -510 | -530 | -515 | -525 |               4 | True              |
|   49 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |          0 |        240 |  195 |  140 |  185 |  240 |   55 |   20 |   15 |   10 |               0 | True              |
|   50 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 |  -30 | -175 | -130 | -185 | -115 | -195 | -120 | -245 |               0 | True              |
|   51 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        285 |          0 |        285 |  185 |  270 |  200 |  255 |  285 |  190 |   55 |   85 |               0 | True              |
|   52 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -460 |        460 | -200 | -315 | -395 | -340 | -395 | -335 | -460 | -395 |               3 | True              |
|   53 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        145 |       -175 |        320 |   90 |  145 |   90 | -160 | -175 | -130 |  -90 | -135 |               0 | True              |
|   54 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         55 |       -185 |        240 |   10 |  -35 |   45 |   55 |   40 |   50 | -185 | -115 |               1 | True              |
|   55 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -200 |        265 |   65 | -110 | -170 | -135 | -200 | -110 | -195 | -175 |               0 | True              |
|   56 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -175 |        275 |  100 |   70 |   40 |   60 |   -0 | -175 | -100 | -110 |               0 | True              |
|   57 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        225 |        -45 |        270 |  200 |  225 |   75 |  -45 |   -5 |   -5 |  -35 |   -0 |               0 | True              |
|   58 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -450 |        450 | -190 | -290 | -235 | -155 | -190 | -175 | -240 | -450 |               0 | True              |
|   59 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -185 |        290 |  105 |   -5 |  -20 |   -0 |  -30 |   -0 | -185 | -160 |               0 | True              |
|   60 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         50 |       -185 |        235 |   50 | -130 |  -55 | -185 | -105 |  -80 | -105 | -110 |               0 | True              |
|   61 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -210 |        320 |   35 |   65 |   45 |  110 |   55 | -130 | -135 | -210 |               0 | True              |
|   62 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         40 |       -185 |        225 |   40 |   30 |  -10 |   25 |  -65 |   10 |  -85 | -185 |               0 | True              |
|   63 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        265 |        -55 |        320 |  140 |   80 |  265 |  210 |  -10 |  -55 |  -35 |   70 |               0 | True              |
|   64 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        340 |          0 |        340 |  180 |  340 |  320 |  115 |  125 |   85 |  155 |  140 |               0 | True              |
|   65 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -625 |        625 | -240 | -505 | -535 | -565 | -565 | -565 | -575 | -625 |               3 | True              |
|   66 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -630 |        630 | -395 | -295 | -390 | -330 | -375 | -410 | -415 | -630 |               2 | True              |
|   67 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        165 |       -220 |        385 |  165 |   65 | -195 | -150 | -170 | -220 | -185 | -175 |               0 | True              |
|   68 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |       -180 |        260 |  -20 |   55 |   80 |   50 |  -15 |   70 |   -5 | -180 |               0 | True              |
|   69 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        300 |         -0 |        300 |  245 |  300 |  280 |  190 |   25 |   35 |   65 |   -0 |              16 | True              |
|   70 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |        -95 |        210 |   95 |   85 |  115 |    5 |  -45 |  -85 |  -70 |  -95 |               0 | True              |
|   71 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -470 |        470 | -245 | -145 | -470 | -400 | -450 | -420 | -460 | -415 |               3 | True              |
|   72 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -615 |        615 | -360 | -450 | -450 | -600 | -590 | -535 | -525 | -615 |               4 | True              |
|   73 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 | -155 | -160 |  -85 | -110 | -100 | -170 | -140 | -225 |               0 | True              |
|   74 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -160 |        320 |  145 |   80 |  160 |  120 |  -35 | -145 | -130 | -160 |               0 | True              |
|   75 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -530 |        530 | -335 | -380 | -320 | -285 | -495 | -530 | -515 | -525 |               4 | True              |
|   76 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 |  -80 | -115 | -140 | -135 | -165 | -195 | -135 |  -75 |               0 | True              |
|   77 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -190 |        260 |   70 | -150 | -190 | -110 |  -15 | -110 |  -95 | -160 |               0 | True              |
|   78 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -535 |        535 | -225 | -255 | -235 | -110 | -315 | -535 | -415 | -465 |               3 | True              |
|   79 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        315 |          0 |        315 |  190 |  265 |  315 |  200 |   70 |  125 |  130 |   20 |               0 | True              |
|   80 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |       -165 |        315 |  100 |   55 |  135 |   55 |  -35 |  150 |   40 | -165 |               0 | True              |
|   81 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |        -70 |        225 |  155 |   75 |  -10 |  -20 |  -10 |   35 |  -65 |  -70 |               0 | True              |
|   82 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         55 |       -260 |        315 |   25 |   55 |  -35 |   20 | -260 | -225 | -190 | -205 |               0 | True              |
|   83 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -225 |        225 |  -75 |  -70 | -125 | -155 | -160 | -225 | -135 | -210 |               1 | True              |
|   84 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         90 |       -210 |        300 |   90 |   75 | -105 |  -35 |  -50 |  -75 |  -40 | -210 |               0 | True              |
|   85 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        250 |        -30 |        280 |  160 |  240 |  220 |  190 |  250 |  240 |   10 |  -30 |               0 | True              |
|   86 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -320 |        320 |  -70 | -100 | -150 | -165 | -120 | -295 | -320 | -315 |               1 | True              |
|   87 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        350 |          0 |        350 |  200 |  350 |  245 |  215 |  225 |  165 |  280 |  190 |              14 | True              |
|   88 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 | -165 |  -30 | -115 | -105 | -170 |  -60 | -105 | -100 |               0 | True              |
|   89 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -390 |        390 | -155 | -100 | -390 | -360 | -345 | -285 | -240 | -315 |               2 | True              |
|   90 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -350 |        350 | -165 |  -85 | -185 | -125 | -130 | -195 | -170 | -350 |               0 | True              |
|   91 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        300 |         -5 |        305 |  180 |  290 |  215 |  215 |  300 |  200 |   -5 |  140 |               0 | True              |
|   92 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        195 |       -200 |        395 |    5 |   50 |   35 |    5 |  195 | -200 | -145 | -190 |               0 | True              |
|   93 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         35 |       -245 |        280 |   -5 |   35 |  -35 | -245 |  -85 | -180 | -145 | -165 |               0 | True              |
|   94 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        285 |        -30 |        315 |  195 |  205 |  285 |  190 |  245 |   15 |  -30 |  -10 |               0 | True              |
|   95 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         95 |       -195 |        290 |   40 |   40 |   55 |   50 |   95 |  -40 |   35 | -195 |               0 | True              |
|   96 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -220 |        305 |   85 | -220 |  -90 |  -75 | -120 | -160 |  -25 | -115 |               0 | True              |
|   97 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -170 |        360 |   80 |  -55 |  -20 |  115 |  190 | -165 | -170 | -100 |               0 | True              |
|   98 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -180 |        190 |   10 | -165 |  -85 |  -90 | -110 | -180 |  -70 | -115 |               0 | True              |
|   99 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         80 |       -230 |        310 |   45 |   -0 |   65 |   25 |   20 |   80 | -200 | -230 |               0 | True              |
|  100 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -435 |        435 | -195 | -180 | -205 | -275 | -265 | -255 | -275 | -435 |               0 | True              |
|  101 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -720 |        720 | -545 | -580 | -435 | -565 | -685 | -720 | -655 | -655 |               6 | True              |
|  102 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -585 |        585 | -445 | -440 | -480 | -480 | -485 | -585 | -435 | -475 |               3 | True              |
|  103 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -720 |        720 | -420 | -375 | -430 | -385 | -350 | -660 | -720 | -595 |               5 | True              |
|  104 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        145 |        -85 |        230 |  -85 |   75 |   75 |   25 |  145 |   60 |  -20 |   50 |               0 | True              |
|  105 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         10 |       -380 |        390 |  -65 | -125 | -165 | -135 |  -55 |   10 | -380 | -300 |               1 | True              |
|  106 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |        -50 |        280 |  120 |  230 |  200 |   65 |  -30 |  -50 |  -40 |   -5 |               0 | True              |
|  107 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -330 |        405 |   75 | -235 | -135 | -205 | -330 | -160 | -135 | -230 |               0 | True              |
|  108 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -235 |        295 |   60 |   55 | -180 | -230 | -215 | -235 | -125 |  -40 |               0 | True              |
|  109 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        200 |       -100 |        300 |  200 |  145 |  -20 |   70 |   35 |  -80 |  -70 | -100 |               0 | True              |
|  110 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         45 |       -170 |        215 |   10 |   -0 |   10 |    5 |   45 |  -25 | -170 | -105 |               0 | True              |
|  111 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        205 |        -95 |        300 |  205 |  120 |  105 |  -60 |  -20 |   45 |   50 |  -95 |               0 | True              |
|  112 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         60 |       -270 |        330 |  -25 |   60 |  -50 |   20 |  -25 | -205 | -270 | -215 |               1 | True              |
|  113 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         90 |       -210 |        300 |   90 |   90 |   80 |   55 |   50 | -200 | -210 |  -95 |               0 | True              |
|  114 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -390 |        390 | -120 | -220 | -150 | -200 | -200 | -360 | -390 | -345 |               2 | True              |
|  115 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         90 |       -295 |        385 |   90 |   55 |   75 | -235 | -175 | -160 | -295 | -130 |               0 | True              |
|  116 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         65 |       -100 |        165 | -100 |  -25 |  -10 |   65 |   45 |  -50 |   -0 | -100 |               0 | True              |
|  117 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 | -170 | -150 | -190 | -165 | -210 | -205 | -150 | -355 |               0 | True              |
|  118 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |        -55 |        185 |  130 |   90 |   -5 |   75 |  -50 |  -35 |  -55 |  -30 |               0 | True              |
|  119 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        170 |       -105 |        275 |  170 |  -75 | -105 |  -35 |  -10 |  -60 |  -30 |  -25 |               0 | True              |
|  120 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -625 |        625 | -280 | -320 | -305 | -275 | -430 | -335 | -625 | -470 |               4 | True              |
|  121 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         25 |       -225 |        250 |   25 |  -55 |  -50 |   -5 |  -90 |   -0 | -225 | -200 |               0 | True              |
|  122 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -605 |        605 | -335 | -350 | -310 | -380 | -470 | -605 | -480 | -540 |               4 | True              |
|  123 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        230 |        -55 |        285 |  175 |   65 |  230 |  190 |  -55 |   45 |  -40 |   90 |               0 | True              |
|  124 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         75 |       -240 |        315 |  -30 |  -45 |   -0 |   75 |  -65 |  -20 | -180 | -240 |               1 | True              |
|  125 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -230 |        240 |   10 | -200 | -225 | -230 | -170 | -200 | -210 | -205 |               0 | True              |
|  126 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -155 | -125 | -150 | -185 | -140 | -225 | -145 | -380 |               0 | True              |
|  127 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         60 |       -370 |        430 |  -55 |    5 |   -5 |  -40 |   60 | -370 | -270 | -235 |               1 | True              |
|  128 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -280 |        280 | -160 | -140 | -190 | -175 |  -55 |  -70 |  -80 | -280 |               0 | True              |
|  129 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -180 |        285 |  105 |  -45 |   10 |  -35 |  -10 |  -35 | -145 | -180 |               0 | True              |
|  130 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        220 |        -15 |        235 |   35 |  180 |  135 |  180 |  175 |  220 |   75 |  -15 |               0 | True              |
|  131 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         15 |       -115 |        130 |  -75 | -115 |  -75 |   15 |   -0 |  -45 | -105 |  -30 |               0 | True              |
|  132 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -360 |        360 | -100 | -195 | -110 |  -75 | -150 |  -70 | -160 | -360 |               0 | True              |
|  133 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -130 |        290 |   40 |  160 |   35 |  150 |  100 |   15 |  -55 | -130 |               0 | True              |
|  134 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -135 |        250 |  115 |  -60 |  -80 |  -25 |  -25 |  -25 |    5 | -135 |               0 | True              |
|  135 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -335 |        335 | -100 |  -30 | -105 | -120 |  -65 |  -85 | -130 | -335 |               0 | True              |
|  136 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -130 |        260 |  130 |   15 |   80 |  -75 |  -30 | -125 | -130 | -110 |               0 | True              |
|  137 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         25 |       -210 |        235 | -120 | -140 |   25 | -125 | -155 | -210 | -195 | -185 |               0 | True              |
|  138 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -325 |        385 |   10 |  -30 | -215 | -190 | -325 | -155 | -135 |   60 |               0 | True              |
|  139 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -220 |        220 |  -85 | -170 | -160 |  -80 |  -55 |  -80 | -120 | -220 |               0 | True              |
|  140 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -740 |        740 | -450 | -470 | -515 | -510 | -475 | -530 | -660 | -740 |               5 | True              |
|  141 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        320 |          0 |        320 |  170 |  155 |  255 |  165 |  320 |   85 |   25 |   80 |               0 | True              |
|  142 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        195 |        -95 |        290 |  195 |  195 |  -95 |  -50 |  -50 |  -15 |  -75 |  -95 |               0 | True              |
|  143 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |          0 |        200 |  185 |  200 |  200 |  195 |  150 |   15 |   15 |   25 |               0 | True              |
|  144 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         75 |       -220 |        295 |   50 |   75 | -220 |    5 |  -85 |  -25 |  -70 |  -80 |               0 | True              |
|  145 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -405 |        405 | -205 | -240 | -260 | -175 | -220 | -300 | -280 | -405 |               0 | True              |
|  146 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         70 |       -215 |        285 |   20 |   60 |   70 |  -35 |  -60 |   50 | -105 | -215 |               0 | True              |
|  147 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         65 |       -235 |        300 |   65 |   55 | -175 | -205 | -185 | -235 | -120 | -130 |               0 | True              |
|  148 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -200 |        320 |   70 |   50 |   25 |   30 |  120 |   -0 |  -85 | -200 |               0 | True              |
|  149 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -390 |        390 | -170 | -240 | -145 | -170 | -145 | -100 | -140 | -390 |               0 | True              |
|  150 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         10 |       -250 |        260 |   10 |   -0 | -170 |  -95 | -165 | -250 | -115 | -180 |               0 | True              |
|  151 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         75 |       -195 |        270 |   35 |   75 | -115 | -155 | -195 | -180 | -130 |  -75 |               0 | True              |
|  152 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         55 |       -240 |        295 |   55 |   -0 |   -5 |   30 | -240 | -175 | -140 | -155 |               0 | True              |
|  153 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -305 |        390 |   85 | -295 | -285 | -300 | -305 | -270 | -290 | -240 |               0 | True              |
|  154 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |       -110 |        315 |  150 |  205 |  -75 |  -80 |  -60 |    5 | -100 | -110 |               0 | True              |
|  155 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -705 |        705 | -460 | -670 | -530 | -625 | -650 | -705 | -630 | -705 |               5 | True              |
|  156 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -525 |        525 | -355 | -290 | -225 | -520 | -430 | -520 | -525 | -505 |               4 | True              |
|  157 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -345 |        345 | -260 | -260 | -285 | -225 | -230 | -275 | -345 | -175 |               1 | True              |
|  158 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -200 |        250 |   50 |   10 |    5 | -100 |  -45 |  -25 | -110 | -200 |               0 | True              |
|  159 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        235 |        -95 |        330 |  100 |  235 |  155 |   85 |  100 |  -45 |  -50 |  -95 |               0 | True              |
|  160 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -100 |        140 |   15 | -100 |  -70 |   40 |  -75 |  -30 |  -45 |  -80 |               0 | True              |
|  161 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -330 |        330 | -160 |  -80 | -330 | -205 | -325 | -295 | -265 | -230 |               2 | True              |
|  162 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         95 |       -220 |        315 |   90 |   15 |   95 | -185 | -220 | -110 |  -90 | -155 |               0 | True              |
|  163 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -570 |        570 | -430 | -390 | -425 | -325 | -330 | -525 | -545 | -570 |               5 | True              |
|  164 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -150 |        315 |  165 |  -70 | -135 | -150 | -125 |  -95 | -150 |  -80 |               0 | True              |
|  165 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -275 |        275 | -195 | -195 | -120 | -195 | -175 | -215 | -250 | -275 |               0 | True              |
|  166 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        135 |       -215 |        350 |   55 |   30 |   25 |  135 |   40 | -165 |  -80 | -215 |               0 | True              |
|  167 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -600 |        600 | -375 | -590 | -475 | -450 | -600 | -520 | -545 | -470 |               4 | True              |
|  168 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        290 |        -55 |        345 |  170 |  175 |  290 |  -25 |  -15 |   30 |   55 |  -55 |               0 | True              |
|  169 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         10 |       -270 |        280 |   -5 |  -50 |   10 |  -65 |  -55 |  -25 | -270 | -205 |               0 | True              |
|  170 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -265 |        350 |   85 | -225 | -170 | -235 | -195 | -265 | -180 | -220 |               0 | True              |
|  171 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -420 |        420 | -235 | -265 | -215 | -200 | -140 | -360 | -420 | -380 |               3 | True              |
|  172 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |        -90 |        205 |   45 |  115 |   85 |   -0 |   70 |  -25 |  -10 |  -90 |               0 | True              |
|  173 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -195 |        350 |  155 |   80 | -120 |  -15 | -190 |  -35 | -195 | -175 |               0 | True              |
|  174 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -570 |        570 | -390 | -400 | -310 | -365 | -270 | -340 | -570 | -565 |               4 | True              |
|  175 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -730 |        730 | -465 | -510 | -510 | -530 | -490 | -435 | -445 | -730 |               3 | True              |
|  176 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -260 |        260 | -180 | -260 |  -75 | -145 |  -75 | -220 | -220 | -165 |               0 | True              |
|  177 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -350 |        350 |  -15 | -150 | -270 | -160 | -350 | -230 | -300 | -140 |               1 | True              |
|  178 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |        -90 |        230 |  115 |  140 |   45 |   -0 |  -70 |   -0 |  -90 |  -35 |               0 | True              |
|  179 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -165 |        195 |   30 | -155 |  -35 | -165 | -115 |  -40 | -110 |  -85 |               0 | True              |
|  180 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -460 |        460 | -310 | -205 | -250 | -240 | -195 | -260 | -445 | -460 |               4 | True              |
|  181 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        350 |        -85 |        435 |  190 |  135 |  350 |  -40 |   -0 |  145 |  -85 |  -65 |               0 | True              |
|  182 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        335 |          0 |        335 |  205 |  295 |  225 |  335 |  100 |   55 |   30 |   60 |               0 | True              |
|  183 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |       -170 |        235 |   65 |  -60 |   30 |   60 |  -20 |   35 |   -5 | -170 |               0 | True              |
|  184 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         35 |       -315 |        350 |  -75 |  -30 |   35 | -160 | -245 | -240 | -200 | -315 |               1 | True              |
|  185 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -185 |        250 |   65 | -100 | -135 |  -80 |  -80 | -185 | -185 |  -35 |               0 | True              |
|  186 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 | -160 | -195 | -155 | -165 |  -35 |  -25 | -125 | -130 |               0 | True              |
|  187 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |       -160 |        330 |   55 |  170 |  -85 | -160 | -120 | -100 |  -60 |  -65 |               0 | True              |
|  188 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |       -190 |        330 |  140 |   75 |   75 | -125 | -190 | -180 | -170 | -185 |               0 | True              |
|  189 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -365 |        365 |  -40 | -365 | -280 | -285 | -210 | -265 | -270 | -205 |               1 | True              |
|  190 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -535 |        535 | -305 | -330 | -510 | -515 | -465 | -450 | -430 | -535 |               3 | True              |
|  191 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -545 |        545 | -355 | -410 | -370 | -355 | -430 | -395 | -340 | -545 |               2 | True              |
|  192 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -765 |        765 | -610 | -490 | -665 | -505 | -620 | -570 | -765 | -755 |               7 | True              |
|  193 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        185 |        -70 |        255 |  185 |  -15 |   10 |  -25 |  -70 |  -30 |   -0 |  -50 |               0 | True              |
|  194 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |        -40 |        270 |  175 |  205 |  230 |  -15 |  -40 |   85 |   90 |   25 |               0 | True              |
|  195 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -175 |        180 | -125 | -125 |    5 |  -30 |  -65 |  -35 |  -60 | -175 |               0 | True              |
|  196 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |       -115 |        265 |   15 |   95 |  130 |   95 |  150 |  110 |  145 | -115 |               0 | True              |
|  197 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         10 |       -340 |        350 |  -25 | -120 |  -40 |  -60 |   10 | -340 | -285 | -270 |               1 | True              |
|  198 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        255 |          0 |        255 |  175 |  255 |   50 |   75 |   45 |   30 |  150 |   40 |               0 | True              |
|  199 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        190 |          0 |        190 |  190 |   80 |   95 |   85 |   45 |   80 |   15 |   65 |              14 | True              |
|  200 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 | -140 | -140 | -110 | -190 | -135 | -225 | -165 | -355 |               0 | True              |
|  201 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         85 |        -85 |        170 |  -45 |   85 |  -30 |  -40 |  -85 |  -25 |   20 |  -55 |               0 | True              |
|  202 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        230 |        -70 |        300 |  200 |  210 |  230 |  175 |  -70 |   25 |  -55 |   30 |               0 | True              |
|  203 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -185 |        265 |   60 |   25 |   50 |   80 | -105 | -100 | -110 | -185 |               0 | True              |
|  204 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |       -160 |        390 |  135 |  230 |  160 | -115 |  -55 | -160 |  -35 |  -30 |               0 | True              |
|  205 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        220 |        -30 |        250 |  170 |  210 |  220 |  190 |  -30 |   15 |   35 |   25 |               0 | True              |
|  206 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -375 |        375 | -110 | -145 | -210 | -205 | -125 | -145 | -170 | -375 |               0 | True              |
|  207 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -215 |        305 |   70 |   90 |   70 |   85 |   55 |  -15 | -155 | -215 |               0 | True              |
|  208 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        330 |          0 |        330 |  285 |  330 |  325 |  305 |  170 |  295 |   80 |   95 |              16 | True              |
|  209 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         95 |       -190 |        285 |   80 |   15 |   35 |    5 |   95 |   15 | -190 | -170 |               0 | True              |
|  210 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        255 |          0 |        255 |  205 |  205 |  255 |  200 |   95 |   15 |   80 |   15 |               0 | True              |
|  211 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -225 | -135 | -140 | -255 | -145 | -130 | -195 | -165 |               1 | True              |
|  212 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -320 |        320 | -240 | -305 | -190 | -320 | -250 | -275 | -215 | -260 |               1 | True              |
|  213 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -220 |        235 |   -5 |  -45 |   15 |  -10 |   -5 |  -30 |   -0 | -220 |               0 | True              |
|  214 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |        -30 |        235 |  185 |  185 |  150 |  205 |  140 |  -20 |  -30 |  -25 |               0 | True              |
|  215 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         30 |       -300 |        330 |   25 |  -55 |  -45 |   30 |  -55 | -200 | -300 | -230 |               0 | True              |
|  216 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -500 |        500 | -435 | -500 | -375 | -455 | -420 | -495 | -330 | -395 |               3 | True              |
|  217 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         35 |       -190 |        225 |   35 | -175 |  -75 |   -5 | -190 | -125 | -100 |  -65 |               0 | True              |
|  218 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 |  -75 | -185 | -135 | -165 | -145 | -135 | -195 |  -85 |               0 | True              |
|  219 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |       -125 |        275 |   30 |   85 |  135 |  100 |  150 |  145 |  135 | -125 |               0 | True              |
|  220 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -165 |        280 |  115 | -155 | -135 |  -25 | -105 | -140 | -145 | -165 |               0 | True              |
|  221 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         25 |        -75 |        100 |  -75 |   -0 |  -55 |  -30 |   15 |  -55 |   25 |  -20 |               0 | True              |
|  222 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -530 |        530 | -290 | -365 | -460 | -350 | -405 | -530 | -525 | -400 |               4 | True              |
|  223 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -295 |        375 |   80 |  -25 |   20 |  -45 | -295 | -265 | -195 | -175 |               0 | True              |
|  224 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         10 |       -290 |        300 |   10 |   -0 | -290 | -200 | -180 | -195 | -200 |  -75 |               0 | True              |
|  225 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |       -210 |        385 |   55 |   55 |   25 |  175 |   70 |  135 | -165 | -210 |               0 | True              |
|  226 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -155 |        155 |  -15 |  -25 | -110 | -125 |  -45 |  -90 | -155 |  -80 |               0 | True              |
|  227 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -145 |        310 |  165 |  -30 |  -60 |  -95 |   -5 | -145 |  -55 |   40 |               0 | True              |
|  228 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -280 |        280 | -130 | -210 | -250 | -220 | -175 | -280 | -225 | -130 |               0 | True              |
|  229 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        215 |        -40 |        255 |  215 |  100 |  195 |  145 |  165 |  -25 |   10 |  -40 |               0 | True              |
|  230 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -160 |        280 |  120 | -125 | -100 |   15 |  -25 |  -45 |  -20 | -160 |               0 | True              |
|  231 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          5 |       -230 |        235 |   -0 |    5 |  -85 | -195 |  -95 | -220 | -230 | -195 |               0 | True              |
|  232 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -310 |        310 | -150 | -230 | -310 | -195 | -165 | -215 | -150 | -190 |               0 | True              |
|  233 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -795 |        795 | -540 | -525 | -570 | -575 | -500 | -505 | -795 | -645 |               6 | True              |
|  234 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -195 |        230 |  -30 | -100 |  -55 |   35 |  -15 |    5 |  -60 | -195 |               0 | True              |
|  235 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        315 |        -15 |        330 |  205 |  315 |  210 |  240 |  195 |  295 |  190 |  -15 |              14 | True              |
|  236 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        155 |        -75 |        230 |   85 |  115 |  135 |  155 |  110 |  155 |  -75 |  -30 |               0 | True              |
|  237 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |       -130 |        285 |   90 |  155 |   80 |   90 |    5 |  -80 |  -25 | -130 |               0 | True              |
|  238 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |        -40 |        220 |   55 |  180 |  120 |   65 |  175 |  145 |  -35 |  -40 |               0 | True              |
|  239 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        245 |        -70 |        315 |  140 |  175 |  245 |  210 |  180 |  180 |  115 |  -70 |               0 | True              |
|  240 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -630 |        630 | -365 | -265 | -460 | -390 | -420 | -380 | -580 | -630 |               4 | True              |
|  241 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -305 |        305 |  -60 |  -65 |  -70 | -110 |  -20 | -305 | -265 | -245 |               1 | True              |
|  242 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -370 |        370 |  -75 |  -60 | -105 | -100 |  -95 |  -25 | -190 | -370 |               0 | True              |
|  243 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |        -20 |        160 |   80 |   20 |   80 |  120 |  140 |  140 |   30 |  -20 |               0 | True              |
|  244 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -270 |        290 |  -85 |   -0 |  -20 | -195 | -110 |  -65 |   20 | -270 |               0 | True              |
|  245 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -540 |        540 | -380 | -480 | -380 | -465 | -340 | -485 | -345 | -540 |               2 | True              |
|  246 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -345 |        345 |  -20 |  -40 | -345 | -315 | -235 | -230 | -335 | -270 |               1 | True              |
|  247 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -505 |        505 | -395 | -440 | -470 | -435 | -500 | -450 | -505 | -435 |               2 | True              |
|  248 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         60 |       -295 |        355 |   60 |   15 |   35 |   55 |   55 |  -85 | -120 | -295 |               0 | True              |
|  249 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        255 |        -40 |        295 |  190 |  215 |  255 |  190 |  -20 |  -20 |  -40 |   75 |               0 | True              |
|  250 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         30 |       -350 |        380 |   30 |  -30 |  -75 | -350 | -280 | -320 | -290 | -325 |               0 | True              |
|  251 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         45 |       -215 |        260 |   45 |  -15 | -205 | -215 | -200 | -155 | -195 | -190 |               0 | True              |
|  252 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -390 |        390 | -130 |  -90 | -165 | -225 | -235 | -200 | -245 | -390 |               0 | True              |
|  253 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -300 |        300 | -175 | -150 | -270 | -220 | -300 | -205 | -255 | -290 |               0 | True              |
|  254 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -160 |        335 |  175 |   60 |  -20 |   70 |   60 | -150 |  -55 | -160 |               0 | True              |
|  255 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         15 |       -280 |        295 |   15 |  -40 | -265 | -230 | -245 | -215 | -245 | -280 |               0 | True              |
|  256 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -285 |        285 | -115 | -120 |  -60 | -180 | -140 |  -65 | -105 | -285 |               0 | True              |
|  257 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        235 |        -10 |        245 |  170 |  190 |  180 |  190 |  235 |  225 |   55 |  -10 |               0 | True              |
|  258 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         45 |       -285 |        330 |   45 |  -80 | -185 | -245 | -220 | -285 | -240 | -240 |               0 | True              |
|  259 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -210 |        210 | -180 | -135 | -210 | -150 | -180 | -190 | -160 | -185 |               0 | True              |
|  260 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -655 |        655 | -510 | -520 | -430 | -655 | -535 | -630 | -600 | -650 |               6 | True              |
|  261 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        260 |        -70 |        330 |  110 |  260 |  165 |  175 |   20 |   25 |  -10 |  -70 |               0 | True              |
|  262 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -215 |        215 | -110 | -190 | -130 | -215 | -170 | -140 | -175 | -165 |               0 | True              |
|  263 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |        -95 |        115 |  -45 |  -75 |  -80 |  -50 |   20 |  -95 |  -80 |  -95 |               0 | True              |
|  264 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -770 |        770 | -505 | -660 | -625 | -705 | -770 | -730 | -680 | -690 |               6 | True              |
|  265 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 |   -5 |  -55 |  -90 |  -55 | -180 | -130 |  -20 | -340 |               0 | True              |
|  266 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        260 |          0 |        260 |  190 |  260 |  220 |  110 |   70 |   15 |   40 |   90 |               0 | True              |
|  267 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -280 |        280 | -145 | -120 | -180 | -165 | -170 | -280 | -115 | -120 |               0 | True              |
|  268 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -345 |        345 |  -15 | -220 | -265 | -265 | -255 | -345 | -240 | -300 |               1 | True              |
|  269 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -210 |        330 |   45 |  120 | -125 | -155 | -190 |  -95 | -135 | -210 |               0 | True              |
|  270 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -160 |        270 |  110 |   60 |   65 |   85 |   70 | -155 | -130 | -160 |               0 | True              |
|  271 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -735 |        735 | -595 | -630 | -525 | -520 | -520 | -605 | -735 | -660 |               6 | True              |
|  272 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        265 |        -20 |        285 |  190 |  140 |  100 |  265 |  195 |  100 |  -20 |   95 |               0 | True              |
|  273 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -605 |        605 | -305 | -500 | -555 | -505 | -605 | -535 | -415 | -525 |               4 | True              |
|  274 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        145 |       -140 |        285 |   75 |  145 |   80 |  -35 |  -95 |  -35 | -140 |  -95 |               0 | True              |
|  275 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -265 |        370 |  105 |   55 |   -0 | -135 |  -10 |  -65 |  -80 | -265 |               0 | True              |
|  276 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |       -125 |        255 |  130 |   80 |  -85 | -100 |  -90 |  -95 | -100 | -125 |               0 | True              |
|  277 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |        -20 |        190 |  130 |  170 |  -20 |   70 |   50 |  100 |   -0 |   45 |               0 | True              |
|  278 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |        -85 |        170 |   75 |   45 |   75 |   80 |   -0 |   20 |   85 |  -85 |               0 | True              |
|  279 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |        -70 |        270 |  150 |   90 |   90 |  155 |  200 |  -15 |  -70 |  -65 |               0 | True              |
|  280 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |       -220 |        300 |   80 |  -85 |  -15 |  -35 |  -40 | -220 | -160 | -190 |               0 | True              |
|  281 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        120 |       -135 |        255 |  120 |   80 |   40 |  -40 | -115 | -135 |  -75 | -120 |               0 | True              |
|  282 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        310 |        -35 |        345 |  175 |  190 |  310 |  300 |   50 |   40 |  -35 |   20 |               0 | True              |
|  283 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -155 |        155 | -115 |  -60 | -140 |  -65 |  -65 | -155 | -115 | -135 |               0 | True              |
|  284 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -350 |        350 | -140 | -140 | -215 | -210 | -250 | -155 | -170 | -350 |               0 | True              |
|  285 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |       -120 |        300 |  140 |  180 |   75 |  -15 |  -90 |   -5 | -120 | -115 |               0 | True              |
|  286 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -630 |        630 | -445 | -360 | -470 | -315 | -430 | -510 | -375 | -630 |               3 | True              |
|  287 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        125 |       -175 |        300 |   35 |  105 |  125 |   30 | -135 | -125 | -175 |  -85 |               0 | True              |
|  288 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         90 |       -245 |        335 |  -20 |   30 |   90 |   10 | -245 | -240 | -210 |  -75 |               0 | True              |
|  289 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        105 |       -110 |        215 |  105 |  -10 | -110 |  -85 | -105 |  -30 |  -40 | -100 |               0 | True              |
|  290 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        165 |        -70 |        235 |  130 |  165 |  105 |   70 |  -55 |  -70 |  -55 |   55 |               0 | True              |
|  291 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        130 |       -275 |        405 |   10 |  -20 |  130 |   60 |  -55 | -195 | -190 | -275 |               0 | True              |
|  292 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -540 |        540 | -240 | -175 | -105 | -155 | -380 | -540 | -450 | -380 |               3 | True              |
|  293 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        335 |       -110 |        445 |  100 |  155 |  165 |  335 |  150 |  215 |  -50 | -110 |               0 | True              |
|  294 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         -0 |       -265 |        265 |   -0 | -120 |  -75 | -235 | -210 | -205 | -265 | -210 |               0 | True              |
|  295 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -490 |        490 | -285 | -320 | -315 | -270 | -345 | -310 | -490 | -475 |               3 | True              |
|  296 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -410 |        410 | -190 | -335 | -410 | -260 | -205 | -185 | -340 | -325 |               0 | True              |
|  297 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -245 |        365 |  120 |   90 | -125 |  -25 |  -55 | -140 |  -90 | -245 |               0 | True              |
|  298 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -245 |        305 |   30 |   60 | -120 | -245 | -175 | -200 | -210 | -230 |               0 | True              |
|  299 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -140 |        180 |  -40 |   40 | -105 | -140 |  -80 |  -90 |  -80 | -100 |               0 | True              |
|  300 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |        -55 |        245 |  165 |   95 |  190 |   -5 |   20 |   15 |  -55 |  -30 |               0 | True              |
|  301 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |        -40 |        240 |  145 |   80 |  200 |  -15 |  -40 |  -20 |   10 |   35 |               0 | True              |
|  302 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |        -85 |        220 |   95 |  105 |  135 |  -85 |  -30 |   10 |   60 |   -0 |               0 | True              |
|  303 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        210 |       -115 |        325 |  210 |   80 |   45 |  100 |  130 |  160 |  165 | -115 |               0 | True              |
|  304 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -360 |        360 | -175 | -110 |  -70 | -260 | -125 |  -45 |  -65 | -360 |               0 | True              |
|  305 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |        -60 |        220 |   65 |   80 |  160 |  130 |  105 |   70 |  100 |  -60 |               0 | True              |
|  306 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -600 |        600 | -280 | -375 | -505 | -505 | -520 | -540 | -600 | -510 |               4 | True              |
|  307 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -475 |        475 | -340 | -290 |  -75 | -395 | -475 | -285 | -385 | -355 |               4 | True              |
|  308 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -690 |        690 | -605 | -610 | -680 | -580 | -560 | -555 | -690 | -670 |               5 | True              |
|  309 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -350 |        350 | -165 | -350 | -165 | -290 | -230 | -220 | -260 | -235 |               0 | True              |
|  310 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        195 |        -45 |        240 |  150 |  160 |  195 |  -45 |  -40 |  -15 |   70 |   45 |               0 | True              |
|  311 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |       -205 |        315 |  110 |   65 |   60 |   95 |   25 |  -45 |   -5 | -205 |               0 | True              |
|  312 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -170 |        220 |  -90 |  -20 |  -75 |  -60 |   -0 | -105 |   50 | -170 |               0 | True              |
|  313 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         70 |       -220 |        290 |   70 |  -45 | -195 |  -95 | -110 | -155 | -220 | -150 |               0 | True              |
|  314 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |       -235 |        315 |   -0 |   40 |   80 | -175 | -160 | -235 | -215 | -155 |               0 | True              |
|  315 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -755 |        755 | -425 | -640 | -720 | -605 | -590 | -750 | -690 | -755 |               5 | True              |
|  316 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         95 |       -200 |        295 |   80 |  -20 |   95 |  -60 | -130 | -200 | -105 | -180 |               0 | True              |
|  317 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -190 | -265 | -240 | -240 | -140 | -185 |  -65 | -165 |               0 | True              |
|  318 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -470 |        470 | -125 | -225 | -290 | -335 | -335 | -410 | -470 | -405 |               2 | True              |
|  319 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -495 |        495 | -110 | -195 | -235 | -140 | -495 | -325 | -405 | -395 |               2 | True              |
|  320 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -165 | -245 | -135 | -130 | -100 | -195 | -210 | -190 |               0 | True              |
|  321 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -360 |        360 | -295 | -130 | -360 | -260 | -265 | -190 | -255 | -275 |               1 | True              |
|  322 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -690 |        690 | -360 | -595 | -690 | -645 | -620 | -620 | -575 | -645 |               4 | True              |
|  323 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -385 |        385 |  -20 | -100 |  -75 | -175 | -120 | -130 | -215 | -385 |               1 | True              |
|  324 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         65 |       -180 |        245 |  -85 |   10 |    5 |  -20 |  -10 |   65 | -175 | -180 |               1 | True              |
|  325 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |        -80 |        250 |  170 |   65 |   25 |   70 |   90 |   90 |   80 |  -80 |               0 | True              |
|  326 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        105 |       -150 |        255 |   95 |  105 | -150 |  -85 | -105 | -145 |  -15 |  -70 |               0 | True              |
|  327 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -690 |        690 | -375 | -380 | -515 | -450 | -690 | -665 | -650 | -665 |               4 | True              |
|  328 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        125 |       -165 |        290 |  125 |  115 |  100 | -105 |  -85 |  -75 | -165 |  -55 |               0 | True              |
|  329 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |       -155 |        350 |  145 |   85 |  195 |  125 |  130 |  105 | -155 |  -85 |               0 | True              |
|  330 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         10 |       -285 |        295 |   10 |  -80 |  -95 | -245 | -285 | -220 | -220 | -195 |               0 | True              |
|  331 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -340 |        340 | -195 | -340 | -220 | -235 | -195 | -235 | -180 | -255 |               0 | True              |
|  332 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        105 |       -100 |        205 |  105 |   10 |   80 |  -90 |  -75 | -100 |  -35 |  -55 |               0 | True              |
|  333 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |       -110 |        290 |  180 |  135 |  120 |   -5 | -105 | -110 |  -90 |  -15 |               0 | True              |
|  334 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -590 |        590 | -245 | -320 | -360 | -385 | -325 | -590 | -545 | -545 |               3 | True              |
|  335 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        205 |        -80 |        285 |  195 |  205 |  105 |  175 |  -20 |  -80 |  -35 |   60 |               0 | True              |
|  336 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -465 |        465 | -300 | -410 | -390 | -390 | -385 | -465 | -415 | -385 |               4 | True              |
|  337 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -425 |        425 | -165 | -125 | -230 | -295 | -175 | -125 | -210 | -425 |               0 | True              |
|  338 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -640 |        640 | -405 | -595 | -545 | -575 | -555 | -640 | -525 | -430 |               5 | True              |
|  339 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -545 |        545 | -355 | -265 | -320 | -285 | -445 | -500 | -545 | -535 |               4 | True              |
|  340 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -495 |        495 | -295 | -290 | -425 | -355 | -275 | -495 | -370 | -325 |               1 | True              |
|  341 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         50 |       -175 |        225 |   35 |    5 |   50 | -140 |  -70 | -175 | -115 |  -90 |               0 | True              |
|  342 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         85 |        -35 |        120 |  -35 |   75 |   -0 |   85 |   55 |   -5 |  -25 |   80 |               0 | True              |
|  343 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -165 |        270 |  -75 |   30 |   35 |   50 |  -45 |  105 |   -5 | -165 |               0 | True              |
|  344 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |       -175 |        285 |   55 |    5 |  100 |  110 |   65 |   40 |   60 | -175 |               0 | True              |
|  345 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -215 |        355 |  140 |   55 |   65 |   10 |  -60 | -115 | -215 | -165 |               0 | True              |
|  346 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -195 |        330 |  135 | -195 |  -75 | -185 | -140 | -140 |  -95 | -120 |               0 | True              |
|  347 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        190 |       -110 |        300 |  180 |  190 |  -15 | -110 |  -10 |  -20 | -100 |    5 |               0 | True              |
|  348 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |       -130 |        330 |  130 |  115 |  185 |  100 |  200 |  145 |  -90 | -130 |               0 | True              |
|  349 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 | -170 | -150 | -165 |  -15 | -140 | -170 | -105 | -120 |               0 | True              |
|  350 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -435 |        435 | -135 | -120 | -130 | -435 | -330 | -305 | -400 | -390 |               2 | True              |
|  351 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |       -305 |        420 |   15 |  115 |   60 | -305 | -140 | -175 | -115 | -125 |               0 | True              |
|  352 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -205 |        285 |   80 |   75 |  -30 |  -50 | -195 |  -75 | -155 | -205 |               0 | True              |
|  353 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -530 |        530 | -255 | -530 | -470 | -515 | -485 | -525 | -490 | -480 |               3 | True              |
|  354 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -545 |        545 | -285 | -285 | -465 | -545 | -490 | -500 | -525 | -455 |               3 | True              |
|  355 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -285 |        285 |  -90 | -155 |  -80 | -150 | -210 |  -65 | -135 | -285 |               0 | True              |
|  356 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -345 |        345 |  -10 | -125 | -260 |  -75 |  -25 | -130 | -345 | -295 |               1 | True              |
|  357 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        105 |       -350 |        455 |  105 | -250 | -265 | -210 | -265 | -350 | -235 | -330 |               0 | True              |
|  358 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        235 |        -55 |        290 |   55 |  235 |  190 |  -20 |   55 |   20 |   -0 |  -55 |               0 | True              |
|  359 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -265 |        265 | -125 | -135 |  -95 |  -95 | -145 |  -95 | -135 | -265 |               0 | True              |
|  360 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        185 |        -45 |        230 |  185 |   90 |  175 |  175 |   85 |   -0 |   10 |  -45 |               0 | True              |
|  361 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        205 |        -10 |        215 |  145 |  205 |  195 |   40 |  125 |   -5 |  -10 |  125 |               0 | True              |
|  362 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -100 |        220 |  120 |  -75 | -100 |  -90 |   20 |   50 |  -60 |  -15 |               0 | True              |
|  363 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -465 |        465 | -180 | -220 | -220 | -245 | -160 | -240 | -275 | -465 |               0 | True              |
|  364 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        285 |        -35 |        320 |  270 |  195 |  185 |  170 |  285 |  285 |  160 |  -35 |              14 | True              |
|  365 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -290 |        290 |  -15 |  -15 | -130 | -265 | -290 | -225 | -200 | -280 |               1 | True              |
|  366 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        265 |        -15 |        280 |   65 |  175 |  265 |  250 |  215 |  195 |    5 |  -15 |               0 | True              |
|  367 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |       -210 |        290 |  -10 |   40 |   10 |  -40 |   80 |  -45 |   -5 | -210 |               0 | True              |
|  368 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -135 |        255 |  120 |   80 | -130 | -130 |  -60 | -135 |  -15 |   -0 |               0 | True              |
|  369 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |        -45 |        220 |   55 |  175 |   60 |  110 |   55 |  105 |   80 |  -45 |               0 | True              |
|  370 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -215 |        215 | -105 |  -90 | -115 | -180 |  -60 | -215 | -145 | -165 |               0 | True              |
|  371 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        105 |       -240 |        345 |  105 |   80 |   85 | -240 | -205 | -205 | -195 | -125 |               0 | True              |
|  372 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |        -65 |        225 |   90 |  110 |   70 |  160 |   95 |   90 |  130 |  -65 |               0 | True              |
|  373 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -375 |        375 | -200 | -340 | -160 | -155 | -290 | -260 | -145 | -375 |               0 | True              |
|  374 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |       -130 |        280 |  150 |  110 |   65 |  -60 |  -95 |  -75 |  -65 | -130 |               0 | True              |
|  375 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -380 |        380 |  -45 | -135 | -380 | -330 | -345 | -300 | -225 | -260 |               1 | True              |
|  376 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |       -155 |        355 |  200 |  145 |  155 |   55 |  125 |  100 | -155 | -155 |               0 | True              |
|  377 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -160 |        275 |  115 |  -20 | -135 | -110 | -120 | -110 |  -55 | -160 |               0 | True              |
|  378 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |        -10 |        100 |   25 |   90 |   -0 |   70 |   -0 |   25 |   30 |  -10 |               0 | True              |
|  379 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        220 |          0 |        220 |  200 |   35 |  220 |   50 |  130 |   75 |  155 |   55 |               0 | True              |
|  380 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -120 |        285 |  165 |  -60 |  -85 | -115 | -120 |  -55 |  -85 | -110 |               0 | True              |
|  381 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         50 |       -270 |        320 |   50 |  -10 |   30 |  -10 | -245 | -270 | -255 | -210 |               0 | True              |
|  382 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -730 |        730 | -435 | -615 | -515 | -580 | -660 | -700 | -700 | -730 |               5 | True              |
|  383 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -220 |        375 |  155 |   35 | -195 | -205 | -185 | -220 | -215 | -130 |               0 | True              |
|  384 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -210 | -175 | -185 | -225 | -110 | -175 | -245 | -175 |               0 | True              |
|  385 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |       -145 |        190 |  -35 |  -20 | -145 |   45 |  -15 | -100 |   -5 |  -60 |               0 | True              |
|  386 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -405 |        405 | -235 | -235 | -300 | -355 | -310 | -265 | -405 | -300 |               1 | True              |
|  387 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -305 |        355 |    5 |  -50 |  -10 |  -10 |    5 |   50 |  -85 | -305 |               0 | True              |
|  388 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -275 |        275 | -155 | -120 | -120 | -180 | -175 | -175 | -275 | -200 |               0 | True              |
|  389 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        255 |          0 |        255 |  180 |  220 |  225 |  255 |  205 |   20 |   35 |   15 |               0 | True              |
|  390 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -475 |        475 | -110 | -110 | -110 | -160 | -155 | -110 |  -90 | -475 |               0 | True              |
|  391 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -115 |        115 |   -5 |  -95 |  -50 |  -80 | -115 |  -40 |  -65 |  -85 |               0 | True              |
|  392 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -560 |        560 | -350 | -250 | -545 | -530 | -560 | -525 | -535 | -555 |               4 | True              |
|  393 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -435 |        435 | -165 | -160 | -340 | -330 | -325 | -435 | -350 | -415 |               2 | True              |
|  394 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -670 |        670 | -400 | -455 | -430 | -615 | -645 | -615 | -630 | -670 |               5 | True              |
|  395 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -490 |        490 | -345 | -190 | -370 | -470 | -395 | -455 | -415 | -490 |               4 | True              |
|  396 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        225 |        -45 |        270 |  150 |  175 |  195 |  225 |   15 |  -15 |  -35 |  -45 |               0 | True              |
|  397 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -175 |        295 |  120 |  -20 | -140 |  -85 | -135 |  -70 | -175 | -110 |               0 | True              |
|  398 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -150 |        285 |  135 |  -65 |  -85 | -150 |  -95 | -105 |  -90 | -100 |               0 | True              |
|  399 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |       -195 |        345 |   60 |  150 |  100 |   40 |  -15 |   45 |   55 | -195 |               0 | True              |
|  400 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        250 |        -35 |        285 |  180 |   70 |  225 |  185 |  145 |  250 |  215 |  -35 |               0 | True              |
|  401 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |          0 |        185 |   90 |  150 |   95 |  160 |  180 |  185 |   95 |   40 |               0 | True              |
|  402 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -440 |        440 | -345 | -390 | -365 | -365 | -440 | -355 | -370 | -350 |               2 | True              |
|  403 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        275 |        -35 |        310 |  160 |  275 |  235 |  195 |  170 |  -15 |  -35 |   90 |               0 | True              |
|  404 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -325 |        325 | -160 | -300 | -140 | -200 | -300 | -325 | -210 | -270 |               0 | True              |
|  405 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -670 |        670 | -340 | -370 | -400 | -465 | -420 | -670 | -580 | -570 |               4 | True              |
|  406 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -470 |        470 | -410 | -410 | -470 | -395 | -275 | -325 | -430 | -435 |               3 | True              |
|  407 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        160 |       -185 |        345 |  160 |  -40 |   15 |  -80 |  -55 | -185 | -100 |   20 |               0 | True              |
|  408 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        300 |        -20 |        320 |  185 |  200 |  300 |   55 |  -10 |   40 |  -20 |    5 |               0 | True              |
|  409 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -125 |        290 |  165 |   45 |  -25 |  -60 | -125 |    5 |  -50 |  -55 |               0 | True              |
|  410 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -395 |        395 | -175 | -275 | -225 | -205 | -200 | -395 | -325 | -355 |               2 | True              |
|  411 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -300 |        300 |  -85 | -130 | -155 | -165 |  -80 | -170 | -110 | -300 |               0 | True              |
|  412 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |       -105 |        185 |   40 |   30 |    5 |   70 |   80 |   50 |   60 | -105 |               0 | True              |
|  413 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        245 |       -145 |        390 |  200 |  245 |  -90 | -100 |  -10 | -110 |  -70 | -145 |               0 | True              |
|  414 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -110 |        320 |  140 |  210 |  110 | -110 | -100 |  -65 |  -55 |  -35 |               0 | True              |
|  415 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        105 |       -240 |        345 |   20 |   75 |   55 |  105 | -180 | -155 | -145 | -240 |               0 | True              |
|  416 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |        -30 |        225 |   80 |   95 |  115 |  195 |  195 |  170 |  -15 |  -30 |               0 | True              |
|  417 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -670 |        670 | -410 | -455 | -415 | -385 | -565 | -490 | -600 | -670 |               5 | True              |
|  418 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -695 |        695 | -510 | -485 | -485 | -460 | -520 | -525 | -470 | -695 |               3 | True              |
|  419 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        145 |       -170 |        315 |  145 |  -50 |   35 | -170 |  -70 |  -20 |  -40 |  -20 |               0 | True              |
|  420 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -295 |        295 | -105 | -120 |  -75 | -150 |  -95 | -195 |  -95 | -295 |               0 | True              |
|  421 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -360 |        360 |  -40 | -280 | -290 | -320 | -280 | -360 | -300 | -295 |               1 | True              |
|  422 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -455 |        455 | -245 | -225 | -125 | -155 | -180 | -200 | -455 | -425 |               3 | True              |
|  423 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -215 |        215 |  -60 |  -55 |  -60 |  -85 |  -30 | -100 |  -35 | -215 |               0 | True              |
|  424 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         25 |       -240 |        265 |   25 |  -65 |   25 | -215 | -240 | -220 | -210 | -240 |               0 | True              |
|  425 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -730 |        730 | -460 | -490 | -430 | -430 | -415 | -730 | -505 | -650 |               5 | True              |
|  426 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         10 |       -360 |        370 |   -0 |  -20 |   10 |  -50 |  -85 | -150 | -360 | -215 |               0 | True              |
|  427 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -270 |        380 |  -20 |  -40 |  110 | -240 | -250 | -215 | -255 | -270 |               1 | True              |
|  428 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         55 |       -180 |        235 |  -85 |  -90 |  -10 |   55 |   50 | -145 | -180 | -115 |               1 | True              |
|  429 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        135 |          0 |        135 |  125 |  115 |  105 |  135 |   90 |   90 |   50 |   35 |               0 | True              |
|  430 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        260 |        -30 |        290 |  155 |  190 |  260 |  -15 |  -30 |   20 |  110 |  -20 |               0 | True              |
|  431 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         85 |       -190 |        275 |   85 |   35 | -155 | -190 | -135 | -145 | -110 | -140 |               0 | True              |
|  432 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        270 |        -10 |        280 |  140 |  160 |  270 |  160 |  260 |  190 |  -10 |    5 |               0 | True              |
|  433 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |        -35 |        205 |  170 |  100 |   75 |   95 |   50 |  140 |   80 |  -35 |               0 | True              |
|  434 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        255 |        -20 |        275 |  160 |  110 |  135 |  255 |  100 |  190 |  -20 |   10 |               0 | True              |
|  435 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -225 |        325 |   55 |   90 |   85 |  100 |  -50 | -115 | -150 | -225 |               0 | True              |
|  436 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |        -30 |        185 |  155 |  -30 |   60 |   80 |   80 |   55 |   35 |   55 |               0 | True              |
|  437 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |        -80 |        230 |  120 |  105 |   85 |   70 |  150 |   40 |  110 |  -80 |               0 | True              |
|  438 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -125 |        245 |  100 |  120 |  -95 |  -80 | -100 | -125 | -100 | -110 |               0 | True              |
|  439 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        130 |       -170 |        300 |   85 |  130 |   65 |   40 | -115 | -170 | -100 | -145 |               0 | True              |
|  440 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -345 |        345 |  -40 |  -55 | -100 | -240 | -265 | -345 | -245 | -255 |               1 | True              |
|  441 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -470 |        470 |  -75 | -205 | -470 | -445 | -415 | -400 | -390 | -390 |               1 | True              |
|  442 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -640 |        640 | -390 | -305 | -375 | -405 | -630 | -560 | -540 | -640 |               5 | True              |
|  443 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |        -60 |        245 |   65 |   70 |  185 |   75 |  130 |  100 |  135 |  -60 |               0 | True              |
|  444 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |       -190 |        300 |  110 |   25 |  -90 | -145 | -140 | -170 | -150 | -190 |               0 | True              |
|  445 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -525 |        525 | -330 | -250 | -445 | -495 | -440 | -490 | -525 | -480 |               4 | True              |
|  446 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        170 |       -140 |        310 |  170 | -140 |  -10 |  -45 |  -15 |  -30 |   25 |  -20 |               0 | True              |
|  447 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -210 |        350 |   90 |  140 |   85 |   95 | -200 | -210 | -115 | -150 |               0 | True              |
|  448 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |       -125 |        255 |  130 |  -60 | -120 | -125 |  -85 |  -50 |  -20 |  -45 |               0 | True              |
|  449 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        105 |       -140 |        245 |   60 |   90 |  100 |   30 |  105 | -130 | -140 |  -80 |               0 | True              |
|  450 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -240 |        240 | -105 | -195 | -240 | -170 | -225 | -230 | -205 | -150 |               0 | True              |
|  451 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |        -75 |        190 |  105 |  115 |   50 |  -70 |   20 |   15 |   10 |  -75 |               0 | True              |
|  452 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        265 |        -70 |        335 |  200 |  255 |  225 |  265 |   90 |  -70 |   25 |  -10 |               0 | True              |
|  453 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        185 |       -165 |        350 |  185 |  165 |  -10 |  -50 | -165 |  -35 | -115 |  -85 |               0 | True              |
|  454 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |       -260 |        325 |   15 |    5 |  -15 |   15 |   60 |   65 |   35 | -260 |               0 | True              |
|  455 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -200 |        250 |   -5 |  -25 |   10 |  -65 |   35 |  -60 |   50 | -200 |               0 | True              |
|  456 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |       -100 |        270 |   80 |   60 |  120 |   65 |  170 |   70 | -100 |  -85 |               0 | True              |
|  457 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        350 |          0 |        350 |  190 |  350 |  250 |  210 |  300 |  275 |  165 |  130 |               0 | True              |
|  458 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |       -120 |        365 |  185 |  245 |  230 |  170 | -120 |  105 |   75 |  -30 |               0 | True              |
|  459 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |        -55 |        170 |  110 |   65 |   40 |   -5 |  105 |  115 |   95 |  -55 |               0 | True              |
|  460 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        205 |        -90 |        295 |  205 |   25 |   10 |  -30 |  -75 |  -10 |  -90 |   15 |               0 | True              |
|  461 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -450 |        450 | -335 | -320 | -315 | -450 | -250 | -300 | -325 | -315 |               2 | True              |
|  462 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         65 |       -280 |        345 |  -60 |   65 | -215 | -200 | -250 | -140 | -280 | -130 |               1 | True              |
|  463 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -330 |        330 | -250 | -220 | -175 | -155 | -285 | -140 | -325 | -330 |               3 | True              |
|  464 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -320 |        330 |   10 | -320 | -275 | -180 | -185 | -280 | -270 | -245 |               0 | True              |
|  465 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |       -245 |        320 |   30 |   20 |   25 |  -15 |   75 |    5 |  -20 | -245 |               0 | True              |
|  466 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         65 |       -180 |        245 |   20 |   25 |   60 |   65 |   55 |   -0 | -160 | -180 |               0 | True              |
|  467 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |       -130 |        305 |   95 |  175 |  100 |   45 |  115 |   50 |  130 | -130 |               0 | True              |
|  468 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        140 |       -100 |        240 |  140 |  -15 | -100 |  -40 |  -40 |  -40 |   35 |  -40 |               0 | True              |
|  469 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        310 |        -20 |        330 |  180 |  265 |  310 |   95 |  180 |  215 |   35 |  -20 |               0 | True              |
|  470 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        170 |       -165 |        335 |  140 |  105 |  170 |  -40 | -165 |   35 | -115 | -115 |               0 | True              |
|  471 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         55 |       -185 |        240 |   55 | -165 | -105 | -105 | -165 | -120 | -125 | -185 |               0 | True              |
|  472 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         -0 |       -295 |        295 |   -0 |  -40 | -205 | -235 | -135 | -295 | -170 | -230 |               0 | True              |
|  473 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         45 |       -320 |        365 |   45 | -230 | -320 | -235 | -290 | -315 | -230 | -265 |               0 | True              |
|  474 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         60 |       -165 |        225 |   30 |   45 |   55 |   60 | -105 | -165 | -160 | -105 |               0 | True              |
|  475 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -70 |        280 |  125 |  100 |  165 |  185 |  155 |  210 |  -70 |   -0 |               0 | True              |
|  476 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -135 |        260 |   15 |  125 |   85 |  -20 |  105 |  110 | -135 | -135 |               0 | True              |
|  477 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |        -80 |        210 |  130 |  -35 |  -35 |  -10 |  -20 |  -80 |   35 |  -55 |               0 | True              |
|  478 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -475 |        475 | -405 | -475 | -435 | -465 | -445 | -455 | -345 | -445 |               3 | True              |
|  479 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -130 |        280 |  150 |  145 |   75 |   90 |   -0 |  -65 | -130 |  -90 |               0 | True              |
|  480 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |       -200 |        280 |   10 |  -15 |   75 |   80 |   30 | -200 | -195 | -170 |               0 | True              |
|  481 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        195 |        -85 |        280 |   25 |  195 |   75 |  -65 |  -85 |  -50 |   -0 |  -70 |               0 | True              |
|  482 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -185 |        260 |   75 |  -55 |  -30 |  -40 | -125 | -105 | -185 | -140 |               0 | True              |
|  483 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |       -210 |        390 |  110 |  165 |   85 |  180 |  -80 | -125 | -210 |  -30 |               0 | True              |
|  484 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        205 |        -50 |        255 |  170 |  110 |  160 |  205 |  165 |   85 |  -50 |   40 |               0 | True              |
|  485 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         -0 |       -255 |        255 |  -95 |  -45 |   -0 |  -35 | -240 | -255 | -155 | -135 |               1 | True              |
|  486 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -160 |        170 |  -45 |  -65 |  -55 | -145 |  -45 | -130 | -160 |   10 |               0 | True              |
|  487 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |       -250 |        370 |   25 |  120 |   10 |   25 |  -25 | -250 | -115 | -175 |               0 | True              |
|  488 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        185 |       -160 |        345 |  -35 |   50 |  130 |  185 |   95 |   65 | -110 | -160 |               1 | True              |
|  489 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -230 |        235 | -145 |  -75 |    5 | -120 |  -15 |  -35 |  -45 | -230 |               0 | True              |
|  490 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |       -300 |        380 |  -30 |  -50 |  -80 |   80 |  -70 | -300 | -280 | -285 |               1 | True              |
|  491 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -200 |        275 |   75 |   50 |   70 |    5 | -140 | -200 | -165 | -195 |               0 | True              |
|  492 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        245 |       -115 |        360 |  150 |  245 | -115 |  -35 |   30 |   10 |  105 |  -10 |               0 | True              |
|  493 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -225 |        435 |   95 |  145 |  210 |  -75 |  -55 | -160 | -225 |  -25 |               0 | True              |
|  494 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        200 |        -10 |        210 |  170 |  200 |  -10 |   30 |    5 |   20 |   -0 |   85 |               0 | True              |
|  495 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -225 |        250 |  -10 |   25 |   20 | -110 |  -90 |  -45 |  -15 | -225 |               0 | True              |
|  496 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |       -155 |        345 |  165 |   95 |  190 |   30 |   15 | -155 |  -45 |  -20 |               0 | True              |
|  497 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |        -60 |        255 |  195 |  150 |  110 |  125 |  115 |  165 |  110 |  -60 |               0 | True              |
|  498 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -420 |        420 | -235 | -220 | -165 | -270 | -420 | -415 | -300 | -340 |               3 | True              |
|  499 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -285 |        285 | -120 | -240 | -180 | -140 | -285 | -185 | -240 | -240 |               1 | True              |
|  500 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -315 |        315 | -105 | -245 | -205 | -195 | -150 | -175 | -255 | -315 |               0 | True              |
|  501 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -170 |        285 |  115 |   20 |  -90 | -145 | -110 | -170 | -105 | -165 |               0 | True              |
|  502 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -705 |        705 | -345 | -360 | -540 | -535 | -705 | -625 | -660 | -650 |               4 | True              |
|  503 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |       -120 |        300 |  100 |  180 |  150 |   70 |  135 |  115 | -120 |  -55 |               0 | True              |
|  504 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        120 |        -25 |        145 |  120 |  120 |   50 |   80 |   35 |   45 |   40 |  -25 |               0 | True              |
|  505 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |       -120 |        200 |  -10 | -120 | -105 | -120 |  -75 |  -55 |   80 |   -0 |               0 | True              |
|  506 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -210 |        295 |   85 | -125 | -130 | -115 |  -65 |  -15 | -210 | -140 |               0 | True              |
|  507 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -235 |        235 | -175 | -145 | -235 | -195 | -220 | -120 | -225 | -200 |               0 | True              |
|  508 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          5 |       -475 |        480 |  -75 | -125 | -140 | -230 |    5 | -225 | -345 | -475 |               1 | True              |
|  509 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        165 |       -120 |        285 |  165 |   95 |  -35 |   35 |   95 |  -95 | -120 |  -60 |               0 | True              |
|  510 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |       -130 |        160 |  -50 |   20 |  -35 | -110 |   30 | -130 | -115 |  -45 |               0 | True              |
|  511 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -310 |        310 | -110 | -150 | -205 | -125 | -110 | -105 | -245 | -310 |               0 | True              |
|  512 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |        -60 |        220 |   75 |  160 |  100 |  100 |   45 |   75 |   15 |  -60 |               0 | True              |
|  513 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |       -170 |        285 |   25 |  115 |   45 |   50 |   45 |  -25 |  -40 | -170 |               0 | True              |
|  514 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -645 |        645 | -380 | -500 | -520 | -645 | -530 | -565 | -535 | -520 |               4 | True              |
|  515 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -225 |        325 |   -0 |  100 |  -65 |  -15 |   55 |   25 |   80 | -225 |               0 | True              |
|  516 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -235 |        265 |   30 | -155 | -235 | -110 | -155 | -170 | -200 | -140 |               0 | True              |
|  517 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -140 |        190 |   25 |   50 | -100 |  -85 | -105 | -140 |  -85 |  -75 |               0 | True              |
|  518 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         50 |       -340 |        390 |   50 | -130 |  -40 |  -30 |  -25 | -240 | -140 | -340 |               0 | True              |
|  519 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -375 |        375 | -260 | -375 | -290 | -270 | -290 | -300 | -350 | -310 |               1 | True              |
|  520 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        195 |        -90 |        285 |  145 |  195 |  165 |  150 |   80 |   55 |  -25 |  -90 |               0 | True              |
|  521 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |        -50 |        210 |   80 |  140 |  110 |   55 |  160 |  125 |  125 |  -50 |               0 | True              |
|  522 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        230 |          0 |        230 |  115 |  135 |  175 |  230 |  125 |  125 |  150 |    5 |               0 | True              |
|  523 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -650 |        650 | -330 | -625 | -495 | -590 | -650 | -560 | -625 | -545 |               4 | True              |
|  524 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |        -15 |        210 |  195 |   85 |  -15 |    5 |   75 |   80 |   80 |   35 |               0 | True              |
|  525 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |        -35 |        175 |   45 |   70 |  140 |  105 |  135 |  105 |  140 |  -35 |               0 | True              |
|  526 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         85 |       -200 |        285 |   50 |  -20 |   85 |   10 | -180 | -180 | -110 | -200 |               0 | True              |
|  527 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -540 |        540 | -250 | -180 | -205 | -160 | -250 | -200 | -485 | -540 |               3 | True              |
|  528 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |        -70 |        265 |  185 |   80 |  110 |  195 |  135 |  165 |  190 |  -70 |               0 | True              |
|  529 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |        -45 |        110 |   40 |    5 |   15 |   65 |   -0 |   55 |  -40 |  -45 |               0 | True              |
|  530 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 | -350 |  -80 | -200 | -155 | -275 | -190 | -235 | -355 |               2 | True              |
|  531 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -265 |        365 |  100 |  -65 | -105 | -265 | -235 | -205 | -255 | -215 |               0 | True              |
|  532 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        110 |       -120 |        230 |   20 |   75 |  100 |   80 |   95 |  110 | -115 | -120 |               0 | True              |
|  533 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -460 |        460 | -235 | -415 | -460 | -410 | -455 | -370 | -450 | -415 |               3 | True              |
|  534 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 |  -85 | -130 | -180 | -100 | -170 |  -60 | -225 | -125 |               0 | True              |
|  535 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        210 |        -85 |        295 |  210 |  -45 |  -50 |  -85 |  -25 |  -40 |  -50 |   70 |              16 | True              |
|  536 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        230 |       -105 |        335 |  115 |  125 |  115 |  230 | -105 |   30 |   -5 |  -65 |               0 | True              |
|  537 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |       -110 |        240 |  130 |  120 |  -40 |  -30 |  -75 |  -10 | -110 |  -55 |               0 | True              |
|  538 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -355 |        355 | -355 | -335 | -255 | -310 | -275 | -230 | -325 | -345 |               2 | True              |
|  539 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -330 |        330 | -230 | -225 | -255 | -255 | -150 | -330 | -205 | -225 |               1 | True              |
|  540 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -490 |        490 | -340 | -340 | -360 | -350 | -290 | -245 | -465 | -490 |               4 | True              |
|  541 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -175 |        360 |  115 |  185 |  180 |  165 |   -5 | -175 |  -10 |  -85 |               0 | True              |
|  542 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -540 |        540 | -270 | -225 | -300 | -355 | -230 | -540 | -360 | -485 |               3 | True              |
|  543 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |       -110 |        300 |  190 |  165 |   40 |   55 |  -70 | -110 |   50 | -105 |               0 | True              |
|  544 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -260 |        260 |  -20 | -155 | -210 | -125 | -175 | -205 | -260 | -255 |               1 | True              |
|  545 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -235 |        260 |  -75 |   25 |    5 |   25 | -120 |  -75 |  -25 | -235 |               0 | True              |
|  546 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         60 |       -290 |        350 |   -0 |    5 |  -15 |   60 |  -35 | -270 | -290 | -265 |               0 | True              |
|  547 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -270 |        270 |  -70 |  -15 |  -20 |  -40 |  -20 | -265 | -180 | -270 |               1 | True              |
|  548 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -580 |        580 | -435 | -455 | -440 | -350 | -345 | -375 | -580 | -465 |               3 | True              |
|  549 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -690 |        690 | -425 | -605 | -625 | -690 | -605 | -550 | -620 | -645 |               5 | True              |
|  550 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        210 |        -60 |        270 |  110 |  210 |  -60 |  -25 |   -0 |  -20 |   -0 |  -15 |               0 | True              |
|  551 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |        -70 |        185 |  115 |  -70 |   10 |   -5 |  -70 |   80 |    5 |   -5 |               0 | True              |
|  552 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        130 |       -185 |        315 |  130 |  -25 |  -10 |   60 |  -85 | -140 |  -60 | -185 |               0 | True              |
|  553 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -505 |        505 | -280 | -345 | -390 | -255 | -245 | -280 | -450 | -505 |               4 | True              |
|  554 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -325 |        325 | -165 | -205 | -220 | -195 | -170 | -225 | -215 | -325 |               0 | True              |
|  555 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -755 |        755 | -465 | -465 | -405 | -455 | -530 | -755 | -610 | -620 |               5 | True              |
|  556 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        195 |       -140 |        335 |  195 |   55 |   85 |   15 |  -15 | -140 |  -40 |  -60 |               0 | True              |
|  557 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        245 |       -105 |        350 |  170 |  245 |  105 |  -40 | -105 |   30 |  -75 |  -40 |               0 | True              |
|  558 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -240 |        340 |  100 | -150 | -115 | -175 | -140 | -160 | -240 | -130 |               0 | True              |
|  559 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -285 |        305 |   20 | -140 | -255 | -160 | -260 | -170 | -285 | -185 |               0 | True              |
|  560 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -255 |        345 |   40 |   90 |   40 |   60 |   65 |   -5 |  -10 | -255 |               0 | True              |
|  561 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        275 |       -110 |        385 |  195 |  205 |  185 |   95 |  275 | -110 |   25 |  -15 |               0 | True              |
|  562 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |        -60 |        210 |  130 |  135 |    5 |   70 |   70 |  150 |   20 |  -60 |               0 | True              |
|  563 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -380 |        380 |  -85 | -335 | -295 | -315 | -310 | -250 | -275 | -380 |               1 | True              |
|  564 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -595 |        595 | -295 | -415 | -215 | -305 | -325 | -375 | -280 | -595 |               2 | True              |
|  565 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -680 |        680 | -340 | -415 | -425 | -340 | -555 | -625 | -590 | -680 |               4 | True              |
|  566 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         95 |       -165 |        260 |   10 |   35 |   60 |  -20 |   95 |    5 |   50 | -165 |               0 | True              |
|  567 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -235 |        350 |   60 |  115 |  -30 | -125 |  -55 | -235 |  -20 | -110 |               0 | True              |
|  568 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -705 |        705 | -415 | -535 | -520 | -645 | -545 | -705 | -660 | -620 |               5 | True              |
|  569 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         25 |       -320 |        345 |   25 |  -15 | -170 | -180 | -320 | -255 | -170 | -220 |               0 | True              |
|  570 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -585 |        585 | -245 | -265 | -375 | -395 | -390 | -535 | -585 | -505 |               3 | True              |
|  571 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -235 |        295 |   60 |  -10 | -225 | -135 | -230 | -235 | -235 | -235 |               0 | True              |
|  572 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -310 |        340 | -130 |  -85 |   30 |  -70 | -120 | -110 |  -40 | -310 |               0 | True              |
|  573 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -375 |        375 | -125 | -100 | -130 | -140 | -300 | -350 | -320 | -375 |               2 | True              |
|  574 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -240 |        360 |  -25 |  120 |   50 |   45 |   90 |   25 | -240 | -185 |               1 | True              |
|  575 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -725 |        725 | -450 | -415 | -410 | -465 | -510 | -725 | -585 | -685 |               5 | True              |
|  576 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        320 |          0 |        320 |  305 |  245 |  215 |  320 |  125 |  115 |  115 |   90 |              16 | True              |
|  577 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |        -60 |        260 |  170 |  130 |  110 |  200 |  150 |   -5 |  -60 |  -10 |               0 | True              |
|  578 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        170 |       -215 |        385 |   55 |  170 |  -25 | -190 | -170 | -215 | -130 | -200 |               0 | True              |
|  579 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -100 |        225 |   45 |  110 |  125 |   65 |   90 |    5 | -100 |  -30 |               0 | True              |
|  580 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        285 |          0 |        285 |  145 |  190 |  285 |  255 |  250 |   25 |   30 |   20 |               0 | True              |
|  581 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -490 |        490 | -235 | -210 | -240 | -390 | -465 | -425 | -490 | -420 |               3 | True              |
|  582 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -265 |        340 |   25 |  -30 |   75 |   -5 |  -75 | -180 | -265 | -230 |               0 | True              |
|  583 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |        -90 |        240 |  110 |  150 |  110 |   90 |   80 |  -20 |  -80 |  -90 |               0 | True              |
|  584 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -295 |        365 |   70 |  -55 |  -15 | -210 | -195 | -260 | -295 | -230 |               0 | True              |
|  585 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        105 |       -100 |        205 |  -25 |  -70 | -100 |   30 |   -5 |   -5 |  -10 |  105 |               0 | True              |
|  586 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        250 |        -90 |        340 |   85 |  250 |   80 |  -55 |  -85 |  -90 |   15 |   -5 |               0 | True              |
|  587 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        125 |       -165 |        290 |  125 |   95 |   55 |   80 |   70 |  105 |   65 | -165 |               0 | True              |
|  588 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |       -105 |        125 |  -20 |  -90 |  -65 |  -50 |  -70 |   20 | -100 | -105 |               0 | True              |
|  589 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        275 |        -80 |        355 |  135 |  245 |  190 |  275 |  -50 |  -30 |  -55 |  -80 |               0 | True              |
|  590 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -415 |        415 | -385 | -415 | -370 | -400 | -360 | -360 | -365 | -345 |               2 | True              |
|  591 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -255 |        255 |  -15 | -150 | -210 | -255 | -245 | -155 | -130 | -125 |               0 | True              |
|  592 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -485 |        485 | -315 | -275 | -150 | -360 | -375 | -455 | -485 | -335 |               4 | True              |
|  593 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         15 |       -250 |        265 |   -5 |   15 | -150 | -210 | -250 | -140 | -160 | -135 |               0 | True              |
|  594 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         85 |       -200 |        285 |   85 |   85 |   -5 |   40 | -115 | -200 | -140 | -165 |               0 | True              |
|  595 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -100 |        295 |  110 |  195 |  130 |  185 |  -10 | -100 | -100 |  -30 |               0 | True              |
|  596 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -555 |        555 | -275 | -325 | -365 | -260 | -305 | -520 | -555 | -530 |               3 | True              |
|  597 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |          0 |        240 |   95 |  110 |  160 |  125 |  240 |   70 |  125 |   10 |               0 | True              |
|  598 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         70 |       -235 |        305 |   70 |   45 |   -5 |  -25 | -145 | -195 | -235 | -165 |               0 | True              |
|  599 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        130 |       -135 |        265 |  130 |   35 |   90 |   80 |   70 |   45 | -135 | -125 |               0 | True              |
|  600 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -230 |        260 |   30 | -120 | -140 | -125 | -210 | -100 | -230 | -145 |               0 | True              |
|  601 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        240 |        -55 |        295 |  205 |  240 |  145 |  -55 |  -10 |   -0 |  -55 |  -30 |               0 | True              |
|  602 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         20 |       -175 |        195 |   10 |   20 |   20 | -175 | -145 |  -55 | -155 | -135 |               0 | True              |
|  603 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         40 |       -315 |        355 |   40 | -150 | -100 | -105 | -315 | -175 | -285 | -290 |               0 | True              |
|  604 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -135 |        285 |   55 |  150 |  -15 |  150 | -130 | -135 | -110 | -100 |               0 | True              |
|  605 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -295 |        295 | -135 | -235 | -280 | -295 | -205 | -195 | -250 | -230 |               2 | True              |
|  606 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         85 |       -250 |        335 |   45 |   85 | -250 | -160 | -105 | -200 |  -75 | -150 |               0 | True              |
|  607 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         15 |        -90 |        105 |   -5 |   15 |  -40 |  -65 |  -90 |   -0 |  -20 |   -0 |              17 | True              |
|  608 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -495 |        495 | -245 | -360 | -455 | -495 | -485 | -440 | -445 | -480 |               3 | True              |
|  609 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -245 |        355 |   70 |  110 |  -40 | -140 | -125 |  -85 | -245 |  -80 |               0 | True              |
|  610 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        300 |        -10 |        310 |  190 |  220 |  215 |  205 |  185 |  300 |  -10 |    5 |               0 | True              |
|  611 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -300 |        310 |   10 | -140 | -200 | -160 | -270 | -300 | -170 | -180 |               0 | True              |
|  612 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |       -105 |        185 |   75 |   80 |  -20 |  -40 |   20 | -105 |   55 |   35 |              16 | True              |
|  613 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -210 |        295 |   85 |  -80 |  -75 | -210 | -205 | -135 | -110 |  -85 |               0 | True              |
|  614 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         10 |       -295 |        305 |  -40 | -110 | -130 |   10 |  -20 | -105 | -235 | -295 |               1 | True              |
|  615 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |       -115 |        290 |  175 |  -60 |  -30 |  -40 | -100 | -115 |  -25 | -100 |               0 | True              |
|  616 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -595 |        595 | -350 | -265 | -455 | -375 | -495 | -445 | -440 | -595 |               4 | True              |
|  617 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -170 |        345 |   40 |   85 |   75 |  175 |   10 | -160 |  -85 | -170 |               0 | True              |
|  618 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -155 |        230 |   75 | -100 |  -20 |  -65 | -155 | -130 |  -65 |  -65 |               0 | True              |
|  619 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -285 |        285 | -125 | -285 | -260 | -175 | -180 | -270 | -270 | -240 |               0 | True              |
|  620 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |        -25 |        265 |  200 |  185 |  240 |  205 |  180 |  225 |  215 |  -25 |               0 | True              |
|  621 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -330 |        380 |   35 |   50 | -230 | -230 | -305 | -330 | -255 | -310 |               0 | True              |
|  622 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |       -155 |        305 |  120 |  150 |  120 | -155 |  -80 |  -35 |  -95 | -105 |               0 | True              |
|  623 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        215 |       -135 |        350 |  120 |  160 |  215 | -135 |  -35 |  -95 |  -70 |    5 |               0 | True              |
|  624 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |        -25 |        195 |  165 |  170 |   75 |  160 |  170 |   60 |  105 |  -25 |               0 | True              |
|  625 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -165 | -195 | -235 | -245 | -240 | -175 | -155 | -185 |               0 | True              |
|  626 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -255 |        255 | -150 |  -80 | -100 |  -35 | -105 | -175 |  -55 | -255 |               0 | True              |
|  627 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -195 | -195 | -130 | -265 | -200 | -130 | -230 | -115 |               0 | True              |
|  628 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        250 |       -165 |        415 |  250 |  -10 |  -40 |   -5 |  -25 |   -0 |   30 | -165 |              16 | True              |
|  629 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |        -65 |        215 |  100 |  120 |  150 |  130 |   55 |   90 |   15 |  -65 |               0 | True              |
|  630 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -340 |        340 | -235 | -270 | -340 | -210 | -260 | -230 | -265 | -170 |               1 | True              |
|  631 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         85 |       -235 |        320 |   85 |  -15 |   30 | -235 | -210 | -185 | -230 | -180 |               0 | True              |
|  632 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -410 |        410 |  -60 | -155 | -170 |  -25 | -215 | -155 | -210 | -410 |               0 | True              |
|  633 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         95 |       -185 |        280 |   95 |   70 | -105 | -185 | -155 | -125 | -120 |  -55 |               0 | True              |
|  634 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -330 |        340 |   10 | -165 | -210 | -240 | -265 | -210 | -330 | -305 |               0 | True              |
|  635 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -350 |        350 | -110 | -170 |  -50 | -135 |  -90 | -140 | -105 | -350 |               0 | True              |
|  636 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         40 |       -235 |        275 |    5 |   40 |   40 |   40 |   35 |   40 | -235 | -130 |               1 | True              |
|  637 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        220 |        -55 |        275 |  160 |  190 |  220 |  195 |  -55 |   20 |   -5 |  -35 |               0 | True              |
|  638 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -260 |        285 |   25 | -130 |   -0 |  -40 |    5 |  -85 |  -20 | -260 |               0 | True              |
|  639 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -510 |        510 | -305 | -255 | -310 | -215 | -435 | -430 | -510 | -475 |               3 | True              |
|  640 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        105 |       -170 |        275 |   80 |   65 |   20 |  105 |  105 | -155 | -170 |  -50 |               0 | True              |
|  641 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        100 |       -115 |        215 |   30 |  100 |   65 |   70 | -115 |  -30 |  -55 |  -10 |               0 | True              |
|  642 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |       -115 |        160 |  -10 |   40 |   15 |  -35 | -115 |   45 |   -0 |   15 |               0 | True              |
|  643 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |        -95 |        270 |  175 |    5 |  -45 |   15 |  -60 |   40 |  -20 |  -95 |               0 | True              |
|  644 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -195 |        330 |  135 |   80 |  -75 |  -20 | -195 |  -85 |  -70 |  -90 |               0 | True              |
|  645 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -200 |        200 |   -5 |  -80 |  -40 | -200 |  -65 |  -10 |  -30 |  -60 |               0 | True              |
|  646 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         90 |       -165 |        255 |   60 |   90 | -140 | -165 | -120 |  -85 | -115 | -135 |               0 | True              |
|  647 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -150 |        220 |   70 | -110 | -130 | -135 | -150 | -145 | -140 |  -65 |               0 | True              |
|  648 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |        -20 |         90 |  -20 |   10 |   40 |    5 |   15 |   30 |   65 |   70 |               0 | True              |
|  649 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -230 |        310 |   80 |   70 | -230 |  -85 | -105 | -120 |  -35 |  -90 |               0 | True              |
|  650 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         80 |       -220 |        300 |   80 | -150 |  -75 | -175 |  -75 | -205 | -105 | -220 |               0 | True              |
|  651 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        360 |          0 |        360 |  145 |  280 |  350 |  360 |  345 |  325 |   35 |  205 |               0 | True              |
|  652 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        160 |       -100 |        260 |  160 |  -70 | -100 |  -20 |  -10 |  -55 |  -10 |   20 |               0 | True              |
|  653 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -140 |        140 | -105 | -120 | -105 |  -80 | -120 | -140 | -120 | -105 |               0 | True              |
|  654 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -195 |        360 |  165 |   50 |   40 | -155 |  -85 | -195 | -190 | -165 |               0 | True              |
|  655 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        125 |        -95 |        220 |   65 |   70 |   75 |   75 |   70 |  110 |  125 |  -95 |               0 | True              |
|  656 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        235 |        -20 |        255 |  180 |  135 |   90 |  105 |   95 |  235 |   40 |  -20 |               0 | True              |
|  657 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -190 |        255 |   65 |  -95 | -110 | -160 | -155 | -150 | -190 | -130 |               0 | True              |
|  658 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -45 |        245 |   45 |  110 |  180 |  140 |  200 |  140 |  150 |  -45 |               0 | True              |
|  659 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |          0 |        220 |   80 |  190 |  210 |  165 |  220 |   30 |   80 |   50 |               0 | True              |
|  660 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -535 |        535 | -370 | -295 | -300 | -295 | -375 | -345 | -535 | -485 |               4 | True              |
|  661 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        205 |       -105 |        310 |  150 |  205 |  190 |   60 |  130 |   50 |  -10 | -105 |               0 | True              |
|  662 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |        -65 |        110 |   -0 |   45 |   45 |   15 |   20 |  -65 |   40 |  -55 |               0 | True              |
|  663 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        310 |          0 |        310 |  200 |  215 |  285 |  310 |  295 |  290 |   75 |  245 |               0 | True              |
|  664 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        260 |        -15 |        275 |  185 |  205 |  105 |  260 |  205 |  -15 |   65 |   45 |               0 | True              |
|  665 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |       -145 |        295 |   85 |   55 |  -30 |  150 |  125 |  110 | -125 | -145 |               0 | True              |
|  666 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -240 |        310 |   70 |  -70 | -125 | -240 | -190 | -180 | -125 |  -85 |               0 | True              |
|  667 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |       -125 |        195 |   -5 |  -80 |  -40 |   -0 |   50 |   70 |   60 | -125 |               0 | True              |
|  668 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -180 | -185 | -145 | -215 | -265 | -200 | -165 | -380 |               0 | True              |
|  669 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -225 |        300 |   65 |    5 |   75 | -225 | -115 |  -50 | -170 | -135 |               0 | True              |
|  670 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        110 |       -140 |        250 |  110 | -120 | -135 | -140 |  -90 |  -75 |  -65 |  -75 |               0 | True              |
|  671 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |        -80 |        170 |   15 |   30 |   80 |   15 |   90 |   -5 |  -70 |  -80 |               0 | True              |
|  672 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -265 |        265 |  -80 |  -40 | -150 |  -90 | -140 | -115 | -105 | -265 |               0 | True              |
|  673 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        145 |       -130 |        275 |  145 |  -55 | -130 | -105 |  -55 |   25 |  -65 |  -70 |               0 | True              |
|  674 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        180 |        -85 |        265 |  150 |  180 |  105 |  115 |  110 |  145 |  140 |  -85 |               0 | True              |
|  675 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         55 |        -90 |        145 |  -60 |  -15 |   10 |   55 |  -55 |   25 |  -90 |   55 |               0 | True              |
|  676 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -160 |        325 |  165 |  -40 |  -45 | -160 |  -95 |  -55 |  -45 | -125 |               0 | True              |
|  677 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -175 |        335 |  125 |  105 |  160 |  155 |   95 |   20 | -175 | -110 |               0 | True              |
|  678 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        210 |        -60 |        270 |  195 |  210 |   75 |   10 |   90 |   80 |   65 |  -60 |               0 | True              |
|  679 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -435 |        435 | -225 | -190 | -290 | -370 | -435 | -430 | -390 | -395 |               3 | True              |
|  680 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -530 |        530 | -320 | -310 | -430 | -530 | -450 | -450 | -515 | -410 |               4 | True              |
|  681 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -285 |        285 |  -85 |  -90 | -160 | -150 | -105 | -215 | -260 | -285 |               1 | True              |
|  682 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -150 |        265 |  115 |  110 | -110 |  -90 |  -80 | -150 | -100 |  -50 |               0 | True              |
|  683 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         15 |       -125 |        140 |  -90 |  -95 | -125 | -120 |  -60 |  -85 |   -5 |   15 |               0 | True              |
|  684 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         15 |       -310 |        325 |   15 |  -70 |  -25 |  -70 |  -15 | -280 | -310 | -300 |               0 | True              |
|  685 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -540 |        540 | -255 | -435 | -540 | -425 | -445 | -400 | -480 | -345 |               3 | True              |
|  686 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -295 |        295 | -120 | -240 | -295 | -250 | -290 | -270 | -240 | -250 |               2 | True              |
|  687 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |       -210 |        320 |  110 |   45 | -210 |  -95 | -165 |  -85 | -145 | -190 |               0 | True              |
|  688 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -275 |        275 | -190 | -155 | -150 | -100 | -275 | -160 | -120 | -105 |               0 | True              |
|  689 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -465 |        465 | -100 | -140 | -180 | -155 | -145 | -255 | -165 | -465 |               0 | True              |
|  690 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        150 |        -40 |        190 |   -5 |   15 |   35 |  -40 |   35 |   65 |  150 |  130 |               0 | True              |
|  691 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        100 |       -145 |        245 |   95 |  -30 |   90 |  100 |  -25 |  -45 |  -75 | -145 |               0 | True              |
|  692 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -445 |        445 | -235 | -215 | -215 | -430 | -425 | -445 | -420 | -400 |               3 | True              |
|  693 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -780 |        780 | -530 | -450 | -580 | -545 | -470 | -530 | -435 | -780 |               4 | True              |
|  694 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -610 |        610 | -335 | -340 | -235 | -300 | -355 | -610 | -525 | -485 |               4 | True              |
|  695 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         20 |       -420 |        440 | -265 |   20 | -270 | -250 | -335 | -290 | -420 | -360 |               3 | True              |
|  696 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         75 |        -55 |        130 |   75 |  -35 |  -45 |   25 |   25 |  -55 |   65 |   60 |              16 | True              |
|  697 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -190 |        345 |  155 |   10 |   95 |  155 |   25 | -190 |  -20 | -145 |               0 | True              |
|  698 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        175 |        -85 |        260 |  175 |  100 |   85 |  -55 |  -65 |  -85 |   10 |   -5 |               0 | True              |
|  699 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        220 |        -20 |        240 |  135 |  220 |  140 |  220 |  195 |  210 |  150 |  -20 |               0 | True              |
|  700 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -675 |        675 | -360 | -465 | -495 | -450 | -440 | -630 | -625 | -675 |               4 | True              |
|  701 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         20 |       -315 |        335 |   20 |  -75 |  -10 | -120 | -315 | -235 | -220 | -295 |               0 | True              |
|  702 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        200 |        -70 |        270 |  160 |  200 |   10 |  -45 |   40 |  -60 |  -70 |   75 |               0 | True              |
|  703 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -480 |        480 |   -5 | -375 | -400 | -375 | -215 | -480 | -390 | -355 |               0 | True              |
|  704 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -295 |        415 |  120 |   55 |  -55 | -100 | -295 | -120 | -115 |  -80 |               0 | True              |
|  705 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -580 |        580 | -360 | -365 | -235 | -580 | -420 | -430 | -455 | -440 |               4 | True              |
|  706 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |        -90 |        285 |  140 |  145 |  170 |  180 |  195 |  185 |  130 |  -90 |               0 | True              |
|  707 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -420 |        420 |  -85 |  -90 |  -60 |  -80 | -155 | -420 | -390 | -375 |               1 | True              |
|  708 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -570 |        570 | -450 | -520 | -475 | -535 | -345 | -525 | -550 | -570 |               3 | True              |
|  709 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -240 |        290 |   20 |   50 | -230 | -175 | -200 | -155 | -240 | -210 |               0 | True              |
|  710 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -575 |        575 | -385 | -550 | -360 | -470 | -405 | -575 | -445 | -455 |               3 | True              |
|  711 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -230 |        230 | -140 |  -95 |  -85 |  -75 |   -5 | -120 |  -85 | -230 |               0 | True              |
|  712 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        290 |        -85 |        375 |  165 |  290 |  275 |  -25 |   80 |  -85 |  -45 |  -40 |               0 | True              |
|  713 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -230 |        290 |   60 | -230 | -210 | -160 |  -85 | -115 | -160 |  -65 |               0 | True              |
|  714 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -140 |        285 |   60 |   90 |  145 |  125 |   40 |  140 | -140 | -110 |               0 | True              |
|  715 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         95 |       -185 |        280 |   95 |   55 | -110 | -120 | -120 | -170 | -185 | -115 |               0 | True              |
|  716 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         95 |       -135 |        230 |    5 |   95 |   30 |  -60 | -135 | -115 | -115 |  -80 |               1 | True              |
|  717 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        195 |        -45 |        240 |  120 |  195 |   85 |  105 |  120 |  -45 |   30 |   20 |               0 | True              |
|  718 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        245 |        -25 |        270 |  190 |  245 |  135 |   95 |  200 |  175 |  -25 |   40 |               0 | True              |
|  719 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         95 |       -150 |        245 |   55 |   75 |   85 |  -25 |   70 |   95 | -150 | -110 |               0 | True              |
|  720 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        290 |        -85 |        375 |  180 |  190 |  200 |  290 |  -20 |   -5 |  -30 |  -85 |               0 | True              |
|  721 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -175 |        315 |   60 |  140 |   95 |   80 | -120 | -175 |  -75 |  -35 |               0 | True              |
|  722 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        170 |        -55 |        225 |   25 |   80 |   45 |  170 |  100 |  -55 |  -50 |  -55 |               0 | True              |
|  723 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -185 |        365 |   20 |  130 |  180 |   75 |   35 |  -25 |  -45 | -185 |               0 | True              |
|  724 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -130 |        245 |   90 |  115 | -120 | -130 |  -95 | -130 |  -95 |  -95 |               0 | True              |
|  725 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |       -130 |        335 |  115 |  120 |  105 |  205 |  135 | -130 |  -10 |  -90 |               0 | True              |
|  726 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -185 |        185 |   -5 | -115 | -185 | -170 | -115 | -120 | -135 | -165 |               0 | True              |
|  727 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |       -130 |        300 |  170 |  105 |    5 | -130 |  -90 | -110 |  -95 | -120 |               0 | True              |
|  728 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         95 |       -100 |        195 | -100 |  -60 |   -0 |  -30 |   95 |  -55 |  -40 |  -30 |               0 | True              |
|  729 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        280 |          0 |        280 |  280 |  250 |  230 |  210 |  260 |  200 |  155 |  185 |              14 | True              |
|  730 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -400 |        400 | -280 | -225 | -295 | -245 | -245 | -255 | -180 | -400 |               1 | True              |
|  731 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -120 |        120 |  -85 |  -75 | -120 |  -80 |  -45 | -120 | -105 | -120 |               0 | True              |
|  732 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |         -0 |        180 |  180 |  105 |  115 |   -0 |   15 |    5 |   10 |   40 |               0 | True              |
|  733 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -165 |        340 |   55 |  175 | -160 |  -95 | -165 |  -55 | -135 | -140 |               0 | True              |
|  734 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |       -110 |        310 |  145 |  190 |  180 |  200 |   85 |  -85 | -110 |  -15 |               0 | True              |
|  735 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -155 |        295 |   35 |  140 |   20 |   90 | -125 | -155 |  -40 |  -80 |               0 | True              |
|  736 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 |  -45 | -135 |  -60 |  -85 |  -80 | -130 | -130 | -205 |               0 | True              |
|  737 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -560 |        560 | -360 | -455 | -340 | -530 | -475 | -560 | -470 | -515 |               4 | True              |
|  738 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -185 |        245 |   60 |  -25 |  -55 |  -25 | -105 |  -60 |  -35 | -185 |               0 | True              |
|  739 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -405 |        405 |  -85 | -145 | -150 | -175 | -110 |  -25 | -400 | -405 |               1 | True              |
|  740 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -465 |        465 | -195 | -410 | -335 | -435 | -360 | -330 | -320 | -465 |               2 | True              |
|  741 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |       -105 |        300 |  130 |  110 |   50 |   25 |  145 |  195 |   70 | -105 |               0 | True              |
|  742 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |       -110 |        280 |  115 |  120 |  170 |  170 |  -40 |  -85 | -110 |  -25 |               0 | True              |
|  743 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        340 |          0 |        340 |  130 |  320 |  340 |  240 |  330 |  290 |  280 |   35 |               0 | True              |
|  744 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -105 |        315 |  110 |  210 |  155 | -105 |  -65 |  -50 |  -10 |  -85 |               0 | True              |
|  745 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -125 |        315 |  100 |  120 |  190 |  160 |  110 | -125 |  -60 |  -70 |               0 | True              |
|  746 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -20 |        220 |  200 |  165 |  150 |   65 |  185 |   90 |   70 |  -20 |               0 | True              |
|  747 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        245 |        -60 |        305 |  175 |  245 |  130 |  170 |  190 |   85 |  -10 |  -60 |               0 | True              |
|  748 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -135 |        315 |   45 |  150 |   90 |  180 |  120 | -115 | -135 |  -10 |               0 | True              |
|  749 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -300 |        330 |  -25 |  -20 |  -40 |  -75 |   30 |  -30 |  -80 | -300 |               0 | True              |
|  750 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -670 |        670 | -375 | -380 | -365 | -585 | -625 | -670 | -510 | -585 |               4 | True              |
|  751 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -645 |        645 | -595 | -550 | -555 | -615 | -645 | -565 | -575 | -545 |               4 | True              |
|  752 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -255 |        380 |   75 |   20 |  -30 |   40 |  125 | -255 | -160 | -230 |               0 | True              |
|  753 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |        -95 |        295 |  200 |  175 |  170 |  -95 |  -70 |  -10 |   90 |  -10 |               0 | True              |
|  754 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |       -120 |        255 |   90 |  120 |  135 |  110 |   45 |   40 | -120 | -120 |               0 | True              |
|  755 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         85 |       -130 |        215 |   25 |  -30 |   10 |   85 |  -35 |   20 |  -75 | -130 |               0 | True              |
|  756 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        185 |        -10 |        195 |  180 |  170 |  150 |  170 |  185 |   -0 |  -10 |   25 |               0 | True              |
|  757 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -565 |        565 | -320 | -515 | -340 | -375 | -275 | -425 | -435 | -565 |               2 | True              |
|  758 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -290 |        290 | -125 |  -65 |  -65 |  -45 | -130 |  -90 |  -75 | -290 |               0 | True              |
|  759 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |       -125 |        305 |  145 |  180 |  -80 |  -70 | -115 | -100 | -125 |  -50 |               0 | True              |
|  760 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -400 |        400 | -130 | -195 | -200 | -170 | -140 | -170 | -400 | -350 |               2 | True              |
|  761 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        285 |        -80 |        365 |  205 |  180 |  285 |  125 |  145 |  195 |  165 |  -80 |               0 | True              |
|  762 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         95 |       -105 |        200 |   65 |   95 |  -85 |   -5 |  -90 |  -65 | -100 | -105 |               0 | True              |
|  763 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        115 |       -190 |        305 |   85 |  115 |   95 |   90 |  100 | -100 | -190 | -130 |               0 | True              |
|  764 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        270 |        -50 |        320 |  165 |  100 |  270 |   50 |   -5 |  -50 |   15 |   30 |               0 | True              |
|  765 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -395 |        395 | -210 | -385 | -375 | -395 | -380 | -370 | -345 | -345 |               3 | True              |
|  766 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |       -195 |        305 |   40 |  110 | -150 | -100 | -195 | -185 |  -80 | -105 |               0 | True              |
|  767 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -185 |        275 |  -30 |   10 |   40 |   90 |   40 |   90 | -185 | -110 |               1 | True              |
|  768 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -215 |        340 |  105 |  125 | -215 | -120 | -195 | -120 | -110 |  -90 |               0 | True              |
|  769 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |       -100 |        255 |  155 |  -55 |   -5 |   -0 |   -0 | -100 |   70 |   20 |               0 | True              |
|  770 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        155 |       -125 |        280 |  105 |  100 |  155 |   25 |   45 |  150 |  -80 | -125 |               0 | True              |
|  771 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -205 |        235 |  -60 |   30 |  -50 |  -20 |  -20 | -105 | -100 | -205 |               0 | True              |
|  772 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -710 |        710 | -355 | -450 | -580 | -655 | -690 | -705 | -710 | -535 |               4 | True              |
|  773 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         95 |       -200 |        295 |   60 |   95 |  -30 |  -25 |  -60 |  -60 | -155 | -200 |               0 | True              |
|  774 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -105 |        235 |   45 |  130 |   85 |  -55 |  -80 |  -95 |  -15 | -105 |               0 | True              |
|  775 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |        -75 |        250 |  110 |  140 |  125 |  100 |  175 |   40 |  -25 |  -75 |               0 | True              |
|  776 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         75 |        -30 |        105 |   25 |  -30 |   -0 |   75 |   60 |    5 |   70 |  -25 |               0 | True              |
|  777 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |        -20 |        190 |   95 |  170 |  115 |   85 |  135 |  110 |  -20 |   20 |               0 | True              |
|  778 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |       -175 |        345 |   10 |  110 |  170 |  100 |  -65 |  -55 |  -80 | -175 |               0 | True              |
|  779 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -280 |        345 |   -0 |   65 |  -60 | -280 | -135 | -160 | -200 | -210 |               0 | True              |
|  780 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -670 |        670 | -445 | -465 | -475 | -645 | -590 | -620 | -670 | -645 |               5 | True              |
|  781 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -120 |        195 |   75 | -100 |  -75 |  -55 | -115 | -110 | -120 | -115 |               0 | True              |
|  782 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        210 |        -90 |        300 |  125 |  120 |  160 |  210 |  -50 |  -50 |  -80 |  -90 |               0 | True              |
|  783 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         65 |        -85 |        150 |  -85 |  -25 |  -25 |   65 |    5 |   10 |  -25 |   25 |               0 | True              |
|  784 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -120 |        210 |  -15 |   35 |   15 |   90 |   40 |    5 |   70 | -120 |               0 | True              |
|  785 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -210 |        345 |   45 |  135 |   25 |  -80 | -105 | -130 | -210 | -165 |               0 | True              |
|  786 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -325 |        325 |  -25 | -145 | -190 | -315 | -320 | -300 | -325 | -230 |               1 | True              |
|  787 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        165 |       -135 |        300 |   90 |    5 |   45 |   75 |   70 |   35 |  165 | -135 |               0 | True              |
|  788 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        140 |       -145 |        285 |   80 |  130 |  120 |   55 |  140 |  120 | -115 | -145 |               0 | True              |
|  789 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        215 |       -105 |        320 |  215 |  135 |  160 |   40 |  130 |  180 |   -0 | -105 |              16 | True              |
|  790 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        155 |        -75 |        230 |  130 |  135 |  115 |  155 |   95 |  125 |  -55 |  -75 |               0 | True              |
|  791 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -165 |        165 | -165 | -155 |  -60 | -125 |  -50 |   -0 |  -20 |   -5 |               0 | True              |
|  792 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        160 |        -50 |        210 |  -50 |  150 |  160 |   60 |   40 |  120 |   45 |   65 |               0 | True              |
|  793 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -230 |        355 |  125 |    5 |   10 |  -25 |   95 |    5 | -230 | -110 |               0 | True              |
|  794 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -615 |        615 | -400 | -360 | -415 | -435 | -395 | -480 | -535 | -615 |               4 | True              |
|  795 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -190 |        190 | -170 |  -25 |  -80 | -160 | -190 | -190 | -135 | -145 |               0 | True              |
|  796 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        115 |       -195 |        310 |   55 |   75 |  115 |   35 |   55 | -195 |  -45 | -180 |               0 | True              |
|  797 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |       -115 |        270 |  115 |  135 |   85 |  150 |  115 |  100 |  155 | -115 |               0 | True              |
|  798 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        115 |        -50 |        165 |  -50 |    5 |  -25 |  -25 |   80 |  -10 |   25 |  115 |               0 | True              |
|  799 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -725 |        725 | -580 | -600 | -615 | -550 | -565 | -530 | -725 | -715 |               6 | True              |
|  800 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |        -70 |        150 |  -40 |   30 |   80 |  -70 |  -10 |  -50 |  -50 |   30 |               0 | True              |
|  801 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |       -100 |        300 |  190 |  110 |  190 |  120 |  200 |  200 | -100 | -100 |               0 | True              |
|  802 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        160 |       -135 |        295 |   40 |  160 |  130 |  110 |   65 | -120 | -135 | -105 |               0 | True              |
|  803 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -500 |        500 | -210 | -275 | -210 | -445 | -420 | -415 | -500 | -465 |               3 | True              |
|  804 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         95 |       -185 |        280 |   45 |   95 |   75 |   40 | -185 | -115 | -180 | -125 |               0 | True              |
|  805 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |        -80 |        230 |  130 |  150 |  -80 |  -80 |  -10 |   -0 |   40 |  -35 |               0 | True              |
|  806 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -400 |        400 | -115 | -210 | -120 | -375 | -385 | -350 | -315 | -400 |               2 | True              |
|  807 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |        -70 |        295 |  105 |  175 |  195 |  150 |  115 |  125 |  225 |  -70 |               0 | True              |
|  808 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         75 |       -160 |        235 |   45 |   75 | -160 | -110 | -140 | -160 | -160 |  -55 |               0 | True              |
|  809 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |        -25 |        225 |  150 |  135 |  200 |  145 |  150 |   70 |  -25 |  -15 |               0 | True              |
|  810 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -390 |        390 | -390 | -305 | -355 | -230 | -310 | -315 | -350 | -240 |               2 | True              |
|  811 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -290 |        290 | -200 | -205 | -150 | -260 | -230 | -170 | -275 | -290 |               0 | True              |
|  812 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -180 |        280 |   95 |  100 | -100 | -140 | -180 |  -75 | -110 |  -70 |               0 | True              |
|  813 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -155 |        255 |   70 |   80 |  100 |   65 |  -15 |  -60 | -155 | -100 |               0 | True              |
|  814 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |       -200 |        355 |   75 |  -15 |  155 |   90 |  -90 | -200 |  -75 | -135 |               0 | True              |
|  815 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -655 |        655 | -365 | -540 | -655 | -595 | -605 | -630 | -640 | -600 |               4 | True              |
|  816 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        165 |        -35 |        200 |  165 |  150 |  -30 |   15 |   35 |   60 |    5 |  -35 |               0 | True              |
|  817 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        100 |        -65 |        165 |  -65 |  100 |   25 |   20 |  -55 |    5 |   50 |   15 |               0 | True              |
|  818 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -425 |        425 | -340 | -310 | -375 | -425 | -350 | -365 | -380 | -300 |               2 | True              |
|  819 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -260 |        260 |  -35 |  -90 |  -95 | -160 |  -70 | -155 |  -85 | -260 |               0 | True              |
|  820 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |        -90 |        190 |  100 |  -65 |  -90 |  -45 |  -20 |  -30 |   25 |  -45 |               0 | True              |
|  821 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |        -85 |        305 |  185 |  220 |  160 |  135 |  165 |  -25 |  -20 |  -85 |               0 | True              |
|  822 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |       -135 |        305 |  170 |   75 |   45 |  115 |  155 |  150 | -100 | -135 |               0 | True              |
|  823 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -440 |        440 | -150 | -280 | -245 | -185 | -215 | -185 | -440 | -425 |               2 | True              |
|  824 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -745 |        745 | -435 | -440 | -405 | -745 | -700 | -690 | -740 | -680 |               5 | True              |
|  825 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         70 |       -215 |        285 |   20 |  -25 |   30 |   70 | -210 | -140 | -160 | -215 |               0 | True              |
|  826 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -405 |        405 | -125 | -180 | -405 | -320 | -355 | -210 | -380 | -345 |               2 | True              |
|  827 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -435 |        435 | -190 | -125 | -185 | -190 | -110 | -130 | -230 | -435 |               0 | True              |
|  828 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        210 |          0 |        210 |  110 |   75 |  165 |  120 |  180 |  185 |  210 |   55 |              15 | True              |
|  829 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -605 |        605 | -430 | -340 | -380 | -355 | -455 | -375 | -470 | -605 |               3 | True              |
|  830 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -220 |        220 | -160 |   -0 |  -45 |  -45 |  -70 |  -90 |  -10 | -220 |               0 | True              |
|  831 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -245 |        275 |   15 |   15 |  -40 |   25 |   30 |  -25 |  -80 | -245 |               0 | True              |
|  832 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        155 |       -195 |        350 |  110 |  155 |   75 |   -0 |   10 |   50 | -195 | -100 |               0 | True              |
|  833 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |       -230 |        340 |   40 |   70 |   30 |   40 |  110 |  105 |   15 | -230 |               0 | True              |
|  834 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |       -165 |        305 |  140 |  110 |  130 |  -40 |  -60 | -115 |  -55 | -165 |               0 | True              |
|  835 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -380 |        410 |  -70 | -155 | -130 |   30 |  -45 | -175 | -120 | -380 |               0 | True              |
|  836 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -465 |        465 | -195 | -255 | -150 | -390 | -465 | -415 | -425 | -445 |               3 | True              |
|  837 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -225 |        325 |  100 |   65 | -225 | -110 | -135 | -115 | -175 | -140 |               0 | True              |
|  838 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        145 |       -225 |        370 |   85 |  145 |  -15 |   10 | -225 | -175 | -100 | -110 |               0 | True              |
|  839 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -400 |        400 | -195 | -260 | -235 | -240 | -200 | -270 | -215 | -400 |               0 | True              |
|  840 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -215 |        340 |  125 |   30 |  -75 | -100 |  -70 | -215 | -175 | -155 |               0 | True              |
|  841 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -205 |        205 |  -20 | -205 |  -85 |  -75 |  -45 |  -35 |  -30 | -205 |               0 | True              |
|  842 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -345 |        345 |  -80 | -190 | -215 | -160 | -140 | -125 | -185 | -345 |               0 | True              |
|  843 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        105 |       -170 |        275 |   50 |  105 |  -85 |  -55 | -170 |  -15 | -130 |  -60 |               0 | True              |
|  844 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -565 |        565 | -305 | -380 | -295 | -355 | -405 | -565 | -535 | -545 |               4 | True              |
|  845 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |       -185 |        380 |  195 | -100 | -185 |  -90 |  -40 | -105 | -105 | -130 |               0 | True              |
|  846 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -745 |        745 | -740 | -600 | -745 | -670 | -675 | -665 | -680 | -730 |               6 | True              |
|  847 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        125 |        -80 |        205 |  105 |   75 |   90 |  125 |   70 |  -15 |   60 |  -80 |               0 | True              |
|  848 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -625 |        625 | -505 | -365 | -440 | -330 | -510 | -440 | -625 | -595 |               5 | True              |
|  849 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -240 |        340 |  100 | -155 | -110 | -240 | -115 | -140 | -145 | -130 |               0 | True              |
|  850 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |        -25 |        225 |  135 |  150 |  170 |  200 |   90 |   25 |  -25 |   30 |               0 | True              |
|  851 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         90 |       -290 |        380 |   90 |  -60 | -260 | -210 | -200 | -180 | -290 | -230 |               0 | True              |
|  852 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        230 |        -90 |        320 |   75 |   55 |  100 |  230 |   60 |   65 |  -90 |  -70 |               0 | True              |
|  853 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -315 |        315 | -100 |  -35 |  -85 | -125 | -120 | -130 | -315 | -265 |               1 | True              |
|  854 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         70 |       -180 |        250 |  -80 |  -80 |  -50 |   -0 |  -70 |   70 | -115 | -180 |               0 | True              |
|  855 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -125 |        185 |   60 | -125 | -100 |  -75 |  -60 | -115 |  -70 | -120 |               0 | True              |
|  856 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         45 |       -240 |        285 |   45 |   10 |  -50 |  -45 |  -40 | -100 |  -90 | -240 |               0 | True              |
|  857 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        125 |       -170 |        295 |   70 |  125 |   60 |  -60 |  -45 | -170 |  -75 |  -70 |               0 | True              |
|  858 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          5 |       -165 |        170 |    5 |  -70 |  -85 | -165 |  -30 |  -70 | -130 |  -80 |               0 | True              |
|  859 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          5 |       -145 |        150 |  -45 |    5 |  -15 | -110 | -145 |  -25 |  -65 | -100 |               0 | True              |
|  860 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -160 |        325 |  110 |  165 |   45 |  -75 |   35 | -160 |  -25 | -120 |               0 | True              |
|  861 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -615 |        615 | -570 | -610 | -520 | -535 | -500 | -485 | -450 | -615 |               4 | True              |
|  862 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -100 |        275 |  165 |  175 |   35 |  -15 |  -90 |   -5 | -100 |  -80 |               0 | True              |
|  863 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        250 |        -65 |        315 |   65 |  165 |  195 |  250 |  155 |  160 |   30 |  -65 |               0 | True              |
|  864 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        250 |        -25 |        275 |  155 |  155 |  250 |  165 |  135 |  165 |  115 |  -25 |               0 | True              |
|  865 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |       -135 |        330 |  180 |   40 |   35 |   65 |  195 |   40 |  -50 | -135 |               0 | True              |
|  866 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |       -140 |        280 |  -10 |  120 |  140 |   60 |   65 |    5 |   80 | -140 |               0 | True              |
|  867 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |       -210 |        290 |   15 |   75 |   80 |  -85 |   50 | -210 | -100 | -160 |               0 | True              |
|  868 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 | -170 | -180 | -195 | -165 | -120 | -215 | -170 | -355 |               0 | True              |
|  869 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |       -100 |        260 |  160 |   45 |  105 |   80 |   95 |   50 |  125 | -100 |               0 | True              |
|  870 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         70 |       -250 |        320 |   70 |  -25 |   25 |   50 |   35 |   70 | -250 | -135 |               0 | True              |
|  871 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |       -150 |        180 |  -80 |  -60 | -150 |   -5 |    5 |  -15 |   30 |  -40 |               0 | True              |
|  872 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        315 |        -50 |        365 |  175 |  200 |  315 |  -30 |  -50 |   25 |   75 |   15 |               0 | True              |
|  873 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 | -170 | -250 |  -75 | -100 | -240 | -245 | -170 | -330 |               0 | True              |
|  874 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        230 |          0 |        230 |  205 |  190 |  180 |  205 |  120 |  230 |   30 |   10 |               0 | True              |
|  875 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -280 |        380 |  -15 |  100 |   65 |   70 |   85 | -250 | -280 |  -75 |               0 | True              |
|  876 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |        -80 |        245 |  125 |  140 |  165 |  -80 |  -80 |  -40 |  -20 |    5 |               0 | True              |
|  877 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -155 |        155 | -150 |  -45 | -105 |  -55 |  -45 |  -40 | -125 | -155 |               0 | True              |
|  878 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -465 |        465 | -165 | -205 | -155 | -160 | -215 | -125 | -110 | -465 |               0 | True              |
|  879 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -590 |        590 | -345 | -435 | -330 | -510 | -570 | -500 | -520 | -590 |               4 | True              |
|  880 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -525 |        525 | -380 | -325 | -375 | -485 | -480 | -525 | -525 | -520 |               4 | True              |
|  881 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 | -145 | -195 | -135 | -155 |  -70 | -170 | -190 |  -95 |               0 | True              |
|  882 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -225 |        225 | -150 |  -50 |  -85 |   -5 | -135 |  -20 |  -60 | -225 |               0 | True              |
|  883 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -125 |        245 |  120 |  105 |   35 |   45 |   65 |   90 | -125 | -115 |               0 | True              |
|  884 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -795 |        795 | -490 | -490 | -520 | -795 | -705 | -780 | -720 | -735 |               5 | True              |
|  885 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        290 |        -45 |        335 |  130 |  210 |  240 |  290 |  255 |   -0 |  -45 |   35 |               0 | True              |
|  886 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        250 |        -65 |        315 |  250 |   80 |  -65 |  -10 |  -45 |  -20 |  -15 |  -60 |              16 | True              |
|  887 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |        -45 |        275 |  105 |  230 |  225 |  -10 |  -45 |  135 |   10 |   50 |               0 | True              |
|  888 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -245 |        320 |   75 |  -10 |   25 |   55 | -130 | -245 | -160 | -220 |               0 | True              |
|  889 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |       -145 |        260 |   35 |  -55 |  115 | -130 |  -60 |   20 |   55 | -145 |               0 | True              |
|  890 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -615 |        615 | -300 | -420 | -365 | -615 | -515 | -530 | -605 | -510 |               4 | True              |
|  891 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -325 |        325 |  -60 | -245 | -260 | -265 | -215 | -325 | -250 | -225 |               1 | True              |
|  892 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -250 |        250 |  -75 |  -55 |  -75 |  -90 |  -25 | -195 | -150 | -250 |               0 | True              |
|  893 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -120 |        270 |  150 |   95 |   20 |   90 |  -90 | -120 |  -50 |  -80 |               0 | True              |
|  894 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         55 |       -255 |        310 |    5 |   55 |   40 |   50 |   20 | -255 | -135 | -165 |               0 | True              |
|  895 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -100 |        290 |   65 |  190 |  110 |  165 |   45 |  -70 |  -95 | -100 |               0 | True              |
|  896 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         85 |       -155 |        240 |   45 |   85 |  -15 |   -5 |   15 | -155 | -150 | -110 |               0 | True              |
|  897 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -220 |        220 |  -10 |  -45 | -180 | -220 | -135 | -185 | -210 | -205 |               1 | True              |
|  898 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        195 |       -165 |        360 |  195 |  130 | -115 | -165 |  -50 | -130 |  -70 | -145 |               0 | True              |
|  899 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -280 |        280 | -125 | -150 | -140 | -130 | -165 |  -45 | -270 | -280 |               2 | True              |
|  900 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -190 |        190 | -110 | -125 |  -95 | -150 |  -80 | -190 | -105 |  -95 |               0 | True              |
|  901 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         25 |       -275 |        300 |   25 | -105 |  -65 |   -5 | -105 | -115 | -230 | -275 |               0 | True              |
|  902 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -145 |        355 |   70 |  210 |  160 |   35 |  -15 |  -80 |  -85 | -145 |               0 | True              |
|  903 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -540 |        540 | -245 | -255 | -540 | -365 | -395 | -410 | -505 | -515 |               3 | True              |
|  904 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         50 |       -265 |        315 |   50 | -185 | -265 | -130 | -190 |  -90 | -210 | -185 |               0 | True              |
|  905 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        250 |        -30 |        280 |  145 |  175 |  250 |   80 |  -30 |  -15 |   10 |  -10 |               0 | True              |
|  906 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |        -95 |        175 |   60 |   80 |   10 |   30 |  -80 |  -85 |  -95 |  -70 |               0 | True              |
|  907 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -235 |        415 |   85 |   10 |   45 |  180 |  125 | -235 |  -85 | -150 |               0 | True              |
|  908 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         55 |       -235 |        290 |   30 |   55 |  -55 | -185 | -235 | -115 | -215 | -100 |               0 | True              |
|  909 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 |  -65 |  -65 | -140 | -140 |  -90 |  -70 | -180 | -355 |               0 | True              |
|  910 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        130 |       -140 |        270 |   80 |  -25 |   10 |  130 |   80 |   40 |   55 | -140 |               0 | True              |
|  911 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         90 |       -205 |        295 |   55 |  -65 |   90 |   55 | -160 | -140 | -150 | -205 |               0 | True              |
|  912 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        330 |          0 |        330 |  165 |  180 |  330 |  235 |  130 |  255 |  225 |   55 |               0 | True              |
|  913 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -175 |        175 |  -95 |   -5 |  -65 |  -65 |  -40 |  -35 | -140 | -175 |               0 | True              |
|  914 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -185 |        330 |   25 |   10 |   85 |   80 |   -0 |  145 | -185 | -160 |               0 | True              |
|  915 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -210 |        275 |   65 |  -50 |   60 | -190 | -180 | -210 | -185 |  -90 |               0 | True              |
|  916 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        135 |       -220 |        355 |   95 |  -25 |   10 |  -10 |  135 | -150 | -220 | -180 |               0 | True              |
|  917 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        195 |        -90 |        285 |  195 |  155 |   15 |   10 |  105 |   35 |  -25 |  -90 |               0 | True              |
|  918 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |        -90 |        270 |  180 |  150 |  120 |  -15 |   -0 |  -90 |   15 |  -15 |               0 | True              |
|  919 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -155 |        350 |   95 |  150 |  165 |  195 |  -15 | -155 |  -30 |  -55 |               0 | True              |
|  920 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -185 |        190 |  -75 |    5 |  -55 |  -55 |  -40 | -100 |  -45 | -185 |               0 | True              |
|  921 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         15 |       -255 |        270 |   15 | -155 | -145 | -135 | -255 | -175 | -160 | -220 |               0 | True              |
|  922 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         -0 |       -385 |        385 |   -0 |  -75 |  -50 |  -35 | -260 | -385 | -230 | -290 |               0 | True              |
|  923 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |       -275 |        480 |   80 |   75 |  205 |  120 |  -10 | -115 | -275 |  -70 |               0 | True              |
|  924 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -145 |        145 |  -85 | -145 |  -95 |  -95 |  -95 |  -70 |  -90 |  -70 |               0 | True              |
|  925 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        270 |          0 |        270 |  200 |  245 |  155 |  180 |  270 |  170 |  180 |   40 |               0 | True              |
|  926 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -530 |        530 | -365 | -525 | -465 | -510 | -500 | -530 | -470 | -460 |               4 | True              |
|  927 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |        -95 |        275 |   90 |  160 |  160 |   60 |  180 |  -30 |  -95 |  -30 |               0 | True              |
|  928 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -180 |        245 |  -20 |  -15 |  -55 |  -40 |   65 | -170 | -140 | -180 |               1 | True              |
|  929 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -425 |        425 | -265 | -220 | -180 | -400 | -400 | -410 | -355 | -425 |               3 | True              |
|  930 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -250 |        250 |  -40 |  -25 |  -90 |  -10 | -155 |  -20 | -120 | -250 |               0 | True              |
|  931 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -430 |        430 | -160 | -185 | -215 | -145 | -195 | -200 | -210 | -430 |               0 | True              |
|  932 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -290 |        290 | -175 | -105 | -170 |  -55 | -185 |  -60 | -120 | -290 |               0 | True              |
|  933 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        290 |         -5 |        295 |  240 |  150 |  150 |  290 |  245 |  290 |  200 |   -5 |              14 | True              |
|  934 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         -0 |       -265 |        265 |   -5 |  -65 |   -0 |  -10 | -105 | -135 | -265 | -210 |               1 | True              |
|  935 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -290 |        355 |   65 | -270 | -290 | -235 | -220 | -215 | -235 | -160 |               0 | True              |
|  936 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        105 |       -205 |        310 |  105 |   25 |   40 |   85 | -190 | -195 | -205 | -160 |               0 | True              |
|  937 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -525 |        525 | -255 | -425 | -495 | -475 | -410 | -525 | -440 | -510 |               3 | True              |
|  938 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -370 |        430 |  -75 |   55 |   60 |   10 |  -95 |  -10 |  -15 | -370 |               0 | True              |
|  939 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        205 |       -140 |        345 |  115 |  150 |  205 |  -15 |  -30 |  -70 | -125 | -140 |               0 | True              |
|  940 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -290 |        365 |   75 | -195 | -290 | -240 | -235 | -250 | -270 | -290 |               0 | True              |
|  941 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -125 |        165 |  -40 | -125 |   40 |  -20 | -100 |  -85 |  -35 |  -55 |               0 | True              |
|  942 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         55 |       -290 |        345 |   55 |   -0 |  -25 | -135 | -165 | -230 | -200 | -290 |               0 | True              |
|  943 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -335 |        370 | -170 |  -85 |  -50 |   35 |  -90 |  -90 |  -70 | -335 |               0 | True              |
|  944 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -180 |        285 |  105 |   40 |   90 |   -5 |   40 |  -15 |   20 | -180 |               0 | True              |
|  945 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        185 |       -120 |        305 |  185 |  -55 |  -40 | -100 |  -35 |  -85 |  -85 | -120 |               0 | True              |
|  946 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -150 |        310 |  160 |  160 |  110 |  135 |   55 |   45 | -150 | -130 |               0 | True              |
|  947 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -625 |        625 | -405 | -370 | -590 | -570 | -625 | -545 | -595 | -425 |               5 | True              |
|  948 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        255 |        -55 |        310 |   90 |  130 |  255 |  170 |   -0 |   65 |   -5 |  -55 |               0 | True              |
|  949 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -380 |        380 | -110 | -135 |  -45 | -135 | -265 | -380 | -260 | -265 |               2 | True              |
|  950 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -570 |        570 | -475 | -380 | -380 | -385 | -465 | -405 | -455 | -570 |               3 | True              |
|  951 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -240 |        385 |   60 |  120 |  -30 |  145 |  -35 |    5 | -160 | -240 |               0 | True              |
|  952 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -680 |        680 | -445 | -465 | -400 | -660 | -630 | -680 | -665 | -675 |               5 | True              |
|  953 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -125 |        285 |   95 |   95 |  155 |  115 |   90 |  160 |  -90 | -125 |               0 | True              |
|  954 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        215 |        -45 |        260 |  170 |  215 |  205 |  -45 |   55 |  -25 |  -30 |   45 |               0 | True              |
|  955 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -280 |        285 |  -90 |    5 |  -20 |  -75 |  -65 |  -20 |  -85 | -280 |               0 | True              |
|  956 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        300 |          0 |        300 |  175 |  295 |  290 |  240 |  300 |  220 |  210 |   25 |               0 | True              |
|  957 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -560 |        560 | -480 | -485 | -510 | -395 | -530 | -450 | -495 | -560 |               3 | True              |
|  958 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -250 |        250 |  -10 |  -30 |  -35 | -100 |  -85 |  -50 |  -50 | -250 |               0 | True              |
|  959 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -430 |        430 | -170 | -290 | -250 | -290 | -235 | -340 | -255 | -430 |               0 | True              |
|  960 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        275 |        -65 |        340 |  170 |  275 |  200 |   55 |  -20 |   25 |   30 |  -65 |               0 | True              |
|  961 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -35 |        235 |   75 |  165 |  175 |   40 |  130 |  160 |  200 |  -35 |               0 | True              |
|  962 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -280 |        280 |  -70 | -155 | -125 |  -90 | -250 | -190 | -205 | -280 |               0 | True              |
|  963 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -480 |        480 |  -95 | -185 | -365 | -405 | -385 | -305 | -340 | -480 |               1 | True              |
|  964 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -175 |        300 |    5 |  -20 |  125 |   20 |  100 |  125 | -135 | -175 |               0 | True              |
|  965 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -125 |        260 |  135 |   50 | -125 | -115 | -125 |  -95 |  -55 |  -55 |               0 | True              |
|  966 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        215 |       -145 |        360 |   55 |  215 |   95 | -145 | -115 | -120 |  -75 | -145 |               0 | True              |
|  967 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         30 |       -305 |        335 |   30 |  -20 | -200 |  -70 |  -40 |  -25 | -305 | -245 |               0 | True              |
|  968 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         10 |       -325 |        335 |   10 |  -35 |  -85 | -100 | -325 | -180 | -160 | -270 |               0 | True              |
|  969 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -195 |        205 | -175 | -115 |  -95 | -150 |   10 | -145 | -115 | -195 |               0 | True              |
|  970 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -665 |        665 | -580 | -665 | -640 | -615 | -620 | -575 | -515 | -590 |               4 | True              |
|  971 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -455 |        455 | -255 | -305 | -315 | -345 | -215 | -285 | -265 | -455 |               1 | True              |
|  972 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -175 |        250 |   75 |  -70 |  -60 | -105 | -175 | -140 |  -80 | -100 |               0 | True              |
|  973 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -160 | -255 | -175 | -100 |  -90 | -155 |  -55 | -125 |               0 | True              |
|  974 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         80 |       -240 |        320 |   80 | -140 | -235 | -135 | -105 | -120 | -240 | -125 |               0 | True              |
|  975 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -375 |        375 | -110 |  -45 | -130 | -255 |  -85 | -175 |  -90 | -375 |               0 | True              |
|  976 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -375 |        375 | -180 | -225 | -195 | -235 | -135 | -225 | -375 | -365 |               2 | True              |
|  977 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -335 |        335 | -150 | -180 | -245 | -245 | -235 | -335 | -150 | -190 |               0 | True              |
|  978 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         90 |       -190 |        280 | -190 |   10 |  -35 |  -75 |  -45 |  -35 |  -40 |   90 |               0 | True              |
|  979 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        235 |       -145 |        380 |  150 |  160 |  175 |  235 |   85 |  -35 |  -25 | -145 |               0 | True              |
|  980 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        195 |          0 |        195 |  190 |  170 |  195 |  160 |  140 |   85 |  160 |  190 |              15 | True              |
|  981 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -245 |        265 |   20 | -180 | -175 | -110 | -130 | -245 | -200 |  -65 |               0 | True              |
|  982 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -105 |  -75 |  -40 | -130 | -110 | -255 | -200 | -185 |               0 | True              |
|  983 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -350 |        350 | -305 | -290 | -320 | -350 | -320 | -300 | -295 | -285 |               2 | True              |
|  984 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -505 |        505 | -330 | -335 | -345 | -250 | -330 | -435 | -260 | -505 |               2 | True              |
|  985 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -510 |        510 | -305 | -305 | -280 | -385 | -265 | -290 | -505 | -510 |               3 | True              |
|  986 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        230 |       -130 |        360 |  165 |  230 |  -15 | -120 | -130 |   10 |   -5 |   10 |               0 | True              |
|  987 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -515 |        515 | -250 | -235 | -165 | -180 | -205 | -180 | -385 | -515 |               3 | True              |
|  988 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |       -135 |        335 |  165 |  170 |   60 |   75 |  200 |  -60 | -135 |   15 |               0 | True              |
|  989 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -125 |        335 |  210 |  180 |  200 |  -75 |  -10 |   30 |  -65 | -125 |               0 | True              |
|  990 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -575 |        575 | -400 | -300 | -405 | -330 | -570 | -550 | -555 | -575 |               4 | True              |
|  991 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |       -140 |        290 |  130 |  130 |  150 |  -30 |  -40 |  -10 | -140 |   20 |               0 | True              |
|  992 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        210 |        -30 |        240 |  210 |  185 |  150 |  115 |  155 |  -20 |  -30 |   20 |               0 | True              |
|  993 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        150 |        -45 |        195 |  -45 |   10 |   35 |  150 |  110 |  115 |   55 |    5 |               0 | True              |
|  994 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         50 |       -200 |        250 |   50 |    5 |   25 |  -10 | -175 | -185 | -170 | -200 |               0 | True              |
|  995 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -150 |        275 |   40 |  100 |   35 |  105 |  125 |   50 | -115 | -150 |               0 | True              |
|  996 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         25 |       -115 |        140 |  -95 |   25 |  -35 |  -35 | -115 |  -60 |  -90 |  -20 |               0 | True              |
|  997 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -175 |        290 |  115 |  -85 |    5 |   -5 | -140 |   35 |  -20 | -175 |               0 | True              |
|  998 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         40 |       -195 |        235 |   40 |  -15 |  -80 |  -85 |  -20 |    5 |   25 | -195 |               0 | True              |
|  999 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |       -105 |        265 |  120 |  140 |  160 |   10 |   20 |  -65 |  -55 | -105 |               0 | True              |
| 1000 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         90 |       -245 |        335 |   90 |   25 |   20 |   50 | -150 | -245 | -220 |  -90 |               0 | True              |
| 1001 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -650 |        650 | -545 | -650 | -570 | -620 | -570 | -525 | -580 | -535 |               4 | True              |
| 1002 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -605 |        605 | -315 | -425 | -260 | -420 | -250 | -370 | -605 | -465 |               4 | True              |
| 1003 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -170 |        315 |  145 |  105 |   75 |  120 |  140 |   85 |  -85 | -170 |               0 | True              |
| 1004 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         50 |       -135 |        185 |   50 | -135 |  -80 |  -70 |  -55 |  -25 |  -15 |   40 |               0 | True              |
| 1005 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -295 |        295 | -205 | -205 | -170 | -185 | -295 | -260 | -280 | -285 |               0 | True              |
| 1006 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -430 |        430 | -145 |  -25 | -150 | -270 | -270 | -430 | -270 | -405 |               2 | True              |
| 1007 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        280 |        -30 |        310 |  280 |   70 |  -30 |   60 |   90 |   20 |  -25 |  120 |              16 | True              |
| 1008 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -415 |        415 | -140 | -120 | -170 | -280 | -345 | -415 | -405 | -290 |               2 | True              |
| 1009 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -560 |        560 | -470 | -460 | -470 | -560 | -420 | -455 | -465 | -435 |               3 | True              |
| 1010 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -265 |        325 |   60 |   55 | -150 | -265 | -155 | -130 | -145 | -110 |               0 | True              |
| 1011 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        235 |       -125 |        360 |  195 |  140 |  195 |  235 |   90 | -125 |  -50 |  -25 |               0 | True              |
| 1012 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -175 |        285 |  110 |   -0 |   60 |   80 |   95 | -175 | -135 |  -30 |               0 | True              |
| 1013 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |       -145 |        325 |  180 |  170 |  160 |  -60 |  -60 |  -25 |   20 | -145 |               0 | True              |
| 1014 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -675 |        675 | -410 | -465 | -410 | -370 | -400 | -470 | -675 | -560 |               5 | True              |
| 1015 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -495 |        495 | -130 | -195 | -300 | -375 | -370 | -470 | -385 | -495 |               2 | True              |
| 1016 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        185 |        -75 |        260 |  110 |  125 |  185 |  -75 |  -40 |    5 |  -50 |  -40 |               0 | True              |
| 1017 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        240 |        -60 |        300 |  110 |  195 |  240 |  150 |  215 |  115 |  -10 |  -60 |               0 | True              |
| 1018 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -400 |        400 | -160 | -130 |  -40 | -145 |  -75 | -390 | -305 | -400 |               2 | True              |
| 1019 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -320 |        325 |  -30 |  -75 |  -75 |  -15 |   -5 | -145 |    5 | -320 |               0 | True              |
| 1020 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        125 |         -0 |        125 |   40 |   25 |   10 |  125 |   65 |   70 |   60 |   -0 |              16 | True              |
| 1021 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        170 |       -150 |        320 |  135 |  130 |  170 |  -85 |  -60 | -110 | -150 |  -70 |               0 | True              |
| 1022 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |       -210 |        385 |   60 |  175 |   35 |   45 |  150 |   75 | -210 | -170 |               0 | True              |
| 1023 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        225 |        -95 |        320 |   85 |  205 |  225 |  -65 |   35 |  -10 |  -95 |  -15 |               0 | True              |
| 1024 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -740 |        740 | -380 | -680 | -695 | -725 | -655 | -700 | -740 | -615 |               4 | True              |
| 1025 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -270 |        335 |   65 |   -5 |   10 |  -95 |  -85 | -270 | -235 | -205 |               0 | True              |
| 1026 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         45 |       -275 |        320 |   45 |   -5 | -115 | -200 | -160 | -195 | -200 | -275 |               0 | True              |
| 1027 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         15 |       -310 |        325 |   15 | -310 | -165 | -230 | -170 | -225 | -170 | -220 |               0 | True              |
| 1028 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -315 |        315 |  -30 | -100 | -110 | -155 |  -30 |  -35 | -275 | -315 |               1 | True              |
| 1029 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         40 |       -240 |        280 |  -70 |   40 |   20 |  -20 | -240 | -130 | -135 | -160 |               1 | True              |
| 1030 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        330 |        -40 |        370 |  175 |  120 |  330 |  105 |  290 |   70 |  -40 |   -0 |               0 | True              |
| 1031 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        175 |       -135 |        310 |  175 |  105 |  140 | -135 |  -80 |  -10 | -125 |  -30 |               0 | True              |
| 1032 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         30 |       -230 |        260 |   30 |  -25 |    5 |  -15 |   -0 | -230 | -185 | -165 |               0 | True              |
| 1033 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -780 |        780 | -510 | -475 | -515 | -575 | -560 | -780 | -760 | -665 |               6 | True              |
| 1034 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 | -150 |  -85 | -155 | -140 | -195 | -150 | -140 | -100 |               0 | True              |
| 1035 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |       -230 |        345 | -115 |  115 |  -30 |   10 |  -55 |  -60 |   70 | -230 |               0 | True              |
| 1036 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        180 |        -80 |        260 |  125 |   80 |  130 |  180 |  145 |  140 |  170 |  -80 |               0 | True              |
| 1037 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -120 |        260 |  140 |  100 |  110 |   95 | -120 |  -75 |  -65 |   -0 |               0 | True              |
| 1038 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |        -75 |        265 |  185 |  105 |  130 |   95 |  190 |  -75 |  -45 |  -55 |               0 | True              |
| 1039 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |        -65 |        260 |  195 |  170 |  150 |  195 |  125 |   35 |  -25 |  -65 |               0 | True              |
| 1040 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -515 |        515 | -300 | -250 | -180 | -205 | -465 | -455 | -490 | -515 |               3 | True              |
| 1041 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -165 | -255 | -130 |  -20 | -105 | -100 | -250 | -125 |               0 | True              |
| 1042 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        275 |        -80 |        355 |  145 |  190 |  275 |  230 |  195 |  -40 |  -80 |  -40 |               0 | True              |
| 1043 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        235 |       -100 |        335 |  115 |  125 |  235 |  -55 |  -15 | -100 |  -60 |  -75 |               0 | True              |
| 1044 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -255 |        255 |  -45 | -225 | -190 | -205 | -170 | -205 | -240 | -255 |               1 | True              |
| 1045 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |       -130 |        175 |  -60 |  -55 |  -95 |   45 | -130 |  -10 |  -85 |   25 |               0 | True              |
| 1046 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        275 |        -45 |        320 |  275 |  120 |  155 |   85 |   10 |  -45 |   35 |  -30 |              16 | True              |
| 1047 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -175 |        185 | -175 |  -15 |  -80 | -130 |  -10 |  -85 |  -55 |   10 |               0 | True              |
| 1048 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        125 |       -265 |        390 |  125 |    5 |   50 |  -30 | -185 | -220 | -265 | -170 |               0 | True              |
| 1049 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -115 |        245 |   15 |  130 |   60 |  -15 |  -55 | -115 |  -65 |  -85 |               0 | True              |
| 1050 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -780 |        780 | -560 | -450 | -610 | -695 | -625 | -575 | -590 | -780 |               4 | True              |
| 1051 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         95 |       -235 |        330 |   55 |  -30 |  -35 |   95 |   15 | -230 | -175 | -235 |               0 | True              |
| 1052 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |        -15 |         85 |    5 |   10 |  -15 |   25 |   25 |    5 |   70 |   65 |               0 | True              |
| 1053 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        315 |          0 |        315 |  150 |  225 |  280 |  115 |  315 |  225 |  195 |   15 |               0 | True              |
| 1054 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -105 |        280 |  100 |  150 |  175 |  170 |  -30 | -105 |  -50 |  -35 |               0 | True              |
| 1055 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -640 |        640 | -270 | -355 | -550 | -640 | -495 | -595 | -580 | -540 |               3 | True              |
| 1056 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -490 |        490 | -425 | -350 | -325 | -340 | -355 | -450 | -480 | -490 |               5 | True              |
| 1057 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -560 |        560 | -210 | -220 | -420 | -560 | -445 | -410 | -490 | -490 |               3 | True              |
| 1058 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -655 |        655 | -400 | -340 | -400 | -335 | -390 | -365 | -655 | -575 |               5 | True              |
| 1059 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -295 |        295 |  -45 | -115 |  -85 |  -10 |  -75 |  -95 | -240 | -295 |               1 | True              |
| 1060 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -295 |        295 | -135 | -145 | -175 | -155 |  -85 | -135 | -295 | -220 |               2 | True              |
| 1061 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         55 |        -80 |        135 |  -80 |  -50 |  -55 |   55 |  -45 |  -20 |  -55 |  -20 |               0 | True              |
| 1062 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -245 |        245 | -150 | -165 |  -35 |  -20 | -150 | -165 |  -25 | -245 |               0 | True              |
| 1063 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |       -290 |        410 |  -50 | -100 |  -65 |  -85 |  120 | -235 | -290 | -195 |               1 | True              |
| 1064 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        155 |        -80 |        235 |  155 |   65 |  120 |  -60 |   20 |  -65 |  -80 |  -10 |               0 | True              |
| 1065 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        270 |        -25 |        295 |  180 |  270 |  205 |  225 |  -25 |   80 |   80 |   25 |               0 | True              |
| 1066 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         15 |       -125 |        140 | -125 | -105 | -120 |  -55 |   15 | -105 |  -70 |  -70 |               0 | True              |
| 1067 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         55 |       -155 |        210 |  -40 |   55 |   15 |  -15 | -110 |  -50 | -145 | -155 |               0 | True              |
| 1068 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         75 |       -165 |        240 |   35 |   70 |  -15 |   55 |   65 |   75 | -165 | -110 |               0 | True              |
| 1069 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -660 |        660 | -380 | -410 | -320 | -435 | -480 | -335 | -370 | -660 |               2 | True              |
| 1070 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -45 |        255 |  145 |   50 |  210 |  180 |  105 |  140 |   30 |  -45 |               0 | True              |
| 1071 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -435 |        435 | -145 | -355 | -340 | -435 | -360 | -400 | -385 | -315 |               2 | True              |
| 1072 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -270 |        270 | -270 | -190 | -220 | -270 | -195 | -210 | -255 | -255 |               1 | True              |
| 1073 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -135 |        290 |  105 |  110 |  155 |  110 |  100 |  -70 | -135 |  -25 |               0 | True              |
| 1074 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -510 |        510 | -295 | -315 | -255 | -410 | -510 | -485 | -415 | -490 |               4 | True              |
| 1075 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        100 |       -215 |        315 |  -15 |   35 |   55 |   15 |  -45 |  100 | -180 | -215 |               0 | True              |
| 1076 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -460 |        460 | -280 | -295 | -400 | -290 | -220 | -355 | -460 | -460 |               3 | True              |
| 1077 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -145 |        145 | -145 |  -65 | -145 | -100 |  -95 | -110 | -100 |  -95 |               0 | True              |
| 1078 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         35 |       -230 |        265 |   35 |  -45 |  -60 | -225 | -160 | -160 | -155 | -230 |               0 | True              |
| 1079 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 | -125 | -155 | -115 | -155 | -170 | -170 |  -80 | -105 |               0 | True              |
| 1080 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -425 |        425 | -270 | -305 | -300 | -315 | -250 | -355 | -205 | -425 |               1 | True              |
| 1081 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -230 |        280 |   50 |  -15 | -230 | -150 | -230 | -185 | -150 | -205 |               0 | True              |
| 1082 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -460 |        460 | -210 | -310 | -305 | -255 | -190 | -300 | -275 | -460 |               0 | True              |
| 1083 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -735 |        735 | -455 | -655 | -735 | -685 | -720 | -630 | -690 | -540 |               5 | True              |
| 1084 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -465 |        465 | -465 | -400 | -380 | -400 | -415 | -360 | -440 | -455 |               3 | True              |
| 1085 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -420 |        420 |  -95 | -160 |  -85 | -125 | -110 | -150 |  -90 | -420 |               0 | True              |
| 1086 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        400 |          0 |        400 |  150 |  235 |  190 |  400 |  230 |  220 |   60 |  120 |               0 | True              |
| 1087 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        185 |       -160 |        345 |  185 |  -30 |  -65 |  -30 | -125 | -160 | -145 |  -85 |               0 | True              |
| 1088 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -155 |        295 |    5 |   40 |   55 |   80 |  140 | -105 | -100 | -155 |               0 | True              |
| 1089 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        315 |          0 |        315 |  175 |  270 |  315 |  280 |  250 |  190 |  175 |   70 |               0 | True              |
| 1090 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         25 |       -285 |        310 |  -85 |   -5 |   25 | -245 | -110 | -285 | -230 | -245 |               1 | True              |
| 1091 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         -0 |       -225 |        225 |  -10 |  -35 |   -0 |  -15 | -180 | -140 | -225 | -215 |               1 | True              |
| 1092 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |       -155 |        310 |  120 |  140 |   70 |   35 |  135 |  145 |  155 | -155 |               0 | True              |
| 1093 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -395 |        395 |  -90 | -220 | -180 |  -45 | -360 | -395 | -360 | -280 |               2 | True              |
| 1094 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        210 |       -145 |        355 |   20 |  210 |  115 |  140 |  145 | -145 |  -45 |  -30 |               0 | True              |
| 1095 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        155 |        -85 |        240 |   80 |  110 |  155 |   50 |  140 |   80 |   50 |  -85 |               0 | True              |
| 1096 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        170 |       -130 |        300 |  170 |    5 | -110 | -130 |  -55 |   15 |  -10 |   25 |               0 | True              |
| 1097 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -200 |        315 |  115 | -145 | -180 | -135 |  -65 | -100 | -160 | -200 |               0 | True              |
| 1098 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        310 |         -0 |        310 |  200 |  220 |  195 |  105 |  255 |  170 |  310 |   -0 |               0 | True              |
| 1099 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -570 |        570 | -335 | -410 | -275 | -400 | -520 | -570 | -555 | -420 |               4 | True              |
| 1100 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         20 |       -320 |        340 |   -0 |  -50 |   20 | -170 | -320 | -290 | -270 | -285 |               1 | True              |
| 1101 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -515 |        515 | -450 | -460 | -505 | -505 | -515 | -505 | -435 | -415 |               3 | True              |
| 1102 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -200 |        230 |   30 |    5 | -200 | -200 | -145 | -120 | -165 | -150 |               0 | True              |
| 1103 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        125 |        -90 |        215 |  -30 |  -90 |  -20 |  110 |   15 |   10 |   45 |  125 |               0 | True              |
| 1104 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -155 |        305 |  150 |    5 | -150 | -155 |  -95 |  -95 | -110 |  -70 |               0 | True              |
| 1105 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -760 |        760 | -390 | -530 | -650 | -735 | -665 | -645 | -735 | -760 |               5 | True              |
| 1106 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -715 |        715 | -470 | -460 | -445 | -715 | -630 | -595 | -700 | -580 |               5 | True              |
| 1107 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -555 |        555 | -310 | -365 | -310 | -415 | -555 | -500 | -530 | -535 |               4 | True              |
| 1108 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        160 |       -155 |        315 |   75 |  110 |  125 |  120 |  160 | -155 |  -65 |  -25 |               0 | True              |
| 1109 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        150 |        -60 |        210 |  -60 |   40 |  150 |   80 |  -20 |   15 |   45 |  105 |               0 | True              |
| 1110 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -130 |        270 |  140 |  130 |  120 |   30 |  -60 |   -0 | -130 |   -5 |               0 | True              |
| 1111 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         30 |       -195 |        225 |   30 |   25 |  -45 | -125 | -195 | -140 |  -80 | -130 |               0 | True              |
| 1112 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -505 |        505 | -100 | -250 | -110 | -250 | -235 | -470 | -440 | -505 |               2 | True              |
| 1113 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        205 |        -80 |        285 |  155 |  205 |  150 |  105 |  130 |  135 |  -80 |   75 |               0 | True              |
| 1114 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        260 |          0 |        260 |   80 |  125 |  260 |  120 |  200 |  140 |  125 |    5 |               0 | True              |
| 1115 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         95 |        -50 |        145 |  -30 |  -50 |  -45 |   15 |   70 |   95 |    5 |   75 |               0 | True              |
| 1116 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -465 |        465 | -275 | -215 | -170 | -185 | -185 | -265 | -465 | -415 |               3 | True              |
| 1117 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -205 |        265 |   30 |   60 |  -90 | -195 |  -25 | -145 |  -70 | -205 |               0 | True              |
| 1118 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -180 |        250 |   30 |   70 |   55 | -180 | -140 | -125 |  -30 | -105 |               0 | True              |
| 1119 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -270 |        370 |   90 |  100 |   95 |   15 |  -20 | -215 | -270 | -100 |               0 | True              |
| 1120 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        265 |          0 |        265 |  200 |  265 |  215 |   90 |  265 |  255 |  240 |   45 |               0 | True              |
| 1121 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |        -65 |         85 |  -50 |  -15 |  -35 |  -65 |   20 |  -25 |  -25 |  -10 |               0 | True              |
| 1122 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -150 |        315 |  165 |  -65 | -105 |  -85 | -120 |  -95 | -105 | -150 |               0 | True              |
| 1123 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        105 |        -75 |        180 |  105 |  105 |  -70 |  -55 |  -65 |  -10 |  -75 |  -25 |               0 | True              |
| 1124 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         60 |       -185 |        245 |  -45 |   60 |  -70 |   40 |   15 |  -80 | -185 | -135 |               1 | True              |
| 1125 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -785 |        785 | -460 | -560 | -550 | -565 | -520 | -785 | -710 | -755 |               5 | True              |
| 1126 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         25 |       -220 |        245 |   25 | -220 | -110 | -185 |  -65 | -130 |  -95 | -220 |               0 | True              |
| 1127 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 |  -50 | -115 | -120 |  -70 | -180 |  -45 | -165 | -180 |               0 | True              |
| 1128 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -460 |        460 | -265 | -190 | -395 | -425 | -355 | -445 | -430 | -460 |               3 | True              |
| 1129 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -365 |        365 | -340 | -295 | -275 | -335 | -285 | -345 | -265 | -365 |               2 | True              |
| 1130 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |       -140 |        315 |  175 | -140 |   20 |  -15 |  -30 |  -60 |  -15 |   50 |               0 | True              |
| 1131 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        230 |       -120 |        350 |   55 |  135 |  190 |  155 |  230 |  -65 | -120 |  -60 |               0 | True              |
| 1132 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -395 |        395 | -255 | -120 | -230 | -240 | -150 | -195 | -225 | -395 |               1 | True              |
| 1133 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 | -180 | -160 | -130 | -160 | -225 | -175 | -205 | -330 |               0 | True              |
| 1134 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |        -65 |        245 |  180 |  135 |  140 |  160 |  145 |  -65 |  -50 |  -45 |               0 | True              |
| 1135 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -240 |        240 |  -20 |  -50 |  -60 | -125 | -225 | -180 | -240 | -210 |               1 | True              |
| 1136 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -325 |        405 |   35 |  -25 |  -50 |   80 | -190 | -270 | -195 | -325 |               0 | True              |
| 1137 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         40 |       -380 |        420 |   15 |   40 | -255 | -310 | -355 | -195 | -280 | -380 |               1 | True              |
| 1138 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -670 |        670 | -380 | -610 | -635 | -615 | -595 | -575 | -670 | -555 |               4 | True              |
| 1139 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -320 |        345 |  -10 |  -15 |   25 |  -75 | -105 |  -30 |  -60 | -320 |               0 | True              |
| 1140 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        225 |        -65 |        290 |  125 |  225 |  190 |  225 |  -35 |  -65 |    5 |   -0 |               0 | True              |
| 1141 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -340 |        340 | -100 |  -65 | -170 | -325 | -340 | -305 | -290 | -255 |               1 | True              |
| 1142 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -495 |        495 | -235 | -365 | -385 | -320 | -395 | -495 | -290 | -390 |               3 | True              |
| 1143 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -165 |        255 |   40 |  -80 |   90 |   -5 |   40 |  -15 |   -5 | -165 |               0 | True              |
| 1144 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -525 |        525 | -295 | -325 | -230 | -200 | -240 | -345 | -460 | -525 |               4 | True              |
| 1145 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -305 |        305 |  -25 | -275 | -200 | -185 | -235 | -305 | -210 | -190 |               1 | True              |
| 1146 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -100 |        285 |  170 |  185 |  145 |  150 |  -60 |  -25 |  -60 | -100 |               0 | True              |
| 1147 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         70 |       -280 |        350 |   70 |  -40 |    5 |   70 |  -30 |  -30 | -210 | -280 |               0 | True              |
| 1148 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |        -55 |        135 |   15 |  -50 |   80 |   15 |  -30 |  -55 |   15 |  -25 |              16 | True              |
| 1149 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        115 |       -280 |        395 |  115 |  -10 |   60 |   15 |   60 |  -25 | -280 | -145 |               0 | True              |
| 1150 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        255 |        -90 |        345 |  210 |  255 |  240 |  160 |  150 |  255 |  -25 |  -90 |               0 | True              |
| 1151 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -260 |        260 |  -35 |  -30 |  -25 | -140 |  -25 | -175 |  -25 | -260 |               0 | True              |
| 1152 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         70 |       -190 |        260 |    5 |  -45 |  -15 |  -50 |   70 | -100 | -170 | -190 |               0 | True              |
| 1153 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -285 |        285 | -110 | -265 | -185 | -260 | -285 | -250 | -240 | -280 |               1 | True              |
| 1154 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         45 |       -305 |        350 |   45 |   -5 |  -80 | -110 |  -65 | -150 | -305 | -225 |               0 | True              |
| 1155 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -770 |        770 | -455 | -460 | -540 | -625 | -510 | -770 | -755 | -770 |               5 | True              |
| 1156 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        315 |          0 |        315 |  215 |  315 |  250 |  125 |   20 |  170 |   85 |   75 |               0 | True              |
| 1157 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -370 |        420 | -100 |  -45 |  -10 |  -35 |   -0 |   50 |  -40 | -370 |               0 | True              |
| 1158 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -520 |        520 | -380 | -160 | -320 | -360 | -430 | -415 | -415 | -520 |               4 | True              |
| 1159 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        305 |        -50 |        355 |  180 |  300 |  305 |  185 |  285 |   20 |   10 |  -50 |               0 | True              |
| 1160 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -270 |        345 |   10 |   20 |   75 | -270 | -115 | -200 | -125 | -185 |               0 | True              |
| 1161 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |        -45 |        240 |  170 |  190 |  185 |  195 |   20 |   -5 |  -30 |  -45 |               0 | True              |
| 1162 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        285 |          0 |        285 |  215 |  195 |  270 |  190 |  220 |  230 |  285 |   20 |               0 | True              |
| 1163 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |       -100 |        270 |  170 |   -0 |   55 |  160 |  145 |   55 |  140 | -100 |               0 | True              |
| 1164 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -580 |        580 | -370 | -330 | -400 | -525 | -580 | -580 | -540 | -510 |               4 | True              |
| 1165 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -255 |        275 |   20 | -255 | -195 | -220 | -120 | -245 | -125 | -185 |               0 | True              |
| 1166 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        265 |        -75 |        340 |  205 |  265 |  250 |  175 |   50 |  -10 |  -40 |  -75 |               0 | True              |
| 1167 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         15 |       -275 |        290 |   15 |  -20 |  -60 | -220 | -275 | -240 | -275 | -245 |               0 | True              |
| 1168 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         55 |       -225 |        280 |   55 | -185 | -120 | -220 | -145 | -105 | -175 | -225 |               0 | True              |
| 1169 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        105 |       -245 |        350 |   80 |  -30 |  105 |   -0 |   30 | -220 | -245 | -135 |               0 | True              |
| 1170 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -190 |        325 |  135 |   85 |   90 |   35 |  -25 | -190 | -150 | -110 |               0 | True              |
| 1171 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        250 |        -85 |        335 |  125 |  125 |   75 |  250 |   40 |  130 |  -35 |  -85 |               0 | True              |
| 1172 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         55 |        -50 |        105 |  -25 |   20 |  -50 |   35 |  -15 |   10 |  -10 |   55 |               0 | True              |
| 1173 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -125 |        250 |   70 |  125 |   25 |   55 |   20 |   70 | -125 |  -80 |               0 | True              |
| 1174 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |       -145 |        375 |   45 |  230 |    5 | -120 | -105 | -115 | -135 | -145 |               0 | True              |
| 1175 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -680 |        680 | -390 | -400 | -280 | -550 | -625 | -680 | -590 | -600 |               4 | True              |
| 1176 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        190 |        -75 |        265 |  150 |  190 |   75 |  -75 |   20 |   15 |   50 |  -40 |               0 | True              |
| 1177 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |        -70 |        250 |   90 |  175 |   35 |   65 |   10 |  180 |  -45 |  -70 |               0 | True              |
| 1178 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        215 |        -85 |        300 |  215 |  160 |  140 |  180 |   40 |  125 |  -75 |  -85 |              16 | True              |
| 1179 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        255 |        -90 |        345 |  195 |  195 |  245 |  255 |  175 |  170 |  -25 |  -90 |               0 | True              |
| 1180 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        335 |          0 |        335 |  190 |  250 |  270 |  335 |  210 |  315 |  240 |   25 |               0 | True              |
| 1181 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 |  -65 | -165 | -100 | -180 | -140 | -125 | -170 | -155 |               0 | True              |
| 1182 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        270 |       -125 |        395 |   -5 |  100 |  135 |   40 |  270 |  130 | -125 | -125 |               1 | True              |
| 1183 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -525 |        525 | -300 | -390 | -370 | -300 | -355 | -340 | -410 | -525 |               3 | True              |
| 1184 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -615 |        615 | -345 | -390 | -390 | -290 | -370 | -480 | -615 | -575 |               4 | True              |
| 1185 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -270 |        270 | -105 | -130 | -165 | -165 | -200 | -195 | -145 | -270 |               0 | True              |
| 1186 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -240 |        305 |   65 |  -50 |  -25 | -150 | -150 | -200 | -240 |  -70 |               0 | True              |
| 1187 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         60 |        -10 |         70 |   40 |   30 |   10 |  -10 |   60 |   -5 |   25 |    5 |              16 | True              |
| 1188 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -250 |        330 |   80 |  -40 | -150 | -120 | -165 | -250 | -215 | -185 |               0 | True              |
| 1189 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -180 |        280 |  100 |  -65 | -130 |  -45 |  -60 | -160 | -180 | -180 |               0 | True              |
| 1190 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 | -105 |  -70 |  -95 | -180 |  -60 | -175 | -130 | -160 |               0 | True              |
| 1191 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -660 |        660 | -370 | -425 | -505 | -490 | -440 | -620 | -660 | -590 |               4 | True              |
| 1192 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -735 |        735 | -665 | -670 | -635 | -735 | -660 | -685 | -690 | -655 |               5 | True              |
| 1193 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -190 |        225 |  -15 |   35 |  -20 |  -50 |  -75 |  -65 |  -15 | -190 |               0 | True              |
| 1194 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        215 |        -85 |        300 |   70 |  215 |  145 |  165 |  180 |   10 |  155 |  -85 |               0 | True              |
| 1195 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |       -180 |        375 |  145 |  155 |   40 |  130 |  195 |  185 | -180 |  -30 |               0 | True              |
| 1196 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |       -155 |        230 |  -60 |   15 |  -65 |   75 |   50 |  -50 |  -75 | -155 |               0 | True              |
| 1197 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -540 |        540 | -505 | -480 | -435 | -475 | -415 | -450 | -390 | -540 |               3 | True              |
| 1198 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        320 |        -60 |        380 |  135 |  320 |  265 |  170 |  -60 |  -15 |   80 |    5 |               0 | True              |
| 1199 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -425 |        425 | -195 | -130 | -100 |  -65 | -210 | -260 | -185 | -425 |               0 | True              |
| 1200 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         95 |        -10 |        105 |  -10 |   55 |   30 |   50 |   80 |   95 |   80 |   40 |               0 | True              |
| 1201 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         55 |       -225 |        280 |   -5 |   55 |  -55 |  -35 |  -55 |   50 | -215 | -225 |               1 | True              |
| 1202 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -510 |        510 | -195 | -315 | -320 | -265 | -320 | -140 | -295 | -510 |               0 | True              |
| 1203 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -750 |        750 | -745 | -715 | -695 | -695 | -750 | -730 | -675 | -680 |               6 | True              |
| 1204 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         35 |        -70 |        105 |  -70 |  -50 |   -0 |   -5 |   -5 |    5 |   35 |   10 |               0 | True              |
| 1205 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -175 |        185 | -120 |   10 | -175 | -135 | -110 |  -80 | -165 | -135 |               0 | True              |
| 1206 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |        -85 |        250 |  165 |  145 |  160 |  -15 |  -35 |  -40 |  -85 |   35 |               0 | True              |
| 1207 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -300 |        300 | -105 |  -50 | -140 | -145 | -165 | -180 | -300 | -260 |               2 | True              |
| 1208 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         20 |       -290 |        310 |   20 |  -10 | -235 | -190 | -290 | -215 | -210 | -225 |               0 | True              |
| 1209 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -235 |        300 |   65 |   25 |  -35 | -185 | -205 | -235 | -185 | -165 |               0 | True              |
| 1210 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 |  -45 | -160 | -130 | -200 | -145 | -225 | -130 | -135 |               0 | True              |
| 1211 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |        -75 |        210 |  120 |  130 |  110 |   95 |  110 |  135 |  -50 |  -75 |               0 | True              |
| 1212 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -205 | -135 | -150 | -195 | -230 | -175 | -160 | -305 |               0 | True              |
| 1213 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        130 |       -125 |        255 |  -15 |   75 |   25 |   65 |   45 |  130 |   30 | -125 |               0 | True              |
| 1214 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        280 |          0 |        280 |  150 |   95 |  235 |  120 |  180 |  280 |   90 |    5 |               0 | True              |
| 1215 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -725 |        725 | -455 | -500 | -460 | -585 | -525 | -640 | -725 | -690 |               5 | True              |
| 1216 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -150 | -155 | -205 | -125 |  -85 | -150 | -155 | -205 |               0 | True              |
| 1217 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -520 |        520 | -255 | -230 | -260 | -370 | -510 | -330 | -415 | -520 |               3 | True              |
| 1218 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -435 |        435 | -215 | -175 | -160 | -330 | -435 | -345 | -415 | -345 |               3 | True              |
| 1219 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -415 |        415 | -105 | -115 | -415 | -315 | -285 | -385 | -405 | -370 |               2 | True              |
| 1220 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -215 |        215 | -175 | -215 | -175 | -110 | -125 | -145 | -205 | -165 |               0 | True              |
| 1221 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -275 |        275 | -260 |  -85 | -275 | -125 | -245 | -220 | -210 | -225 |               1 | True              |
| 1222 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        165 |       -115 |        280 |   60 |  165 |   -5 |   30 |    5 |  -10 | -115 |  -65 |               0 | True              |
| 1223 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -510 |        510 | -375 | -390 | -330 | -410 | -260 | -505 | -510 | -485 |               4 | True              |
| 1224 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -295 |        295 | -140 | -295 | -135 | -210 | -135 |  -75 | -210 | -130 |               0 | True              |
| 1225 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -480 |        480 | -230 | -210 | -200 | -260 | -215 | -465 | -480 | -450 |               3 | True              |
| 1226 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -520 |        520 | -210 | -275 | -480 | -465 | -415 | -445 | -520 | -420 |               3 | True              |
| 1227 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        115 |       -150 |        265 |  110 |  115 |   40 |  115 |  -65 |   10 | -150 |  -70 |               0 | True              |
| 1228 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |       -105 |        305 |  160 |  180 |  200 |  -10 | -105 |   75 |  -90 |   25 |               0 | True              |
| 1229 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -490 |        490 | -225 | -245 | -430 | -375 | -430 | -460 | -490 | -395 |               3 | True              |
| 1230 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -500 |        500 | -160 | -230 | -250 | -255 | -135 | -260 | -115 | -500 |               0 | True              |
| 1231 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -370 |        370 | -145 | -120 | -120 | -105 | -190 | -340 | -370 | -360 |               2 | True              |
| 1232 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |       -160 |        240 | -160 |  -75 |   30 |   35 |   15 |   80 |   45 |  -30 |               0 | True              |
| 1233 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -680 |        680 | -585 | -680 | -595 | -675 | -580 | -670 | -520 | -615 |               4 | True              |
| 1234 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |        -70 |        140 |   -5 |  -70 |  -10 |   30 |  -70 |  -40 |   70 |    5 |               0 | True              |
| 1235 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -245 |        275 |   30 | -245 | -180 |  -95 | -230 | -105 | -120 | -180 |               0 | True              |
| 1236 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 | -100 | -170 |  -80 |  -65 |  -25 |  -25 | -135 | -115 |               0 | True              |
| 1237 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |       -135 |        315 |  165 |  165 |  105 |  180 |  -25 | -135 |  -65 |   60 |               0 | True              |
| 1238 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -210 |        210 | -115 | -210 |  -65 | -210 | -160 | -145 | -115 | -115 |               0 | True              |
| 1239 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        220 |        -50 |        270 |  110 |  200 |  155 |  130 |  145 |  220 |  105 |  -50 |               0 | True              |
| 1240 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        260 |       -100 |        360 |  160 |  260 |  115 |  -10 | -100 |  -35 |  -90 |   40 |               0 | True              |
| 1241 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -770 |        770 | -460 | -485 | -340 | -770 | -650 | -710 | -655 | -730 |               5 | True              |
| 1242 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -790 |        790 | -565 | -700 | -715 | -745 | -745 | -785 | -645 | -790 |               6 | True              |
| 1243 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -240 |        240 |  -75 |  -40 |  -75 |  -80 |  -40 |  -20 | -215 | -240 |               1 | True              |
| 1244 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |       -160 |        315 |  145 |  130 |  125 |   85 |  130 |  155 |  100 | -160 |               0 | True              |
| 1245 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -670 |        670 | -315 | -455 | -455 | -375 | -385 | -570 | -670 | -605 |               4 | True              |
| 1246 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         35 |       -345 |        380 |  -15 | -145 |   35 |  -45 | -275 | -260 | -240 | -345 |               1 | True              |
| 1247 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -50 |        260 |  185 |  105 |  210 |  190 |   50 |  185 |  -50 |   25 |               0 | True              |
| 1248 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        290 |        -80 |        370 |  135 |  125 |  175 |  140 |  190 |  205 |  290 |  -80 |               0 | True              |
| 1249 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        305 |          0 |        305 |  175 |  305 |  250 |  210 |  235 |   70 |  115 |   40 |               0 | True              |
| 1250 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        110 |       -150 |        260 |  110 |  -40 | -150 |  -85 |  -70 |  -70 |  -70 | -135 |               0 | True              |
| 1251 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -650 |        650 | -400 | -345 | -345 | -355 | -650 | -555 | -595 | -540 |               5 | True              |
| 1252 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -685 |        685 | -290 | -625 | -685 | -620 | -595 | -685 | -645 | -630 |               3 | True              |
| 1253 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -245 |        245 |  -30 |  -10 |  -70 | -210 | -245 | -225 | -230 | -165 |               1 | True              |
| 1254 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -205 |        220 |    5 |    5 |   15 |   -5 | -100 |  -30 |  -35 | -205 |               0 | True              |
| 1255 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |       -135 |        275 |  140 |   25 |  120 | -135 |  -50 | -120 |  -95 |  -40 |               0 | True              |
| 1256 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -240 |        240 | -205 | -240 | -205 | -200 | -195 | -105 | -205 | -225 |               1 | True              |
| 1257 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -105 |        265 |   55 |  160 |  120 |   20 |  115 |   90 | -105 |  -20 |               0 | True              |
| 1258 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -555 |        555 | -355 | -510 | -545 | -555 | -530 | -505 | -440 | -455 |               4 | True              |
| 1259 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -445 |        445 | -275 | -295 | -175 | -285 | -270 | -195 | -220 | -445 |               1 | True              |
| 1260 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -785 |        785 | -555 | -595 | -535 | -510 | -500 | -745 | -735 | -785 |               6 | True              |
| 1261 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 | -115 | -215 |  -80 | -225 | -165 | -170 | -190 | -135 |               0 | True              |
| 1262 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        275 |         -0 |        275 |  235 |  275 |  190 |  245 |  175 |  180 |  215 |   -0 |              14 | True              |
| 1263 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        245 |        -40 |        285 |  130 |  245 |  225 |   90 |  -35 |  -30 |  -30 |  -40 |               0 | True              |
| 1264 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -325 |        325 | -325 | -245 | -215 | -265 | -155 | -245 | -275 | -200 |               2 | True              |
| 1265 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -530 |        530 | -220 | -485 | -445 | -460 | -395 | -480 | -530 | -460 |               3 | True              |
| 1266 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -145 |        285 |   50 |   95 |   80 |  140 | -105 | -145 | -125 |  -55 |               0 | True              |
| 1267 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -335 |        335 | -120 | -115 | -235 | -310 | -225 | -210 | -180 | -335 |               2 | True              |
| 1268 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |        -60 |        130 |  -55 |   65 |  -10 |  -25 |  -10 |  -60 |   70 |   -0 |               0 | True              |
| 1269 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |       -200 |        345 |   70 |  110 |   20 |  145 |  140 | -200 |  -45 |  -75 |               0 | True              |
| 1270 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |        -75 |        180 |   -5 |   15 |  105 |   35 |  105 |   95 |  105 |  -75 |               0 | True              |
| 1271 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        125 |       -140 |        265 |   85 |  125 |   20 |   70 | -110 | -140 | -125 |  -95 |               0 | True              |
| 1272 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -620 |        620 | -280 | -450 | -615 | -540 | -585 | -465 | -620 | -560 |               3 | True              |
| 1273 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -180 |        280 |   95 |  100 | -180 |   40 |  -55 |  -65 | -120 | -105 |               0 | True              |
| 1274 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -385 |        385 | -340 | -325 | -355 | -350 | -240 | -360 | -365 | -385 |               2 | True              |
| 1275 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |       -115 |        295 |  165 |  180 |   80 |  110 |  150 |  165 | -115 |  -45 |               0 | True              |
| 1276 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |        -90 |        275 |   60 |  185 |  105 |  130 |  -90 |  -70 |  -60 |  -50 |               0 | True              |
| 1277 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |       -135 |        310 |  160 |  -50 |  175 |  140 |   75 |  110 | -135 | -105 |               0 | True              |
| 1278 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |        -30 |        205 |   90 |  110 |  150 |  105 |   95 |   25 |  175 |  -30 |               0 | True              |
| 1279 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -425 |        425 | -260 | -205 | -230 | -270 | -290 | -265 | -425 | -395 |               3 | True              |
| 1280 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         45 |       -295 |        340 |  -20 |  -90 |   15 |   30 |   45 | -295 | -185 | -295 |               1 | True              |
| 1281 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -190 |        295 |   50 |  100 |   90 |  105 |   80 |   90 | -190 | -150 |               0 | True              |
| 1282 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -365 |        365 | -195 | -205 | -290 | -220 | -150 | -225 | -365 | -355 |               0 | True              |
| 1283 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -275 |        275 |  -75 | -200 | -165 |  -10 | -130 |  -70 |  -85 | -275 |               0 | True              |
| 1284 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        185 |       -140 |        325 |  130 |  135 |  185 |  110 |  110 | -140 |  -25 | -100 |               0 | True              |
| 1285 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -795 |        795 | -415 | -360 | -625 | -540 | -795 | -585 | -635 | -590 |               5 | True              |
| 1286 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         40 |       -290 |        330 |  -10 |   40 | -120 |  -45 | -290 | -155 | -220 | -165 |               0 | True              |
| 1287 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         65 |       -195 |        260 |   65 |   15 | -150 | -165 | -150 | -195 | -120 | -110 |               0 | True              |
| 1288 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -605 |        605 | -605 | -535 | -550 | -515 | -530 | -545 | -525 | -510 |               4 | True              |
| 1289 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         10 |       -200 |        210 |  -50 |  -25 |   10 |  -80 |  -50 |  -60 |  -85 | -200 |               0 | True              |
| 1290 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -120 |        270 |  150 |   95 |  -70 |  -45 |  -50 |  -55 | -120 |  -90 |               0 | True              |
| 1291 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         45 |       -220 |        265 |   45 | -105 | -215 | -140 | -150 | -160 | -145 | -220 |               0 | True              |
| 1292 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        105 |       -170 |        275 |  105 |   -5 |   20 |   25 |   80 | -155 | -160 | -170 |               0 | True              |
| 1293 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        260 |          0 |        260 |  215 |  145 |  260 |  155 |  195 |  190 |   20 |   20 |               0 | True              |
| 1294 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        105 |       -245 |        350 |  105 | -170 | -145 | -180 | -150 | -245 | -230 | -245 |               0 | True              |
| 1295 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -250 |        250 | -195 | -250 |  -95 | -230 | -150 | -215 |  -80 | -180 |               0 | True              |
| 1296 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -300 |        300 | -160 |  -90 | -170 | -185 | -180 | -170 |  -90 | -300 |               0 | True              |
| 1297 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |       -110 |        265 |   65 |   65 |  155 |  110 |  100 |   65 |  100 | -110 |               0 | True              |
| 1298 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -285 |        310 | -105 |  -80 |   25 |   10 |  -90 |   25 |  -40 | -285 |               0 | True              |
| 1299 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        240 |        -25 |        265 |  120 |  190 |  160 |  240 |  145 |  125 |  -25 |  -25 |               0 | True              |
| 1300 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         25 |       -340 |        365 |  -40 |  -70 |   25 |  -65 | -230 | -340 | -325 | -300 |               1 | True              |
| 1301 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -800 |        800 | -595 | -610 | -555 | -575 | -570 | -800 | -735 | -720 |               6 | True              |
| 1302 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        230 |       -120 |        350 |   85 |  230 |  -95 |   45 |  -30 |   75 | -120 |  -35 |               0 | True              |
| 1303 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -165 |        225 |   35 |   35 |    5 |   60 |   25 |  -35 |   45 | -165 |               0 | True              |
| 1304 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         55 |       -165 |        220 |   55 | -150 | -165 |  -80 | -145 |  -95 |  -25 | -150 |               0 | True              |
| 1305 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -410 |        410 | -185 | -280 | -225 | -275 | -310 | -390 | -410 | -385 |               2 | True              |
| 1306 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -625 |        625 | -425 | -265 | -485 | -475 | -390 | -445 | -420 | -625 |               3 | True              |
| 1307 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        240 |        -60 |        300 |  105 |  100 |  240 |  120 |  105 |  -60 |  -40 |   35 |               0 | True              |
| 1308 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        210 |       -135 |        345 |  210 | -135 |  -60 |   10 |  -70 |   60 |  -50 |   50 |               0 | True              |
| 1309 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -230 |        305 |   25 |   -0 |   -0 |   75 | -125 | -225 | -155 | -230 |               0 | True              |
| 1310 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        225 |        -20 |        245 |  100 |  205 |  225 |   70 |   -0 |  -20 |   20 |   75 |               0 | True              |
| 1311 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -310 |        310 | -120 | -155 | -260 | -220 | -185 |  -45 | -155 | -310 |               0 | True              |
| 1312 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         45 |       -195 |        240 |  -95 |  -80 |  -85 |   35 |  -65 |  -65 |   45 | -195 |               0 | True              |
| 1313 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -430 |        430 | -200 | -160 | -225 | -170 | -365 | -430 | -395 | -320 |               3 | True              |
| 1314 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -350 |        350 | -120 | -100 |  -80 | -105 |  -40 | -160 | -350 | -185 |               2 | True              |
| 1315 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -325 |        405 |   80 | -110 | -230 | -325 | -230 | -285 | -230 | -180 |               0 | True              |
| 1316 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        235 |       -115 |        350 |  190 |  235 |  170 | -115 |  -65 |   35 |  -35 |  -60 |               0 | True              |
| 1317 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -130 |        295 |  165 |  -20 | -130 |   40 |   35 |   20 |  -20 |   95 |               0 | True              |
| 1318 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        105 |       -285 |        390 |    5 |  105 |  -80 | -210 | -145 | -285 | -110 | -135 |               0 | True              |
| 1319 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        235 |       -110 |        345 |  130 |  235 |   70 |  -30 |  -65 |  -65 |  -90 | -110 |               0 | True              |
| 1320 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        245 |       -195 |        440 |   45 |  135 |   65 |  -60 |  245 |  -40 | -135 | -195 |               0 | True              |
| 1321 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |        -70 |        220 |  140 |   75 |   80 |  150 |  -60 |  -70 |    5 |  -40 |               0 | True              |
| 1322 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -310 |        370 |  -50 |   15 |   60 |   25 |  -95 |   40 |  -45 | -310 |               0 | True              |
| 1323 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -220 |        300 |   45 |   80 |  -30 |   40 | -135 | -170 | -220 | -215 |               0 | True              |
| 1324 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        320 |          0 |        320 |  190 |  320 |  260 |  285 |  100 |  215 |   65 |   70 |               0 | True              |
| 1325 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        285 |          0 |        285 |  145 |  240 |  240 |  230 |  275 |  145 |  285 |   25 |               0 | True              |
| 1326 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -275 |        275 | -185 | -260 | -245 | -215 | -260 | -240 | -220 | -275 |               0 | True              |
| 1327 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -235 |        235 | -195 | -140 | -155 | -140 | -225 | -215 | -205 | -235 |               0 | True              |
| 1328 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -140 |        315 |   60 |  175 |  120 |   40 |  175 |  -75 |  -50 | -140 |               0 | True              |
| 1329 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -140 |        275 |  135 |  -25 |  -20 | -140 |  -80 |  -75 |  -65 | -115 |               0 | True              |
| 1330 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         45 |       -240 |        285 |   35 |   -5 |   45 | -240 | -215 | -185 | -220 | -215 |               0 | True              |
| 1331 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         50 |       -240 |        290 |   50 |   15 |  -30 | -215 |  -75 | -240 | -170 | -105 |               0 | True              |
| 1332 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |       -160 |        290 |  105 |  130 |  -50 |  -50 | -160 |  -45 |  -40 | -135 |               0 | True              |
| 1333 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -175 |        310 |  135 |   90 |  105 |  -70 | -100 | -175 | -140 | -150 |               0 | True              |
| 1334 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        315 |        -60 |        375 |  180 |  195 |  315 |  110 |  -10 |  -60 |   25 |  -15 |               0 | True              |
| 1335 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -520 |        520 | -330 | -260 | -335 | -395 | -410 | -360 | -385 | -520 |               2 | True              |
| 1336 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         20 |       -360 |        380 |  -75 |  -20 |   20 |  -20 | -140 | -140 | -225 | -360 |               1 | True              |
| 1337 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -295 |        430 |  135 |  -70 | -200 | -190 | -210 | -285 | -295 | -195 |               0 | True              |
| 1338 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        250 |        -10 |        260 |  135 |  250 |  160 |  155 |  150 |  245 |  195 |  -10 |               0 | True              |
| 1339 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -70 |        270 |  105 |   90 |  195 |  150 |  200 |   20 |  105 |  -70 |               0 | True              |
| 1340 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -315 |        315 | -160 | -160 | -135 | -185 | -195 | -225 | -140 | -315 |               0 | True              |
| 1341 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         50 |       -240 |        290 |   50 | -120 |  -50 | -100 |  -25 |  -60 | -240 | -230 |               0 | True              |
| 1342 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -645 |        645 | -375 | -445 | -500 | -505 | -605 | -555 | -575 | -645 |               4 | True              |
| 1343 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |       -180 |        350 |    5 |   -0 |  170 |   25 |  -85 | -155 | -135 | -180 |               0 | True              |
| 1344 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        215 |        -35 |        250 |  165 |   70 |  140 |  165 |  215 |  210 |  135 |  -35 |               0 | True              |
| 1345 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |        -90 |        270 |   45 |  180 |   -0 |  -20 |   10 |  -90 |  -75 |  -60 |               0 | True              |
| 1346 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -550 |        550 | -130 | -280 | -305 | -235 | -360 | -415 | -550 | -350 |               2 | True              |
| 1347 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         20 |       -235 |        255 |   15 |   20 |  -80 |  -85 | -170 | -235 | -115 | -115 |               0 | True              |
| 1348 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        245 |       -100 |        345 |  160 |  155 |  245 |  205 |  175 |  185 |  -65 | -100 |               0 | True              |
| 1349 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         65 |       -275 |        340 |   65 |   10 |   30 |   25 | -220 | -165 | -125 | -275 |               0 | True              |
| 1350 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -645 |        645 | -365 | -415 | -405 | -620 | -645 | -610 | -610 | -615 |               4 | True              |
| 1351 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         35 |       -325 |        360 |   35 | -145 | -325 | -255 | -270 | -325 | -310 | -255 |               0 | True              |
| 1352 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        120 |       -175 |        295 |  -70 |   50 |  120 |   40 | -110 | -120 | -175 | -145 |               1 | True              |
| 1353 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -780 |        780 | -520 | -545 | -410 | -420 | -635 | -620 | -575 | -780 |               4 | True              |
| 1354 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -230 |        310 |   80 |  -30 | -135 | -230 | -145 | -190 | -210 |  -60 |               0 | True              |
| 1355 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -255 |        255 |  -10 | -115 |   -5 |  -60 | -135 |  -70 | -115 | -255 |               0 | True              |
| 1356 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        335 |        -45 |        380 |  205 |  335 |  145 |  -45 |  125 |   90 |   45 |   40 |               0 | True              |
| 1357 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -195 |        350 |   85 |  155 |   25 |   20 |   -5 | -165 |  -75 | -195 |               0 | True              |
| 1358 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -345 |        345 | -285 | -230 | -165 | -345 | -250 | -305 | -285 | -185 |               2 | True              |
| 1359 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |        -60 |        230 |  130 |  170 |  100 |  130 |   95 |  100 |   35 |  -60 |               0 | True              |
| 1360 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        255 |        -70 |        325 |   95 |  130 |  190 |  140 |  255 |   70 |  -55 |  -70 |               0 | True              |
| 1361 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |        -90 |        225 |  125 |  135 |  -85 |   80 |  -15 |  -40 |  -90 |   -0 |               0 | True              |
| 1362 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -315 |        315 | -165 | -140 |  -65 |  -55 |  -15 | -110 | -105 | -315 |               0 | True              |
| 1363 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -635 |        635 | -315 | -600 | -470 | -525 | -635 | -470 | -630 | -455 |               4 | True              |
| 1364 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        165 |       -115 |        280 |  165 |  105 |  115 |  125 |   35 |  -35 | -115 |  -95 |               0 | True              |
| 1365 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -750 |        750 | -425 | -465 | -450 | -495 | -650 | -750 | -625 | -690 |               5 | True              |
| 1366 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         10 |       -285 |        295 |   -5 |   10 |   10 |  -30 |  -55 |   -0 | -225 | -285 |               0 | True              |
| 1367 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        155 |       -200 |        355 |   40 |  155 |  125 | -170 | -125 |  -40 | -200 | -135 |               0 | True              |
| 1368 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -285 |        285 | -140 | -115 | -165 | -285 |  -75 | -150 | -180 |  -40 |               0 | True              |
| 1369 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -310 |        310 |  -60 |  -70 |  -95 |  -60 |  -45 | -265 | -310 | -215 |               1 | True              |
| 1370 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        220 |       -120 |        340 |  200 |  165 |  220 |  -40 | -110 | -120 |    5 |   85 |               0 | True              |
| 1371 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -285 |        285 |  -30 | -125 | -125 | -195 |  -75 |  -75 |  -70 | -285 |               0 | True              |
| 1372 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        205 |       -215 |        420 |  205 |   75 |   95 |   -5 |   65 |   35 | -215 | -140 |               0 | True              |
| 1373 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -120 |        245 |   50 |   95 |   35 |  125 |   95 |  110 | -120 | -115 |               0 | True              |
| 1374 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -155 |        155 | -150 |  -40 |  -95 |  -80 |  -95 |  -40 |  -85 | -155 |               0 | True              |
| 1375 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         75 |        -35 |        110 |  -35 |   75 |   35 |  -20 |   50 |    5 |  -25 |  -35 |               0 | True              |
| 1376 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |        -60 |        265 |  205 |   80 |   90 |  180 |  125 |  -35 |  -60 |  -45 |               0 | True              |
| 1377 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |       -155 |        310 |  155 |  -85 | -155 | -115 |  -70 |  -85 | -130 |  -85 |               0 | True              |
| 1378 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -505 |        505 | -275 | -325 | -245 | -335 | -280 | -385 | -505 | -405 |               3 | True              |
| 1379 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -345 |        345 |  -30 | -110 |  -90 |  -55 |  -10 | -145 | -290 | -345 |               0 | True              |
| 1380 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -620 |        620 | -590 | -590 | -600 | -540 | -620 | -505 | -510 | -600 |               4 | True              |
| 1381 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -490 |        490 | -480 | -490 | -475 | -480 | -480 | -440 | -440 | -490 |               3 | True              |
| 1382 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -630 |        630 | -395 | -385 | -440 | -490 | -345 | -380 | -620 | -630 |               4 | True              |
| 1383 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |       -135 |        200 |   -0 |  -30 |   65 |  -90 |  -80 |  -25 |   20 | -135 |               0 | True              |
| 1384 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 |  -95 | -165 | -125 | -180 | -135 | -195 | -115 | -115 |               0 | True              |
| 1385 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -715 |        715 | -555 | -480 | -480 | -510 | -460 | -715 | -715 | -655 |               6 | True              |
| 1386 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -670 |        670 | -470 | -380 | -450 | -670 | -615 | -595 | -530 | -570 |               5 | True              |
| 1387 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -505 |        505 | -245 | -365 | -355 | -360 | -405 | -505 | -465 | -420 |               3 | True              |
| 1388 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -185 |        290 |   35 |  -20 |  105 |   65 |   50 |   30 | -175 | -185 |               0 | True              |
| 1389 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |       -180 |        370 |  190 |   85 |  180 |  170 |  -80 | -180 | -135 | -160 |               0 | True              |
| 1390 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -160 |        170 |   10 | -140 |  -80 | -160 |  -65 |  -85 | -145 |  -95 |               0 | True              |
| 1391 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -365 |        365 | -155 | -235 | -225 | -205 | -205 | -125 | -225 | -365 |               0 | True              |
| 1392 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -215 |        305 |   90 |   -0 |   -5 |   90 |   55 |   55 |   50 | -215 |               0 | True              |
| 1393 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |       -160 |        190 |  -90 |  -95 |   30 |  -75 |  -35 |  -60 | -160 |  -75 |               0 | True              |
| 1394 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |        -55 |        135 |  -55 |   10 |  -20 |  -40 |   20 |  -35 |   80 |  -10 |               0 | True              |
| 1395 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        150 |       -225 |        375 |  150 |  -80 |  -40 |  -80 |  -30 | -225 |  -25 |  -95 |               0 | True              |
| 1396 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -140 |        275 |  135 |   90 |  -85 |  -95 | -140 |  -85 |   35 |  -45 |               0 | True              |
| 1397 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -450 |        450 | -225 | -290 | -220 | -445 | -360 | -360 | -450 | -340 |               3 | True              |
| 1398 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |        -20 |        195 |   95 |  130 |  175 |  170 |  170 |   -0 |  -20 |  -20 |               0 | True              |
| 1399 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -675 |        675 | -420 | -615 | -600 | -580 | -630 | -545 | -675 | -575 |               5 | True              |
| 1400 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         85 |       -210 |        295 |   60 |   10 |   85 | -145 | -210 | -160 | -150 | -175 |               0 | True              |
| 1401 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        240 |         -5 |        245 |  160 |  240 |  170 |  200 |  145 |  220 |   -5 |   15 |               0 | True              |
| 1402 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -170 |        330 |  160 |  145 |   75 |  -10 |  -65 | -150 | -105 | -170 |               0 | True              |
| 1403 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |        -65 |        240 |  175 |  170 |   75 |  115 |  135 |  150 |  -65 |  -35 |               0 | True              |
| 1404 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |        -65 |        260 |  150 |   90 |  195 |  120 |   70 |  170 |   -0 |  -65 |               0 | True              |
| 1405 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -150 |        220 |   70 | -135 |  -80 |  -40 |  -55 |  -75 | -150 |  -65 |               0 | True              |
| 1406 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -470 |        470 | -295 | -200 | -200 | -320 | -225 | -435 | -470 | -365 |               4 | True              |
| 1407 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -250 |        275 |   -0 |   10 | -195 |  -30 |  -45 |   25 |  -75 | -250 |               0 | True              |
| 1408 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -140 |        255 |   60 |  115 |  -45 | -140 | -130 |  -25 |  -95 |  -75 |               0 | True              |
| 1409 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -575 |        575 | -460 | -575 | -425 | -460 | -395 | -445 | -520 | -500 |               3 | True              |
| 1410 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |        -55 |        250 |  185 |  120 |  145 |  130 |   45 |  195 |  -50 |  -55 |               0 | True              |
| 1411 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         55 |       -220 |        275 |   55 | -220 |  -75 | -120 | -170 | -170 | -150 | -210 |               0 | True              |
| 1412 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -540 |        540 | -300 | -245 | -185 | -265 | -540 | -500 | -535 | -495 |               4 | True              |
| 1413 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        115 |       -200 |        315 |   60 |  -15 |   80 |   45 |   50 |  115 | -200 | -145 |               0 | True              |
| 1414 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -595 |        595 | -375 | -370 | -185 | -320 | -375 | -390 | -370 | -595 |               2 | True              |
| 1415 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |        -30 |        270 |  175 |  240 |  105 |  175 |   60 |  160 |  150 |  -30 |               0 | True              |
| 1416 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -410 |        410 | -245 | -160 | -250 | -120 | -160 | -210 | -220 | -410 |               1 | True              |
| 1417 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -445 |        445 | -290 | -160 | -370 | -375 | -385 | -355 | -360 | -445 |               3 | True              |
| 1418 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -190 |        330 |  140 |  115 |   25 |   65 |   55 | -130 | -190 |  -80 |               0 | True              |
| 1419 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        205 |        -70 |        275 |  190 |  125 |  125 |  205 |  -25 |   85 |    5 |  -70 |               0 | True              |
| 1420 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |        -50 |        225 |  175 |   65 |   25 |   10 |  140 |   90 |  140 |  -50 |               0 | True              |
| 1421 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        145 |        -90 |        235 |  125 |  100 |  145 |    5 |  -90 |  -65 |  -80 |  -30 |               0 | True              |
| 1422 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        290 |         -0 |        290 |  220 |  290 |   95 |   60 |  135 |   40 |  150 |   -0 |               0 | True              |
| 1423 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -185 |        320 |  135 |  -20 |  -85 |  -95 |  -75 |   20 | -100 | -185 |               0 | True              |
| 1424 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |       -165 |        280 |  115 |  100 |    5 | -125 | -105 |  -15 | -125 | -165 |               0 | True              |
| 1425 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -480 |        480 | -295 | -170 | -270 | -220 | -185 | -300 | -215 | -480 |               1 | True              |
| 1426 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -655 |        655 | -495 | -335 | -615 | -575 | -640 | -595 | -655 | -635 |               5 | True              |
| 1427 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        225 |       -120 |        345 |  125 |   80 |  190 |  225 |   65 | -115 | -120 | -115 |               0 | True              |
| 1428 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |       -195 |        350 |   45 |  -40 |  155 |   85 | -105 |  -75 | -110 | -195 |               0 | True              |
| 1429 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |       -150 |        305 |   50 |  155 |   60 |  -55 |   15 |   70 |   35 | -150 |               0 | True              |
| 1430 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |       -125 |        305 |  180 |   30 |  110 | -125 |  -20 |   10 |  -20 | -125 |               0 | True              |
| 1431 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -420 |        420 | -270 | -355 | -270 | -420 | -345 | -325 | -410 | -280 |               1 | True              |
| 1432 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        290 |          0 |        290 |  190 |  260 |  290 |   50 |   20 |  180 |  110 |  180 |               0 | True              |
| 1433 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        305 |         -0 |        305 |  155 |  165 |  170 |  305 |  230 |  235 |  240 |   -0 |               0 | True              |
| 1434 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -110 |        215 |   75 |   35 |   25 |   80 |  105 |   10 |   20 | -110 |               0 | True              |
| 1435 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |        -80 |        220 |   80 |   50 |  105 |   95 |  140 |   80 |  105 |  -80 |               0 | True              |
| 1436 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |       -205 |        380 |  175 | -145 | -120 | -205 |  -55 |  -20 | -160 |  -25 |               0 | True              |
| 1437 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -325 |        325 | -165 |  -65 | -325 | -315 | -250 | -305 | -305 | -225 |               2 | True              |
| 1438 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |       -165 |        295 |  100 |  130 |  -80 |  -95 | -165 |  -85 | -120 |  -65 |               0 | True              |
| 1439 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |       -210 |        365 |   80 |   85 |   90 |   65 |   70 |   80 |  155 | -210 |               0 | True              |
| 1440 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -230 |        365 |  135 | -205 |  -80 |  -20 | -125 | -230 | -115 | -100 |               0 | True              |
| 1441 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -305 |        305 |  -20 |  -65 | -140 |  -70 |  -70 | -305 |  -85 | -240 |               0 | True              |
| 1442 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |          0 |        200 |  160 |   75 |  200 |  150 |  160 |  140 |  150 |    5 |               0 | True              |
| 1443 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -100 |        260 |  135 |  155 |  100 |  160 |  -80 |  -30 | -100 |  -40 |               0 | True              |
| 1444 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |       -210 |        345 |  120 |  135 |   10 |  -10 |   75 |   65 | -180 | -210 |               0 | True              |
| 1445 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |       -115 |        270 |  140 |  140 |  155 |   80 |  -85 |  -50 |  -80 | -115 |               0 | True              |
| 1446 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -385 |        385 | -115 |  -85 |  -80 | -180 |   -0 | -125 |  -90 | -385 |               0 | True              |
| 1447 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         90 |       -225 |        315 |   70 |   70 |  -45 |   60 |   90 | -225 | -145 | -180 |               0 | True              |
| 1448 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |       -220 |        355 |  -10 |  -10 |  -20 |  135 |  -20 |   -5 | -160 | -220 |               0 | True              |
| 1449 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        185 |       -230 |        415 |  185 |   25 |   60 |  115 |  -10 |   50 | -230 |  -10 |               0 | True              |
| 1450 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        315 |        -40 |        355 |  170 |  110 |  315 |  255 |  200 |  115 |  100 |  -40 |               0 | True              |
| 1451 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -120 |        205 |   85 |   15 |  -65 | -100 |  -15 |  -45 | -120 |  -40 |               0 | True              |
| 1452 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |        -80 |        270 |  190 |   40 |  115 |  -30 |  -80 |   -0 |   85 |   40 |               0 | True              |
| 1453 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -190 |        190 | -120 |  -95 | -110 | -125 | -190 | -190 |  -90 | -160 |               0 | True              |
| 1454 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        160 |        -55 |        215 |  160 |   -0 |   35 |   50 |  -30 |   -0 |  -55 |   20 |               0 | True              |
| 1455 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |        -90 |        265 |  145 |  175 |  115 |  100 |  140 |   90 |   80 |  -90 |               0 | True              |
| 1456 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -645 |        645 | -365 | -600 | -645 | -580 | -630 | -585 | -465 | -540 |               4 | True              |
| 1457 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        240 |        -15 |        255 |  190 |  195 |   85 |  240 |  190 |  155 |   25 |  -15 |               0 | True              |
| 1458 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -120 |        180 |   60 | -105 |  -50 |  -60 |  -95 | -120 |  -15 |  -60 |               0 | True              |
| 1459 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -150 |        290 |  140 |  130 |  -70 |  -25 |  -85 |   40 | -100 | -150 |               0 | True              |
| 1460 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -175 | -255 | -235 | -215 | -290 | -305 | -250 | -380 |               0 | True              |
| 1461 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -360 |        360 | -310 | -330 | -240 | -245 | -320 | -310 | -360 | -335 |               2 | True              |
| 1462 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -165 |        165 | -165 | -145 |  -85 | -130 | -100 |  -80 |  -75 |  -95 |               0 | True              |
| 1463 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |        -75 |         75 |  -35 |  -35 |  -35 |  -75 |  -35 |  -30 |  -45 |  -10 |               0 | True              |
| 1464 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -265 |        265 | -115 | -145 | -185 | -100 | -100 | -195 |  -85 | -265 |               0 | True              |
| 1465 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        155 |       -130 |        285 |  120 |  155 |  115 |  -45 |  -15 | -130 |  -35 |  -40 |               0 | True              |
| 1466 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -735 |        735 | -480 | -480 | -725 | -660 | -690 | -695 | -710 | -735 |               5 | True              |
| 1467 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        165 |       -205 |        370 |  165 |   45 |   85 |  145 |  -80 |  -55 |  -95 | -205 |               0 | True              |
| 1468 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         25 |        -85 |        110 |  -85 |  -55 |  -30 |  -15 |   25 |   -0 |  -45 |   25 |               0 | True              |
| 1469 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        220 |        -90 |        310 |  160 |  220 |  170 |  180 |  180 |   85 |  -55 |  -90 |               0 | True              |
| 1470 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -270 |        335 |   40 |   10 |   65 |  -65 |  -25 | -195 | -270 | -165 |               0 | True              |
| 1471 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |        -40 |        210 |   60 |   85 |  120 |   85 |   45 |   85 |  170 |  -40 |               0 | True              |
| 1472 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         50 |       -295 |        345 |   45 |   40 |   50 |  -40 |   30 | -295 | -260 | -120 |               0 | True              |
| 1473 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |        -95 |        340 |  115 |  245 |  160 |   70 |  -70 |  -95 |  -20 |  -30 |               0 | True              |
| 1474 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |       -210 |        350 |  110 |   25 |  100 |  120 |   75 |   30 |  140 | -210 |               0 | True              |
| 1475 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -105 |        245 |  140 |  130 |  -70 | -105 |   55 |   50 |   -5 |  110 |               0 | True              |
| 1476 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -625 |        625 | -365 | -485 | -535 | -460 | -625 | -525 | -550 | -500 |               4 | True              |
| 1477 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -210 |        240 |  -50 |  -15 |   30 |  -20 |    5 |  -30 |  -35 | -210 |               0 | True              |
| 1478 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -575 |        575 | -410 | -435 | -365 | -315 | -400 | -375 | -530 | -575 |               4 | True              |
| 1479 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         85 |       -240 |        325 |   35 |   70 |   30 |   20 |  -40 |   85 | -240 | -110 |               0 | True              |
| 1480 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -385 |        385 | -385 | -320 | -365 | -340 | -340 | -300 | -355 | -365 |               2 | True              |
| 1481 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -225 |        305 |   80 |  -20 |   25 |  -45 | -150 | -225 | -165 | -190 |               0 | True              |
| 1482 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |       -240 |        390 |  100 |   65 |   40 |   25 |  -15 |  150 | -120 | -240 |               0 | True              |
| 1483 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        185 |       -135 |        320 |  185 |  110 |  140 |  -70 |  -60 |   15 | -135 |  -65 |               0 | True              |
| 1484 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -255 |        255 |  -65 |  -25 |  -95 |  -10 |  -20 | -255 | -160 | -205 |               1 | True              |
| 1485 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        205 |          0 |        205 |  205 |  165 |  195 |  100 |   75 |  115 |  105 |  105 |               0 | True              |
| 1486 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -160 |        160 | -150 |  -15 |  -65 | -130 | -160 |  -15 | -120 | -110 |               0 | True              |
| 1487 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -170 | -250 | -235 | -215 | -265 | -200 | -210 | -160 |               0 | True              |
| 1488 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -565 |        565 | -380 | -370 | -470 | -565 | -555 | -485 | -540 | -435 |               4 | True              |
| 1489 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         55 |       -190 |        245 |   25 |   25 |   55 |  -40 |  -15 |   40 |   35 | -190 |               0 | True              |
| 1490 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        290 |          0 |        290 |  155 |  290 |  260 |  245 |  270 |  265 |  235 |   60 |               0 | True              |
| 1491 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        100 |       -210 |        310 |   10 |   75 |   75 |  100 | -195 | -160 | -120 | -210 |               0 | True              |
| 1492 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -250 |        250 | -190 | -210 | -230 | -180 | -200 | -145 | -250 | -205 |               0 | True              |
| 1493 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |        -90 |        280 |   25 |  175 |  190 |  120 |   95 |   95 |  165 |  -90 |               0 | True              |
| 1494 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -560 |        560 | -475 | -560 | -545 | -365 | -450 | -520 | -505 | -450 |               3 | True              |
| 1495 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |        -90 |        135 |   -0 |  -90 |  -80 |   45 |  -70 |  -85 |   -5 |  -20 |               0 | True              |
| 1496 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -780 |        780 | -540 | -515 | -555 | -515 | -630 | -550 | -770 | -780 |               6 | True              |
| 1497 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |       -285 |        475 |   60 |  -25 |   40 |   15 |   25 |  190 |   80 | -285 |               0 | True              |
| 1498 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -625 |        625 | -325 | -455 | -625 | -530 | -545 | -550 | -620 | -590 |               4 | True              |
| 1499 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |        -70 |        225 |  155 |  -20 |   50 |  -40 |  -15 |  -35 |  -70 |  -70 |               0 | True              |
| 1500 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -475 |        475 | -425 | -405 | -475 | -400 | -425 | -460 | -465 | -475 |               3 | True              |
| 1501 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -390 |        390 | -245 | -175 | -140 |  -80 | -160 | -260 | -390 | -340 |               3 | True              |
| 1502 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -455 |        455 | -185 | -245 | -175 | -420 | -400 | -335 | -455 | -425 |               2 | True              |
| 1503 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -185 |        310 |   70 |  125 |  -95 | -165 |  -95 | -115 | -150 | -185 |               0 | True              |
| 1504 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        270 |          0 |        270 |   75 |  155 |  210 |  270 |  215 |  225 |  125 |   15 |               0 | True              |
| 1505 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -555 |        555 | -200 | -290 | -240 | -430 | -500 | -505 | -555 | -530 |               3 | True              |
| 1506 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -570 |        570 | -290 | -350 | -385 | -355 | -520 | -570 | -525 | -515 |               4 | True              |
| 1507 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -655 |        655 | -370 | -425 | -430 | -635 | -620 | -655 | -545 | -630 |               4 | True              |
| 1508 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        165 |       -120 |        285 |   40 |  -15 |  155 |  120 |  165 |  155 | -120 |  -80 |               0 | True              |
| 1509 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -150 |        150 | -110 | -125 |  -50 |  -30 |  -65 |  -55 | -150 | -135 |               0 | True              |
| 1510 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -205 |        325 |   50 |  120 | -115 | -135 | -115 |  -95 | -205 |  -90 |               0 | True              |
| 1511 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -705 |        705 | -540 | -675 | -705 | -700 | -565 | -650 | -635 | -610 |               4 | True              |
| 1512 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -300 |        300 | -140 | -220 | -225 | -130 | -300 | -135 | -150 | -130 |               0 | True              |
| 1513 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -680 |        680 | -470 | -515 | -395 | -470 | -460 | -570 | -625 | -680 |               5 | True              |
| 1514 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -190 |        330 |   80 |   65 |   55 |  140 |   75 |  -85 | -190 | -100 |               0 | True              |
| 1515 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -655 |        655 | -250 | -345 | -530 | -655 | -555 | -655 | -550 | -510 |               3 | True              |
| 1516 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         90 |        -20 |        110 |   -0 |   90 |   25 |   15 |   65 |   10 |  -20 |   15 |               0 | True              |
| 1517 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -615 |        615 | -485 | -545 | -535 | -565 | -540 | -515 | -510 | -615 |               4 | True              |
| 1518 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         35 |       -250 |        285 |   35 |  -10 |  -30 |   30 |   25 | -250 | -195 | -215 |               0 | True              |
| 1519 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        235 |        -70 |        305 |  165 |  175 |  210 |  120 |  235 |  -50 |   25 |  -70 |               0 | True              |
| 1520 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        205 |        -60 |        265 |  150 |  170 |  205 |  -20 |   -0 |   -5 |  -60 |   25 |               0 | True              |
| 1521 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -305 |        310 |    5 |    5 |  -10 |  -40 |  -85 |  -70 |  -35 | -305 |               0 | True              |
| 1522 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -655 |        655 | -410 | -365 | -465 | -395 | -510 | -500 | -500 | -655 |               2 | True              |
| 1523 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         -0 |       -325 |        325 |   -0 |  -85 | -150 | -170 | -290 | -235 | -325 | -290 |               0 | True              |
| 1524 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -200 |        285 |   85 |   40 |  -30 |   10 |   45 |   50 |  -20 | -200 |               0 | True              |
| 1525 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |          0 |        225 |  190 |  225 |  215 |  225 |  190 |  185 |  170 |   60 |               0 | True              |
| 1526 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        265 |        -50 |        315 |   75 |  140 |  125 |  265 |  -50 |  -20 |  -15 |  -25 |               0 | True              |
| 1527 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -355 |        355 | -240 | -240 | -235 | -265 | -355 | -305 | -155 | -265 |               1 | True              |
| 1528 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 | -130 | -120 | -140 | -155 |  -90 |  -90 |  -85 | -330 |               0 | True              |
| 1529 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -465 |        465 | -260 | -240 | -140 | -165 | -220 | -385 | -465 | -425 |               3 | True              |
| 1530 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 | -175 | -155 | -225 | -135 | -195 | -140 |  -95 | -170 |               0 | True              |
| 1531 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -240 |        375 |   15 |  -25 |  135 | -180 | -130 | -115 | -240 | -175 |               0 | True              |
| 1532 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -360 |        360 | -260 | -360 | -255 | -210 | -225 | -320 | -300 | -335 |               1 | True              |
| 1533 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -455 |        455 | -370 | -330 | -275 | -290 | -300 | -355 | -455 | -450 |               4 | True              |
| 1534 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -415 |        415 | -135 | -415 | -335 | -320 | -370 | -365 | -335 | -330 |               2 | True              |
| 1535 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -150 | -205 | -195 | -295 | -280 | -305 | -220 | -300 |               0 | True              |
| 1536 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |       -235 |        310 |   75 |   55 |  -55 |   60 |  -75 |   15 |  -15 | -235 |               0 | True              |
| 1537 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -205 |        235 |   -5 |  -15 |   30 |   10 |  -30 | -105 |  -95 | -205 |               0 | True              |
| 1538 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         65 |       -240 |        305 |  -95 |   10 |  -45 |   40 |   40 |   65 | -130 | -240 |               1 | True              |
| 1539 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -160 |        295 |  135 | -160 | -105 | -115 | -140 | -125 |  -55 | -110 |               0 | True              |
| 1540 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -220 |        220 |  -40 | -165 | -170 |  -90 | -210 | -195 | -220 | -160 |               1 | True              |
| 1541 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -265 |        295 | -100 |  -40 |  -25 |  -15 | -110 |  -95 |   30 | -265 |               0 | True              |
| 1542 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -560 |        560 | -505 | -540 | -490 | -485 | -560 | -535 | -455 | -545 |               3 | True              |
| 1543 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        105 |       -230 |        335 |   40 |  105 |   45 |   20 | -230 | -155 | -160 | -105 |               0 | True              |
| 1544 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -140 |        295 |   35 |   20 |  155 |  115 |   35 | -140 | -100 | -120 |               0 | True              |
| 1545 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        300 |          0 |        300 |  185 |  245 |  300 |  165 |   95 |   80 |    5 |   25 |               0 | True              |
| 1546 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -345 |        345 | -150 |  -70 | -140 |  -75 |  -90 | -250 | -165 | -345 |               0 | True              |
| 1547 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -240 |        240 | -175 | -195 | -240 | -210 | -210 | -110 | -205 | -115 |               0 | True              |
| 1548 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        235 |          0 |        235 |  200 |  195 |  235 |  225 |  145 |   60 |  125 |   70 |               0 | True              |
| 1549 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |         -0 |        190 |   85 |  120 |   40 |  135 |  190 |   30 |  145 |   -0 |               0 | True              |
| 1550 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -450 |        450 | -170 |  -75 | -345 | -385 | -450 | -335 | -435 | -355 |               2 | True              |
| 1551 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        185 |        -85 |        270 |  150 |  185 |  130 |  -15 |  -65 |  -85 |  -30 |   50 |               0 | True              |
| 1552 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -335 |        335 | -245 | -235 | -335 | -285 | -240 | -190 | -280 | -255 |               1 | True              |
| 1553 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        210 |       -170 |        380 |  210 | -135 | -110 | -140 |  -75 | -170 |  -55 |   -5 |               0 | True              |
| 1554 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        285 |        -90 |        375 |  175 |  285 |   10 |   55 |  -90 |  -25 |   20 |   20 |               0 | True              |
| 1555 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        150 |       -170 |        320 |  150 | -160 |  -85 |  -40 |  -95 |  -75 | -170 |  -10 |               0 | True              |
| 1556 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        165 |       -130 |        295 |  120 |  145 |   60 |   95 |   75 |   65 |  165 | -130 |               0 | True              |
| 1557 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -495 |        495 | -420 | -450 | -495 | -435 | -455 | -290 | -450 | -410 |               3 | True              |
| 1558 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         55 |       -205 |        260 |  -50 | -100 |   -5 |   55 | -190 | -165 | -205 | -200 |               1 | True              |
| 1559 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -625 |        625 | -340 | -320 | -460 | -355 | -500 | -565 | -525 | -625 |               4 | True              |
| 1560 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -790 |        790 | -555 | -645 | -620 | -565 | -535 | -725 | -745 | -790 |               6 | True              |
| 1561 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -175 |        285 |  105 |   30 |  110 |  -15 |   70 |  -75 | -175 |  -15 |               0 | True              |
| 1562 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -675 |        675 | -410 | -495 | -530 | -585 | -625 | -600 | -585 | -675 |               5 | True              |
| 1563 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |        -95 |        215 |   55 |   80 |  110 |   70 |  120 |   95 |  -80 |  -95 |               0 | True              |
| 1564 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -385 |        385 |  -95 | -150 | -155 |  -90 |  -80 | -115 | -365 | -385 |               1 | True              |
| 1565 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -185 |        185 | -185 | -175 | -155 | -100 |  -55 |  -95 | -155 |  -90 |               0 | True              |
| 1566 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -585 |        585 | -395 | -480 | -415 | -380 | -365 | -335 | -330 | -585 |               2 | True              |
| 1567 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -150 |        150 |  -90 | -100 | -150 | -130 |   -5 |  -10 | -140 | -120 |               0 | True              |
| 1568 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -540 |        540 | -480 | -445 | -405 | -380 | -540 | -470 | -415 | -520 |               3 | True              |
| 1569 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        225 |       -140 |        365 |  135 |  165 |  225 |  140 |  180 | -140 |  -90 |  -30 |               0 | True              |
| 1570 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -260 |        295 |  -70 | -100 |  -90 |  -55 |  -25 |   35 |  -60 | -260 |               0 | True              |
| 1571 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        160 |       -145 |        305 |  160 |  -35 |  -45 |  -75 |  -25 | -145 |  -50 |  -55 |               0 | True              |
| 1572 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |       -235 |        345 |    5 |  110 | -105 | -235 |  -50 |  -25 | -140 |  -80 |               1 | True              |
| 1573 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -220 |        220 |  -40 |  -10 |  -15 | -195 | -150 | -205 | -220 | -210 |               1 | True              |
| 1574 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -640 |        640 | -295 | -275 | -630 | -585 | -605 | -500 | -600 | -640 |               4 | True              |
| 1575 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -605 |        605 | -600 | -605 | -560 | -565 | -520 | -500 | -510 | -605 |               4 | True              |
| 1576 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        230 |        -50 |        280 |  180 |  180 |  160 |  230 |  210 |   -0 |  -50 |  -15 |               0 | True              |
| 1577 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        185 |       -115 |        300 |  135 |  155 |   55 |   90 |  185 | -115 |  -30 | -105 |               0 | True              |
| 1578 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        320 |          0 |        320 |  210 |  265 |  260 |  225 |  215 |  320 |  220 |   65 |               0 | True              |
| 1579 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -375 |        375 | -155 |  -65 |  -95 |  -70 | -340 | -355 | -275 | -375 |               2 | True              |
| 1580 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         70 |       -195 |        265 |   70 |   30 |   55 |   55 |   20 |   30 |  -50 | -195 |               0 | True              |
| 1581 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        190 |       -180 |        370 |  190 |  185 |   55 |  -10 |  -45 |  -35 |   -5 | -180 |               0 | True              |
| 1582 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -235 |        345 |   65 |   10 |  110 |   65 | -235 | -180 | -200 | -220 |               0 | True              |
| 1583 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |       -165 |        315 |   10 |  150 |  110 |  100 |   40 | -165 |  -25 |  -75 |               0 | True              |
| 1584 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -260 |        280 |   20 | -200 | -190 | -190 | -185 | -240 | -150 | -260 |               0 | True              |
| 1585 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        320 |        -90 |        410 |  170 |  320 |  255 |  190 |  305 |  225 |  165 |  -90 |               0 | True              |
| 1586 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -310 |        310 |  -55 | -100 |  -85 |  -90 |  -95 | -100 | -100 | -310 |               0 | True              |
| 1587 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -650 |        650 | -425 | -550 | -435 | -430 | -520 | -445 | -440 | -650 |               3 | True              |
| 1588 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         95 |       -265 |        360 |  -65 | -125 |   95 |  -80 |  -50 | -100 | -265 | -200 |               1 | True              |
| 1589 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |        -45 |        275 |  180 |  225 |  230 |   -5 |  -45 |   20 |  -35 |   30 |               0 | True              |
| 1590 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |        -60 |        290 |  190 |  230 |   85 |  -45 |   20 |    5 |  -60 |  -50 |               0 | True              |
| 1591 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |       -145 |        340 |  130 |  195 |  140 |  120 |  100 |  160 |  175 | -145 |               0 | True              |
| 1592 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -125 |        315 |   80 |   85 |  190 |   90 |  105 | -120 | -125 |  -50 |               0 | True              |
| 1593 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -280 |        280 |  -75 | -205 | -210 | -160 | -280 | -245 | -265 | -175 |               0 | True              |
| 1594 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -200 |        200 | -135 | -115 | -125 | -185 | -150 | -105 | -185 | -200 |               0 | True              |
| 1595 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        110 |        -80 |        190 |  110 |  -35 |  -30 |  -65 |  -80 |  -25 |  -50 |  -40 |               0 | True              |
| 1596 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |       -100 |        295 |   75 |   15 |    5 |   35 |  105 |   95 |  195 | -100 |               0 | True              |
| 1597 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -115 |        255 |  115 |  140 |  -20 |  -30 |   -0 | -115 | -115 |   20 |               0 | True              |
| 1598 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        170 |       -155 |        325 |  140 |  120 |  170 |  115 |   90 | -155 |  -75 | -140 |               0 | True              |
| 1599 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -230 |        315 |  -35 |  -60 |   85 |   15 |  -30 |    5 |   65 | -230 |               0 | True              |
| 1600 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -215 |        280 |   65 | -170 |  -90 | -125 | -215 | -105 | -135 | -200 |               0 | True              |
| 1601 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -585 |        585 | -410 | -385 | -525 | -465 | -515 | -515 | -510 | -585 |               3 | True              |
| 1602 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |        -80 |        230 |  140 |  105 |   90 |  150 |  -70 |   25 |  -80 |  -65 |               0 | True              |
| 1603 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        275 |          0 |        275 |  150 |  230 |  240 |  205 |  255 |  275 |  260 |   25 |               0 | True              |
| 1604 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         90 |        -65 |        155 |   90 |   60 |  -20 |  -55 |   -5 |  -65 |   40 |  -30 |               0 | True              |
| 1605 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -170 | -145 | -305 | -220 | -165 | -130 | -130 | -295 |               0 | True              |
| 1606 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -210 |        270 |   60 |   45 | -210 | -175 | -135 | -150 |  -65 | -185 |               0 | True              |
| 1607 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        235 |       -130 |        365 |  235 |   80 |  -15 |   75 |  -45 |  -80 | -130 | -105 |              17 | True              |
| 1608 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -425 |        425 | -160 | -120 | -165 | -140 | -235 | -180 | -100 | -425 |               0 | True              |
| 1609 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -425 |        425 | -150 | -135 | -145 | -150 | -350 | -385 | -330 | -425 |               2 | True              |
| 1610 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -530 |        530 | -185 | -315 | -340 | -305 | -385 | -320 | -415 | -530 |               1 | True              |
| 1611 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        255 |        -60 |        315 |  185 |  255 |  155 |  230 |   10 |  -60 |  -25 |  -35 |               0 | True              |
| 1612 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -565 |        565 | -390 | -335 | -285 | -240 | -365 | -360 | -565 | -460 |               4 | True              |
| 1613 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 |  -45 |  -75 | -105 | -110 |  -90 |  -10 |  -45 | -205 |               0 | True              |
| 1614 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         75 |       -110 |        185 |   70 |   75 | -110 |   45 |  -25 |  -40 |  -65 |  -75 |               0 | True              |
| 1615 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -220 |        280 |   60 | -210 | -140 | -150 | -135 | -165 | -220 |  -80 |               0 | True              |
| 1616 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -315 |        340 |  -60 |   25 | -100 |  -85 |   -0 |  -55 |   15 | -315 |               0 | True              |
| 1617 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -120 |        260 |   85 |   75 |  100 |  140 |   95 |  -40 |  -40 | -120 |               0 | True              |
| 1618 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -215 |        315 |   -0 |  -15 |   20 |  -15 | -135 |   30 |  100 | -215 |               0 | True              |
| 1619 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        260 |        -25 |        285 |  180 |  260 |   65 |  125 |   70 |  -25 |   -5 |   10 |               0 | True              |
| 1620 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -465 |        465 | -320 | -325 | -350 | -325 | -300 | -350 | -350 | -465 |               2 | True              |
| 1621 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        140 |       -170 |        310 |  140 |   85 |   60 |   30 |   60 |  -80 | -115 | -170 |               0 | True              |
| 1622 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          5 |       -200 |        205 |  -75 |  -55 | -195 |  -10 | -145 | -200 |    5 |  -20 |               0 | True              |
| 1623 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -360 |        360 | -130 |  -90 | -225 | -290 | -330 | -245 | -265 | -360 |               2 | True              |
| 1624 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -335 |        420 |  -75 |  -90 |  -95 | -105 | -100 |   85 |  -35 | -335 |               0 | True              |
| 1625 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -165 |        165 | -115 |  -25 |  -25 |  -30 | -100 | -105 |  -55 | -165 |               0 | True              |
| 1626 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -205 |        255 |   45 |   25 |  -80 |   15 |   -5 |   50 |  -25 | -205 |               0 | True              |
| 1627 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |       -255 |        365 |   60 |  -10 |  110 |   65 |  -50 |   -5 |   30 | -255 |               0 | True              |
| 1628 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -185 |        185 | -125 | -145 | -110 | -115 | -155 |  -55 |  -45 | -185 |               0 | True              |
| 1629 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         50 |       -225 |        275 |   50 |   45 |   -5 | -225 | -150 | -210 | -160 | -145 |               0 | True              |
| 1630 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          5 |       -300 |        305 |    5 |  -45 | -155 | -220 | -300 | -270 | -260 | -225 |               0 | True              |
| 1631 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        350 |          0 |        350 |  260 |  270 |  295 |  205 |  225 |  225 |  350 |   70 |              14 | True              |
| 1632 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        165 |       -205 |        370 |  165 |   95 |  -95 | -120 | -205 |  -40 | -115 |  -45 |               0 | True              |
| 1633 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -610 |        610 | -430 | -345 | -480 | -440 | -465 | -530 | -610 | -560 |               5 | True              |
| 1634 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -160 |        200 |   -0 |  -50 |  -15 | -160 |  -40 |   40 |  -75 |  -60 |               0 | True              |
| 1635 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        350 |          0 |        350 |  190 |  265 |  295 |  280 |  350 |  290 |  260 |  105 |               0 | True              |
| 1636 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        315 |          0 |        315 |  135 |  215 |  180 |  315 |  155 |  175 |  130 |   25 |               0 | True              |
| 1637 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -420 |        420 | -180 | -295 | -150 | -340 | -225 | -210 | -280 | -420 |               0 | True              |
| 1638 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -230 |        315 |   80 |   65 |   40 |   55 |   70 |   85 |  -10 | -230 |               0 | True              |
| 1639 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        215 |        -35 |        250 |  105 |  170 |  175 |  180 |  215 |  -20 |  -35 |   80 |               0 | True              |
| 1640 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |        -70 |        245 |   55 |  115 |  175 |  145 |   95 |  -70 |  -50 |   35 |               0 | True              |
| 1641 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         45 |       -280 |        325 |   45 |  -30 |   30 |  -85 |   -0 | -250 | -280 | -245 |               0 | True              |
| 1642 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -270 |        270 |  -75 | -145 | -155 | -110 |  -45 | -195 |  -90 | -270 |               0 | True              |
| 1643 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        185 |        -70 |        255 |  185 |  -70 |   10 |  -70 |   10 |   35 |   35 |   20 |               0 | True              |
| 1644 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        240 |         -5 |        245 |  180 |  190 |  220 |  200 |  240 |  235 |   -0 |   -5 |               0 | True              |
| 1645 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -520 |        520 | -225 | -285 | -235 | -425 | -440 | -520 | -455 | -480 |               3 | True              |
| 1646 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        115 |       -135 |        250 |  -25 |   35 |   40 |  115 |  -25 | -120 | -135 |  -65 |               1 | True              |
| 1647 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         95 |       -215 |        310 |   95 | -105 | -120 | -120 | -155 | -100 | -215 | -140 |               0 | True              |
| 1648 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -225 |        315 |   45 |   60 |   45 |   75 |   45 |   90 |  -10 | -225 |               0 | True              |
| 1649 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        275 |         -5 |        280 |  190 |  275 |  125 |  240 |   65 |   -5 |   50 |   15 |               0 | True              |
| 1650 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |       -100 |        130 |  -20 |   10 | -100 |  -10 |  -85 |  -65 |  -10 |   30 |               0 | True              |
| 1651 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -185 |        320 |  135 |   45 |  -15 |  -85 | -185 | -160 | -155 | -155 |               0 | True              |
| 1652 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -270 |        270 | -100 |  -50 | -200 | -200 | -135 | -200 | -265 | -270 |               1 | True              |
| 1653 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -60 |        285 |  185 |  225 |  170 |  185 |  200 |   60 |  -10 |  -60 |               0 | True              |
| 1654 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 | -200 | -180 |  -85 |  -65 | -180 | -170 | -225 | -180 |               0 | True              |
| 1655 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -265 |        365 |   35 |   80 |   30 |  -80 |  100 | -265 | -265 | -200 |               0 | True              |
| 1656 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -525 |        525 | -310 | -370 | -325 | -275 | -355 | -290 | -410 | -525 |               4 | True              |
| 1657 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -120 |        295 |   55 |   75 |  115 |  130 |  175 | -120 |  -75 |  -65 |               0 | True              |
| 1658 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        265 |          0 |        265 |  195 |  225 |  250 |  235 |  265 |   65 |   20 |    5 |               0 | True              |
| 1659 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         95 |       -225 |        320 |   50 |  -30 |   95 |   50 | -105 | -180 | -225 | -185 |               0 | True              |
| 1660 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -280 |        300 |   20 | -230 | -165 | -145 | -280 | -280 | -210 | -270 |               0 | True              |
| 1661 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -170 |        255 |  -70 |   85 |  -45 |   -0 |  -55 |  -30 |  -20 | -170 |               0 | True              |
| 1662 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -370 |        370 | -140 |  -40 |  -60 |  -75 | -120 |  -95 | -355 | -370 |               2 | True              |
| 1663 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -725 |        725 | -430 | -505 | -475 | -600 | -525 | -530 | -670 | -725 |               5 | True              |
| 1664 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         95 |       -175 |        270 |    5 |   10 |   65 |   95 | -175 | -160 | -145 | -175 |               0 | True              |
| 1665 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        135 |          0 |        135 |  115 |   55 |  130 |  135 |   85 |   15 |   45 |   40 |               0 | True              |
| 1666 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        185 |       -105 |        290 |  185 |  -30 |   55 |   70 |  145 |   35 | -105 |  -65 |               0 | True              |
| 1667 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        170 |        -60 |        230 |  150 |  130 |  130 |  170 |  155 |   55 |  -60 |  -30 |               0 | True              |
| 1668 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        235 |         -5 |        240 |  195 |  200 |  195 |  235 |  195 |   65 |   -5 |   50 |               0 | True              |
| 1669 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -630 |        630 | -345 | -275 | -295 | -435 | -550 | -545 | -630 | -530 |               4 | True              |
| 1670 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        265 |          0 |        265 |  205 |  180 |  265 |  150 |  160 |  215 |   40 |   15 |               0 | True              |
| 1671 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |          0 |        185 |  165 |  110 |  135 |  185 |   65 |  150 |  140 |   25 |               0 | True              |
| 1672 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -620 |        620 | -290 | -535 | -500 | -585 | -580 | -615 | -620 | -605 |               4 | True              |
| 1673 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |       -135 |        180 |  -85 | -105 |  -40 | -135 |   45 |  -25 |  -30 |   20 |               0 | True              |
| 1674 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -190 |        190 | -130 |  -80 |  -70 |  -20 |  -30 |  -20 |   -0 | -190 |               0 | True              |
| 1675 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -175 |        295 |  120 |  -40 | -140 | -175 | -120 | -155 |  -25 | -165 |               0 | True              |
| 1676 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -410 |        410 | -140 | -185 | -375 | -400 | -330 | -385 | -410 | -335 |               2 | True              |
| 1677 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -445 |        445 | -345 | -360 | -415 | -410 | -355 | -445 | -395 | -400 |               2 | True              |
| 1678 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -200 |        260 |   25 |   60 | -190 | -145 |  -55 | -135 |   -5 | -200 |               0 | True              |
| 1679 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        245 |        -85 |        330 |  170 |  165 |  245 |   40 |  -85 |   10 |  -65 |  -15 |               0 | True              |
| 1680 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -250 |        250 | -145 | -215 | -195 | -170 |  -90 | -100 | -115 | -250 |               0 | True              |
| 1681 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -740 |        740 | -445 | -415 | -685 | -645 | -740 | -570 | -600 | -630 |               5 | True              |
| 1682 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |        -65 |        245 |  105 |  160 |  180 |  125 |  -25 |  -25 |  -65 |  -10 |               0 | True              |
| 1683 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        165 |       -140 |        305 |  165 |  105 |  110 |  105 |   80 |  -70 | -140 |  -80 |               0 | True              |
| 1684 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -335 |        335 |  -50 |   -5 |  -30 |   -0 |  -50 | -110 |  -25 | -335 |               0 | True              |
| 1685 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -305 |        305 | -140 | -240 | -255 | -175 | -185 | -305 | -255 | -250 |               0 | True              |
| 1686 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -735 |        735 | -420 | -430 | -535 | -735 | -725 | -630 | -700 | -720 |               5 | True              |
| 1687 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -375 |        375 | -175 | -130 |  -95 | -265 | -320 | -375 | -350 | -265 |               2 | True              |
| 1688 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -190 |        315 |  110 |   15 |  125 |   95 |  120 | -190 | -120 | -155 |               0 | True              |
| 1689 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -185 | -165 | -225 | -255 | -180 | -190 | -155 | -245 |               0 | True              |
| 1690 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -215 |        215 | -150 | -105 | -215 | -175 |  -90 | -200 | -105 | -115 |               0 | True              |
| 1691 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        230 |       -110 |        340 |  125 |   85 |   75 |  230 |   45 |   55 |  225 | -110 |               0 | True              |
| 1692 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |        -80 |        275 |  195 |  -55 |  -20 |   -0 |  -80 |   70 |  -25 |  -25 |               0 | True              |
| 1693 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -150 |        315 |  110 |  -30 |  165 |  -85 |  -65 |  -60 |  -80 | -150 |               0 | True              |
| 1694 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -295 |        295 |  -90 | -155 | -105 | -130 | -220 | -260 | -295 | -270 |               1 | True              |
| 1695 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |        -70 |        270 |   75 |  165 |  160 |  200 |  170 |   85 |  -50 |  -70 |               0 | True              |
| 1696 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -600 |        600 | -265 | -365 | -600 | -550 | -570 | -510 | -560 | -510 |               3 | True              |
| 1697 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        130 |        -55 |        185 |    5 |   70 |    5 |  -55 |  130 |   70 |   -5 |  -15 |               0 | True              |
| 1698 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |        -65 |        180 |  105 |  115 |  105 |   30 |   45 |  -65 |  -20 |  -35 |               0 | True              |
| 1699 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -335 |        335 | -130 | -125 | -170 | -215 | -100 |  -60 | -250 | -335 |               0 | True              |
| 1700 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |       -100 |        280 |  180 |  150 |  175 |  130 |  -10 | -100 |    5 |   -5 |               0 | True              |
| 1701 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         55 |       -295 |        350 |   55 | -195 | -170 | -130 | -135 | -170 | -170 | -295 |               0 | True              |
| 1702 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -505 |        505 | -200 | -330 | -270 | -305 | -235 | -350 | -320 | -505 |               0 | True              |
| 1703 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -515 |        515 | -135 | -290 | -515 | -485 | -455 | -435 | -465 | -460 |               2 | True              |
| 1704 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         50 |       -220 |        270 |   20 |   40 |  -80 |   50 |    5 |  -85 | -220 | -150 |               0 | True              |
| 1705 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -140 |        140 |  -80 |  -80 | -140 |  -75 |  -40 |  -60 |  -90 | -130 |               0 | True              |
| 1706 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        220 |        -65 |        285 |  160 |  200 |  220 |  145 |  -65 |   20 |   30 |   -0 |               0 | True              |
| 1707 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -730 |        730 | -575 | -560 | -535 | -525 | -505 | -535 | -715 | -730 |               6 | True              |
| 1708 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -180 |        245 |   65 | -145 |  -55 | -110 |  -95 |  -80 | -180 | -100 |               0 | True              |
| 1709 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        115 |       -215 |        330 |   10 |  -25 |  115 |   95 | -140 | -105 | -215 | -130 |               0 | True              |
| 1710 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -770 |        770 | -550 | -315 | -510 | -635 | -705 | -595 | -655 | -770 |               6 | True              |
| 1711 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -220 |        310 |   90 |   65 |   30 |  -15 |  -30 |   25 | -220 | -170 |               0 | True              |
| 1712 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         70 |       -175 |        245 |   65 |   70 |  -90 |  -70 |  -90 |  -65 | -175 | -135 |               0 | True              |
| 1713 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         70 |       -295 |        365 |  -35 |   70 | -170 | -295 | -175 | -175 | -240 | -210 |               1 | True              |
| 1714 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        185 |       -110 |        295 |  145 |  185 |  150 |  -90 |  -90 |   -0 | -110 |  -60 |               0 | True              |
| 1715 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -735 |        735 | -575 | -735 | -605 | -580 | -565 | -690 | -625 | -630 |               4 | True              |
| 1716 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -535 |        535 | -400 | -420 | -450 | -480 | -535 | -510 | -480 | -510 |               3 | True              |
| 1717 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -305 |        335 |   10 |   30 | -210 | -235 | -305 | -215 | -255 | -260 |               0 | True              |
| 1718 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          5 |       -290 |        295 |  -15 |   -0 |    5 |  -15 | -185 | -265 | -290 | -215 |               1 | True              |
| 1719 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        105 |       -220 |        325 |   50 |   20 |    5 |  105 |   10 | -150 | -170 | -220 |               0 | True              |
| 1720 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         80 |       -160 |        240 |   80 |   30 |  -50 |   50 |   25 |   45 | -105 | -160 |               0 | True              |
| 1721 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        100 |       -185 |        285 |  100 |   85 |   75 | -135 | -100 | -185 |  -90 |  -80 |               0 | True              |
| 1722 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -455 |        455 | -200 | -320 | -225 | -435 | -410 | -455 | -410 | -430 |               3 | True              |
| 1723 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        305 |          0 |        305 |  220 |  210 |  305 |  290 |   90 |   75 |   20 |  140 |               0 | True              |
| 1724 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -290 |        290 | -185 | -145 | -200 | -230 |  -15 | -215 | -180 | -290 |               0 | True              |
| 1725 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -460 |        460 | -410 | -360 | -420 | -350 | -450 | -450 | -405 | -460 |               3 | True              |
| 1726 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -220 |        245 |  -60 |  -25 |  -70 |   15 |  -25 |  -70 |   25 | -220 |               0 | True              |
| 1727 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -315 |        315 | -125 | -115 |  -15 |  -90 |  -85 | -140 |  -85 | -315 |               0 | True              |
| 1728 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        165 |        -60 |        225 |  165 |  165 |   -0 |   35 |  -60 |  -15 |  -30 |   70 |               0 | True              |
| 1729 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         50 |       -225 |        275 |   50 |   15 |  -30 | -225 | -140 | -155 |  -30 |  -90 |               0 | True              |
| 1730 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        100 |       -170 |        270 |   75 |   15 |   15 |  -15 |  100 |   95 | -170 |  -80 |               0 | True              |
| 1731 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -145 |        320 |   30 |  105 |  100 |  175 | -115 |  -70 | -145 |  -25 |               0 | True              |
| 1732 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |        -45 |        290 |  200 |  125 |  245 |  145 |   -0 |  -45 |  -20 |  -45 |              17 | True              |
| 1733 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -200 |        200 | -125 | -145 | -130 | -110 | -190 | -110 | -200 | -125 |               0 | True              |
| 1734 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -385 |        385 | -120 | -255 | -265 | -360 | -335 | -240 | -240 | -385 |               0 | True              |
| 1735 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -620 |        620 | -460 | -490 | -610 | -510 | -620 | -620 | -560 | -620 |               5 | True              |
| 1736 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        130 |       -155 |        285 |  105 |  100 |   90 |  130 |  130 |  -85 | -155 | -125 |               0 | True              |
| 1737 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        145 |       -200 |        345 |  145 |   45 |  100 |   35 | -125 | -165 | -200 | -125 |               0 | True              |
| 1738 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 |  -75 |  -75 |  -55 | -125 |  -50 |  -35 | -180 | -135 |               0 | True              |
| 1739 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        320 |        -80 |        400 |  215 |  300 |  320 |  305 |  215 |  -80 |  100 |  100 |              16 | True              |
| 1740 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -215 |        305 |   90 |  -25 |  -55 |   60 | -120 |  -30 | -175 | -215 |               0 | True              |
| 1741 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -300 |        300 | -160 |  -70 |  -20 |  -60 |  -70 |  -60 | -125 | -300 |               0 | True              |
| 1742 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -315 |        315 | -210 | -195 | -150 | -235 | -200 | -195 | -315 | -130 |               0 | True              |
| 1743 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         90 |       -140 |        230 |   90 |   70 |   85 |  -30 |  -35 | -140 |  -55 |  -60 |               0 | True              |
| 1744 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -530 |        530 | -255 | -285 | -300 | -350 | -400 | -275 | -530 | -460 |               3 | True              |
| 1745 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -365 |        365 | -105 |  -95 | -125 | -205 |  -95 | -100 | -255 | -365 |               1 | True              |
| 1746 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |       -100 |        300 |  145 |  180 |  200 |  165 |   95 |   15 |  -85 | -100 |               0 | True              |
| 1747 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -260 |        260 | -155 | -145 | -145 | -125 | -145 |  -65 |  -75 | -260 |               0 | True              |
| 1748 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -740 |        740 | -555 | -600 | -485 | -545 | -610 | -535 | -740 | -715 |               6 | True              |
| 1749 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -225 |        400 |  100 |   -0 |  175 |   25 | -175 | -120 | -225 | -100 |               0 | True              |
| 1750 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 |  -85 | -255 | -215 | -220 | -140 | -180 | -205 | -235 |               0 | True              |
| 1751 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -495 |        495 | -430 | -455 | -480 | -495 | -465 | -450 | -385 | -435 |               3 | True              |
| 1752 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        195 |       -125 |        320 |  195 |  140 |   -5 |  -40 |  -15 | -125 |  -65 |  -35 |               0 | True              |
| 1753 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -635 |        635 | -535 | -560 | -565 | -595 | -540 | -635 | -515 | -585 |               4 | True              |
| 1754 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         10 |       -425 |        435 | -140 | -145 |   10 |  -90 | -100 | -425 | -325 | -365 |               2 | True              |
| 1755 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -640 |        640 | -390 | -415 | -585 | -640 | -560 | -570 | -615 | -560 |               4 | True              |
| 1756 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        235 |        -75 |        310 |  145 |  235 |  -75 |   30 |   10 |  -15 |  115 |  -10 |               0 | True              |
| 1757 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        330 |        -30 |        360 |  210 |  330 |  270 |  130 |   40 |   65 |   25 |  -30 |               0 | True              |
| 1758 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -585 |        585 | -220 | -320 | -525 | -585 | -435 | -515 | -460 | -515 |               3 | True              |
| 1759 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -180 | -255 | -220 | -175 | -175 | -180 | -250 | -200 |               0 | True              |
| 1760 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -440 |        440 | -230 | -205 | -205 | -195 | -440 | -375 | -320 | -330 |               3 | True              |
| 1761 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -125 |        125 |  -95 |  -80 | -125 |  -45 |  -10 |  -70 |  -55 |  -65 |               0 | True              |
| 1762 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        300 |          0 |        300 |  190 |  190 |  185 |  300 |  230 |  210 |   40 |   20 |               0 | True              |
| 1763 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |       -165 |        280 |  -40 |   70 |  -70 |   10 |  105 |  115 |   45 | -165 |               0 | True              |
| 1764 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -310 |        310 | -145 |  -80 |  -25 |  -10 | -150 | -235 | -280 | -310 |               2 | True              |
| 1765 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        340 |        -30 |        370 |  175 |  340 |  195 |  200 |  120 |  145 |  -30 |   -5 |               0 | True              |
| 1766 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -245 |        295 |   35 |  -90 |  -75 |  -35 |   50 |  -95 |  -35 | -245 |               0 | True              |
| 1767 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -150 |        215 |   65 |  -60 | -150 | -125 |  -35 |  -95 |  -65 |  -35 |               0 | True              |
| 1768 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -570 |        570 | -305 | -275 | -325 | -270 | -240 | -565 | -570 | -540 |               4 | True              |
| 1769 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |       -120 |        290 |  170 |  115 | -110 |  -45 |  -75 |  -95 | -120 |   -5 |               0 | True              |
| 1770 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        225 |       -105 |        330 |  105 |  190 |  185 |  225 | -105 |  -45 |   55 |  -65 |               0 | True              |
| 1771 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -155 |        280 |  125 |  105 |  -35 | -155 |  -75 | -110 |  -70 | -125 |               0 | True              |
| 1772 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -215 |        340 |  110 |  125 |   35 |   95 |   95 | -100 | -125 | -215 |               0 | True              |
| 1773 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |       -220 |        380 |  160 |  140 |   70 | -195 |  -85 |  -75 | -220 | -170 |               0 | True              |
| 1774 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -345 |        345 | -180 | -240 | -130 | -225 | -260 | -205 | -110 | -345 |               0 | True              |
| 1775 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        225 |       -105 |        330 |  165 |  135 |  125 |  225 | -105 |  160 |  -80 |  -90 |               0 | True              |
| 1776 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |        -65 |        245 |  150 |   95 |  170 |  180 |  160 |  -65 |   50 |   20 |               0 | True              |
| 1777 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -360 |        360 | -150 | -155 |  -50 |  -75 |  -25 |  -90 |  -35 | -360 |               0 | True              |
| 1778 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        155 |         -0 |        155 |   10 |   60 |   70 |  155 |   60 |   -0 |   95 |   55 |               0 | True              |
| 1779 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |        -55 |        125 |  -55 |   -0 |  -40 |  -35 |   -0 |   15 |   70 |  -50 |               0 | True              |
| 1780 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -75 |        300 |  185 |  225 |  155 |  125 |  125 |  135 |  -75 |  -55 |               0 | True              |
| 1781 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        200 |       -110 |        310 |  200 |   25 |  145 |  140 | -110 |  -40 |  -90 |   20 |              17 | True              |
| 1782 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -160 |        220 |   60 |   40 |   30 |  -25 |   20 |   30 |   -5 | -160 |               0 | True              |
| 1783 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        240 |        -25 |        265 |  185 |  240 |  200 |   -0 |   55 |   40 |   15 |  -25 |               0 | True              |
| 1784 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |       -185 |        315 |   55 |  130 |  -90 | -155 | -185 | -120 |  -30 |  -95 |               0 | True              |
| 1785 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        260 |          0 |        260 |  195 |  250 |  260 |  225 |  245 |  245 |   90 |   60 |               0 | True              |
| 1786 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -190 |        225 |   10 |   -0 |   -5 |   35 |    5 |   30 |   20 | -190 |               0 | True              |
| 1787 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         30 |       -405 |        435 |  -80 | -100 |   30 | -120 | -125 | -100 | -405 | -300 |               1 | True              |
| 1788 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -445 |        445 | -240 | -185 | -230 | -205 | -260 | -185 | -305 | -445 |               1 | True              |
| 1789 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        240 |         -5 |        245 |  160 |  160 |  215 |  240 |  210 |   60 |   -5 |   -5 |               0 | True              |
| 1790 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         20 |       -310 |        330 |  -80 |  -60 | -115 |   15 |   20 | -310 | -225 | -280 |               1 | True              |
| 1791 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -305 |        305 | -110 |  -15 |  -55 |  -50 | -285 | -260 | -305 | -225 |               1 | True              |
| 1792 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -120 |        230 |   85 |   90 |  110 |  110 |  -70 | -120 | -100 |   -5 |               0 | True              |
| 1793 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         85 |       -160 |        245 |   40 |   20 |  -35 | -110 |   85 | -160 | -135 | -100 |               0 | True              |
| 1794 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -175 |        265 |   70 |   60 |  -45 |  -20 |   90 |   -5 |  -25 | -175 |               0 | True              |
| 1795 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -625 |        625 | -390 | -310 | -305 | -625 | -425 | -535 | -520 | -535 |               4 | True              |
| 1796 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |        -60 |        240 |  110 |  180 |  140 |  -60 |    5 |  -10 |   45 |  -50 |               0 | True              |
| 1797 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |       -115 |        265 |  150 |  105 |  135 |  135 |  125 | -115 | -105 |  -90 |               0 | True              |
| 1798 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -505 |        505 | -395 | -315 | -240 | -405 | -480 | -505 | -445 | -475 |               4 | True              |
| 1799 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -160 |        160 | -105 |  -60 | -160 | -100 |   -5 |  -25 | -100 |  -55 |               0 | True              |
| 1800 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -785 |        785 | -555 | -440 | -485 | -440 | -480 | -505 | -445 | -785 |               4 | True              |
| 1801 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -690 |        690 | -435 | -470 | -435 | -620 | -675 | -665 | -690 | -645 |               5 | True              |
| 1802 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -635 |        635 | -280 | -485 | -540 | -600 | -535 | -520 | -560 | -635 |               3 | True              |
| 1803 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -435 |        435 | -220 | -205 | -395 | -435 | -385 | -340 | -425 | -365 |               3 | True              |
| 1804 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |        -70 |        240 |  135 |  165 |  160 |  170 |  145 |   15 |  -70 |  -60 |               0 | True              |
| 1805 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -225 |        255 |   30 |  -25 | -190 | -225 | -140 | -140 | -225 | -190 |               0 | True              |
| 1806 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -140 |        295 |  140 |   55 |   35 |  155 |  115 |  -65 | -140 | -135 |               0 | True              |
| 1807 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -235 |        370 |   70 |  135 |  -80 | -210 | -155 | -150 | -235 |  -95 |               0 | True              |
| 1808 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |        -85 |        290 |  205 |  170 |  -85 |  -15 |  -25 |  -30 |   -5 |  -80 |               0 | True              |
| 1809 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        215 |        -95 |        310 |  175 |  120 |  170 |  215 |  190 |   70 |  -95 |   70 |               0 | True              |
| 1810 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -305 |        365 |   60 | -140 | -170 |  -85 | -305 | -165 | -175 | -140 |               0 | True              |
| 1811 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -620 |        620 | -515 | -495 | -580 | -595 | -470 | -620 | -470 | -490 |               3 | True              |
| 1812 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -150 |        290 |  140 |   60 |  -80 |  -70 | -115 | -150 |  -40 |  -10 |               0 | True              |
| 1813 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -135 |        225 |   -0 |   55 |   90 |   25 |   10 |   20 | -115 | -135 |               1 | True              |
| 1814 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -780 |        780 | -780 | -610 | -660 | -660 | -765 | -750 | -620 | -675 |               6 | True              |
| 1815 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |        -25 |        185 |  135 |  145 |  115 |  110 |  160 |  145 |  -25 |  -25 |               0 | True              |
| 1816 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |       -130 |        275 |   80 |   75 |  110 |  130 |  145 |   35 |  -15 | -130 |               0 | True              |
| 1817 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |        -95 |        240 |    5 |  105 |  140 |   60 |  145 |   70 |  -95 |  -60 |               0 | True              |
| 1818 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -615 |        615 | -390 | -470 | -510 | -420 | -475 | -490 | -370 | -615 |               2 | True              |
| 1819 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 | -135 | -130 |   -5 |  -95 |  -25 |  -85 |  -15 |  -30 |               0 | True              |
| 1820 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         45 |       -250 |        295 |  -10 |   45 | -195 | -250 | -240 | -200 | -225 | -175 |               0 | True              |
| 1821 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -10 |        220 |  185 |  135 |  210 |  100 |  115 |  165 |  -10 |  -10 |               0 | True              |
| 1822 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -345 |        345 |  -55 | -140 | -100 | -105 | -250 | -345 | -310 | -215 |               1 | True              |
| 1823 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -585 |        585 | -310 | -240 | -340 | -420 | -485 | -585 | -560 | -545 |               4 | True              |
| 1824 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -155 |        155 |  -65 |  -85 |  -80 |  -55 | -120 | -115 | -155 |  -60 |               0 | True              |
| 1825 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -490 |        490 | -105 | -490 | -385 | -345 | -345 | -350 | -405 | -380 |               2 | True              |
| 1826 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -195 |        205 |   10 | -115 |  -85 | -100 | -120 |  -50 | -195 | -120 |               0 | True              |
| 1827 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |        -95 |        260 |  165 |  -80 |  -65 |  -45 |  -50 |  -25 |  -95 |  -40 |               0 | True              |
| 1828 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -425 |        425 | -195 | -295 | -220 | -205 | -170 | -205 | -415 | -425 |               3 | True              |
| 1829 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |       -100 |        290 |   35 |  150 |   45 |  190 |  -45 |  -10 | -100 |  -85 |               0 | True              |
| 1830 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -550 |        550 | -300 | -185 | -220 | -240 | -175 | -550 | -430 | -460 |               4 | True              |
| 1831 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         55 |       -120 |        175 |  -50 |   55 |  -55 |   -0 | -120 | -105 |   10 |  -45 |               0 | True              |
| 1832 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         45 |       -195 |        240 |   45 |   35 |  -20 |   -5 | -175 | -110 |  -85 | -195 |               0 | True              |
| 1833 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -650 |        650 | -430 | -435 | -290 | -615 | -575 | -650 | -635 | -545 |               5 | True              |
| 1834 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         10 |       -275 |        285 |   10 |  -20 |  -80 |  -75 |  -40 | -275 | -205 | -215 |               0 | True              |
| 1835 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         15 |       -320 |        335 |   15 |  -30 |  -10 | -165 | -225 | -320 | -180 | -215 |               0 | True              |
| 1836 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -595 |        595 | -315 | -330 | -300 | -495 | -455 | -535 | -555 | -595 |               4 | True              |
| 1837 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -200 |        260 |   60 | -160 | -120 | -200 | -195 |   -5 |  -95 | -135 |               0 | True              |
| 1838 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -300 |        300 | -170 | -205 | -285 | -270 | -180 | -165 | -175 | -300 |               0 | True              |
| 1839 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         40 |       -260 |        300 |   15 |   40 |  -25 |  -90 |   35 | -260 | -250 | -180 |               0 | True              |
| 1840 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -385 |        385 | -385 | -270 | -335 | -265 | -290 | -285 | -335 | -355 |               3 | True              |
| 1841 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |       -265 |        345 |   80 |  -15 |   55 | -170 | -220 | -185 | -265 | -190 |               0 | True              |
| 1842 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         45 |       -180 |        225 |   45 | -170 | -165 |  -90 | -110 | -100 | -140 | -180 |               0 | True              |
| 1843 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |       -250 |        370 |   40 |  -20 |  120 |  100 |   85 | -210 | -250 | -150 |               0 | True              |
| 1844 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         45 |       -130 |        175 |   45 |  -30 |  -20 |   10 |   45 |  -35 | -130 |  -95 |               0 | True              |
| 1845 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -155 |        280 |  125 |   75 |  -65 | -155 |  -50 | -140 | -115 | -140 |               0 | True              |
| 1846 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -380 |        380 | -220 | -190 | -215 | -120 |  -70 | -150 | -355 | -380 |               3 | True              |
| 1847 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -150 |        340 |  105 |  190 |  100 |  160 |   70 |  -90 |  -50 | -150 |               0 | True              |
| 1848 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         -0 |       -240 |        240 |   -0 |  -65 | -240 | -160 | -200 | -200 | -205 | -200 |               0 | True              |
| 1849 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -800 |        800 | -585 | -515 | -570 | -575 | -495 | -770 | -750 | -800 |               6 | True              |
| 1850 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        320 |         -0 |        320 |  175 |  320 |  145 |  210 |  315 |  200 |  125 |   -0 |               0 | True              |
| 1851 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -690 |        690 | -490 | -525 | -500 | -570 | -565 | -570 | -480 | -690 |               3 | True              |
| 1852 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -190 |  -55 | -145 | -115 | -215 |  -45 | -255 | -130 |               1 | True              |
| 1853 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |        -90 |        250 |   90 |  160 |  100 |   50 |  -60 |  -15 |   -0 |  -90 |               0 | True              |
| 1854 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -215 |        315 |   85 |  -20 |    5 |   -0 |  100 | -135 | -215 | -115 |               0 | True              |
| 1855 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -260 |        385 |  125 |   35 | -165 | -185 | -260 | -140 | -210 | -205 |               0 | True              |
| 1856 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -135 |        315 |  180 |  160 |   85 |  145 |  155 |  -50 |  -30 | -135 |               0 | True              |
| 1857 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -340 |        340 |  -75 | -140 | -160 | -235 | -240 | -340 | -335 | -305 |               1 | True              |
| 1858 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         60 |       -245 |        305 |   55 |   45 |  -25 |   60 | -130 | -245 | -205 | -200 |               0 | True              |
| 1859 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -150 |        250 |  100 |   85 |  -65 | -105 |  -95 | -105 | -150 | -115 |               0 | True              |
| 1860 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -235 |        340 |   90 |  -50 |   25 |  -40 |   20 |  -40 |  105 | -235 |               0 | True              |
| 1861 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -320 |        320 |  -65 | -150 |  -55 | -175 |  -55 |  -90 |  -65 | -320 |               0 | True              |
| 1862 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -375 |        375 | -135 | -225 | -250 | -250 | -120 | -135 | -245 | -375 |               0 | True              |
| 1863 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -615 |        615 | -385 | -595 | -610 | -530 | -540 | -475 | -585 | -615 |               4 | True              |
| 1864 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        185 |        -80 |        265 |  185 |   35 |   15 |  -80 |   35 |  -60 |  -45 |   15 |               0 | True              |
| 1865 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        355 |          0 |        355 |  260 |  215 |  355 |  215 |  285 |  210 |  320 |   80 |              14 | True              |
| 1866 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -555 |        555 | -450 | -525 | -545 | -555 | -490 | -485 | -515 | -460 |               3 | True              |
| 1867 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -135 |        310 |   95 |  175 |   20 |  135 |  -25 |  -70 | -135 |   95 |               0 | True              |
| 1868 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         10 |       -230 |        240 |   10 |  -75 |  -40 |  -30 | -100 | -140 |  -75 | -230 |               0 | True              |
| 1869 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          5 |       -300 |        305 |    5 | -275 | -205 | -100 | -170 | -145 | -235 | -300 |               0 | True              |
| 1870 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         95 |       -170 |        265 |   95 |   75 |   70 |   65 |  -80 | -105 | -170 | -145 |               0 | True              |
| 1871 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        300 |          0 |        300 |  210 |  155 |  300 |  255 |   45 |  110 |  130 |  135 |               0 | True              |
| 1872 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         25 |       -265 |        290 |   25 |  -45 |  -30 | -240 | -265 | -180 | -230 | -145 |               0 | True              |
| 1873 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -220 |        285 |   65 | -120 | -110 | -150 | -135 |   40 | -220 |  -55 |               0 | True              |
| 1874 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -70 |        295 |  145 |   85 |   35 |  175 |  125 |  225 |  -70 |  -60 |               0 | True              |
| 1875 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -515 |        515 | -305 | -415 | -370 | -350 | -445 | -370 | -515 | -360 |               2 | True              |
| 1876 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         75 |       -210 |        285 |   75 |   70 |   50 |   55 |   30 |   -5 | -115 | -210 |               0 | True              |
| 1877 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |       -150 |        395 |  165 |  190 |   45 |  245 | -150 |  -65 | -125 |  -70 |               0 | True              |
| 1878 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |          0 |        190 |  190 |   40 |   75 |   95 |   55 |   80 |   15 |   90 |               0 | True              |
| 1879 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 | -125 | -240 | -160 | -265 |  -90 | -135 | -155 | -340 |               0 | True              |
| 1880 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -120 |        295 |  170 |   85 |  175 |   60 | -120 |  -85 | -105 |  -85 |               0 | True              |
| 1881 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        200 |        -15 |        215 |  200 |   15 |   15 |   60 |  -15 |  -10 |   80 |   50 |               0 | True              |
| 1882 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -155 |        255 |   60 |  100 | -155 |  -95 | -110 | -135 |  -90 | -125 |               0 | True              |
| 1883 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        325 |        -55 |        380 |  160 |  325 |  230 |  190 |  240 |  180 |  155 |  -55 |               0 | True              |
| 1884 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -615 |        615 | -365 | -445 | -615 | -565 | -525 | -575 | -605 | -555 |               4 | True              |
| 1885 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -530 |        530 | -245 | -220 | -245 | -275 | -415 | -490 | -515 | -530 |               3 | True              |
| 1886 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         95 |        -80 |        175 |   35 |  -30 |   95 |   70 |  -15 |  -80 |  -35 |   50 |              16 | True              |
| 1887 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -85 |        295 |   85 |  205 |  115 |  210 |   75 |  140 |  -85 |  -85 |               0 | True              |
| 1888 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -400 |        400 | -215 | -255 | -180 | -180 | -185 | -125 | -400 | -330 |               3 | True              |
| 1889 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        260 |       -150 |        410 |  115 |  160 |  115 |  195 |  205 |  260 | -150 |   40 |               0 | True              |
| 1890 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -495 |        495 | -245 | -225 | -485 | -480 | -360 | -495 | -430 | -465 |               3 | True              |
| 1891 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -215 |        350 |   30 |   70 |  135 | -215 | -160 | -125 | -205 | -185 |               0 | True              |
| 1892 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -360 |        360 |  -95 | -205 | -245 |  -90 | -150 |  -75 |  -90 | -360 |               0 | True              |
| 1893 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -165 |        315 |  135 |  150 |  -75 | -120 |  -70 | -165 |  -95 | -145 |               0 | True              |
| 1894 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        160 |       -125 |        285 |  155 |  130 |  110 |   70 |  160 | -120 | -125 |   40 |               0 | True              |
| 1895 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |       -215 |        355 |   10 |  140 |  120 |   55 |  -35 | -115 |  -75 | -215 |               0 | True              |
| 1896 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -360 |        360 |  -35 |  -60 |  -20 | -150 | -360 | -275 | -220 | -300 |               1 | True              |
| 1897 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |        -65 |        255 |  190 |  -20 |  -30 |   35 |  -65 |   65 |   50 |  -45 |               0 | True              |
| 1898 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         90 |       -215 |        305 |   35 |   50 |   90 | -215 | -150 | -110 |  -40 |  -80 |               0 | True              |
| 1899 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -160 |        230 |   70 |    5 |  -90 |  -90 |    5 |  -10 |  -45 | -160 |               0 | True              |
| 1900 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -450 |        450 | -160 | -100 | -415 | -305 | -400 | -415 | -450 | -320 |               2 | True              |
| 1901 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |       -135 |        295 |  160 |  150 |  155 |  -20 |   10 |  -35 |   25 | -135 |               0 | True              |
| 1902 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -190 |        320 |   60 |  130 |   80 | -160 | -125 |   -5 | -190 | -135 |               0 | True              |
| 1903 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -445 |        445 |  -65 | -125 | -135 | -195 | -180 |  -80 | -135 | -445 |               0 | True              |
| 1904 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         90 |       -225 |        315 |   90 | -155 | -135 | -225 | -105 | -210 | -190 | -120 |               0 | True              |
| 1905 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |        -85 |        235 |   55 |  125 |  130 |  150 |  135 |  -50 |  -40 |  -85 |               0 | True              |
| 1906 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -600 |        600 | -280 | -280 | -280 | -595 | -495 | -425 | -600 | -580 |               3 | True              |
| 1907 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -400 |        400 | -115 | -175 | -180 |  -35 | -355 | -400 | -330 | -290 |               2 | True              |
| 1908 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         90 |       -145 |        235 |   90 | -115 |   15 |  -65 |   -5 |  -70 | -115 | -145 |               0 | True              |
| 1909 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -290 |        290 | -110 | -110 | -145 | -115 | -135 | -170 |  -75 | -290 |               0 | True              |
| 1910 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -100 |        100 |  -60 |  -10 |  -50 |  -75 |  -20 | -100 |  -30 |  -25 |               0 | True              |
| 1911 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -355 |        355 |  -45 | -125 |  -80 | -355 | -235 | -260 | -185 | -215 |               1 | True              |
| 1912 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         50 |        -75 |        125 |   -0 |   50 |  -30 |   50 |  -50 |  -75 |   -0 |  -30 |              16 | True              |
| 1913 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         40 |       -160 |        200 |   40 | -125 |  -90 | -150 | -125 | -130 | -160 | -145 |               0 | True              |
| 1914 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         45 |       -135 |        180 |   35 | -100 | -135 |  -95 |   45 |   30 |  -30 |  -55 |               0 | True              |
| 1915 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        220 |       -115 |        335 |  195 |   55 |  160 |  220 |  -25 |  -95 | -115 |  -25 |               0 | True              |
| 1916 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -365 |        365 |  -55 | -125 | -235 | -165 |  -80 | -130 |  -70 | -365 |               0 | True              |
| 1917 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |        -85 |        280 |  105 |  110 |   50 |  195 |  125 |  175 |  -85 |  -45 |               0 | True              |
| 1918 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        300 |        -80 |        380 |   90 |  255 |  300 |  150 |  220 |  225 |  -70 |  -80 |               0 | True              |
| 1919 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -580 |        580 | -295 | -390 | -570 | -450 | -520 | -530 | -545 | -580 |               3 | True              |
| 1920 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -195 |        245 |  -20 |   50 | -155 | -155 | -190 | -180 | -180 | -195 |               1 | True              |
| 1921 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |       -120 |        310 |  120 |  190 |  180 |  145 |  145 |  120 |  130 | -120 |               0 | True              |
| 1922 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -420 |        420 | -135 | -335 | -415 | -420 | -340 | -335 | -335 | -360 |               2 | True              |
| 1923 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        330 |          0 |        330 |  165 |  250 |  330 |  235 |  270 |  245 |   30 |  165 |               0 | True              |
| 1924 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         45 |       -275 |        320 |   45 |  -80 |   45 | -185 | -110 | -275 | -170 | -115 |               0 | True              |
| 1925 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |       -235 |        310 |  -55 |   10 |  -65 |  -55 |   75 |   50 |   65 | -235 |               0 | True              |
| 1926 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -585 |        585 | -205 | -405 | -330 | -330 | -355 | -270 | -585 | -490 |               3 | True              |
| 1927 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        205 |        -15 |        220 |  155 |  205 |  195 |  190 |  160 |   35 |   30 |  -15 |               0 | True              |
| 1928 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         60 |       -130 |        190 |  -20 |  -95 |  -85 | -100 |   60 |  -25 |  -45 | -130 |               0 | True              |
| 1929 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        215 |         -5 |        220 |  180 |  215 |   45 |   55 |   40 |   -5 |   85 |   20 |               0 | True              |
| 1930 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        265 |        -45 |        310 |  195 |  265 |   -5 |   35 |   35 |   90 |  -45 |   -5 |               0 | True              |
| 1931 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -350 |        350 |  -65 | -350 | -285 | -240 | -235 | -220 | -285 | -320 |               1 | True              |
| 1932 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        350 |          0 |        350 |  210 |  300 |  350 |  290 |  265 |   55 |   70 |  160 |               0 | True              |
| 1933 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         70 |       -280 |        350 |   70 |  -15 |   35 |  -10 |  -40 | -280 | -210 | -210 |               0 | True              |
| 1934 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         80 |       -215 |        295 |   50 |   35 |   -0 |  -20 |   80 |  -40 | -215 | -175 |               0 | True              |
| 1935 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -185 |        245 |  -40 |   40 |  -40 |   60 |   -0 |   25 |  -10 | -185 |               0 | True              |
| 1936 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -375 |        375 | -360 | -275 | -215 | -210 | -225 | -335 | -230 | -375 |               2 | True              |
| 1937 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -355 |        355 | -190 | -215 | -170 | -355 | -195 |  -90 | -175 | -185 |               0 | True              |
| 1938 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -660 |        660 | -435 | -475 | -415 | -395 | -465 | -585 | -660 | -585 |               5 | True              |
| 1939 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -670 |        670 | -460 | -670 | -550 | -525 | -505 | -655 | -550 | -505 |               3 | True              |
| 1940 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |       -165 |        280 |  100 |   65 |  115 | -100 | -165 |  -35 | -110 | -135 |               0 | True              |
| 1941 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         10 |       -410 |        420 |   10 |  -85 | -130 |  -20 | -115 | -410 | -365 | -255 |               0 | True              |
| 1942 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         75 |       -155 |        230 |  -55 | -155 |   75 |  -15 |  -40 |    5 |    5 | -105 |               0 | True              |
| 1943 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -105 | -130 |  -85 |  -60 | -160 | -205 | -195 | -195 |               0 | True              |
| 1944 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -470 |        470 | -200 |  -75 |  -60 | -470 | -410 | -325 | -380 | -340 |               3 | True              |
| 1945 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        285 |        -10 |        295 |  140 |  285 |  260 |  -10 |  130 |   95 |  120 |   85 |               0 | True              |
| 1946 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        250 |        -50 |        300 |  180 |  120 |  110 |  250 |  120 |  200 |  190 |  -50 |               0 | True              |
| 1947 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |         -5 |        195 |  185 |  130 |  145 |  145 |  145 |  150 |  190 |   -5 |               0 | True              |
| 1948 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         55 |       -145 |        200 |  -70 | -145 |  -90 |  -70 |   -0 |   15 |  -55 |   55 |               0 | True              |
| 1949 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        265 |        -90 |        355 |  165 |  155 |  230 |  265 |  -70 |  -60 |  -90 |  -55 |               0 | True              |
| 1950 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        295 |        -50 |        345 |  190 |  225 |  245 |  265 |  260 |  295 |  215 |  -50 |               0 | True              |
| 1951 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -305 |        305 | -200 | -215 | -145 | -305 | -215 | -175 | -180 | -175 |               0 | True              |
| 1952 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |        -80 |        280 |  105 |   15 |  160 |   35 |  200 |   55 |  -65 |  -80 |               0 | True              |
| 1953 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -800 |        800 | -705 | -765 | -715 | -685 | -750 | -795 | -765 | -800 |               5 | True              |
| 1954 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |        -65 |        210 |  105 |   35 |   60 |  145 |  130 |  -25 |  -30 |  -65 |               0 | True              |
| 1955 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -525 |        525 | -525 | -505 | -395 | -405 | -490 | -440 | -425 | -455 |               4 | True              |
| 1956 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -530 |        530 | -380 | -505 | -530 | -450 | -470 | -465 | -500 | -480 |               2 | True              |
| 1957 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        210 |        -45 |        255 |  205 |  210 |  -45 |   75 |  -15 |   -0 |   10 |  -25 |               0 | True              |
| 1958 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -575 |        575 | -290 | -310 | -210 | -370 | -505 | -400 | -565 | -575 |               4 | True              |
| 1959 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |       -155 |        335 |   95 |  155 |  110 |  180 |  175 |   45 | -155 |  -35 |               0 | True              |
| 1960 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |       -175 |        305 |  130 | -130 | -125 | -140 |  -90 |  -70 | -175 |  -60 |               0 | True              |
| 1961 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        125 |       -150 |        275 |   55 |  125 |   35 |   85 | -150 | -145 | -115 | -140 |               0 | True              |
| 1962 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -155 |        275 |  120 |  110 |  -70 | -155 | -105 | -110 | -105 |  -40 |               0 | True              |
| 1963 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |       -195 |        325 |  130 | -155 |  -40 | -110 |   75 |   25 | -115 | -195 |               0 | True              |
| 1964 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -245 |        245 |  -10 |  -60 | -100 | -100 |  -15 | -140 |  -50 | -245 |               0 | True              |
| 1965 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -340 |        355 |  -75 | -115 | -100 |  -50 |  -50 |   15 | -105 | -340 |               0 | True              |
| 1966 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        270 |        -95 |        365 |  130 |  220 |  200 |  255 |  155 |  270 |  -95 |  105 |               0 | True              |
| 1967 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -610 |        610 | -560 | -415 | -465 | -400 | -560 | -610 | -480 | -505 |               4 | True              |
| 1968 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -740 |        740 | -525 | -495 | -520 | -530 | -490 | -470 | -440 | -740 |               4 | True              |
| 1969 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         70 |       -230 |        300 |   -5 |   70 |  -15 |  -50 |  -20 |  -10 |   20 | -230 |               0 | True              |
| 1970 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -390 |        390 | -270 | -275 | -235 | -255 | -260 | -230 | -300 | -390 |               1 | True              |
| 1971 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -140 |        225 |   85 |   75 |   40 |   30 |   -0 |   40 |   45 | -140 |               0 | True              |
| 1972 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -135 | -260 | -195 | -190 | -215 | -225 | -145 | -265 |               0 | True              |
| 1973 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -620 |        620 | -610 | -535 | -555 | -620 | -560 | -605 | -580 | -560 |               4 | True              |
| 1974 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -475 |        475 | -200 | -235 | -330 | -305 | -245 | -220 | -335 | -475 |               0 | True              |
| 1975 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        145 |       -105 |        250 |  145 |  115 |   -0 |  -20 |  -30 |  -10 | -105 |  -50 |               0 | True              |
| 1976 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -180 |        335 |  155 |   50 |  -45 | -130 | -180 | -170 | -150 | -170 |               0 | True              |
| 1977 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -350 |        350 |  -70 | -350 | -190 | -315 | -290 | -200 | -245 | -130 |               1 | True              |
| 1978 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -250 |        250 | -230 | -165 | -205 | -245 | -215 | -235 | -160 | -250 |               1 | True              |
| 1979 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -215 |        215 | -140 | -120 | -195 | -180 | -215 | -115 | -215 | -130 |               0 | True              |
| 1980 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -175 |        325 |  150 |  120 |   45 |  105 |  -60 | -175 |  -35 | -145 |               0 | True              |
| 1981 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        245 |        -90 |        335 |  245 |  -65 |   75 |   90 |  -35 |   60 |  -90 |  135 |              16 | True              |
| 1982 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |       -165 |        310 |  110 |  145 |   15 |   45 |   60 | -165 | -105 |  -95 |               0 | True              |
| 1983 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -415 |        415 | -190 | -160 | -230 | -165 | -135 | -200 |  -80 | -415 |               0 | True              |
| 1984 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -130 |        340 |  135 |  210 |  195 |  -60 |  -60 | -130 |  -65 | -105 |               0 | True              |
| 1985 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         40 |       -190 |        230 |   10 |  -50 |   40 | -145 | -100 | -190 |  -90 | -160 |               1 | True              |
| 1986 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        235 |        -20 |        255 |  145 |  175 |  200 |  155 |  120 |  130 |  235 |  -20 |               0 | True              |
| 1987 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -155 |        285 |  130 |   55 |  120 | -155 | -115 |  -35 | -125 | -115 |               0 | True              |
| 1988 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -295 |        295 |  -35 |  -20 |  -75 | -280 | -295 | -260 | -240 | -175 |               1 | True              |
| 1989 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         45 |       -250 |        295 |   20 |   20 |  -70 |   45 | -140 | -185 | -215 | -250 |               0 | True              |
| 1990 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        230 |        -55 |        285 |  110 |  165 |  205 |  230 |  150 |  150 |   -0 |  -55 |               0 | True              |
| 1991 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |        -70 |        155 |   85 |   85 |   10 |  -45 |   50 |   70 |  -40 |  -70 |               0 | True              |
| 1992 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -200 |        200 |  -55 |  -10 |  -80 |  -80 |  -85 |  -55 | -155 | -200 |               0 | True              |
| 1993 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        205 |       -120 |        325 |  120 |   35 |   35 |  100 |  205 |   45 | -120 |    5 |               0 | True              |
| 1994 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -560 |        560 | -310 | -370 | -510 | -520 | -560 | -480 | -515 | -525 |               4 | True              |
| 1995 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -370 |        370 | -100 | -160 | -175 | -145 | -145 | -130 | -100 | -370 |               0 | True              |
| 1996 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         95 |       -265 |        360 |  -65 |  -60 |   95 |   65 |  -20 |  -70 | -190 | -265 |               1 | True              |
| 1997 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |       -145 |        320 |  175 | -145 | -130 |  -90 |  -30 |  -60 |   -5 |  -45 |               0 | True              |
| 1998 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         95 |       -145 |        240 |   60 |   55 |   60 |   95 |   85 |  -10 |  -40 | -145 |               0 | True              |
| 1999 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -200 |        290 |   90 |   30 |   10 |  -25 |   65 |   40 | -200 | -145 |               0 | True              |
| 2000 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        120 |       -160 |        280 |  105 |  100 |  120 |   35 | -160 |  -70 | -105 |   -5 |               0 | True              |

# CRR results

|      | rings_are_locked_to      | mrm_crr_equivalent   |   max_cool |   max_heat |   max_diff |     0 |     1 |     2 |     3 |     4 |     5 |     6 |     7 |
|-----:|:-------------------------|:---------------------|-----------:|-----------:|-----------:|------:|------:|------:|------:|------:|------:|------:|------:|
|    0 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -705 |        705 |  -425 |  -545 |  -470 |  -430 |  -595 |  -550 |  -470 |  -705 |
|    1 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -825 |        825 |  -755 |  -825 |  -800 |  -685 |  -740 |  -670 |  -705 |  -815 |
|    2 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        140 |        -70 |        210 |   115 |   120 |   140 |   100 |    65 |   105 |    -0 |   -70 |
|    3 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        220 |        -75 |        295 |   165 |   130 |   220 |   140 |   160 |   210 |   200 |   -75 |
|    4 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        290 |          0 |        290 |   285 |   145 |   210 |   160 |   290 |    25 |    70 |    85 |
|    5 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -785 |        785 |  -510 |  -680 |  -660 |  -575 |  -600 |  -580 |  -770 |  -785 |
|    6 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        265 |          0 |        265 |   250 |   130 |   265 |   150 |   120 |   185 |   115 |   190 |
|    7 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -840 |        840 |  -600 |  -575 |  -525 |  -675 |  -795 |  -765 |  -715 |  -840 |
|    8 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -520 |        520 |  -310 |  -315 |  -235 |  -470 |  -390 |  -485 |  -520 |  -455 |
|    9 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -660 |        660 |  -480 |  -605 |  -575 |  -490 |  -550 |  -530 |  -505 |  -660 |
|   10 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        280 |          0 |        280 |   280 |   255 |   190 |   250 |   230 |   260 |   210 |    20 |
|   11 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        350 |          0 |        350 |   350 |   270 |   335 |   270 |   115 |   130 |   205 |   140 |
|   12 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        215 |       -220 |        435 |    95 |    20 |    55 |   215 |  -185 |   -70 |  -220 |  -145 |
|   13 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        400 |          0 |        400 |   305 |   400 |   130 |    65 |   150 |   155 |   180 |   195 |
|   14 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        215 |       -120 |        335 |   215 |   170 |    70 |   130 |   -80 |  -120 |  -115 |   -75 |
|   15 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         10 |       -265 |        275 |    10 |   -95 |   -10 |  -265 |  -165 |  -245 |  -245 |  -150 |
|   16 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         25 |       -135 |        160 |    25 |  -135 |   -15 |  -100 |   -90 |  -105 |   -55 |   -95 |
|   17 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -850 |        850 |  -745 |  -720 |  -540 |  -775 |  -655 |  -655 |  -725 |  -850 |
|   18 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -830 |        830 |  -415 |  -750 |  -830 |  -720 |  -735 |  -720 |  -705 |  -720 |
|   19 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1065 |       1065 |  -710 |  -925 | -1065 |  -875 | -1000 | -1005 |  -965 |  -980 |
|   20 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |        -80 |        315 |   180 |    75 |   135 |   235 |   180 |   105 |    65 |   -80 |
|   21 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -280 |        280 |   -35 |  -100 |  -260 |  -280 |  -265 |  -265 |  -170 |  -240 |
|   22 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        335 |          0 |        335 |   295 |   175 |   330 |   335 |   265 |    65 |   125 |   145 |
|   23 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        345 |          0 |        345 |   250 |   280 |   345 |   285 |   290 |   125 |   160 |    95 |
|   24 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         35 |       -290 |        325 |   -45 |    35 |   -25 |    35 |    25 |  -190 |  -290 |  -215 |
|   25 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        105 |       -170 |        275 |    20 |   105 |    70 |    40 |   -15 |    45 |   100 |  -170 |
|   26 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1120 |       1120 |  -740 |  -880 |  -920 |  -785 |  -825 |  -775 |  -800 | -1120 |
|   27 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        200 |       -125 |        325 |    -5 |   200 |    95 |   -95 |   -95 |  -125 |   -65 |  -100 |
|   28 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -680 |        680 |  -345 |  -465 |  -410 |  -410 |  -445 |  -435 |  -680 |  -615 |
|   29 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -925 |        925 |  -745 |  -675 |  -725 |  -700 |  -705 |  -765 |  -760 |  -925 |
|   30 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        360 |          0 |        360 |   360 |   330 |   150 |   105 |   210 |   115 |   140 |   150 |
|   31 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1050 |       1050 |  -955 |  -895 |  -840 |  -785 |  -845 |  -890 |  -950 | -1050 |
|   32 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        360 |          0 |        360 |   335 |   310 |   330 |   300 |   280 |   360 |   110 |   135 |
|   33 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1150 |       1150 |  -965 |  -940 |  -985 |  -945 |  -870 |  -830 | -1120 | -1150 |
|   34 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        360 |          0 |        360 |   360 |    50 |   150 |   195 |   145 |   100 |   135 |    95 |
|   35 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -875 |        875 |  -675 |  -665 |  -690 |  -775 |  -750 |  -710 |  -800 |  -875 |
|   36 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -395 |        395 |   -70 |   -75 |   -65 |  -310 |  -395 |  -295 |  -280 |  -195 |
|   37 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -510 |        510 |  -250 |  -235 |  -485 |  -455 |  -435 |  -450 |  -510 |  -455 |
|   38 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        330 |          0 |        330 |   290 |   330 |   275 |   315 |   250 |   305 |   135 |   255 |
|   39 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        145 |        -85 |        230 |   145 |   -35 |   -65 |    -5 |   -55 |   -30 |    55 |   -85 |
|   40 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -570 |        570 |  -300 |  -415 |  -285 |  -290 |  -260 |  -495 |  -445 |  -570 |
|   41 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        365 |          0 |        365 |   365 |    85 |   155 |   110 |    95 |    80 |   115 |   160 |
|   42 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1010 |       1010 |  -805 |  -770 |  -690 |  -855 |  -950 |  -885 |  -925 | -1010 |
|   43 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1030 |       1030 |  -815 |  -750 |  -870 |  -900 |  -865 | -1030 | -1005 | -1015 |
|   44 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -720 |        720 |  -660 |  -700 |  -695 |  -720 |  -575 |  -635 |  -525 |  -625 |
|   45 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        175 |        -80 |        255 |   115 |   175 |    95 |    15 |    -0 |   -10 |   -80 |    75 |
|   46 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   295 |   300 |   345 |   400 |   310 |   275 |   345 |   135 |
|   47 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1060 |       1060 |  -765 | -1060 |  -975 | -1035 |  -960 | -1000 |  -965 | -1000 |
|   48 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |          0 |        265 |   265 |   180 |   140 |   110 |    45 |    40 |    15 |    70 |
|   49 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1065 |       1065 |  -920 |  -905 |  -855 |  -850 | -1065 | -1035 |  -995 | -1030 |
|   50 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        315 |          0 |        315 |   315 |   165 |   220 |   230 |   235 |   240 |   310 |   240 |
|   51 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1030 |       1030 |  -890 |  -750 |  -820 |  -785 |  -850 |  -810 | -1020 | -1030 |
|   52 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        390 |          0 |        390 |   390 |   160 |   110 |   145 |   220 |   115 |   100 |   240 |
|   53 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        270 |          0 |        270 |   165 |   270 |   240 |    20 |    20 |     5 |    10 |    90 |
|   54 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        175 |        -10 |        185 |   165 |   165 |   175 |   125 |   145 |    90 |    -0 |   -10 |
|   55 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -345 |        345 |   -80 |  -280 |  -320 |  -315 |  -210 |  -345 |  -290 |  -265 |
|   56 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -555 |        555 |  -440 |  -390 |  -360 |  -315 |  -285 |  -520 |  -555 |  -500 |
|   57 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        195 |        -30 |        225 |   195 |   120 |   170 |    20 |    20 |     5 |    10 |   -30 |
|   58 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -325 |        325 |  -165 |  -180 |  -115 |  -135 |  -135 |  -120 |  -120 |  -325 |
|   59 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1070 |       1070 |  -745 |  -805 |  -835 |  -795 |  -835 |  -760 | -1005 | -1070 |
|   60 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -710 |        710 |  -460 |  -710 |  -675 |  -670 |  -670 |  -590 |  -650 |  -685 |
|   61 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        235 |          0 |        235 |   200 |   235 |   140 |   180 |   170 |    50 |    70 |   100 |
|   62 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -715 |        715 |  -520 |  -515 |  -535 |  -560 |  -580 |  -590 |  -550 |  -715 |
|   63 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -640 |        640 |  -425 |  -495 |  -415 |  -445 |  -620 |  -640 |  -620 |  -585 |
|   64 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -990 |        990 |  -695 |  -615 |  -730 |  -985 |  -885 |  -860 |  -990 |  -865 |
|   65 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        400 |          0 |        400 |   400 |   165 |    90 |    85 |   105 |   150 |   105 |   125 |
|   66 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        330 |        -15 |        345 |   225 |   285 |   245 |   330 |   255 |   210 |   275 |   -15 |
|   67 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -860 |        860 |  -580 |  -590 |  -840 |  -795 |  -755 |  -860 |  -770 |  -810 |
|   68 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -825 |        825 |  -625 |  -540 |  -495 |  -590 |  -540 |  -580 |  -550 |  -825 |
|   69 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1105 |       1105 |  -730 |  -825 |  -800 |  -835 | -1105 |  -940 |  -975 |  -950 |
|   70 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -925 |        925 |  -595 |  -640 |  -705 |  -905 |  -900 |  -850 |  -925 |  -855 |
|   71 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        305 |        -15 |        320 |   150 |   305 |    35 |   120 |    35 |   -15 |    50 |   165 |
|   72 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        340 |          0 |        340 |   270 |   285 |   340 |   105 |    80 |    80 |    25 |    55 |
|   73 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         75 |        -70 |        145 |     5 |   -55 |   -70 |    25 |   -65 |    -5 |    75 |   -30 |
|   74 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -565 |        565 |  -350 |  -340 |  -275 |  -295 |  -505 |  -480 |  -565 |  -525 |
|   75 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        345 |          0 |        345 |   300 |   345 |   225 |   310 |   150 |   115 |   130 |   110 |
|   76 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -325 |        325 |  -100 |  -325 |  -110 |  -165 |  -210 |  -155 |  -175 |  -195 |
|   77 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -330 |        330 |  -150 |  -240 |  -330 |  -315 |  -305 |  -295 |  -290 |  -250 |
|   78 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        375 |          0 |        375 |   310 |   240 |   260 |   375 |   280 |   160 |   120 |   185 |
|   79 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -715 |        715 |  -375 |  -455 |  -450 |  -415 |  -675 |  -595 |  -715 |  -685 |
|   80 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        125 |       -100 |        225 |   100 |   100 |    30 |   115 |    25 |    50 |   125 |  -100 |
|   81 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -620 |        620 |  -315 |  -540 |  -550 |  -430 |  -575 |  -585 |  -620 |  -440 |
|   82 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -955 |        955 |  -670 |  -690 |  -720 |  -700 |  -905 |  -850 |  -955 |  -820 |
|   83 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        400 |          0 |        400 |   400 |   220 |   180 |   140 |   205 |   280 |   245 |   165 |
|   84 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |        -10 |        280 |   270 |   205 |    65 |   105 |    -0 |    55 |   130 |   -10 |
|   85 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        325 |          0 |        325 |   325 |   285 |   180 |   205 |   235 |   315 |    70 |    50 |
|   86 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        340 |        -55 |        395 |   310 |   340 |   260 |   265 |   195 |    75 |   120 |   -55 |
|   87 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -815 |        815 |  -755 |  -725 |  -745 |  -815 |  -655 |  -745 |  -725 |  -795 |
|   88 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        310 |          0 |        310 |   240 |   215 |   200 |   310 |   220 |   260 |   240 |   205 |
|   89 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        395 |          0 |        395 |   325 |   395 |    50 |   125 |    85 |   205 |   120 |   110 |
|   90 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        130 |       -225 |        355 |    30 |    65 |   130 |    70 |    50 |   -70 |    20 |  -225 |
|   91 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -995 |        995 |  -820 |  -845 |  -815 |  -780 |  -815 |  -735 |  -925 |  -995 |
|   92 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -475 |        475 |  -170 |  -235 |  -260 |  -220 |  -255 |  -425 |  -375 |  -475 |
|   93 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -690 |        690 |  -390 |  -395 |  -565 |  -605 |  -615 |  -690 |  -585 |  -630 |
|   94 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1180 |       1180 |  -880 |  -900 |  -855 |  -965 | -1005 | -1055 | -1075 | -1180 |
|   95 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1155 |       1155 |  -975 |  -875 | -1005 | -1020 |  -895 |  -995 | -1020 | -1155 |
|   96 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        170 |       -155 |        325 |   170 |  -155 |   -60 |  -125 |   -55 |  -100 |  -100 |  -105 |
|   97 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -505 |        505 |  -250 |  -180 |  -200 |   -80 |  -230 |  -505 |  -355 |  -300 |
|   98 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        180 |          0 |        180 |   180 |    20 |    85 |   150 |   160 |    55 |    20 |    50 |
|   99 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         40 |       -365 |        405 |  -195 |  -190 |  -140 |  -195 |  -210 |    40 |  -365 |  -345 |
|  100 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1110 |       1110 |  -830 |  -965 |  -915 |  -980 |  -965 |  -970 | -1025 | -1110 |
|  101 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |        -30 |        255 |   135 |    65 |   225 |   120 |   -15 |    40 |   -25 |   -30 |
|  102 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        265 |          0 |        265 |   220 |   265 |   195 |   160 |   150 |   145 |   145 |   140 |
|  103 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        365 |          0 |        365 |   290 |   365 |   270 |   245 |   365 |    75 |    40 |    35 |
|  104 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -465 |        465 |  -450 |  -370 |  -280 |  -440 |  -430 |  -340 |  -465 |  -355 |
|  105 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        285 |        -75 |        360 |   180 |   185 |   130 |   145 |   175 |   285 |   -75 |   -40 |
|  106 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -760 |        760 |  -510 |  -510 |  -435 |  -695 |  -750 |  -725 |  -760 |  -725 |
|  107 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1155 |       1155 |  -825 | -1080 | -1035 | -1055 | -1095 | -1100 | -1155 | -1050 |
|  108 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -820 |        820 |  -490 |  -555 |  -820 |  -775 |  -810 |  -770 |  -745 |  -725 |
|  109 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -450 |        450 |  -225 |  -145 |  -280 |  -400 |  -365 |  -390 |  -365 |  -450 |
|  110 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        115 |       -280 |        395 |    10 |    60 |   115 |    35 |    35 |    70 |  -280 |  -175 |
|  111 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -440 |        440 |  -155 |  -155 |  -245 |  -315 |  -440 |  -340 |  -415 |  -420 |
|  112 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        305 |          0 |        305 |   290 |   260 |   260 |   305 |   220 |   155 |    35 |   165 |
|  113 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1025 |       1025 |  -770 |  -715 |  -855 |  -765 |  -685 | -1025 |  -960 |  -925 |
|  114 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        265 |        -10 |        275 |   220 |   220 |   265 |   200 |    80 |    20 |   -10 |   135 |
|  115 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        110 |       -150 |        260 |    65 |    40 |   110 |  -150 |   -10 |    -0 |  -135 |  -135 |
|  116 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        140 |          0 |        140 |    25 |   140 |    50 |    20 |    75 |   135 |    45 |   140 |
|  117 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -895 |        895 |  -705 |  -770 |  -845 |  -775 |  -815 |  -810 |  -815 |  -895 |
|  118 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -860 |        860 |  -620 |  -575 |  -755 |  -860 |  -815 |  -825 |  -830 |  -795 |
|  119 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1160 |       1160 |  -890 | -1130 | -1135 | -1160 |  -980 | -1150 | -1120 | -1060 |
|  120 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        380 |          0 |        380 |   315 |   320 |   380 |   350 |   325 |   280 |    40 |   165 |
|  121 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1125 |       1125 |  -915 |  -905 |  -860 |  -945 |  -865 |  -875 | -1125 | -1035 |
|  122 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        350 |        -20 |        370 |   350 |   295 |   255 |   290 |    40 |   -20 |    50 |    45 |
|  123 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -475 |        475 |  -265 |  -260 |  -325 |  -260 |  -455 |  -370 |  -475 |  -430 |
|  124 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        345 |          0 |        345 |   345 |   260 |   270 |   245 |   265 |   210 |   115 |    60 |
|  125 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        180 |       -110 |        290 |   180 |  -110 |    20 |   -20 |     5 |   -10 |    40 |   -15 |
|  126 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        145 |       -140 |        285 |   145 |   120 |   140 |    60 |   130 |    65 |   130 |  -140 |
|  127 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        305 |        -10 |        315 |   305 |   175 |   175 |   175 |   185 |    65 |    -5 |   -10 |
|  128 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -745 |        745 |  -435 |  -610 |  -500 |  -470 |  -525 |  -550 |  -625 |  -745 |
|  129 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -325 |        325 |  -105 |   -65 |  -115 |   -70 |   -10 |   -85 |  -285 |  -325 |
|  130 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -425 |        425 |  -260 |  -265 |  -255 |  -295 |  -240 |  -245 |  -290 |  -425 |
|  131 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -360 |        360 |  -330 |  -265 |  -290 |  -360 |  -300 |  -240 |  -310 |  -295 |
|  132 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -790 |        790 |  -545 |  -560 |  -605 |  -615 |  -580 |  -630 |  -615 |  -790 |
|  133 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         75 |       -220 |        295 |    25 |   -25 |    20 |   -60 |    55 |    75 |  -220 |  -215 |
|  134 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -645 |        645 |  -490 |  -570 |  -605 |  -620 |  -615 |  -610 |  -645 |  -605 |
|  135 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1060 |       1060 |  -870 |  -775 |  -940 |  -810 |  -825 |  -875 |  -870 | -1060 |
|  136 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        380 |         -0 |        380 |   325 |   380 |   280 |    35 |    50 |   170 |   100 |    -0 |
|  137 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        115 |        -55 |        170 |    90 |    70 |   100 |    50 |   -55 |    45 |   115 |    65 |
|  138 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -550 |        550 |  -275 |  -300 |  -525 |  -535 |  -550 |  -465 |  -525 |  -540 |
|  139 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   325 |   380 |   400 |   305 |   330 |   320 |   355 |   185 |
|  140 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |        -40 |        285 |   140 |   205 |   220 |   120 |   160 |   245 |   -40 |    45 |
|  141 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1175 |       1175 |  -905 |  -985 |  -945 |  -965 |  -940 | -1130 | -1175 | -1160 |
|  142 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -890 |        890 |  -510 |  -655 |  -820 |  -825 |  -775 |  -845 |  -890 |  -840 |
|  143 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1000 |       1000 |  -910 |  -850 |  -755 |  -815 |  -745 | -1000 |  -995 |  -925 |
|  144 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1190 |       1190 |  -935 |  -860 | -1115 | -1190 | -1135 | -1135 | -1105 | -1145 |
|  145 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -920 |  -755 |  -775 |  -770 |  -755 |  -790 |  -750 |  -990 |
|  146 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         25 |       -210 |        235 |     5 |   -35 |   -20 |    25 |     5 |    15 |   -95 |  -210 |
|  147 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -760 |        760 |  -625 |  -430 |  -735 |  -685 |  -760 |  -745 |  -760 |  -620 |
|  148 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -830 |        830 |  -715 |  -655 |  -620 |  -530 |  -630 |  -680 |  -830 |  -815 |
|  149 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -600 |        600 |  -455 |  -380 |  -335 |  -300 |  -325 |  -295 |  -340 |  -600 |
|  150 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        400 |          0 |        400 |   325 |   400 |   200 |   200 |   180 |   210 |   135 |   210 |
|  151 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1045 |       1045 |  -715 |  -720 |  -820 | -1015 |  -990 |  -970 | -1045 |  -960 |
|  152 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        325 |          0 |        325 |   320 |   310 |   310 |   325 |   130 |    80 |   205 |    85 |
|  153 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        330 |          0 |        330 |   330 |    45 |   160 |   175 |   235 |   125 |   140 |    45 |
|  154 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -380 |        380 |   -85 |   -90 |  -270 |  -270 |  -260 |  -295 |  -205 |  -380 |
|  155 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        300 |         -5 |        305 |   300 |    65 |    -5 |    40 |    -0 |    75 |    -0 |    85 |
|  156 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |          0 |        265 |   250 |   265 |   255 |    75 |    10 |    45 |    80 |    25 |
|  157 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        355 |          0 |        355 |   175 |   265 |   305 |   255 |   230 |   355 |   275 |   250 |
|  158 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |          0 |        310 |   235 |   310 |   305 |   185 |   205 |   225 |   260 |    20 |
|  159 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1120 |       1120 |  -815 |  -920 |  -860 |  -740 |  -865 | -1120 | -1095 |  -985 |
|  160 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        365 |          0 |        365 |   365 |   305 |   300 |   295 |   300 |   305 |   315 |   235 |
|  161 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        250 |          0 |        250 |   170 |   250 |    65 |   125 |    70 |    45 |    85 |    35 |
|  162 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        240 |       -160 |        400 |   225 |   240 |   240 |  -110 |  -130 |    15 |  -160 |   -45 |
|  163 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |        -60 |        335 |   245 |   275 |   165 |   165 |   265 |   105 |   -60 |    -5 |
|  164 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        195 |        -10 |        205 |   195 |    40 |    20 |    65 |    40 |    30 |    10 |   -10 |
|  165 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 | -1010 |  -925 |  -890 |  -860 | -1020 |  -845 |  -960 | -1080 |
|  166 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -815 |        815 |  -590 |  -525 |  -645 |  -540 |  -655 |  -815 |  -770 |  -785 |
|  167 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        310 |       -100 |        410 |   310 |   -25 |    55 |    65 |  -100 |   -40 |    20 |   -20 |
|  168 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        335 |          0 |        335 |   315 |   285 |   335 |    95 |    75 |   200 |   205 |   160 |
|  169 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -440 |        440 |  -205 |  -110 |  -135 |  -205 |  -225 |  -175 |  -440 |  -340 |
|  170 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -350 |        350 |   -65 |  -350 |  -280 |  -350 |  -275 |  -235 |  -225 |  -275 |
|  171 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        235 |          0 |        235 |   215 |   130 |   235 |   165 |   130 |    10 |     5 |    15 |
|  172 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -595 |        595 |  -420 |  -440 |  -375 |  -415 |  -405 |  -375 |  -550 |  -595 |
|  173 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -665 |        665 |  -360 |  -495 |  -540 |  -655 |  -570 |  -610 |  -605 |  -665 |
|  174 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        285 |        -45 |        330 |   230 |   180 |   275 |   250 |   215 |   285 |   -20 |   -45 |
|  175 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        290 |        -75 |        365 |   190 |   200 |   200 |   290 |   260 |   270 |   180 |   -75 |
|  176 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -185 |        185 |   -75 |  -110 |    -5 |   -70 |   -25 |  -185 |  -180 |  -140 |
|  177 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |          0 |        265 |   265 |    90 |    90 |   110 |   100 |   120 |   115 |    75 |
|  178 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -885 |        885 |  -500 |  -595 |  -580 |  -795 |  -790 |  -800 |  -885 |  -850 |
|  179 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         95 |       -110 |        205 |    95 |  -110 |   -95 |   -90 |   -60 |   -60 |   -95 |   -60 |
|  180 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        220 |        -35 |        255 |   210 |   105 |   120 |   150 |   205 |   220 |   -35 |   -25 |
|  181 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1170 |       1170 |  -840 |  -935 |  -820 | -1125 | -1105 |  -995 | -1170 | -1050 |
|  182 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        340 |          0 |        340 |   340 |   250 |   190 |   285 |   125 |    80 |    35 |   105 |
|  183 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          5 |       -310 |        315 |  -150 |  -100 |     5 |   -35 |   -50 |    -5 |   -25 |  -310 |
|  184 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        330 |          0 |        330 |   230 |   325 |   330 |   300 |    75 |    50 |    10 |    90 |
|  185 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -955 |        955 |  -615 |  -935 |  -870 |  -930 |  -860 |  -890 |  -955 |  -930 |
|  186 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -160 |        160 |  -100 |   -45 |  -160 |   -65 |   -60 |  -115 |   -90 |   -25 |
|  187 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -265 |        265 |   -15 |   -35 |  -125 |  -150 |  -265 |  -220 |  -220 |  -175 |
|  188 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -890 |        890 |  -765 |  -790 |  -730 |  -890 |  -855 |  -795 |  -820 |  -800 |
|  189 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |        -70 |        335 |   265 |   -25 |   -45 |    70 |   -70 |    55 |    45 |    50 |
|  190 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        280 |         -5 |        285 |   280 |   225 |   100 |    85 |    35 |    25 |    -5 |    40 |
|  191 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        305 |          0 |        305 |   145 |   130 |   200 |   180 |   305 |   250 |   195 |    40 |
|  192 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        230 |        -25 |        255 |    95 |   230 |   130 |   150 |   180 |   155 |   -15 |   -25 |
|  193 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        155 |       -160 |        315 |   155 |  -105 |   -60 |  -160 |   -30 |   -85 |   -70 |  -125 |
|  194 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -790 |        790 |  -550 |  -410 |  -460 |  -790 |  -730 |  -660 |  -675 |  -680 |
|  195 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -885 |        885 |  -725 |  -735 |  -680 |  -780 |  -635 |  -695 |  -745 |  -885 |
|  196 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -755 |        755 |  -435 |  -420 |  -530 |  -465 |  -485 |  -425 |  -480 |  -755 |
|  197 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        375 |          0 |        375 |   285 |   255 |   315 |   375 |   270 |    80 |    70 |    20 |
|  198 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -485 |        485 |  -270 |  -150 |  -460 |  -455 |  -400 |  -340 |  -355 |  -485 |
|  199 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -975 |        975 |  -850 |  -840 |  -835 |  -975 |  -965 |  -875 |  -900 |  -875 |
|  200 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -820 |        820 |  -665 |  -600 |  -680 |  -610 |  -740 |  -655 |  -585 |  -820 |
|  201 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -680 |        680 |  -530 |  -675 |  -655 |  -615 |  -600 |  -680 |  -580 |  -615 |
|  202 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1135 |       1135 |  -945 |  -855 |  -810 |  -770 | -1015 | -1125 | -1135 | -1045 |
|  203 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        130 |       -155 |        285 |    60 |   105 |    90 |   130 |  -155 |   -70 |   -75 |  -130 |
|  204 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -735 |        735 |  -555 |  -510 |  -420 |  -655 |  -605 |  -635 |  -735 |  -665 |
|  205 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -570 |        570 |  -340 |  -365 |  -320 |  -440 |  -570 |  -520 |  -535 |  -480 |
|  206 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -660 |        660 |  -500 |  -540 |  -480 |  -465 |  -420 |  -545 |  -415 |  -660 |
|  207 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -500 |        500 |  -295 |  -395 |  -365 |  -330 |  -215 |  -295 |  -500 |  -485 |
|  208 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1045 |       1045 |  -730 |  -685 |  -855 |  -655 |  -790 |  -795 | -1045 |  -980 |
|  209 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        260 |          0 |        260 |   210 |   185 |   185 |   260 |   165 |   205 |    70 |    40 |
|  210 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |       -110 |        335 |   150 |   215 |   180 |   225 |   -15 |   -55 |    30 |  -110 |
|  211 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        380 |          0 |        380 |   345 |   230 |   225 |   190 |   210 |   335 |   380 |   345 |
|  212 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        345 |          0 |        345 |   310 |   225 |   245 |   240 |   345 |   230 |   295 |   225 |
|  213 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -795 |        795 |  -640 |  -665 |  -600 |  -630 |  -655 |  -685 |  -715 |  -795 |
|  214 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |        -75 |        360 |   170 |   150 |   285 |   210 |   105 |   -40 |   -75 |   -35 |
|  215 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        220 |       -100 |        320 |   220 |    30 |   155 |   195 |   175 |   -55 |    10 |  -100 |
|  216 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        200 |          0 |        200 |   200 |   145 |     5 |   110 |   120 |   145 |   100 |   125 |
|  217 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -635 |        635 |  -450 |  -620 |  -595 |  -610 |  -595 |  -595 |  -635 |  -615 |
|  218 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -455 |        455 |  -370 |  -450 |  -355 |  -455 |  -305 |  -450 |  -440 |  -305 |
|  219 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        100 |       -290 |        390 |    -5 |   -30 |   -10 |    85 |    60 |    15 |   100 |  -290 |
|  220 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        380 |          0 |        380 |   380 |   170 |   220 |   220 |   140 |   215 |   145 |   165 |
|  221 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -930 |        930 |  -825 |  -890 |  -930 |  -890 |  -900 |  -870 |  -910 |  -860 |
|  222 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        355 |        -10 |        365 |   235 |   355 |   240 |   255 |   190 |   -10 |    75 |    10 |
|  223 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -565 |        565 |  -335 |  -300 |  -270 |  -355 |  -470 |  -565 |  -480 |  -540 |
|  224 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -955 |        955 |  -695 |  -695 |  -910 |  -855 |  -825 |  -900 |  -955 |  -955 |
|  225 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        145 |       -325 |        470 |    75 |   100 |    60 |     5 |   145 |   -10 |  -215 |  -325 |
|  226 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        260 |          0 |        260 |   155 |   260 |   245 |   100 |   125 |   130 |   165 |   190 |
|  227 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -655 |        655 |  -385 |  -655 |  -605 |  -530 |  -495 |  -540 |  -605 |  -420 |
|  228 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1190 |       1190 | -1165 | -1140 | -1175 | -1150 | -1150 | -1150 | -1170 | -1190 |
|  229 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |          0 |        370 |   235 |   235 |   370 |   250 |   290 |    50 |    45 |   165 |
|  230 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         50 |       -170 |        220 |    50 |  -170 |  -105 |  -125 |   -25 |  -125 |  -100 |  -155 |
|  231 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |          0 |        360 |   305 |   350 |   360 |   175 |   155 |   205 |   200 |   145 |
|  232 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -510 |        510 |  -450 |  -475 |  -455 |  -510 |  -355 |  -490 |  -390 |  -375 |
|  233 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |        -80 |        325 |   145 |   205 |   245 |   150 |   170 |   145 |   -80 |    -5 |
|  234 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -585 |        585 |  -460 |  -395 |  -455 |  -485 |  -515 |  -400 |  -435 |  -585 |
|  235 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1155 |       1155 |  -890 |  -775 |  -845 |  -905 |  -965 |  -945 |  -815 | -1155 |
|  236 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        300 |        -30 |        330 |   170 |   160 |   270 |   130 |   300 |   170 |   -30 |    55 |
|  237 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -465 |        465 |  -185 |  -205 |  -240 |  -225 |  -465 |  -430 |  -385 |  -395 |
|  238 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -255 |        255 |   -65 |    -5 |   -80 |   -80 |   -60 |   -35 |  -255 |  -195 |
|  239 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1030 |       1030 |  -720 |  -840 |  -805 |  -810 |  -810 |  -780 |  -880 | -1030 |
|  240 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        395 |          0 |        395 |   350 |   395 |   325 |   385 |   320 |   325 |   105 |   140 |
|  241 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        315 |        -30 |        345 |   220 |   280 |   275 |   270 |   315 |   -30 |    60 |    40 |
|  242 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        290 |        -40 |        330 |   245 |   220 |   240 |   160 |   260 |   290 |   215 |   -40 |
|  243 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |       -135 |        335 |   160 |    85 |   200 |    90 |    80 |   115 |    95 |  -135 |
|  244 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         25 |       -285 |        310 |  -140 |    -5 |  -105 |    25 |   -20 |   -10 |    -0 |  -285 |
|  245 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        245 |       -120 |        365 |   205 |   170 |   190 |   245 |   130 |   140 |   115 |  -120 |
|  246 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        180 |       -100 |        280 |   180 |   155 |    15 |   -85 |   -55 |   -75 |  -100 |    -0 |
|  247 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        390 |          0 |        390 |   325 |   285 |   390 |   390 |   245 |   315 |   350 |   195 |
|  248 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -560 |        560 |  -270 |  -345 |  -245 |  -240 |  -325 |  -205 |  -560 |  -500 |
|  249 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        285 |        -55 |        340 |   220 |   190 |   285 |   170 |    30 |    10 |    -0 |   -55 |
|  250 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -460 |        460 |  -155 |  -200 |  -245 |  -445 |  -460 |  -385 |  -275 |  -440 |
|  251 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         35 |       -410 |        445 |    35 |   -15 |  -270 |  -210 |  -215 |  -410 |  -350 |  -270 |
|  252 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -775 |        775 |  -505 |  -595 |  -500 |  -595 |  -550 |  -545 |  -555 |  -775 |
|  253 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -465 |        465 |  -425 |  -410 |  -410 |  -430 |  -430 |  -465 |  -380 |  -415 |
|  254 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -885 |        885 |  -535 |  -630 |  -540 |  -615 |  -540 |  -885 |  -765 |  -750 |
|  255 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1105 |       1105 |  -780 |  -845 | -1020 |  -985 | -1060 | -1065 | -1040 | -1105 |
|  256 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -430 |        430 |  -270 |  -315 |  -265 |  -190 |  -210 |  -220 |  -165 |  -430 |
|  257 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1115 |       1115 |  -925 |  -880 |  -960 |  -840 |  -850 |  -940 | -1115 | -1095 |
|  258 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -305 |        305 |   -65 |   -65 |  -230 |  -305 |  -195 |  -200 |  -215 |  -180 |
|  259 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -795 |        795 |  -660 |  -720 |  -795 |  -705 |  -740 |  -760 |  -735 |  -750 |
|  260 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        255 |       -110 |        365 |   255 |   160 |   180 |   -95 |   -60 |   -75 |   -25 |  -110 |
|  261 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -710 |        710 |  -450 |  -490 |  -400 |  -495 |  -630 |  -665 |  -620 |  -710 |
|  262 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -320 |        320 |  -220 |  -300 |  -320 |  -255 |  -275 |  -280 |  -220 |  -255 |
|  263 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -210 |        210 |   -75 |  -175 |  -200 |  -205 |   -65 |  -195 |  -210 |  -185 |
|  264 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        195 |       -105 |        300 |   195 |   -80 |    55 |    20 |  -105 |   -50 |    -5 |   -90 |
|  265 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1100 |       1100 |  -860 |  -805 |  -905 |  -880 |  -880 |  -960 |  -930 | -1100 |
|  266 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |        -60 |        355 |   205 |   295 |   215 |   -60 |    70 |    70 |    -0 |   -45 |
|  267 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -295 |        295 |  -220 |  -175 |  -145 |  -235 |  -115 |  -295 |  -240 |  -195 |
|  268 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        370 |      -1190 |       1560 | -1190 |   370 |   310 |   350 |   305 |   295 |   345 |   365 |
|  269 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        145 |       -195 |        340 |   145 |    85 |   -60 |   -15 |   -95 |   -75 |  -195 |   -55 |
|  270 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -915 |        915 |  -545 |  -570 |  -585 |  -660 |  -645 |  -860 |  -840 |  -915 |
|  271 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        280 |          0 |        280 |   135 |   190 |   200 |   260 |   280 |   245 |    85 |    50 |
|  272 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -460 |        460 |  -180 |  -165 |  -255 |  -205 |  -230 |  -445 |  -460 |  -360 |
|  273 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        290 |          0 |        290 |   290 |   145 |   185 |    85 |   100 |   180 |   175 |   135 |
|  274 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        350 |          0 |        350 |   280 |   350 |   275 |    95 |   185 |    10 |    75 |   115 |
|  275 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1175 |       1175 |  -910 |  -900 |  -955 |  -945 |  -890 |  -915 | -1175 | -1015 |
|  276 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        260 |        -10 |        270 |   260 |   190 |    20 |    45 |   -10 |    -5 |    10 |    55 |
|  277 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -860 |        860 |  -540 |  -540 |  -750 |  -725 |  -830 |  -765 |  -780 |  -860 |
|  278 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -590 |        590 |  -355 |  -445 |  -460 |  -390 |  -360 |  -540 |  -435 |  -590 |
|  279 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -855 |        855 |  -640 |  -580 |  -605 |  -645 |  -670 |  -800 |  -805 |  -855 |
|  280 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1140 |       1140 |  -965 | -1000 |  -920 |  -890 |  -910 | -1140 | -1080 | -1055 |
|  281 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -710 |        710 |  -530 |  -555 |  -500 |  -710 |  -695 |  -690 |  -610 |  -700 |
|  282 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1190 |       1190 | -1015 |  -945 | -1045 | -1005 | -1080 | -1135 | -1140 | -1190 |
|  283 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -750 |        750 |  -680 |  -610 |  -680 |  -690 |  -750 |  -695 |  -620 |  -700 |
|  284 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -785 |        785 |  -575 |  -565 |  -695 |  -585 |  -535 |  -550 |  -635 |  -785 |
|  285 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        145 |       -120 |        265 |   145 |   105 |    90 |    10 |   -30 |  -120 |  -105 |   -80 |
|  286 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |          0 |        200 |   200 |    75 |   180 |   150 |   155 |   180 |   130 |    15 |
|  287 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -425 |        425 |  -155 |  -150 |  -120 |  -115 |  -310 |  -425 |  -310 |  -340 |
|  288 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1200 |       1200 |  -810 |  -855 |  -840 |  -790 | -1010 | -1200 |  -965 |  -960 |
|  289 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -485 |        485 |  -280 |  -440 |  -360 |  -395 |  -485 |  -435 |  -415 |  -310 |
|  290 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -605 |        605 |  -285 |  -345 |  -325 |  -380 |  -605 |  -515 |  -545 |  -595 |
|  291 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         20 |       -260 |        280 |   -35 |    20 |   -75 |  -120 |   -35 |  -260 |  -130 |  -245 |
|  292 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        400 |          0 |        400 |   305 |   400 |   300 |   315 |   185 |   135 |   180 |   200 |
|  293 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -830 |        830 |  -555 |  -625 |  -545 |  -595 |  -505 |  -595 |  -830 |  -805 |
|  294 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -670 |        670 |  -520 |  -515 |  -500 |  -645 |  -655 |  -655 |  -670 |  -620 |
|  295 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        320 |          0 |        320 |   225 |   290 |   320 |   285 |   270 |   300 |    90 |    75 |
|  296 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        360 |          0 |        360 |   290 |   300 |   275 |   360 |   190 |   200 |   280 |   320 |
|  297 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        130 |       -165 |        295 |    90 |   130 |   -75 |   -90 |  -160 |  -165 |   -55 |  -115 |
|  298 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -800 |        800 |  -495 |  -540 |  -660 |  -670 |  -800 |  -735 |  -695 |  -735 |
|  299 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -735 |        735 |  -710 |  -730 |  -640 |  -705 |  -735 |  -675 |  -630 |  -680 |
|  300 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1050 |       1050 |  -750 |  -840 |  -760 | -1010 |  -980 |  -915 | -1050 |  -930 |
|  301 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          5 |       -305 |        310 |   -45 |     5 |   -85 |  -260 |  -270 |  -305 |  -180 |  -240 |
|  302 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1135 |       1135 |  -800 |  -870 |  -865 | -1010 | -1035 |  -970 | -1135 | -1035 |
|  303 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         85 |       -330 |        415 |  -175 |    50 |  -195 |  -180 |   -65 |    85 |  -120 |  -330 |
|  304 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -795 |        795 |  -600 |  -550 |  -495 |  -635 |  -585 |  -640 |  -515 |  -795 |
|  305 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -330 |        330 |  -165 |  -110 |   -15 |   -45 |  -235 |  -105 |  -160 |  -330 |
|  306 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        305 |          0 |        305 |   305 |   280 |   125 |    50 |    25 |    10 |    30 |    45 |
|  307 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        355 |        -10 |        365 |   355 |   210 |   255 |   -10 |    20 |    -0 |    -5 |    30 |
|  308 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        140 |          0 |        140 |   125 |   130 |    40 |    55 |   100 |   140 |    25 |   140 |
|  309 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         45 |       -125 |        170 |    45 |   -40 |   -60 |   -25 |   -70 |   -75 |   -35 |  -125 |
|  310 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -680 |        680 |  -455 |  -450 |  -385 |  -610 |  -650 |  -680 |  -645 |  -560 |
|  311 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -450 |        450 |  -205 |  -265 |  -270 |  -260 |  -225 |  -230 |  -305 |  -450 |
|  312 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -525 |        525 |  -430 |  -455 |  -430 |  -345 |  -390 |  -430 |  -390 |  -525 |
|  313 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -730 |        730 |  -420 |  -430 |  -575 |  -590 |  -645 |  -730 |  -635 |  -515 |
|  314 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -395 |        395 |   -85 |  -195 |  -255 |  -330 |  -320 |  -395 |  -330 |  -380 |
|  315 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        180 |        -30 |        210 |   180 |    45 |    95 |    80 |    90 |    90 |   105 |   -30 |
|  316 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        180 |        -30 |        210 |   180 |   110 |   160 |    50 |   -30 |    40 |   -25 |    65 |
|  317 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1135 |       1135 | -1135 | -1125 | -1005 | -1105 | -1105 | -1110 | -1130 | -1125 |
|  318 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        265 |        -70 |        335 |   125 |   160 |   125 |   265 |   140 |    10 |   -70 |    15 |
|  319 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        320 |        -25 |        345 |   285 |   155 |   260 |   320 |   195 |   -25 |    20 |    80 |
|  320 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         40 |       -145 |        185 |   -20 |   -45 |   -20 |  -145 |  -110 |   -60 |   -70 |    40 |
|  321 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        235 |          0 |        235 |    10 |   235 |   130 |    90 |   185 |   130 |    50 |   120 |
|  322 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |        -40 |        280 |   240 |    95 |    35 |    95 |    65 |    75 |   -40 |    95 |
|  323 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |        -75 |        430 |   355 |   285 |   305 |   200 |   275 |   255 |    95 |   -75 |
|  324 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        280 |        -30 |        310 |   250 |   280 |   275 |   250 |   255 |   210 |   100 |   -30 |
|  325 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 | -1005 |  -840 |  -950 |  -930 |  -965 |  -975 |  -920 | -1080 |
|  326 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1175 |       1175 |  -945 |  -965 | -1175 | -1100 | -1150 | -1160 | -1065 | -1010 |
|  327 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        345 |          0 |        345 |   280 |   240 |   255 |   345 |    10 |    35 |    45 |    70 |
|  328 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1170 |       1170 |  -780 |  -880 |  -915 | -1130 | -1095 | -1150 | -1170 | -1060 |
|  329 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -280 |        280 |   -30 |   -70 |  -100 |  -115 |  -135 |  -140 |  -245 |  -280 |
|  330 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -470 |        470 |  -320 |  -220 |  -305 |  -185 |  -465 |  -470 |  -410 |  -460 |
|  331 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1055 |       1055 |  -905 |  -985 |  -920 |  -970 |  -980 | -1040 | -1000 | -1055 |
|  332 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -595 |        595 |  -275 |  -410 |  -365 |  -500 |  -595 |  -450 |  -515 |  -445 |
|  333 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -860 |        860 |  -540 |  -550 |  -535 |  -845 |  -760 |  -745 |  -860 |  -695 |
|  334 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        295 |          0 |        295 |   295 |   170 |   255 |   295 |   250 |    20 |    55 |    40 |
|  335 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        265 |          0 |        265 |   265 |   250 |   250 |   210 |     5 |    75 |    60 |    35 |
|  336 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        345 |          0 |        345 |   345 |   135 |   100 |    80 |   115 |   135 |   150 |   185 |
|  337 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        100 |       -215 |        315 |   -90 |    50 |   100 |    30 |   -70 |   -65 |   -45 |  -215 |
|  338 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |        -60 |        315 |   255 |   -60 |   -10 |    50 |    25 |    50 |    10 |    80 |
|  339 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |          0 |        370 |   240 |   280 |   370 |   360 |   310 |    80 |   150 |    80 |
|  340 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        180 |        -10 |        190 |    40 |    40 |    60 |    85 |   180 |    40 |   -10 |    20 |
|  341 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -535 |        535 |  -175 |  -205 |  -125 |  -535 |  -480 |  -445 |  -380 |  -345 |
|  342 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        115 |        -30 |        145 |    70 |    55 |   100 |   110 |   -30 |   115 |    55 |   100 |
|  343 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -925 |        925 |  -680 |  -590 |  -710 |  -600 |  -800 |  -605 |  -650 |  -925 |
|  344 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        385 |      -1170 |       1555 | -1165 | -1065 | -1170 | -1165 | -1140 | -1165 | -1080 |   385 |
|  345 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -290 |        290 |  -145 |  -145 |  -150 |  -160 |  -260 |  -290 |  -245 |  -275 |
|  346 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -475 |        475 |  -150 |  -460 |  -465 |  -425 |  -390 |  -395 |  -435 |  -475 |
|  347 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -875 |        875 |  -555 |  -650 |  -765 |  -875 |  -805 |  -775 |  -810 |  -845 |
|  348 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        300 |          0 |        300 |   300 |   250 |   230 |   215 |   245 |   225 |    25 |   105 |
|  349 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -180 |        180 |   -90 |  -135 |   -80 |  -180 |   -85 |   -95 |   -80 |   -65 |
|  350 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        400 |          0 |        400 |   400 |   305 |   240 |   100 |    75 |   130 |   140 |    70 |
|  351 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1045 |       1045 |  -715 |  -845 |  -865 |  -970 | -1045 |  -975 | -1020 |  -985 |
|  352 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -505 |        505 |  -285 |  -220 |  -245 |  -180 |  -385 |  -410 |  -505 |  -480 |
|  353 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        290 |          0 |        290 |   290 |   115 |   115 |    85 |    70 |   135 |   140 |   175 |
|  354 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        320 |          0 |        320 |   265 |   320 |   180 |    70 |   125 |   160 |   115 |   185 |
|  355 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -245 |        245 |  -210 |  -230 |  -180 |  -190 |  -160 |  -150 |  -205 |  -245 |
|  356 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |          0 |        350 |   300 |   250 |   335 |   350 |   265 |   250 |    15 |   130 |
|  357 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        115 |       -195 |        310 |   115 |  -195 |  -130 |   -55 |  -135 |   -30 |   -10 |  -170 |
|  358 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -570 |        570 |  -220 |  -290 |  -305 |  -460 |  -570 |  -520 |  -505 |  -475 |
|  359 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -645 |        645 |  -440 |  -465 |  -500 |  -445 |  -495 |  -575 |  -500 |  -645 |
|  360 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1055 |       1055 |  -745 |  -835 |  -725 |  -755 |  -760 |  -955 |  -950 | -1055 |
|  361 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -390 |        390 |  -115 |  -185 |  -230 |  -300 |  -240 |  -390 |  -305 |  -355 |
|  362 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -665 |        665 |  -305 |  -600 |  -550 |  -580 |  -590 |  -605 |  -665 |  -560 |
|  363 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         15 |       -320 |        335 |  -125 |  -140 |   -80 |    -5 |  -140 |    15 |   -95 |  -320 |
|  364 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -955 |        955 |  -865 |  -805 |  -785 |  -785 |  -820 |  -735 |  -865 |  -955 |
|  365 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        270 |        -80 |        350 |   210 |   210 |   270 |   -55 |    70 |   -80 |   -65 |    50 |
|  366 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1155 |       1155 |  -915 |  -970 | -1060 | -1025 |  -975 |  -955 | -1155 | -1120 |
|  367 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        375 |          0 |        375 |   275 |   270 |   375 |   325 |   345 |   295 |   295 |   140 |
|  368 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1080 |       1080 |  -810 |  -775 | -1040 |  -960 | -1050 | -1015 |  -915 | -1080 |
|  369 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        255 |        -15 |        270 |    90 |   145 |    45 |   130 |   255 |   100 |   160 |   -15 |
|  370 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -340 |        340 |  -300 |  -240 |  -340 |  -125 |  -275 |  -135 |  -290 |  -135 |
|  371 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -845 |        845 |  -660 |  -640 |  -605 |  -795 |  -820 |  -820 |  -845 |  -840 |
|  372 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        155 |        -45 |        200 |   150 |   120 |    80 |   155 |    -0 |     5 |   125 |   -45 |
|  373 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         -0 |       -255 |        255 |  -110 |  -160 |  -130 |   -50 |  -110 |    -0 |   -40 |  -255 |
|  374 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -425 |        425 |  -160 |  -180 |  -230 |  -390 |  -360 |  -425 |  -380 |  -330 |
|  375 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        305 |          0 |        305 |   275 |   305 |   105 |    80 |   120 |    85 |   120 |   145 |
|  376 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        320 |          0 |        320 |   250 |   245 |   305 |   320 |   225 |   250 |     5 |     5 |
|  377 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -655 |        655 |  -380 |  -390 |  -615 |  -565 |  -570 |  -655 |  -560 |  -500 |
|  378 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        185 |        -15 |        200 |   105 |   185 |   160 |   125 |   120 |    70 |   120 |   -15 |
|  379 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        230 |         -0 |        230 |   230 |   130 |    35 |   110 |   110 |    60 |   105 |    -0 |
|  380 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        150 |       -230 |        380 |   150 |  -140 |  -145 |   -65 |  -100 |  -230 |  -135 |  -160 |
|  381 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -525 |        525 |  -255 |  -295 |  -160 |  -225 |  -505 |  -500 |  -470 |  -525 |
|  382 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        345 |        -40 |        385 |   345 |   330 |   195 |   245 |    70 |   115 |   100 |   -40 |
|  383 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -350 |        350 |  -150 |   -45 |  -345 |  -305 |  -230 |  -350 |  -240 |  -325 |
|  384 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -340 |        340 |  -140 |  -230 |  -340 |  -220 |  -200 |  -295 |  -100 |  -215 |
|  385 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -440 |        440 |  -390 |  -440 |  -425 |  -345 |  -330 |  -280 |  -295 |  -385 |
|  386 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        340 |          0 |        340 |   190 |   335 |   150 |   275 |   225 |   330 |   340 |   210 |
|  387 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -360 |        360 |  -215 |  -175 |  -195 |  -165 |  -270 |  -150 |  -245 |  -360 |
|  388 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -465 |        465 |  -385 |  -370 |  -340 |  -440 |  -270 |  -370 |  -415 |  -465 |
|  389 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        130 |       -250 |        380 |    95 |   130 |    30 |    25 |   -60 |  -250 |  -185 |  -125 |
|  390 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -970 |        970 |  -805 |  -845 |  -795 |  -880 |  -800 |  -815 |  -830 |  -970 |
|  391 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         95 |        -30 |        125 |    85 |    95 |    90 |    30 |   -30 |    35 |   -30 |    -0 |
|  392 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        385 |          0 |        385 |   365 |   385 |   210 |   155 |   225 |   200 |   200 |   300 |
|  393 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        275 |        -70 |        345 |   270 |   275 |    40 |   -70 |    95 |   110 |    70 |    60 |
|  394 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        325 |          0 |        325 |   210 |   325 |   160 |    65 |    45 |    75 |    55 |    75 |
|  395 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |        -25 |        290 |   240 |   230 |   265 |    75 |    20 |    65 |    45 |   -25 |
|  396 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        215 |        -85 |        300 |   150 |   215 |   180 |   140 |    65 |   -85 |    85 |     5 |
|  397 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |       -140 |        405 |   195 |   265 |   -20 |   -55 |    90 |  -140 |   -40 |    30 |
|  398 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -420 |        420 |  -145 |  -305 |  -195 |  -360 |  -420 |  -410 |  -300 |  -300 |
|  399 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        105 |       -155 |        260 |    10 |    20 |   105 |    55 |   -45 |    60 |    20 |  -155 |
|  400 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -400 |        400 |  -300 |  -240 |  -210 |  -210 |  -210 |  -225 |  -135 |  -400 |
|  401 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1045 |       1045 |  -800 |  -735 |  -760 |  -795 |  -715 |  -790 |  -835 | -1045 |
|  402 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        110 |        -50 |        160 |    65 |    55 |   -15 |   -50 |   110 |   -50 |   -20 |   -35 |
|  403 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -300 |        300 |   -25 |   -85 |   -20 |  -130 |   -55 |  -270 |  -220 |  -300 |
|  404 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         70 |        -20 |         90 |   -20 |    70 |    10 |    -5 |    20 |    -0 |   -20 |   -15 |
|  405 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        335 |        -30 |        365 |   135 |   200 |   335 |   100 |   235 |    30 |    55 |   -30 |
|  406 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        305 |          0 |        305 |   230 |   205 |   305 |   200 |   300 |   295 |   200 |   235 |
|  407 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1050 |       1050 |  -760 |  -935 |  -940 | -1050 | -1000 |  -965 | -1005 |  -970 |
|  408 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        125 |       -180 |        305 |   110 |   110 |   125 |  -130 |   -50 |  -100 |   -95 |  -180 |
|  409 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        190 |       -125 |        315 |   190 |    35 |    -5 |  -125 |     5 |   -10 |   -15 |   -40 |
|  410 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        280 |          0 |        280 |   190 |   205 |   205 |   165 |   280 |    15 |     5 |    45 |
|  411 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        350 |          0 |        350 |   350 |   330 |   215 |   240 |   270 |   245 |   260 |    45 |
|  412 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -850 |        850 |  -680 |  -610 |  -665 |  -755 |  -725 |  -685 |  -590 |  -850 |
|  413 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        310 |        -30 |        340 |   285 |   310 |    20 |   -30 |    60 |    60 |    40 |    35 |
|  414 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        125 |       -195 |        320 |   115 |   125 |    95 |    35 |   -70 |   -20 |  -195 |   -75 |
|  415 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -690 |        690 |  -520 |  -510 |  -405 |  -435 |  -670 |  -690 |  -675 |  -635 |
|  416 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -310 |        310 |  -100 |   -60 |  -150 |  -120 |   -75 |  -145 |  -265 |  -310 |
|  417 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        290 |        -75 |        365 |   200 |   170 |   165 |   290 |   -25 |   -20 |   -75 |   -60 |
|  418 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        265 |        -50 |        315 |   210 |   150 |   265 |   140 |   250 |   185 |   155 |   -50 |
|  419 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -995 |        995 |  -835 |  -910 |  -980 |  -995 |  -980 |  -890 |  -985 |  -990 |
|  420 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1105 |       1105 |  -695 |  -660 |  -800 |  -735 |  -725 |  -850 |  -690 | -1105 |
|  421 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        215 |        -50 |        265 |   210 |   110 |    -5 |   125 |   115 |   215 |   160 |   -50 |
|  422 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        295 |          0 |        295 |   295 |   225 |   190 |   275 |   290 |   235 |    55 |    45 |
|  423 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -770 |        770 |  -650 |  -625 |  -580 |  -625 |  -630 |  -560 |  -615 |  -770 |
|  424 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -925 |        925 |  -655 |  -595 |  -735 |  -900 |  -925 |  -815 |  -850 |  -860 |
|  425 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        345 |          0 |        345 |   155 |   300 |   225 |   265 |   345 |    55 |    85 |    55 |
|  426 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        125 |       -160 |        285 |    15 |    75 |   120 |   125 |    85 |    -5 |  -160 |   -50 |
|  427 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        330 |          0 |        330 |   330 |   295 |   320 |   100 |    90 |   170 |   120 |   140 |
|  428 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        335 |          0 |        335 |   245 |   325 |   240 |   330 |   335 |   120 |   210 |   115 |
|  429 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -675 |        675 |  -440 |  -445 |  -335 |  -455 |  -515 |  -370 |  -455 |  -675 |
|  430 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -865 |        865 |  -660 |  -585 |  -665 |  -865 |  -815 |  -835 |  -820 |  -760 |
|  431 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        255 |          0 |        255 |   255 |   225 |   135 |    80 |    50 |    95 |    95 |   105 |
|  432 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -760 |        760 |  -580 |  -540 |  -605 |  -535 |  -645 |  -730 |  -750 |  -760 |
|  433 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -295 |        295 |  -125 |   -80 |   -50 |  -115 |   -50 |   -95 |   -15 |  -295 |
|  434 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -515 |        515 |  -145 |  -155 |  -255 |   -85 |   -95 |  -125 |  -515 |  -445 |
|  435 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -860 |        860 |  -545 |  -555 |  -510 |  -555 |  -560 |  -860 |  -790 |  -750 |
|  436 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -830 |        830 |  -615 |  -745 |  -720 |  -815 |  -745 |  -775 |  -830 |  -805 |
|  437 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        285 |          0 |        285 |   285 |   125 |   240 |   160 |   155 |   195 |   145 |    50 |
|  438 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        340 |        -10 |        350 |   310 |   340 |   100 |   105 |   -10 |   135 |   160 |   145 |
|  439 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        185 |        -70 |        255 |   170 |   125 |   185 |    55 |    70 |    10 |   -70 |    20 |
|  440 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        285 |          0 |        285 |   250 |   225 |   285 |    15 |    50 |   130 |    70 |    45 |
|  441 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        345 |          0 |        345 |   345 |   285 |   215 |   235 |   250 |   175 |   160 |   235 |
|  442 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        190 |        -95 |        285 |   155 |   180 |    95 |   190 |   -95 |    20 |   -85 |   -55 |
|  443 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -455 |        455 |  -310 |  -160 |  -375 |  -320 |  -275 |  -350 |  -245 |  -455 |
|  444 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -945 |        945 |  -720 |  -750 |  -880 |  -900 |  -905 |  -940 |  -835 |  -945 |
|  445 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        290 |          0 |        290 |   285 |   290 |   130 |   105 |    95 |   230 |   175 |   140 |
|  446 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        105 |       -120 |        225 |   105 |  -120 |    20 |    -5 |  -110 |   -50 |   -40 |   -30 |
|  447 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |        -10 |        235 |   190 |   225 |   155 |   225 |   -10 |   105 |    -5 |    75 |
|  448 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -325 |        325 |   -25 |  -300 |  -325 |  -300 |  -300 |  -220 |  -290 |  -235 |
|  449 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1180 |       1180 |  -900 |  -855 |  -975 |  -860 |  -935 | -1055 | -1180 | -1170 |
|  450 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -285 |        285 |  -280 |  -265 |  -190 |  -250 |  -285 |  -265 |  -190 |  -125 |
|  451 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          5 |       -310 |        315 |  -115 |     5 |  -275 |  -210 |  -240 |  -310 |  -225 |  -190 |
|  452 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1165 |       1165 |  -865 |  -805 |  -815 |  -900 | -1060 | -1095 | -1035 | -1165 |
|  453 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1160 |       1160 |  -945 |  -895 | -1045 | -1110 | -1020 | -1095 | -1050 | -1160 |
|  454 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -850 |        850 |  -735 |  -790 |  -850 |  -755 |  -775 |  -660 |  -715 |  -850 |
|  455 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -790 |  -780 |  -850 |  -735 |  -815 |  -710 |  -785 |  -860 |
|  456 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |        -30 |        380 |   225 |   235 |   230 |   280 |   350 |   230 |   -30 |    95 |
|  457 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -375 |        375 |  -215 |  -100 |  -160 |  -255 |  -125 |  -215 |  -205 |  -375 |
|  458 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -925 |        925 |  -700 |  -745 |  -635 |  -620 |  -910 |  -910 |  -885 |  -925 |
|  459 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1165 |       1165 | -1010 | -1000 | -1020 |  -975 |  -910 |  -975 | -1005 | -1165 |
|  460 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -480 |        480 |  -215 |  -460 |  -400 |  -480 |  -480 |  -410 |  -430 |  -470 |
|  461 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        260 |          0 |        260 |   165 |   165 |   175 |   165 |   210 |   260 |   185 |   180 |
|  462 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |        -35 |        300 |   200 |   265 |   -20 |    50 |   125 |   -10 |   -35 |    40 |
|  463 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        375 |          0 |        375 |   375 |   275 |   350 |   315 |   290 |   345 |    85 |   210 |
|  464 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        375 |          0 |        375 |   315 |   225 |   215 |   375 |   200 |   295 |   205 |   300 |
|  465 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        350 |          0 |        350 |   245 |   240 |   350 |   200 |   180 |   215 |   200 |    45 |
|  466 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -750 |        750 |  -645 |  -480 |  -465 |  -590 |  -605 |  -455 |  -750 |  -725 |
|  467 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1055 |       1055 |  -915 |  -870 |  -895 |  -945 |  -915 |  -935 |  -925 | -1055 |
|  468 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -455 |        455 |  -215 |  -310 |  -430 |  -315 |  -330 |  -455 |  -310 |  -445 |
|  469 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -950 |        950 |  -655 |  -665 |  -755 |  -770 |  -745 |  -630 |  -865 |  -950 |
|  470 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        370 |          0 |        370 |   370 |   370 |   325 |   185 |   230 |   220 |   255 |   195 |
|  471 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -435 |        435 |   -90 |  -330 |  -340 |  -290 |  -335 |  -335 |  -435 |  -380 |
|  472 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        325 |        -50 |        375 |   205 |   325 |   -10 |   -10 |   105 |    35 |    70 |   -50 |
|  473 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        225 |        -15 |        240 |   225 |    80 |    90 |    50 |    35 |   -15 |    50 |    40 |
|  474 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -445 |        445 |  -135 |  -110 |  -130 |  -260 |  -445 |  -350 |  -285 |  -355 |
|  475 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        200 |       -105 |        305 |   115 |   175 |    80 |    70 |   200 |   130 |   -10 |  -105 |
|  476 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -930 |        930 |  -685 |  -680 |  -675 |  -680 |  -625 |  -680 |  -930 |  -865 |
|  477 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |        -55 |        310 |   255 |   -55 |   -15 |   115 |    30 |    65 |    45 |    35 |
|  478 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         75 |       -100 |        175 |    75 |   -65 |    30 |    35 |   -20 |    -0 |  -100 |    60 |
|  479 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1005 |       1005 |  -720 |  -730 |  -730 |  -795 |  -915 |  -910 | -1005 |  -975 |
|  480 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |          0 |        285 |   285 |   125 |   280 |   285 |   220 |   155 |    35 |   110 |
|  481 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -700 |        700 |  -525 |  -400 |  -510 |  -570 |  -605 |  -600 |  -525 |  -700 |
|  482 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1190 |       1190 |  -910 |  -890 |  -855 |  -830 | -1025 | -1190 | -1060 | -1105 |
|  483 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -605 |        605 |  -330 |  -360 |  -340 |  -255 |  -605 |  -470 |  -515 |  -510 |
|  484 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        250 |         -0 |        250 |   235 |   220 |   195 |   250 |   205 |   185 |    -0 |    15 |
|  485 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        285 |        -55 |        340 |   190 |   250 |   230 |   285 |   -55 |   -15 |    70 |    10 |
|  486 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -690 |        690 |  -645 |  -670 |  -560 |  -690 |  -590 |  -650 |  -585 |  -510 |
|  487 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -880 |        880 |  -540 |  -555 |  -595 |  -690 |  -680 |  -750 |  -835 |  -880 |
|  488 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1135 |       1135 | -1035 | -1020 |  -900 |  -925 |  -950 |  -960 | -1135 | -1100 |
|  489 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1140 |       1140 |  -935 |  -905 |  -905 |  -850 |  -960 |  -980 |  -940 | -1140 |
|  490 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        375 |          0 |        375 |   300 |   235 |   340 |   235 |   375 |   165 |    25 |   150 |
|  491 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1150 |       1150 |  -900 |  -940 |  -815 |  -910 | -1030 | -1115 | -1080 | -1150 |
|  492 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -635 |        635 |  -425 |  -355 |  -530 |  -615 |  -600 |  -590 |  -635 |  -585 |
|  493 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -635 |        635 |  -355 |  -240 |  -320 |  -560 |  -475 |  -635 |  -570 |  -530 |
|  494 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -335 |        335 |   -20 |   -15 |  -160 |  -275 |  -205 |  -335 |  -280 |  -270 |
|  495 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -485 |        485 |  -200 |  -255 |  -250 |  -305 |  -285 |  -260 |  -280 |  -485 |
|  496 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -545 |        545 |  -260 |  -275 |  -325 |  -490 |  -545 |  -390 |  -425 |  -460 |
|  497 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -855 |  -850 |  -780 |  -825 |  -925 |  -820 |  -815 | -1080 |
|  498 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        210 |        -20 |        230 |   200 |   165 |   210 |    10 |   -20 |    65 |   -10 |    -0 |
|  499 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |        -20 |        285 |   265 |    75 |    70 |   140 |   -20 |    55 |    65 |    35 |
|  500 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1190 |       1190 | -1075 | -1020 | -1035 | -1065 | -1070 | -1050 | -1075 | -1190 |
|  501 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -995 |        995 |  -785 |  -775 |  -960 |  -995 |  -960 |  -935 |  -985 |  -955 |
|  502 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        390 |          0 |        390 |   390 |   385 |    75 |   140 |   165 |   110 |    65 |   160 |
|  503 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -535 |        535 |  -285 |  -295 |  -345 |  -340 |  -290 |  -230 |  -495 |  -535 |
|  504 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        350 |      -1145 |       1495 | -1090 | -1080 | -1145 | -1110 | -1145 | -1100 | -1065 |   350 |
|  505 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         60 |        -45 |        105 |   -45 |    20 |    -5 |    -5 |    20 |    50 |    60 |   -45 |
|  506 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -495 |        495 |  -195 |  -410 |  -345 |  -495 |  -355 |  -385 |  -440 |  -435 |
|  507 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -770 |        770 |  -595 |  -640 |  -770 |  -580 |  -605 |  -635 |  -565 |  -655 |
|  508 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        230 |        -55 |        285 |   155 |   175 |   215 |   130 |   130 |   230 |   -25 |   -55 |
|  509 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -555 |        555 |  -255 |  -305 |  -240 |  -235 |  -255 |  -500 |  -530 |  -555 |
|  510 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -835 |        835 |  -815 |  -790 |  -795 |  -775 |  -805 |  -835 |  -735 |  -790 |
|  511 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        295 |          0 |        295 |   200 |   205 |   295 |   145 |   270 |   215 |   220 |   110 |
|  512 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1140 |       1140 |  -910 |  -900 |  -905 |  -900 |  -925 |  -950 | -1125 | -1140 |
|  513 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -350 |        350 |  -165 |  -175 |  -200 |  -220 |  -200 |  -260 |  -190 |  -350 |
|  514 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        355 |          0 |        355 |   355 |   180 |   155 |   175 |   200 |   150 |   315 |   225 |
|  515 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        270 |        -55 |        325 |   120 |   215 |   270 |    90 |   225 |   235 |   175 |   -55 |
|  516 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1200 |       1200 | -1020 | -1170 | -1185 | -1125 | -1200 | -1105 | -1140 | -1200 |
|  517 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -715 |        715 |  -430 |  -435 |  -705 |  -715 |  -485 |  -630 |  -560 |  -600 |
|  518 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -760 |        760 |  -455 |  -365 |  -425 |  -375 |  -405 |  -760 |  -625 |  -545 |
|  519 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |          0 |        155 |   155 |    35 |    60 |    65 |    80 |   130 |    80 |   140 |
|  520 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -880 |        880 |  -690 |  -645 |  -685 |  -625 |  -660 |  -845 |  -795 |  -880 |
|  521 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -335 |        335 |  -155 |  -180 |  -155 |  -195 |  -115 |  -135 |   -80 |  -335 |
|  522 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        390 |         -0 |        390 |   250 |   260 |   240 |   390 |   305 |   315 |   275 |    -0 |
|  523 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        320 |          0 |        320 |   320 |   145 |    90 |    75 |   150 |    30 |   150 |   155 |
|  524 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -720 |        720 |  -475 |  -720 |  -615 |  -675 |  -605 |  -645 |  -715 |  -625 |
|  525 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -655 |        655 |  -385 |  -575 |  -345 |  -430 |  -455 |  -425 |  -385 |  -655 |
|  526 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -645 |        645 |  -450 |  -440 |  -440 |  -430 |  -560 |  -495 |  -645 |  -640 |
|  527 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        380 |          0 |        380 |   330 |   380 |   345 |   325 |   280 |   335 |   195 |   140 |
|  528 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -585 |        585 |  -380 |  -355 |  -385 |  -365 |  -390 |  -475 |  -395 |  -585 |
|  529 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        280 |          0 |        280 |   165 |   200 |   175 |   115 |   280 |    80 |    50 |   105 |
|  530 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        265 |        -15 |        280 |   100 |   225 |   165 |   265 |   215 |   205 |   265 |   -15 |
|  531 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        370 |          0 |        370 |   270 |   370 |    95 |   115 |   225 |   185 |    95 |   140 |
|  532 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1105 |       1105 |  -785 |  -800 |  -840 |  -845 |  -875 |  -915 | -1105 | -1015 |
|  533 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        315 |          0 |        315 |   315 |   115 |     5 |    85 |   105 |   155 |    90 |   225 |
|  534 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -230 |        230 |  -155 |  -200 |  -130 |  -140 |  -230 |  -145 |    -5 |  -190 |
|  535 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1080 |       1080 |  -710 |  -915 |  -920 |  -925 | -1065 | -1080 |  -940 |  -950 |
|  536 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -985 |        985 |  -820 |  -815 |  -685 |  -775 |  -950 |  -925 |  -955 |  -985 |
|  537 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        325 |          0 |        325 |   325 |   310 |   110 |   140 |   170 |    95 |   125 |    95 |
|  538 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        305 |          0 |        305 |   270 |   290 |   305 |   240 |   250 |   290 |   300 |   210 |
|  539 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        130 |          0 |        130 |    90 |   130 |   120 |    95 |    60 |   130 |   110 |    90 |
|  540 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        240 |        -25 |        265 |   200 |   230 |   240 |   130 |   200 |   190 |   -25 |    35 |
|  541 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        375 |          0 |        375 |   375 |   295 |   260 |   265 |   115 |   230 |    90 |   185 |
|  542 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        230 |        -15 |        245 |   170 |   165 |   175 |   230 |    85 |   110 |   -15 |    60 |
|  543 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |        -60 |        320 |   210 |   235 |   260 |    10 |   -60 |    -5 |    50 |   -45 |
|  544 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        170 |        -95 |        265 |   170 |     5 |   -75 |    25 |   -70 |   105 |   -95 |   -10 |
|  545 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -295 |        295 |  -170 |  -105 |   -80 |   -75 |  -115 |   -75 |  -125 |  -295 |
|  546 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        195 |       -270 |        465 |    55 |    40 |    80 |    -5 |   195 |  -270 |  -130 |  -220 |
|  547 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        180 |        -40 |        220 |   145 |    85 |   180 |    80 |    45 |    -0 |   -30 |   -40 |
|  548 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |        -20 |        175 |    30 |    20 |   115 |    25 |   155 |    40 |   -20 |    90 |
|  549 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        245 |         -0 |        245 |   245 |   175 |   100 |   105 |   115 |    -0 |   165 |    40 |
|  550 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -380 |        380 |  -120 |  -190 |  -335 |  -295 |  -335 |  -335 |  -380 |  -335 |
|  551 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1030 |       1030 |  -800 |  -965 | -1030 |  -860 |  -875 | -1020 | -1030 |  -945 |
|  552 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        390 |        -20 |        410 |   390 |   270 |   170 |   255 |   -15 |   130 |   110 |   -20 |
|  553 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        320 |          0 |        320 |   320 |   310 |   290 |   320 |   310 |   275 |    75 |   110 |
|  554 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -785 |        785 |  -650 |  -680 |  -680 |  -650 |  -605 |  -750 |  -610 |  -785 |
|  555 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        375 |        -25 |        400 |   230 |   285 |   230 |   375 |   255 |    70 |   -25 |    50 |
|  556 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -870 |        870 |  -660 |  -540 |  -510 |  -630 |  -650 |  -835 |  -870 |  -785 |
|  557 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -820 |        820 |  -520 |  -500 |  -570 |  -770 |  -820 |  -745 |  -595 |  -770 |
|  558 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        275 |        -90 |        365 |   275 |   115 |   -10 |   -90 |   110 |    10 |   -25 |   -85 |
|  559 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1055 |       1055 |  -800 |  -865 | -1055 |  -895 |  -975 |  -985 | -1030 |  -970 |
|  560 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -985 |        985 |  -720 |  -740 |  -670 |  -680 |  -715 |  -755 |  -725 |  -985 |
|  561 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        105 |       -270 |        375 |    25 |     5 |    80 |    -5 |   105 |   -40 |  -270 |  -165 |
|  562 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        320 |          0 |        320 |   285 |   320 |   220 |   290 |   300 |   195 |   205 |    65 |
|  563 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        195 |       -100 |        295 |   195 |   -40 |   -45 |  -100 |   -70 |   -60 |   -95 |   -45 |
|  564 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        230 |        -45 |        275 |   195 |   190 |   185 |    75 |   210 |   230 |   175 |   -45 |
|  565 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        365 |        -20 |        385 |   310 |   365 |   320 |   225 |    55 |   -20 |    70 |    50 |
|  566 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -410 |        410 |  -270 |  -280 |  -215 |  -220 |  -220 |  -315 |  -205 |  -410 |
|  567 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1185 |       1185 |  -865 |  -860 | -1185 | -1115 | -1155 | -1165 | -1120 | -1080 |
|  568 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        240 |        -80 |        320 |   240 |   165 |   205 |    15 |    30 |   -80 |    40 |   -25 |
|  569 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -400 |        400 |  -180 |  -145 |  -365 |  -375 |  -355 |  -385 |  -400 |  -330 |
|  570 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |          0 |        260 |   220 |   260 |   260 |    80 |    60 |    65 |    80 |    50 |
|  571 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -925 |        925 |  -645 |  -515 |  -895 |  -725 |  -760 |  -860 |  -810 |  -925 |
|  572 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -295 |        295 |  -140 |  -175 |   -15 |  -120 |  -135 |  -130 |  -100 |  -295 |
|  573 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        285 |        -55 |        340 |   205 |   225 |   175 |   285 |    15 |   -10 |    85 |   -55 |
|  574 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        260 |       -135 |        395 |   195 |   255 |   170 |   260 |    30 |   200 |   -50 |  -135 |
|  575 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        220 |        -70 |        290 |   220 |   220 |   150 |   120 |    85 |    10 |   -70 |    35 |
|  576 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1115 |       1115 |  -825 |  -740 |  -865 |  -725 |  -900 | -1110 |  -995 | -1115 |
|  577 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -795 |        795 |  -680 |  -520 |  -520 |  -515 |  -560 |  -720 |  -760 |  -795 |
|  578 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -615 |        615 |  -350 |  -395 |  -270 |  -590 |  -560 |  -615 |  -540 |  -490 |
|  579 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1180 |       1180 |  -920 | -1010 | -1020 |  -920 |  -855 |  -910 | -1030 | -1180 |
|  580 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -320 |        320 |   -20 |   -80 |   -15 |  -110 |   -60 |  -320 |  -235 |  -255 |
|  581 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |         -0 |        265 |   110 |   170 |   265 |    -0 |     5 |    50 |     5 |    15 |
|  582 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        330 |        -45 |        375 |   160 |   220 |   160 |   330 |    70 |    80 |   -45 |    15 |
|  583 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        190 |        -75 |        265 |    70 |   190 |   165 |   150 |   100 |     5 |   -30 |   -75 |
|  584 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1195 |       1195 |  -925 |  -920 |  -940 | -1170 | -1195 | -1135 | -1020 | -1085 |
|  585 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        215 |          0 |        215 |   175 |   155 |    60 |   160 |   105 |    80 |   215 |   215 |
|  586 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -420 |        420 |   -90 |  -140 |  -105 |  -420 |  -370 |  -310 |  -335 |  -340 |
|  587 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -995 |        995 |  -790 |  -730 |  -685 |  -720 |  -725 |  -700 |  -730 |  -995 |
|  588 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -670 |        670 |  -610 |  -610 |  -620 |  -560 |  -650 |  -670 |  -595 |  -590 |
|  589 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        175 |        -70 |        245 |   175 |   165 |   130 |   160 |   -70 |   -45 |    20 |    10 |
|  590 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        325 |          0 |        325 |   290 |   215 |   305 |   325 |   270 |   200 |   280 |   300 |
|  591 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         25 |       -105 |        130 |    25 |   -90 |   -10 |    -5 |  -105 |   -55 |   -90 |   -55 |
|  592 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        215 |        -65 |        280 |   145 |   135 |   215 |   -55 |   -65 |   -30 |   -10 |    45 |
|  593 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -395 |        395 |  -210 |   -35 |  -395 |  -255 |  -265 |  -275 |  -365 |  -385 |
|  594 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -815 |        815 |  -530 |  -570 |  -480 |  -615 |  -815 |  -795 |  -750 |  -770 |
|  595 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -605 |        605 |  -315 |  -320 |  -320 |  -375 |  -555 |  -535 |  -595 |  -605 |
|  596 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        335 |          0 |        335 |   335 |   270 |   280 |   260 |   320 |    85 |    45 |    65 |
|  597 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -300 |        300 |  -215 |  -150 |  -200 |  -215 |  -190 |  -185 |  -245 |  -300 |
|  598 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        245 |        -90 |        335 |   130 |   235 |   135 |   245 |    45 |   -35 |   -90 |    20 |
|  599 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        390 |          0 |        390 |   375 |   305 |   390 |   305 |   320 |   315 |   195 |    85 |
|  600 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -620 |        620 |  -400 |  -620 |  -620 |  -470 |  -495 |  -530 |  -610 |  -540 |
|  601 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |         -5 |        265 |   245 |   250 |   260 |    30 |    25 |    40 |    45 |    -5 |
|  602 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -545 |        545 |  -235 |  -175 |  -165 |  -465 |  -530 |  -545 |  -530 |  -485 |
|  603 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        310 |          0 |        310 |   310 |   255 |   215 |   215 |    60 |   115 |    85 |    30 |
|  604 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1080 |       1080 |  -865 |  -820 |  -750 |  -800 | -1050 | -1060 | -1005 | -1080 |
|  605 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        200 |        -35 |        235 |   200 |    70 |    75 |   160 |    65 |    35 |   -35 |    60 |
|  606 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         80 |       -230 |        310 |    80 |   -20 |  -140 |  -140 |  -155 |  -115 |  -230 |  -140 |
|  607 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1025 |       1025 |  -870 |  -810 |  -980 | -1025 |  -885 |  -815 |  -890 |  -905 |
|  608 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        375 |          0 |        375 |   375 |   360 |   180 |   250 |   165 |   205 |   145 |   150 |
|  609 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        315 |          0 |        315 |   200 |   200 |   315 |    60 |    35 |    95 |    80 |   155 |
|  610 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -385 |        385 |  -160 |  -190 |   -90 |  -205 |  -145 |  -185 |  -385 |  -315 |
|  611 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        295 |          0 |        295 |   295 |    90 |   105 |   190 |   175 |    90 |    95 |   125 |
|  612 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1010 |       1010 | -1010 |  -910 |  -905 | -1005 |  -905 |  -965 |  -930 |  -965 |
|  613 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |        -25 |        290 |   265 |   135 |    85 |   105 |   120 |   -25 |    25 |    50 |
|  614 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |          0 |        350 |   350 |   250 |   300 |   315 |   275 |   250 |   145 |    80 |
|  615 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        120 |        -45 |        165 |   120 |    55 |   -45 |   -40 |    70 |   -30 |     5 |   -45 |
|  616 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        390 |          0 |        390 |   380 |   390 |   170 |   140 |    70 |   135 |   115 |   150 |
|  617 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        320 |          0 |        320 |   235 |   320 |   270 |   310 |   240 |    80 |    65 |    10 |
|  618 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -735 |        735 |  -445 |  -700 |  -715 |  -675 |  -660 |  -710 |  -735 |  -645 |
|  619 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         -0 |       -110 |        110 |   -70 |   -30 |  -105 |    -0 |  -110 |   -40 |   -80 |   -85 |
|  620 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -320 |        320 |  -130 |  -120 |  -145 |  -165 |  -125 |  -110 |  -150 |  -320 |
|  621 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        370 |        -35 |        405 |   240 |   370 |    35 |    25 |   -35 |    30 |    75 |    80 |
|  622 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -335 |        335 |   -50 |   -55 |   -85 |  -260 |  -210 |  -150 |  -275 |  -335 |
|  623 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1015 |       1015 |  -750 |  -745 |  -685 |  -940 |  -980 | -1015 |  -990 |  -955 |
|  624 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        270 |          0 |        270 |   270 |   175 |   270 |   190 |   200 |   205 |   245 |    70 |
|  625 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1115 |       1115 | -1090 | -1025 | -1070 | -1115 | -1090 | -1010 | -1085 | -1000 |
|  626 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -635 |        635 |  -385 |  -420 |  -485 |  -330 |  -490 |  -330 |  -475 |  -635 |
|  627 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1175 |       1175 | -1050 | -1155 | -1095 | -1175 | -1175 | -1170 | -1145 | -1120 |
|  628 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1130 |       1130 |  -840 |  -875 |  -985 |  -920 |  -930 | -1130 | -1015 | -1020 |
|  629 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         75 |       -310 |        385 |  -135 |  -115 |   -85 |   -60 |   -60 |    75 |  -205 |  -310 |
|  630 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        225 |          0 |        225 |   200 |   145 |   205 |   175 |   225 |   110 |    55 |   115 |
|  631 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -840 |        840 |  -590 |  -625 |  -520 |  -840 |  -765 |  -800 |  -750 |  -800 |
|  632 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -885 |        885 |  -590 |  -605 |  -645 |  -505 |  -585 |  -670 |  -525 |  -885 |
|  633 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          5 |       -260 |        265 |     5 |   -60 |  -215 |  -205 |  -230 |  -260 |  -170 |  -210 |
|  634 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        210 |       -145 |        355 |   210 |   -90 |  -145 |   -95 |  -100 |  -130 |   -90 |  -110 |
|  635 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        220 |        -55 |        275 |    90 |   105 |   200 |   145 |   125 |   220 |    80 |   -55 |
|  636 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -445 |        445 |  -125 |  -190 |  -135 |  -100 |  -160 |  -325 |  -320 |  -445 |
|  637 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1065 |       1065 |  -730 |  -895 |  -795 |  -825 |  -920 |  -995 | -1010 | -1065 |
|  638 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -950 |        950 |  -815 |  -795 |  -720 |  -810 |  -795 |  -735 |  -785 |  -950 |
|  639 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        270 |          0 |        270 |   270 |   150 |   220 |   185 |    30 |    60 |    40 |   100 |
|  640 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1000 |       1000 |  -760 |  -820 |  -785 |  -780 |  -745 |  -945 |  -960 | -1000 |
|  641 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -600 |        600 |  -425 |  -370 |  -390 |  -400 |  -600 |  -600 |  -545 |  -570 |
|  642 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -920 |        920 |  -920 |  -865 |  -810 |  -870 |  -885 |  -785 |  -840 |  -855 |
|  643 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        385 |          0 |        385 |   385 |   135 |   220 |   215 |   210 |   245 |   130 |   145 |
|  644 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -690 |        690 |  -445 |  -475 |  -690 |  -645 |  -680 |  -510 |  -630 |  -640 |
|  645 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -380 |        380 |  -310 |  -310 |  -380 |  -355 |  -285 |  -320 |  -380 |  -365 |
|  646 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -350 |        350 |  -125 |  -140 |  -265 |  -300 |  -250 |  -350 |  -335 |  -305 |
|  647 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        190 |         -0 |        190 |   190 |   130 |    85 |    90 |    40 |    -0 |   140 |    35 |
|  648 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -655 |        655 |  -615 |  -550 |  -620 |  -640 |  -530 |  -550 |  -620 |  -655 |
|  649 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        310 |        -45 |        355 |   250 |   310 |   -25 |    15 |   -45 |   -10 |    15 |    35 |
|  650 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1060 |       1060 |  -790 | -1060 |  -945 |  -985 |  -985 |  -975 |  -870 |  -945 |
|  651 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1110 |       1110 |  -900 |  -795 |  -840 |  -840 |  -735 |  -795 | -1110 | -1095 |
|  652 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -340 |        340 |   -60 |  -275 |  -320 |  -195 |  -230 |  -275 |  -340 |  -230 |
|  653 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        175 |          0 |        175 |    75 |   175 |    55 |   115 |    70 |   120 |   160 |   140 |
|  654 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        110 |       -165 |        275 |   110 |   110 |    30 |  -135 |  -115 |   -40 |  -165 |  -115 |
|  655 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1165 |       1165 | -1040 | -1085 | -1040 | -1060 | -1120 | -1035 | -1105 | -1165 |
|  656 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -985 |        985 |  -730 |  -670 |  -790 |  -640 |  -665 |  -690 |  -985 |  -840 |
|  657 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         80 |       -230 |        310 |    80 |   -65 |  -225 |  -175 |  -175 |  -195 |  -110 |  -230 |
|  658 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        370 |          0 |        370 |   370 |   290 |   240 |   295 |   280 |   235 |   360 |   155 |
|  659 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -340 |        340 |  -110 |  -175 |  -135 |  -205 |  -130 |  -335 |  -340 |  -290 |
|  660 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        200 |        -80 |        280 |    30 |   145 |   175 |   155 |   200 |   120 |   -80 |   -80 |
|  661 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -710 |        710 |  -465 |  -435 |  -420 |  -370 |  -425 |  -410 |  -710 |  -520 |
|  662 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -610 |        610 |  -450 |  -585 |  -605 |  -610 |  -585 |  -555 |  -545 |  -595 |
|  663 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -315 |        315 |   -30 |  -130 |  -155 |  -140 |   -35 |   -55 |  -315 |  -225 |
|  664 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         25 |       -180 |        205 |    25 |   -30 |    -5 |    15 |   -30 |  -180 |  -105 |   -50 |
|  665 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          5 |       -310 |        315 |   -75 |   -95 |   -60 |     5 |   -75 |   -30 |  -170 |  -310 |
|  666 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        375 |      -1195 |       1570 | -1195 |   295 |   375 |   230 |   360 |   325 |   320 |   280 |
|  667 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -430 |        430 |  -355 |  -310 |  -270 |  -335 |  -320 |  -320 |  -410 |  -430 |
|  668 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        360 |          0 |        360 |   280 |   355 |   300 |   275 |   340 |   260 |   360 |   145 |
|  669 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -870 |        870 |  -605 |  -735 |  -580 |  -835 |  -790 |  -775 |  -755 |  -870 |
|  670 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1010 |       1010 |  -645 |  -840 | -1010 |  -935 |  -845 |  -830 |  -935 |  -935 |
|  671 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        165 |       -100 |        265 |   145 |   165 |    40 |    90 |    90 |    35 |   -30 |  -100 |
|  672 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         35 |       -260 |        295 |   -85 |   -90 |   -45 |    30 |   -80 |    35 |    -5 |  -260 |
|  673 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -790 |        790 |  -610 |  -775 |  -625 |  -745 |  -670 |  -740 |  -730 |  -790 |
|  674 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -735 |        735 |  -520 |  -470 |  -485 |  -490 |  -505 |  -470 |  -510 |  -735 |
|  675 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -155 |        155 |  -100 |   -70 |  -155 |  -120 |  -145 |  -100 |   -70 |   -95 |
|  676 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -510 |        510 |  -155 |  -385 |  -370 |  -440 |  -415 |  -510 |  -420 |  -410 |
|  677 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1020 |       1020 |  -875 |  -805 |  -815 |  -810 |  -890 |  -850 |  -965 | -1020 |
|  678 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1200 |       1200 |  -910 |  -960 | -1145 | -1120 | -1110 | -1125 | -1125 | -1200 |
|  679 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        340 |          0 |        340 |   340 |   185 |    60 |   130 |    25 |    30 |   150 |   145 |
|  680 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        355 |          0 |        355 |   345 |   355 |   160 |   185 |   105 |   175 |   215 |   145 |
|  681 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        155 |         -0 |        155 |   155 |   150 |    10 |    60 |    35 |    -0 |     5 |   100 |
|  682 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -420 |        420 |  -220 |   -90 |  -330 |  -420 |  -320 |  -415 |  -405 |  -330 |
|  683 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1130 |       1130 | -1120 | -1035 | -1065 | -1050 | -1130 |  -975 | -1080 | -1090 |
|  684 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         50 |       -345 |        395 |    35 |  -145 |   -85 |    50 |   -85 |  -295 |  -345 |  -225 |
|  685 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        305 |          0 |        305 |   305 |   210 |   120 |   125 |    45 |   105 |    75 |   215 |
|  686 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        215 |       -110 |        325 |   215 |  -110 |   -35 |   -25 |   -20 |   -65 |   -45 |   -70 |
|  687 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1070 |       1070 |  -630 |  -710 |  -870 | -1070 |  -975 |  -900 |  -885 |  -955 |
|  688 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1010 |       1010 |  -945 | -1000 |  -860 |  -920 | -1010 |  -905 |  -945 |  -890 |
|  689 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -800 |        800 |  -400 |  -535 |  -565 |  -370 |  -530 |  -570 |  -455 |  -800 |
|  690 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -505 |        505 |  -465 |  -380 |  -470 |  -505 |  -410 |  -420 |  -480 |  -480 |
|  691 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -540 |        540 |  -360 |  -335 |  -320 |  -355 |  -265 |  -430 |  -500 |  -540 |
|  692 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |        -25 |        285 |   260 |   195 |   235 |    85 |   -25 |   110 |    50 |    40 |
|  693 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        265 |        -90 |        355 |   265 |   195 |    70 |   215 |    80 |    80 |   100 |   -90 |
|  694 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        365 |          0 |        365 |   210 |   325 |   365 |   305 |   160 |    25 |    60 |   100 |
|  695 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        230 |       -115 |        345 |   230 |   165 |   -60 |  -115 |    20 |   -30 |   -70 |    50 |
|  696 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -905 |        905 |  -850 |  -780 |  -905 |  -845 |  -760 |  -845 |  -835 |  -885 |
|  697 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         10 |       -335 |        345 |    10 |  -110 |   -95 |   -75 |  -125 |  -260 |  -335 |  -265 |
|  698 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -950 |        950 |  -605 |  -585 |  -735 |  -890 |  -785 |  -805 |  -860 |  -950 |
|  699 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -440 |        440 |  -265 |  -205 |  -280 |  -250 |  -215 |  -175 |  -100 |  -440 |
|  700 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        305 |        -60 |        365 |   150 |   230 |   300 |   305 |   185 |   -60 |    40 |    50 |
|  701 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -465 |        465 |   -15 |  -215 |  -145 |  -350 |  -400 |  -465 |  -325 |  -420 |
|  702 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1090 |       1090 |  -750 |  -860 |  -955 |  -980 | -1050 | -1045 | -1040 | -1090 |
|  703 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        280 |          0 |        280 |   280 |   160 |    95 |    55 |    75 |   150 |    75 |    50 |
|  704 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        315 |          0 |        315 |   305 |   315 |    45 |   120 |    50 |    50 |   100 |    15 |
|  705 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |        -70 |        330 |   260 |    70 |   195 |   -70 |   -30 |   -25 |   -45 |   -15 |
|  706 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -905 |        905 |  -490 |  -605 |  -615 |  -505 |  -620 |  -555 |  -605 |  -905 |
|  707 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        375 |          0 |        375 |   375 |   375 |   355 |   340 |   320 |    80 |   110 |    90 |
|  708 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        235 |          0 |        235 |   185 |   105 |   215 |    75 |   115 |   225 |   230 |   235 |
|  709 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1050 |       1050 |  -690 |  -820 |  -915 | -1010 |  -950 | -1050 |  -965 |  -910 |
|  710 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        320 |          0 |        320 |   270 |   250 |   255 |   260 |   185 |   220 |   220 |   320 |
|  711 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   400 |   380 |   385 |   310 |   300 |   365 |   325 |   145 |
|  712 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -965 |        965 |  -770 |  -680 |  -780 |  -900 |  -955 |  -965 |  -920 |  -940 |
|  713 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -975 |        975 |  -720 |  -885 |  -905 |  -900 |  -800 |  -885 |  -875 |  -975 |
|  714 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1060 |       1060 |  -720 |  -825 |  -770 |  -795 |  -860 |  -820 | -1060 |  -945 |
|  715 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        215 |       -125 |        340 |   100 |   215 |   -50 |  -100 |   -25 |    25 |  -125 |    55 |
|  716 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |        -30 |        310 |   270 |   280 |   195 |   -30 |    -5 |   -30 |   115 |    15 |
|  717 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         65 |       -240 |        305 |    65 |   -25 |    65 |   -50 |   -25 |  -240 |  -185 |  -195 |
|  718 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -450 |        450 |  -165 |  -215 |  -260 |  -205 |  -335 |  -240 |  -335 |  -450 |
|  719 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        390 |          0 |        390 |   355 |   390 |   390 |   345 |   390 |   300 |   130 |   185 |
|  720 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -835 |        835 |  -660 |  -590 |  -615 |  -535 |  -835 |  -800 |  -815 |  -800 |
|  721 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -370 |        370 |  -185 |   -85 |  -120 |  -230 |  -330 |  -325 |  -370 |  -285 |
|  722 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         80 |       -190 |        270 |    60 |    40 |    50 |    45 |    80 |  -190 |   -75 |   -35 |
|  723 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -825 |        825 |  -605 |  -680 |  -570 |  -645 |  -565 |  -730 |  -825 |  -770 |
|  724 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -670 |        670 |  -475 |  -410 |  -670 |  -565 |  -620 |  -620 |  -600 |  -610 |
|  725 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        160 |       -100 |        260 |   135 |   160 |   115 |   125 |    85 |  -100 |   -50 |   -95 |
|  726 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         65 |       -175 |        240 |    65 |   -75 |  -105 |  -125 |  -170 |  -150 |  -175 |   -90 |
|  727 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |          0 |        270 |   250 |   270 |    40 |    80 |    90 |    65 |    80 |   135 |
|  728 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |          0 |        155 |    85 |   130 |   145 |   115 |   110 |   100 |   155 |   125 |
|  729 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -860 |        860 |  -820 |  -730 |  -790 |  -645 |  -725 |  -860 |  -695 |  -825 |
|  730 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        215 |        -25 |        240 |   205 |   175 |   145 |   195 |   160 |   215 |   190 |   -25 |
|  731 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        245 |          0 |        245 |   140 |   165 |   130 |   225 |   245 |   195 |   210 |    85 |
|  732 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -740 |        740 |  -460 |  -540 |  -615 |  -650 |  -630 |  -740 |  -660 |  -670 |
|  733 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        370 |          0 |        370 |   370 |   350 |   195 |   100 |   120 |   195 |   170 |   110 |
|  734 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -590 |        590 |  -355 |  -340 |  -320 |  -390 |  -205 |  -590 |  -365 |  -460 |
|  735 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        200 |        -20 |        220 |   155 |   200 |   120 |   125 |   -20 |    -5 |    -0 |    10 |
|  736 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -235 |        235 |  -210 |  -175 |  -160 |   -85 |  -185 |  -125 |  -130 |  -235 |
|  737 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |          0 |        295 |   205 |   180 |   295 |    80 |   100 |    55 |    85 |    80 |
|  738 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        340 |          0 |        340 |   340 |    85 |    85 |   130 |    30 |    80 |    20 |    80 |
|  739 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        295 |          0 |        295 |   255 |   205 |   280 |   200 |   295 |   230 |    55 |    75 |
|  740 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        325 |          0 |        325 |   325 |   265 |   175 |    65 |   150 |   210 |   215 |   185 |
|  741 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -425 |        425 |  -195 |  -155 |  -110 |  -210 |  -225 |  -310 |  -225 |  -425 |
|  742 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        325 |        -25 |        350 |   255 |   310 |   315 |   325 |    35 |    10 |    60 |   -25 |
|  743 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -425 |        425 |  -160 |  -245 |  -165 |  -230 |  -220 |  -205 |  -195 |  -425 |
|  744 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -310 |        310 |   -60 |   -25 |   -30 |  -210 |  -310 |  -270 |  -305 |  -260 |
|  745 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -550 |        550 |  -340 |  -370 |  -265 |  -220 |  -255 |  -550 |  -435 |  -525 |
|  746 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        145 |        -90 |        235 |    40 |   130 |   145 |   125 |   145 |   145 |    85 |   -90 |
|  747 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -590 |        590 |  -320 |  -360 |  -355 |  -300 |  -300 |  -365 |  -590 |  -520 |
|  748 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1200 |       1200 |  -900 | -1005 | -1050 |  -940 |  -995 | -1135 | -1070 | -1200 |
|  749 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -640 |        640 |  -540 |  -465 |  -495 |  -430 |  -430 |  -420 |  -505 |  -640 |
|  750 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        390 |         -5 |        395 |   290 |   345 |   390 |   145 |   145 |    85 |    70 |    -5 |
|  751 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        235 |          0 |        235 |   120 |   200 |   235 |   145 |   210 |    55 |   215 |   195 |
|  752 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -740 |        740 |  -465 |  -525 |  -550 |  -590 |  -565 |  -660 |  -740 |  -695 |
|  753 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        365 |          0 |        365 |   335 |   365 |   315 |    60 |    85 |    70 |   120 |   185 |
|  754 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        375 |          0 |        375 |   245 |   320 |   375 |   295 |   320 |   340 |   150 |   140 |
|  755 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         50 |       -355 |        405 |   -40 |  -100 |    50 |   -95 |  -175 |   -30 |  -295 |  -355 |
|  756 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         85 |       -190 |        275 |    85 |    10 |   -10 |   -15 |   -40 |  -190 |  -140 |  -180 |
|  757 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |          0 |        275 |   175 |   275 |   225 |   175 |   145 |   260 |   130 |    45 |
|  758 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         30 |       -215 |        245 |   -20 |   -80 |    20 |   -30 |    -5 |   -25 |    30 |  -215 |
|  759 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1160 |       1160 |  -820 |  -870 |  -995 | -1030 | -1040 | -1035 | -1160 |  -990 |
|  760 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        320 |        -35 |        355 |   225 |   320 |   280 |   275 |   230 |   245 |   -35 |   -10 |
|  761 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1155 |       1155 |  -950 |  -955 | -1005 |  -900 |  -930 |  -955 | -1025 | -1155 |
|  762 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -915 |        915 |  -605 |  -675 |  -785 |  -910 |  -865 |  -845 |  -915 |  -860 |
|  763 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -800 |        800 |  -585 |  -495 |  -640 |  -585 |  -665 |  -800 |  -730 |  -780 |
|  764 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |        -35 |        330 |   295 |   140 |   200 |    35 |   -35 |    10 |    -0 |    10 |
|  765 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        130 |        -55 |        185 |   130 |    50 |   -55 |    60 |   -15 |   -20 |   -20 |   120 |
|  766 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -545 |        545 |  -215 |  -205 |  -545 |  -460 |  -370 |  -385 |  -380 |  -440 |
|  767 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |          0 |        355 |   280 |   350 |   295 |   320 |   355 |   255 |   150 |   185 |
|  768 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        360 |          0 |        360 |   360 |   355 |   110 |   215 |   205 |   300 |   165 |   145 |
|  769 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -435 |        435 |  -190 |  -385 |  -280 |  -375 |  -400 |  -435 |  -350 |  -380 |
|  770 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        230 |          0 |        230 |   200 |   150 |   205 |   230 |   205 |   175 |   130 |    30 |
|  771 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |        -25 |        265 |   185 |   135 |    90 |   135 |   240 |   135 |   190 |   -25 |
|  772 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        330 |        -35 |        365 |   275 |   330 |   135 |   -35 |    90 |   110 |   240 |   150 |
|  773 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -815 |        815 |  -590 |  -465 |  -690 |  -815 |  -675 |  -655 |  -730 |  -755 |
|  774 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        290 |         -0 |        290 |   290 |   270 |   250 |   100 |    50 |    -0 |   135 |    60 |
|  775 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        200 |        -60 |        260 |    60 |   200 |   135 |   110 |    80 |    90 |   -45 |   -60 |
|  776 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -560 |        560 |  -490 |  -415 |  -465 |  -560 |  -455 |  -510 |  -555 |  -485 |
|  777 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -465 |        465 |  -160 |  -100 |  -315 |  -250 |  -275 |  -210 |  -270 |  -465 |
|  778 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -910 |        910 |  -695 |  -680 |  -640 |  -635 |  -910 |  -895 |  -855 |  -825 |
|  779 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1050 |       1050 |  -820 |  -775 |  -810 |  -995 | -1050 | -1005 |  -900 |  -985 |
|  780 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        235 |        -90 |        325 |   235 |    35 |    80 |   -20 |    60 |     5 |   -90 |     5 |
|  781 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -525 |        525 |  -275 |  -425 |  -445 |  -470 |  -475 |  -495 |  -480 |  -525 |
|  782 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -460 |        460 |  -175 |  -155 |  -235 |  -110 |  -320 |  -380 |  -460 |  -360 |
|  783 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -980 |        980 |  -975 |  -885 |  -980 |  -910 |  -855 |  -965 |  -885 |  -945 |
|  784 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        395 |          0 |        395 |   255 |   240 |   320 |   395 |   295 |   230 |   380 |   110 |
|  785 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        375 |          0 |        375 |   340 |   375 |   300 |   195 |   280 |   165 |    45 |   130 |
|  786 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |          0 |        360 |   360 |   310 |   295 |    90 |    35 |   155 |    55 |    90 |
|  787 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -285 |        285 |  -190 |  -130 |  -140 |  -130 |  -175 |   -75 |   -75 |  -285 |
|  788 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1055 |       1055 |  -840 |  -790 |  -730 |  -805 |  -780 |  -815 | -1055 | -1030 |
|  789 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -965 |        965 |  -775 |  -760 |  -760 |  -795 |  -740 |  -795 |  -935 |  -965 |
|  790 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        260 |        -50 |        310 |   190 |   195 |   130 |   110 |   260 |   240 |   -45 |   -50 |
|  791 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -765 |        765 |  -665 |  -630 |  -730 |  -645 |  -665 |  -665 |  -765 |  -745 |
|  792 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         10 |       -175 |        185 |   -95 |  -115 |    10 |    -0 |   -15 |  -175 |   -60 |   -60 |
|  793 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -355 |        355 |  -180 |   -65 |  -105 |   -70 |   -30 |   -75 |  -280 |  -355 |
|  794 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        225 |        -70 |        295 |    90 |   225 |   150 |   145 |   210 |   -20 |   -50 |   -70 |
|  795 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -275 |        275 |  -260 |  -175 |  -265 |  -150 |  -225 |  -275 |  -185 |  -190 |
|  796 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |          0 |        370 |   235 |   350 |   325 |   315 |   370 |   180 |   185 |   140 |
|  797 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        365 |          0 |        365 |   280 |   300 |   290 |   270 |   365 |   225 |   245 |   160 |
|  798 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         20 |       -130 |        150 |   -90 |    -0 |  -130 |   -45 |   -40 |    20 |  -105 |   -25 |
|  799 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        270 |         -5 |        275 |   160 |   140 |   270 |   255 |   240 |   150 |    75 |    -5 |
|  800 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -835 |        835 |  -715 |  -790 |  -835 |  -805 |  -780 |  -750 |  -700 |  -725 |
|  801 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -825 |        825 |  -620 |  -630 |  -660 |  -575 |  -630 |  -580 |  -825 |  -825 |
|  802 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        120 |       -175 |        295 |   120 |   -85 |    45 |    45 |    35 |  -175 |  -100 |  -165 |
|  803 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |          0 |        280 |   280 |   250 |   270 |    30 |   115 |    20 |    25 |    65 |
|  804 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        170 |       -120 |        290 |   170 |    80 |    30 |   170 |  -115 |  -120 |   -85 |   -80 |
|  805 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -290 |        290 |   -60 |   -90 |  -180 |  -155 |  -290 |  -230 |  -185 |  -225 |
|  806 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        240 |       -110 |        350 |   240 |   215 |   105 |   -10 |   -35 |   -80 |  -110 |    -0 |
|  807 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -705 |        705 |  -525 |  -490 |  -545 |  -440 |  -550 |  -485 |  -430 |  -705 |
|  808 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -405 |        405 |  -105 |  -135 |  -320 |  -405 |  -290 |  -355 |  -360 |  -320 |
|  809 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -525 |        525 |  -225 |  -315 |  -365 |  -330 |  -345 |  -315 |  -490 |  -525 |
|  810 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        340 |          0 |        340 |   300 |   320 |   270 |   325 |   335 |   235 |   340 |   335 |
|  811 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -560 |        560 |  -460 |  -465 |  -455 |  -515 |  -435 |  -535 |  -475 |  -560 |
|  812 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        295 |          0 |        295 |   190 |   295 |   125 |   180 |   115 |   105 |   160 |    75 |
|  813 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        150 |       -140 |        290 |   150 |   140 |   135 |    85 |    85 |  -140 |   -55 |   -55 |
|  814 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |        -30 |        255 |    95 |   145 |   225 |    95 |   -30 |    30 |    -5 |    10 |
|  815 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        200 |        -55 |        255 |   200 |    15 |   -55 |    15 |    -0 |   -15 |   -35 |   -50 |
|  816 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -350 |        350 |   -85 |   -30 |  -250 |  -255 |  -350 |  -280 |  -290 |  -330 |
|  817 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -435 |        435 |  -325 |  -435 |  -380 |  -340 |  -335 |  -345 |  -335 |  -410 |
|  818 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         50 |       -120 |        170 |     5 |     5 |   -20 |   -45 |    50 |    45 |  -120 |   -40 |
|  819 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -930 |        930 |  -665 |  -860 |  -725 |  -770 |  -750 |  -870 |  -755 |  -930 |
|  820 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -855 |        855 |  -575 |  -750 |  -855 |  -790 |  -725 |  -775 |  -850 |  -760 |
|  821 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        210 |       -105 |        315 |   190 |    55 |   210 |   200 |   200 |   -10 |   -50 |  -105 |
|  822 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -860 |        860 |  -580 |  -550 |  -660 |  -575 |  -570 |  -675 |  -860 |  -825 |
|  823 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        255 |        -95 |        350 |   205 |   255 |   175 |   105 |   105 |   195 |   -95 |   -35 |
|  824 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        275 |        -40 |        315 |    90 |   275 |   120 |   -35 |   -20 |    20 |   -40 |   -20 |
|  825 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1180 |       1180 |  -975 |  -950 |  -985 |  -915 | -1135 | -1140 | -1170 | -1180 |
|  826 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        300 |         -0 |        300 |   245 |   300 |   140 |    -0 |    80 |    10 |   135 |    -0 |
|  827 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |          0 |        310 |   300 |   235 |   170 |   310 |   300 |   225 |   210 |    10 |
|  828 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -900 |        900 |  -790 |  -845 |  -780 |  -790 |  -800 |  -790 |  -690 |  -900 |
|  829 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |        -40 |        280 |   210 |   240 |   220 |   150 |    90 |   115 |   125 |   -40 |
|  830 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -485 |        485 |  -275 |  -165 |  -240 |  -180 |  -215 |  -195 |  -175 |  -485 |
|  831 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |        -35 |        345 |   175 |   150 |   155 |   150 |   310 |   105 |   235 |   -35 |
|  832 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        390 |          0 |        390 |   360 |   390 |   310 |   375 |   330 |   390 |   170 |   200 |
|  833 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         45 |       -120 |        165 |    35 |   -10 |    20 |    15 |   -20 |    45 |    45 |  -120 |
|  834 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -580 |        580 |  -300 |  -270 |  -355 |  -580 |  -455 |  -445 |  -515 |  -495 |
|  835 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -235 |        235 |  -105 |  -115 |  -110 |  -135 |  -110 |  -155 |  -145 |  -235 |
|  836 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        355 |          0 |        355 |   340 |   355 |   350 |   195 |   195 |   200 |   175 |   145 |
|  837 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -595 |        595 |  -385 |  -295 |  -585 |  -520 |  -545 |  -575 |  -595 |  -470 |
|  838 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -890 |        890 |  -775 |  -625 |  -610 |  -595 |  -890 |  -795 |  -815 |  -810 |
|  839 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |        -50 |        290 |    35 |   135 |   185 |   180 |   145 |   240 |   115 |   -50 |
|  840 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -865 |        865 |  -630 |  -690 |  -865 |  -815 |  -865 |  -750 |  -815 |  -770 |
|  841 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -450 |        450 |  -150 |  -175 |  -175 |   -90 |  -170 |  -145 |  -180 |  -450 |
|  842 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        260 |         -0 |        260 |   150 |   190 |   260 |   120 |    90 |    90 |   165 |    -0 |
|  843 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1195 |       1195 | -1010 |  -830 | -1140 | -1195 | -1120 | -1100 | -1125 | -1190 |
|  844 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        290 |         -5 |        295 |   250 |   185 |   180 |   290 |   235 |    -5 |   105 |    20 |
|  845 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |        -80 |        335 |   255 |    50 |     5 |   -20 |   -10 |   -35 |    10 |   -80 |
|  846 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         85 |        -35 |        120 |    25 |   -25 |    70 |    55 |   -35 |    85 |   -15 |   -15 |
|  847 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -955 |        955 |  -710 |  -785 |  -685 |  -735 |  -730 |  -755 |  -720 |  -955 |
|  848 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |       -135 |        370 |   145 |   145 |   140 |   145 |    60 |   235 |   -60 |  -135 |
|  849 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -410 |        410 |  -150 |  -270 |  -300 |  -385 |  -410 |  -290 |  -225 |  -390 |
|  850 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -320 |        320 |    -5 |  -170 |  -120 |   -90 |   -75 |  -260 |  -320 |  -290 |
|  851 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -535 |        535 |  -330 |  -285 |  -480 |  -495 |  -520 |  -535 |  -530 |  -530 |
|  852 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -655 |        655 |  -435 |  -490 |  -485 |  -385 |  -545 |  -460 |  -655 |  -630 |
|  853 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        255 |        -80 |        335 |    25 |    20 |   215 |   155 |   255 |   175 |   -25 |   -80 |
|  854 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -630 |        630 |  -515 |  -490 |  -415 |  -435 |  -345 |  -465 |  -395 |  -630 |
|  855 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -515 |        515 |  -105 |  -485 |  -370 |  -445 |  -395 |  -380 |  -515 |  -430 |
|  856 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1090 |       1090 |  -925 |  -865 |  -835 |  -890 |  -900 | -1010 |  -910 | -1090 |
|  857 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -720 |        720 |  -420 |  -435 |  -470 |  -570 |  -640 |  -665 |  -580 |  -720 |
|  858 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -435 |        435 |  -185 |  -350 |  -340 |  -400 |  -425 |  -405 |  -435 |  -390 |
|  859 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        210 |          0 |        210 |   130 |   160 |   200 |   210 |    75 |   190 |   165 |   170 |
|  860 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -345 |        345 |   -65 |  -135 |  -130 |  -345 |  -210 |  -265 |  -315 |  -310 |
|  861 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        305 |          0 |        305 |   305 |   270 |   135 |   270 |   165 |   225 |   200 |    10 |
|  862 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1145 |       1145 |  -810 |  -770 | -1080 | -1075 | -1130 | -1145 | -1130 | -1105 |
|  863 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        230 |        -50 |        280 |   205 |   160 |   215 |   215 |   205 |   230 |   -40 |   -50 |
|  864 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -575 |        575 |  -420 |  -475 |  -350 |  -490 |  -350 |  -430 |  -495 |  -575 |
|  865 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -525 |        525 |  -355 |  -395 |  -375 |  -280 |  -435 |  -305 |  -485 |  -525 |
|  866 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        230 |        -20 |        250 |   185 |   140 |   200 |   150 |   230 |   180 |   125 |   -20 |
|  867 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1015 |       1015 |  -805 |  -715 |  -845 |  -830 |  -850 | -1005 |  -970 | -1015 |
|  868 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -345 |        345 |  -105 |  -180 |   -65 |  -205 |   -85 |  -155 |  -125 |  -345 |
|  869 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -770 |        770 |  -440 |  -485 |  -585 |  -525 |  -635 |  -430 |  -455 |  -770 |
|  870 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -330 |        330 |   -70 |   -60 |   -65 |  -125 |   -85 |  -160 |  -280 |  -330 |
|  871 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -745 |        745 |  -595 |  -645 |  -745 |  -570 |  -605 |  -640 |  -645 |  -635 |
|  872 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -565 |        565 |  -240 |  -345 |  -305 |  -425 |  -565 |  -450 |  -485 |  -510 |
|  873 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -925 |        925 |  -685 |  -745 |  -655 |  -745 |  -665 |  -735 |  -755 |  -925 |
|  874 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -675 |        675 |  -475 |  -405 |  -490 |  -455 |  -435 |  -405 |  -670 |  -675 |
|  875 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -320 |        320 |  -165 |   -70 |  -105 |  -125 |   -60 |  -320 |  -305 |  -310 |
|  876 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -415 |        415 |  -145 |  -245 |  -135 |  -355 |  -415 |  -410 |  -390 |  -335 |
|  877 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -630 |        630 |  -470 |  -530 |  -470 |  -530 |  -445 |  -460 |  -420 |  -630 |
|  878 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -450 |        450 |  -165 |  -290 |  -325 |  -200 |  -215 |  -190 |  -235 |  -450 |
|  879 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        345 |          0 |        345 |   330 |   340 |   345 |   205 |   125 |   170 |   220 |   170 |
|  880 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        255 |        -20 |        275 |   235 |   255 |   255 |   -20 |   125 |    90 |    95 |   150 |
|  881 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -875 |        875 |  -790 |  -805 |  -800 |  -875 |  -850 |  -800 |  -870 |  -870 |
|  882 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -210 |        210 |   -70 |   -10 |   -45 |   -70 |   -15 |  -175 |  -110 |  -210 |
|  883 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -860 |        860 |  -680 |  -635 |  -765 |  -565 |  -680 |  -630 |  -860 |  -815 |
|  884 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        255 |        -15 |        270 |   255 |   180 |   230 |    15 |    85 |     5 |   -15 |   155 |
|  885 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -630 |        630 |  -385 |  -440 |  -430 |  -385 |  -375 |  -515 |  -630 |  -530 |
|  886 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1075 |       1075 |  -785 |  -835 | -1000 | -1075 |  -985 | -1020 |  -975 | -1045 |
|  887 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        400 |          0 |        400 |   320 |   400 |   375 |   280 |   300 |   220 |   295 |   260 |
|  888 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1090 |       1090 |  -710 |  -695 |  -735 |  -825 |  -970 | -1040 | -1090 | -1090 |
|  889 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1035 |       1035 |  -910 |  -845 |  -790 |  -920 | -1035 |  -950 | -1025 |  -920 |
|  890 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        350 |          0 |        350 |   215 |   270 |   350 |    35 |   125 |    80 |    75 |    50 |
|  891 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        365 |          0 |        365 |   365 |   145 |   105 |   160 |   135 |   185 |   240 |    85 |
|  892 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        190 |        -50 |        240 |   190 |    60 |   120 |   185 |   105 |   130 |   165 |   -50 |
|  893 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        160 |       -150 |        310 |    60 |   160 |    95 |   135 |  -120 |   -55 |  -150 |  -135 |
|  894 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        300 |         -5 |        305 |   165 |   245 |   265 |   300 |   285 |    30 |    -5 |    95 |
|  895 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -765 |        765 |  -475 |  -480 |  -505 |  -560 |  -590 |  -700 |  -740 |  -765 |
|  896 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        255 |          0 |        255 |   240 |   255 |   220 |   175 |   205 |    75 |    80 |    90 |
|  897 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        235 |        -90 |        325 |   235 |   220 |    30 |   -90 |   -60 |    -0 |   -25 |    95 |
|  898 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1085 |       1085 |  -890 |  -790 | -1035 |  -995 | -1030 | -1085 | -1045 | -1045 |
|  899 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        200 |        -90 |        290 |    15 |   155 |   130 |    50 |   145 |   200 |   -90 |   -65 |
|  900 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -490 |        490 |  -465 |  -420 |  -375 |  -465 |  -490 |  -415 |  -480 |  -485 |
|  901 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1150 |       1150 |  -940 |  -920 |  -990 |  -950 |  -905 |  -985 | -1140 | -1150 |
|  902 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        105 |       -240 |        345 |   105 |   -35 |   -30 |  -220 |  -175 |  -190 |  -200 |  -240 |
|  903 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        100 |       -105 |        205 |   100 |    90 |    50 |   -65 |    20 |  -105 |  -100 |   -85 |
|  904 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1145 |       1145 |  -925 | -1120 | -1125 | -1075 | -1035 | -1145 | -1130 | -1020 |
|  905 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1020 |       1020 |  -580 |  -700 |  -725 |  -740 |  -890 |  -890 |  -930 | -1020 |
|  906 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -875 |        875 |  -655 |  -685 |  -610 |  -730 |  -865 |  -825 |  -875 |  -710 |
|  907 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -620 |        620 |  -330 |  -365 |  -280 |  -375 |  -405 |  -525 |  -550 |  -620 |
|  908 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -505 |        505 |  -375 |  -300 |  -370 |  -345 |  -475 |  -465 |  -485 |  -505 |
|  909 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1195 |       1195 |  -975 | -1095 |  -995 | -1015 |  -930 | -1130 | -1030 | -1195 |
|  910 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        205 |       -150 |        355 |   125 |   105 |   195 |   205 |    85 |   185 |   135 |  -150 |
|  911 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -655 |        655 |  -445 |  -435 |  -425 |  -410 |  -655 |  -530 |  -530 |  -490 |
|  912 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -870 |        870 |  -720 |  -745 |  -745 |  -870 |  -760 |  -780 |  -700 |  -845 |
|  913 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         70 |       -215 |        285 |    55 |    60 |    45 |    70 |    70 |    70 |    10 |  -215 |
|  914 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -330 |        330 |  -110 |  -145 |  -115 |   -95 |   -70 |  -195 |  -330 |  -275 |
|  915 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        400 |          0 |        400 |   360 |   325 |   400 |   110 |   175 |   150 |   210 |   225 |
|  916 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -445 |        445 |  -165 |  -105 |  -220 |  -150 |  -115 |  -280 |  -445 |  -430 |
|  917 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -835 |        835 |  -590 |  -515 |  -770 |  -685 |  -670 |  -810 |  -835 |  -745 |
|  918 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -590 |        590 |  -290 |  -350 |  -370 |  -465 |  -545 |  -590 |  -540 |  -450 |
|  919 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         -0 |       -240 |        240 |    -0 |   -30 |   -55 |  -135 |  -220 |  -240 |  -190 |  -175 |
|  920 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        340 |          0 |        340 |   260 |   170 |   255 |   245 |   340 |   245 |   260 |   115 |
|  921 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1140 |       1140 |  -805 | -1015 | -1085 | -1005 | -1065 | -1140 | -1065 |  -975 |
|  922 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -355 |        355 |  -110 |  -155 |   -65 |   -35 |  -355 |  -225 |  -245 |  -280 |
|  923 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        355 |          0 |        355 |   280 |   250 |   305 |   320 |   355 |   160 |   125 |   205 |
|  924 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        140 |        -95 |        235 |    50 |   -95 |    70 |    -0 |    95 |   140 |   -50 |    75 |
|  925 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   310 |   260 |   280 |   230 |   400 |   275 |   245 |   100 |
|  926 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        330 |          0 |        330 |   330 |   155 |   125 |   110 |   125 |   160 |   215 |    60 |
|  927 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -920 |        920 |  -600 |  -700 |  -600 |  -610 |  -645 |  -790 |  -920 |  -825 |
|  928 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        245 |        -65 |        310 |   175 |   160 |   230 |   225 |   245 |   -30 |    15 |   -65 |
|  929 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        375 |          0 |        375 |   370 |   375 |   305 |    45 |   200 |   170 |   160 |   215 |
|  930 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -665 |        665 |  -480 |  -465 |  -590 |  -480 |  -540 |  -485 |  -545 |  -665 |
|  931 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -895 |        895 |  -780 |  -740 |  -735 |  -555 |  -755 |  -625 |  -765 |  -895 |
|  932 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -615 |        615 |  -245 |  -380 |  -420 |  -310 |  -400 |  -415 |  -410 |  -615 |
|  933 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1040 |       1040 |  -745 |  -750 |  -815 |  -785 |  -895 |  -770 |  -795 | -1040 |
|  934 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        380 |          0 |        380 |   275 |   380 |   355 |   310 |   255 |    10 |   130 |   100 |
|  935 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        150 |       -120 |        270 |   150 |  -100 |   -50 |  -100 |   -45 |   -60 |  -120 |  -105 |
|  936 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -960 |        960 |  -655 |  -705 |  -685 |  -735 |  -840 |  -830 |  -960 |  -890 |
|  937 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        365 |        -10 |        375 |   365 |   150 |   185 |   175 |    90 |    15 |    10 |   -10 |
|  938 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1130 |       1130 |  -975 |  -890 |  -945 |  -965 |  -925 |  -960 |  -910 | -1130 |
|  939 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -440 |        440 |  -230 |  -130 |  -150 |  -440 |  -405 |  -360 |  -325 |  -345 |
|  940 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        165 |        -65 |        230 |   165 |   -25 |    45 |   -65 |    55 |    35 |    35 |   -50 |
|  941 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -655 |        655 |  -555 |  -655 |  -620 |  -650 |  -600 |  -500 |  -655 |  -595 |
|  942 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        310 |          0 |        310 |   255 |   280 |   310 |    60 |   150 |    95 |   125 |    15 |
|  943 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -280 |        280 |  -190 |  -130 |  -155 |   -50 |  -105 |  -215 |  -190 |  -280 |
|  944 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -510 |        510 |  -230 |  -290 |  -305 |  -320 |  -270 |  -340 |  -310 |  -510 |
|  945 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        135 |       -130 |        265 |   135 |   -10 |    25 |   -15 |   -95 |   -85 |   -85 |  -130 |
|  946 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -535 |        535 |  -305 |  -320 |  -295 |  -270 |  -500 |  -535 |  -455 |  -415 |
|  947 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        325 |          0 |        325 |   325 |   285 |   145 |   155 |   125 |   175 |   195 |   225 |
|  948 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -505 |        505 |  -325 |  -245 |  -220 |  -245 |  -505 |  -480 |  -455 |  -490 |
|  949 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        240 |        -85 |        325 |   220 |   155 |   240 |   170 |    80 |    65 |   -85 |   -15 |
|  950 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        250 |          0 |        250 |   190 |   205 |   150 |   250 |   175 |   150 |   215 |    25 |
|  951 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -885 |        885 |  -625 |  -600 |  -650 |  -635 |  -630 |  -585 |  -885 |  -850 |
|  952 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        180 |        -80 |        260 |   180 |   110 |   110 |   -15 |   -50 |   -65 |   -80 |    25 |
|  953 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1015 |       1015 |  -820 |  -790 |  -815 |  -830 |  -890 |  -795 |  -995 | -1015 |
|  954 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -745 |        745 |  -520 |  -450 |  -555 |  -630 |  -745 |  -710 |  -670 |  -725 |
|  955 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -395 |        395 |  -160 |  -105 |  -125 |   -90 |  -110 |   -95 |  -190 |  -395 |
|  956 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -520 |        520 |  -250 |  -300 |  -310 |  -325 |  -225 |  -310 |  -285 |  -520 |
|  957 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |        -35 |        235 |   140 |    85 |   105 |   175 |   200 |   115 |   115 |   -35 |
|  958 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -615 |        615 |  -455 |  -410 |  -505 |  -515 |  -525 |  -505 |  -550 |  -615 |
|  959 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -545 |        545 |  -235 |  -170 |  -240 |  -265 |  -240 |  -215 |  -225 |  -545 |
|  960 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1200 |       1200 |  -970 | -1065 | -1040 | -1175 | -1115 | -1145 | -1200 | -1165 |
|  961 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         90 |       -205 |        295 |   -15 |    85 |   -15 |   -15 |    90 |   -10 |   -40 |  -205 |
|  962 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -260 |        260 |   -70 |   -25 |  -135 |  -230 |  -165 |  -110 |   -60 |  -260 |
|  963 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        325 |          0 |        325 |   300 |   325 |   200 |   240 |    95 |   105 |   165 |   125 |
|  964 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -480 |        480 |  -350 |  -285 |  -355 |  -295 |  -360 |  -290 |  -480 |  -480 |
|  965 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -710 |        710 |  -430 |  -460 |  -710 |  -650 |  -590 |  -630 |  -595 |  -520 |
|  966 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -680 |        680 |  -420 |  -260 |  -345 |  -680 |  -600 |  -610 |  -625 |  -605 |
|  967 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -925 |        925 |  -785 |  -885 |  -835 |  -805 |  -860 |  -865 |  -890 |  -925 |
|  968 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        210 |       -190 |        400 |   210 |   150 |   150 |   115 |   -55 |  -190 |   -95 |    10 |
|  969 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -690 |        690 |  -670 |  -565 |  -690 |  -615 |  -675 |  -630 |  -685 |  -685 |
|  970 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        210 |          0 |        210 |   105 |   115 |   125 |   155 |   135 |   115 |    70 |   210 |
|  971 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        335 |          0 |        335 |   335 |   205 |   245 |   320 |   235 |   220 |   295 |   155 |
|  972 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1070 |       1070 |  -795 |  -960 |  -910 |  -960 |  -945 | -1030 | -1070 |  -920 |
|  973 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         90 |        -10 |        100 |    60 |    -0 |   -10 |    60 |    50 |    90 |    70 |    70 |
|  974 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        190 |        -65 |        255 |   190 |    45 |    45 |     5 |   -65 |    15 |    -0 |    75 |
|  975 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -595 |        595 |  -300 |  -405 |  -310 |  -315 |  -240 |  -315 |  -285 |  -595 |
|  976 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        375 |          0 |        375 |   210 |   375 |   210 |   290 |   310 |   325 |    20 |    85 |
|  977 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -630 |        630 |  -555 |  -540 |  -550 |  -545 |  -535 |  -545 |  -480 |  -630 |
|  978 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -785 |        785 |  -785 |  -700 |  -705 |  -700 |  -760 |  -700 |  -690 |  -735 |
|  979 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -435 |        435 |  -190 |  -280 |  -250 |  -125 |  -180 |  -390 |  -385 |  -435 |
|  980 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -860 |        860 |  -750 |  -780 |  -860 |  -825 |  -855 |  -835 |  -835 |  -815 |
|  981 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        325 |         -0 |        325 |   325 |    95 |    70 |    95 |   185 |    -0 |    75 |   160 |
|  982 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1030 |       1030 |  -895 |  -845 |  -940 | -1000 |  -905 |  -900 | -1030 |  -920 |
|  983 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        355 |          0 |        355 |   355 |   350 |   255 |   290 |   175 |   330 |   190 |   195 |
|  984 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        395 |        -25 |        420 |   320 |   260 |   320 |   325 |   350 |   340 |   395 |   -25 |
|  985 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        340 |          0 |        340 |   265 |   310 |   285 |   255 |   265 |   340 |   185 |    75 |
|  986 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1070 |       1070 |  -850 |  -870 | -1010 | -1020 | -1070 | -1070 | -1060 |  -990 |
|  987 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |       -105 |        320 |   110 |   150 |    85 |   215 |   160 |   190 |   -65 |  -105 |
|  988 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1025 |       1025 |  -815 |  -790 |  -865 |  -760 |  -820 | -1010 | -1025 |  -925 |
|  989 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         75 |       -235 |        310 |    45 |    35 |    75 |  -200 |  -160 |  -110 |  -150 |  -235 |
|  990 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        215 |       -160 |        375 |   160 |   215 |   155 |    95 |    30 |   -25 |  -160 |    10 |
|  991 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1180 |       1180 |  -850 |  -865 |  -760 |  -955 | -1180 |  -995 | -1085 |  -955 |
|  992 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        265 |       -135 |        400 |   200 |   180 |   205 |   265 |   225 |  -135 |    85 |    -5 |
|  993 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         65 |       -125 |        190 |   -15 |   -55 |    65 |  -125 |    -5 |   -75 |    -5 |    30 |
|  994 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        365 |          0 |        365 |   350 |   295 |   365 |   315 |   155 |   165 |   125 |    70 |
|  995 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -750 |        750 |  -565 |  -505 |  -485 |  -505 |  -510 |  -575 |  -715 |  -750 |
|  996 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -250 |        250 |   -70 |  -115 |  -250 |  -125 |  -190 |   -70 |  -110 |  -105 |
|  997 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -255 |        255 |   -90 |  -215 |  -255 |  -255 |  -225 |  -215 |  -230 |  -230 |
|  998 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        100 |       -180 |        280 |   100 |    10 |   -50 |    -5 |   -30 |    35 |     5 |  -180 |
|  999 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        305 |          0 |        305 |   235 |   250 |   305 |   125 |    30 |    80 |   155 |   155 |
| 1000 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -980 |        980 |  -605 |  -740 |  -650 |  -745 |  -905 |  -980 |  -835 |  -855 |
| 1001 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |          0 |        205 |   100 |   205 |   185 |     5 |    65 |    70 |    25 |   180 |
| 1002 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        370 |          0 |        370 |   370 |   325 |   310 |   330 |   370 |   255 |    75 |    55 |
| 1003 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        225 |          0 |        225 |   170 |   190 |   155 |    65 |   225 |   190 |     5 |    10 |
| 1004 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        120 |       -190 |        310 |   120 |  -190 |   -75 |  -100 |  -110 |   -95 |  -145 |   -10 |
| 1005 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -950 |        950 |  -775 |  -950 |  -835 |  -870 |  -735 |  -845 |  -800 |  -765 |
| 1006 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        390 |         -0 |        390 |   355 |   390 |   295 |   115 |    20 |    90 |    95 |    -0 |
| 1007 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1040 |       1040 |  -890 | -1040 |  -980 | -1000 | -1005 |  -935 | -1020 | -1015 |
| 1008 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |         -0 |        360 |   360 |   245 |   340 |    -0 |    60 |    45 |    -0 |    35 |
| 1009 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        125 |         -0 |        125 |    45 |   105 |    50 |    -0 |    -0 |    25 |    95 |   125 |
| 1010 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1035 |       1035 |  -800 |  -765 | -1025 |  -970 | -1035 |  -980 | -1010 |  -935 |
| 1011 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -975 |        975 |  -665 |  -655 |  -835 |  -760 |  -640 |  -860 |  -975 |  -950 |
| 1012 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -885 |        885 |  -640 |  -645 |  -610 |  -650 |  -540 |  -850 |  -845 |  -885 |
| 1013 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        165 |       -210 |        375 |   110 |   110 |   165 |  -105 |  -175 |  -210 |   -15 |   -25 |
| 1014 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        285 |          0 |        285 |   200 |   205 |   205 |   285 |   285 |   285 |    35 |    60 |
| 1015 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        300 |       -135 |        435 |    60 |   300 |    15 |   -10 |     5 |    -0 |   -45 |  -135 |
| 1016 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -715 |        715 |  -455 |  -410 |  -440 |  -630 |  -640 |  -715 |  -590 |  -635 |
| 1017 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        270 |         -0 |        270 |   225 |   260 |   270 |   230 |   210 |   240 |    -0 |    25 |
| 1018 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        290 |       -145 |        435 |   -15 |   150 |   185 |   135 |   290 |  -145 |    -0 |   -80 |
| 1019 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -660 |        660 |  -335 |  -420 |  -395 |  -345 |  -405 |  -350 |  -480 |  -660 |
| 1020 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1110 |       1110 | -1005 |  -995 |  -985 |  -950 |  -980 | -1110 | -1050 |  -955 |
| 1021 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        310 |         -0 |        310 |   295 |   220 |   310 |    15 |    -0 |    75 |    15 |    50 |
| 1022 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -730 |        730 |  -425 |  -475 |  -500 |  -495 |  -505 |  -490 |  -730 |  -690 |
| 1023 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         75 |       -255 |        330 |   -65 |    75 |   -25 |  -255 |  -130 |  -210 |  -215 |  -225 |
| 1024 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        395 |          0 |        395 |   395 |   145 |   275 |   210 |   180 |   145 |   100 |   130 |
| 1025 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |          0 |        370 |   275 |   290 |   315 |   370 |   275 |    55 |   130 |    55 |
| 1026 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -840 |        840 |  -660 |  -555 |  -795 |  -795 |  -760 |  -840 |  -785 |  -835 |
| 1027 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -330 |        330 |   -85 |  -255 |  -210 |  -270 |  -300 |  -280 |  -240 |  -330 |
| 1028 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        270 |       -125 |        395 |   215 |   225 |   260 |   205 |   235 |   270 |  -125 |   140 |
| 1029 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        290 |        -10 |        300 |   225 |   240 |   280 |   290 |   105 |    55 |   -10 |    95 |
| 1030 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -995 |        995 |  -730 |  -710 |  -610 |  -625 |  -570 |  -875 |  -995 |  -815 |
| 1031 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -820 |        820 |  -565 |  -590 |  -575 |  -820 |  -785 |  -760 |  -790 |  -720 |
| 1032 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        340 |          0 |        340 |   340 |   220 |   240 |   290 |   305 |    95 |    95 |    40 |
| 1033 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        255 |        -65 |        320 |   250 |   205 |   135 |   255 |   250 |   -15 |    35 |   -65 |
| 1034 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1090 |       1090 | -1085 | -1015 | -1075 |  -995 | -1065 | -1090 |  -990 | -1080 |
| 1035 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1150 |       1150 |  -800 |  -870 |  -845 |  -985 | -1030 |  -945 |  -945 | -1150 |
| 1036 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -940 |        940 |  -760 |  -715 |  -775 |  -725 |  -820 |  -710 |  -735 |  -940 |
| 1037 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -960 |        960 |  -755 |  -745 |  -650 |  -715 |  -870 |  -960 |  -935 |  -930 |
| 1038 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -985 |        985 |  -755 |  -720 |  -675 |  -875 |  -730 |  -910 |  -985 |  -950 |
| 1039 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -735 |        735 |  -430 |  -455 |  -530 |  -415 |  -620 |  -635 |  -670 |  -735 |
| 1040 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        280 |        -20 |        300 |   205 |   280 |   220 |   180 |    75 |   -20 |    40 |    40 |
| 1041 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -535 |        535 |  -495 |  -535 |  -490 |  -480 |  -470 |  -445 |  -510 |  -480 |
| 1042 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1080 |       1080 |  -865 |  -820 |  -940 |  -875 |  -830 | -1080 |  -960 |  -990 |
| 1043 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -645 |        645 |  -315 |  -390 |  -435 |  -645 |  -615 |  -575 |  -535 |  -635 |
| 1044 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        315 |          0 |        315 |   315 |    35 |    60 |    60 |   105 |    90 |   115 |   120 |
| 1045 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1060 |       1060 | -1060 |  -975 |  -940 | -1040 | -1015 | -1055 |  -960 | -1000 |
| 1046 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1090 |       1090 |  -735 | -1025 |  -970 |  -995 | -1035 |  -915 | -1090 |  -905 |
| 1047 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -905 |        905 |  -905 |  -815 |  -830 |  -750 |  -865 |  -855 |  -815 |  -805 |
| 1048 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -820 |        820 |  -520 |  -585 |  -670 |  -560 |  -700 |  -820 |  -735 |  -790 |
| 1049 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -470 |        470 |  -270 |  -250 |  -245 |  -440 |  -405 |  -465 |  -470 |  -400 |
| 1050 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        205 |        -95 |        300 |   200 |    85 |   100 |    95 |   185 |   140 |   205 |   -95 |
| 1051 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1170 |       1170 |  -905 |  -795 |  -860 |  -845 |  -880 | -1095 | -1090 | -1170 |
| 1052 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -590 |        590 |  -485 |  -460 |  -590 |  -555 |  -525 |  -510 |  -515 |  -455 |
| 1053 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -430 |        430 |  -380 |  -295 |  -245 |  -275 |  -180 |  -270 |  -185 |  -430 |
| 1054 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -950 |        950 |  -760 |  -825 |  -710 |  -705 |  -915 |  -875 |  -945 |  -950 |
| 1055 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        360 |          0 |        360 |   360 |   295 |    90 |   230 |   260 |   185 |   255 |   185 |
| 1056 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        180 |       -100 |        280 |   180 |   160 |   145 |   150 |   155 |   125 |  -100 |    25 |
| 1057 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        350 |         -0 |        350 |   290 |   350 |   100 |    60 |    -0 |    70 |    50 |   110 |
| 1058 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |         -0 |        350 |   345 |   310 |   220 |   270 |   225 |   350 |   170 |    -0 |
| 1059 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        230 |       -115 |        345 |   150 |   140 |   100 |   230 |   230 |   230 |   -40 |  -115 |
| 1060 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        185 |       -180 |        365 |   115 |   125 |   160 |   115 |   185 |    75 |  -145 |  -180 |
| 1061 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -650 |        650 |  -570 |  -650 |  -555 |  -480 |  -545 |  -565 |  -540 |  -585 |
| 1062 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         55 |       -285 |        340 |   -10 |    -5 |    10 |   -80 |    35 |    55 |   -70 |  -285 |
| 1063 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        360 |          0 |        360 |   275 |   160 |   360 |   360 |   250 |    40 |    65 |   185 |
| 1064 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -670 |        670 |  -400 |  -285 |  -405 |  -585 |  -560 |  -575 |  -670 |  -630 |
| 1065 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -500 |        500 |  -270 |  -270 |  -390 |  -200 |  -370 |  -500 |  -395 |  -495 |
| 1066 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -975 |        975 |  -850 |  -860 |  -975 |  -860 |  -830 |  -895 |  -975 |  -960 |
| 1067 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -415 |        415 |   -55 |  -170 |   -80 |  -150 |  -130 |  -130 |  -120 |  -415 |
| 1068 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        110 |       -185 |        295 |    -5 |   105 |   110 |   -45 |    45 |   -10 |  -160 |  -185 |
| 1069 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |        -55 |        330 |   155 |   120 |   270 |   180 |   275 |   155 |   190 |   -55 |
| 1070 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1135 |       1135 |  -730 |  -925 |  -935 |  -940 |  -860 |  -930 | -1080 | -1135 |
| 1071 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        180 |       -115 |        295 |   180 |    40 |    10 |    60 |  -115 |   -15 |   -15 |    10 |
| 1072 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        395 |          0 |        395 |   225 |   255 |   310 |   225 |   280 |   260 |   395 |   245 |
| 1073 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1095 |       1095 |  -890 |  -880 |  -880 |  -880 |  -875 |  -985 | -1095 | -1050 |
| 1074 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        370 |          0 |        370 |   325 |   370 |   295 |   220 |    70 |    45 |   110 |   105 |
| 1075 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        280 |        -10 |        290 |   160 |   280 |   185 |   250 |   260 |   155 |   -10 |    50 |
| 1076 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        365 |          0 |        365 |   230 |   305 |   230 |   365 |   310 |   365 |    35 |    50 |
| 1077 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -975 |        975 |  -965 |  -950 |  -975 |  -950 |  -850 |  -880 |  -855 |  -895 |
| 1078 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -375 |        375 |  -185 |  -105 |  -180 |  -320 |  -300 |  -375 |  -335 |  -295 |
| 1079 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -865 |        865 |  -670 |  -785 |  -765 |  -865 |  -705 |  -750 |  -855 |  -805 |
| 1080 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |          0 |        275 |   255 |   155 |   270 |   260 |   185 |   205 |   275 |    75 |
| 1081 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -460 |        460 |  -150 |  -165 |  -360 |  -415 |  -460 |  -430 |  -355 |  -425 |
| 1082 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -450 |        450 |  -245 |  -270 |  -330 |  -340 |  -250 |  -375 |  -280 |  -450 |
| 1083 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |       -135 |        390 |   255 |   -90 |   -20 |     5 |    20 |   -10 |  -135 |   -55 |
| 1084 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         55 |       -125 |        180 |    50 |    55 |   -65 |   -30 |   -55 |  -125 |    45 |    -5 |
| 1085 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1075 |       1075 |  -795 |  -730 |  -745 |  -720 |  -745 |  -810 |  -765 | -1075 |
| 1086 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -325 |        325 |  -130 |  -255 |  -145 |   -65 |   -45 |   -60 |  -325 |  -215 |
| 1087 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        395 |      -1165 |       1560 | -1165 |   325 |   395 |   310 |   285 |   225 |   320 |   225 |
| 1088 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -595 |        595 |  -505 |  -280 |  -415 |  -305 |  -340 |  -590 |  -595 |  -545 |
| 1089 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -835 |        835 |  -580 |  -560 |  -700 |  -630 |  -580 |  -610 |  -835 |  -825 |
| 1090 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        385 |          0 |        385 |   300 |   385 |   350 |    85 |   145 |    95 |   210 |   130 |
| 1091 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        215 |        -40 |        255 |   200 |   165 |   100 |   215 |    15 |   -40 |    -0 |   -25 |
| 1092 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -530 |        530 |  -285 |  -420 |  -380 |  -375 |  -405 |  -365 |  -430 |  -530 |
| 1093 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        265 |        -75 |        340 |   265 |   175 |   190 |   155 |    -0 |   -75 |   -10 |   -25 |
| 1094 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -785 |        785 |  -510 |  -545 |  -455 |  -560 |  -560 |  -785 |  -785 |  -695 |
| 1095 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -465 |        465 |  -130 |  -150 |  -275 |  -195 |   -75 |   -30 |  -370 |  -465 |
| 1096 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        365 |          0 |        365 |   365 |   350 |   205 |   230 |   125 |   290 |   245 |   210 |
| 1097 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        290 |       -120 |        410 |   290 |    25 |  -120 |   -60 |    40 |   125 |   -95 |   -20 |
| 1098 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          5 |       -330 |        335 |   -10 |   -35 |  -120 |    -0 |     5 |   -40 |   -35 |  -330 |
| 1099 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        355 |          0 |        355 |   240 |   340 |   265 |   355 |   105 |   205 |   110 |   220 |
| 1100 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |          0 |        260 |   230 |   260 |   230 |    70 |     5 |    55 |    25 |    85 |
| 1101 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         35 |        -70 |        105 |    35 |   -35 |    25 |    35 |    -0 |   -70 |    25 |    10 |
| 1102 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1080 |       1080 |  -875 |  -635 | -1060 | -1010 | -1075 | -1030 | -1035 | -1080 |
| 1103 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         45 |        -40 |         85 |    45 |    20 |    15 |   -30 |   -10 |   -30 |    20 |   -40 |
| 1104 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        340 |          0 |        340 |   340 |   315 |   195 |    80 |   120 |   170 |   185 |   255 |
| 1105 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        305 |       -110 |        415 |   225 |   305 |    15 |    50 |  -110 |   -65 |   -35 |    80 |
| 1106 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        225 |       -130 |        355 |   225 |   215 |   100 |   -65 |    10 |  -130 |    65 |    -5 |
| 1107 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        280 |        -75 |        355 |   280 |   230 |   230 |   110 |   -75 |    25 |    45 |   -30 |
| 1108 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          5 |       -170 |        175 |    -5 |     5 |   -80 |   -30 |   -45 |  -170 |  -130 |  -155 |
| 1109 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -330 |        330 |  -295 |  -230 |  -120 |  -310 |  -325 |  -195 |  -330 |  -260 |
| 1110 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -880 |        880 |  -635 |  -640 |  -675 |  -595 |  -770 |  -875 |  -830 |  -880 |
| 1111 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        130 |       -240 |        370 |    95 |    55 |   130 |  -200 |  -130 |   -75 |  -240 |  -130 |
| 1112 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        330 |          0 |        330 |   275 |   260 |   130 |   330 |   290 |    50 |    25 |   125 |
| 1113 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -465 |        465 |  -130 |  -275 |  -260 |  -300 |  -145 |  -270 |  -465 |  -360 |
| 1114 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1120 |       1120 | -1020 |  -925 |  -965 |  -995 |  -925 |  -875 |  -840 | -1120 |
| 1115 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -970 |        970 |  -945 |  -890 |  -955 |  -890 |  -955 |  -930 |  -970 |  -865 |
| 1116 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        365 |          0 |        365 |   365 |   150 |   255 |   250 |   200 |   280 |    55 |   140 |
| 1117 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1185 |       1185 | -1015 |  -980 | -1115 | -1135 | -1145 | -1145 | -1160 | -1185 |
| 1118 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -450 |        450 |  -210 |   -70 |  -245 |  -250 |  -405 |  -450 |  -275 |  -400 |
| 1119 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        165 |       -120 |        285 |    25 |   115 |    65 |   165 |    95 |  -120 |  -120 |   -60 |
| 1120 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        340 |          0 |        340 |   215 |   315 |   320 |   280 |   310 |   340 |   280 |   165 |
| 1121 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -370 |        370 |  -305 |  -330 |  -370 |  -330 |  -335 |  -365 |  -365 |  -345 |
| 1122 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -330 |        330 |  -120 |  -330 |  -220 |  -270 |  -210 |  -275 |  -205 |  -280 |
| 1123 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -785 |        785 |  -545 |  -470 |  -685 |  -785 |  -705 |  -660 |  -770 |  -740 |
| 1124 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |       -115 |        330 |   145 |   140 |   160 |   135 |    80 |   215 |  -115 |   -15 |
| 1125 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        260 |         -0 |        260 |   165 |   210 |   250 |   240 |   260 |     5 |    50 |    -0 |
| 1126 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -990 |        990 |  -680 |  -905 |  -990 |  -965 |  -950 |  -980 |  -935 |  -980 |
| 1127 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -485 |        485 |  -415 |  -395 |  -395 |  -455 |  -465 |  -445 |  -390 |  -485 |
| 1128 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        250 |        -55 |        305 |   175 |   250 |   -35 |    -5 |   -15 |    25 |   -55 |    80 |
| 1129 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        290 |          0 |        290 |   255 |   275 |   220 |   290 |   230 |   225 |   270 |   230 |
| 1130 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -655 |        655 |  -475 |  -625 |  -645 |  -630 |  -645 |  -655 |  -570 |  -585 |
| 1131 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        380 |          0 |        380 |   245 |   380 |   330 |   325 |   340 |   155 |   130 |   125 |
| 1132 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        290 |          0 |        290 |   245 |   250 |   195 |   260 |   240 |   290 |   255 |    20 |
| 1133 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -505 |        505 |  -290 |  -270 |  -215 |  -180 |  -185 |  -215 |  -190 |  -505 |
| 1134 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -520 |        520 |  -290 |  -230 |  -330 |  -265 |  -300 |  -465 |  -485 |  -520 |
| 1135 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        270 |        -35 |        305 |   260 |   230 |   230 |   270 |    95 |   -35 |   125 |    40 |
| 1136 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -555 |        555 |  -220 |  -250 |  -295 |  -305 |  -545 |  -555 |  -465 |  -515 |
| 1137 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        325 |          0 |        325 |   325 |   310 |   220 |   105 |   270 |   185 |   205 |   235 |
| 1138 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        290 |          0 |        290 |   290 |   130 |   150 |   145 |    65 |    55 |   125 |    75 |
| 1139 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -875 |        875 |  -690 |  -555 |  -645 |  -550 |  -680 |  -655 |  -670 |  -875 |
| 1140 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        355 |          0 |        355 |   355 |   355 |   340 |   350 |   150 |   230 |   215 |   185 |
| 1141 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        220 |        -90 |        310 |   175 |   220 |   170 |    15 |   -90 |    25 |    85 |   -45 |
| 1142 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        175 |          0 |        175 |   175 |   110 |    25 |    65 |    35 |    35 |    40 |    10 |
| 1143 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1110 |       1110 | -1075 |  -890 |  -835 |  -835 |  -920 |  -945 |  -940 | -1110 |
| 1144 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |        -55 |        270 |   105 |   215 |   205 |   175 |   195 |   165 |   -55 |   -45 |
| 1145 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        235 |       -120 |        355 |   235 |   -25 |   -45 |   -40 |  -120 |   -10 |     5 |   -25 |
| 1146 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        380 |          0 |        380 |   320 |   320 |   265 |   380 |   125 |    70 |    90 |   225 |
| 1147 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1065 |       1065 |  -790 |  -905 |  -810 |  -790 |  -890 |  -870 | -1020 | -1065 |
| 1148 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1065 |       1065 |  -995 | -1040 |  -975 | -1065 |  -975 | -1050 |  -975 | -1030 |
| 1149 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -355 |        355 |  -100 |  -145 |   -25 |   -65 |   -55 |   -40 |  -355 |  -235 |
| 1150 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -365 |        365 |  -140 |  -120 |  -150 |  -210 |  -150 |  -200 |  -365 |  -335 |
| 1151 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -740 |        740 |  -395 |  -470 |  -450 |  -415 |  -625 |  -450 |  -445 |  -740 |
| 1152 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         10 |       -240 |        250 |    10 |    -5 |   -10 |   -20 |   -40 |  -225 |  -235 |  -240 |
| 1153 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        245 |        -35 |        280 |   245 |   -10 |    90 |   -35 |   -25 |   100 |    90 |   115 |
| 1154 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -800 |        800 |  -515 |  -480 |  -595 |  -500 |  -580 |  -650 |  -800 |  -725 |
| 1155 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        355 |          0 |        355 |   345 |   285 |   355 |   280 |   150 |   155 |    95 |    85 |
| 1156 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        195 |        -85 |        280 |   195 |   185 |   185 |     5 |   -65 |   -85 |   -30 |    45 |
| 1157 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        260 |        -25 |        285 |   200 |   145 |   260 |    70 |   225 |   140 |   170 |   -25 |
| 1158 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        255 |        -20 |        275 |   245 |   240 |   220 |   255 |    60 |   -20 |   155 |    50 |
| 1159 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1135 |       1135 |  -915 |  -900 | -1005 |  -950 |  -910 | -1120 | -1120 | -1135 |
| 1160 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        155 |       -120 |        275 |    85 |   155 |   130 |  -110 |   -75 |  -120 |    15 |   -80 |
| 1161 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        135 |        -85 |        220 |    65 |   130 |   135 |    75 |   -55 |   -80 |   -85 |   -40 |
| 1162 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         65 |        -70 |        135 |    -0 |    25 |    30 |    20 |    45 |    65 |   -15 |   -70 |
| 1163 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -660 |        660 |  -450 |  -425 |  -455 |  -405 |  -425 |  -510 |  -460 |  -660 |
| 1164 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        320 |          0 |        320 |   290 |   265 |   320 |   100 |    55 |   145 |   130 |   115 |
| 1165 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -745 |        745 |  -520 |  -735 |  -670 |  -630 |  -745 |  -745 |  -705 |  -745 |
| 1166 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -600 |        600 |  -325 |  -290 |  -395 |  -290 |  -565 |  -465 |  -600 |  -560 |
| 1167 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -645 |        645 |  -380 |  -445 |  -470 |  -595 |  -625 |  -595 |  -635 |  -645 |
| 1168 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        305 |         -5 |        310 |   305 |   130 |    40 |    60 |    55 |    -0 |    -5 |    50 |
| 1169 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        320 |          0 |        320 |   275 |   310 |   270 |   320 |   195 |    55 |    20 |    10 |
| 1170 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -865 |        865 |  -595 |  -610 |  -530 |  -625 |  -760 |  -865 |  -865 |  -780 |
| 1171 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -980 |        980 |  -835 |  -855 |  -885 |  -885 |  -835 |  -785 |  -980 |  -975 |
| 1172 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         75 |        -65 |        140 |    10 |   -65 |   -25 |   -15 |    75 |    35 |   -50 |   -15 |
| 1173 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        115 |       -165 |        280 |    20 |    60 |    65 |    85 |   115 |    50 |  -165 |   -50 |
| 1174 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         15 |       -350 |        365 |   -10 |   -70 |    15 |   -80 |  -240 |  -350 |  -230 |  -205 |
| 1175 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        205 |       -100 |        305 |   205 |   145 |    90 |   -30 |  -100 |   -65 |    30 |    15 |
| 1176 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -385 |        385 |  -140 |   -60 |  -305 |  -360 |  -270 |  -385 |  -325 |  -235 |
| 1177 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -495 |        495 |  -280 |  -290 |  -245 |  -180 |  -385 |  -215 |  -460 |  -495 |
| 1178 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -885 |        885 |  -675 |  -715 |  -705 |  -700 |  -745 |  -825 |  -780 |  -885 |
| 1179 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -590 |        590 |  -265 |  -325 |  -330 |  -395 |  -325 |  -335 |  -555 |  -590 |
| 1180 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -655 |        655 |  -585 |  -560 |  -590 |  -490 |  -525 |  -560 |  -580 |  -655 |
| 1181 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        160 |          0 |        160 |   100 |   115 |   130 |   160 |   130 |   140 |   145 |   125 |
| 1182 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -560 |        560 |  -290 |  -340 |  -340 |  -330 |  -370 |  -225 |  -520 |  -560 |
| 1183 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        400 |          0 |        400 |   400 |   300 |   345 |   345 |   390 |   245 |   195 |   135 |
| 1184 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        210 |       -160 |        370 |   100 |   210 |   170 |   145 |   115 |   -20 |  -115 |  -160 |
| 1185 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        335 |          0 |        335 |   320 |   235 |   335 |   265 |   290 |   265 |   215 |   135 |
| 1186 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -540 |        540 |  -335 |  -370 |  -315 |  -475 |  -415 |  -540 |  -420 |  -515 |
| 1187 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1040 |       1040 |  -940 | -1025 |  -965 |  -935 |  -965 |  -985 | -1025 | -1040 |
| 1188 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        110 |       -120 |        230 |   100 |   110 |   -55 |  -120 |   -50 |   -70 |   -10 |   -25 |
| 1189 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -820 |        820 |  -530 |  -790 |  -765 |  -775 |  -820 |  -715 |  -765 |  -685 |
| 1190 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -925 |        925 |  -780 |  -845 |  -735 |  -900 |  -830 |  -830 |  -835 |  -925 |
| 1191 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        250 |        -60 |        310 |   230 |    45 |   160 |   250 |   115 |   -10 |   -20 |   -60 |
| 1192 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         90 |        -65 |        155 |    50 |    40 |    40 |   -65 |   -10 |    20 |    35 |    90 |
| 1193 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        345 |          0 |        345 |   220 |   290 |   265 |   345 |   270 |   260 |   240 |    70 |
| 1194 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -710 |        710 |  -490 |  -445 |  -465 |  -400 |  -420 |  -470 |  -450 |  -710 |
| 1195 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         60 |       -200 |        260 |    15 |    60 |     5 |   -25 |   -90 |   -40 |  -195 |  -200 |
| 1196 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -345 |        345 |   -35 |  -105 |  -175 |   -55 |  -115 |   -90 |  -100 |  -345 |
| 1197 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        305 |          0 |        305 |   305 |   185 |   285 |   265 |   205 |   300 |   250 |    85 |
| 1198 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -375 |        375 |  -105 |  -130 |   -65 |  -120 |  -375 |  -305 |  -330 |  -245 |
| 1199 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -590 |        590 |  -465 |  -310 |  -475 |  -340 |  -375 |  -390 |  -385 |  -590 |
| 1200 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        230 |          0 |        230 |   225 |   225 |   145 |   180 |   225 |   230 |   160 |   145 |
| 1201 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        200 |        -35 |        235 |   185 |   100 |   160 |   165 |   200 |    95 |    10 |   -35 |
| 1202 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         25 |       -310 |        335 |    15 |   -55 |  -155 |   -70 |  -145 |  -180 |    25 |  -310 |
| 1203 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        160 |        -25 |        185 |    10 |   160 |   -15 |   -25 |    20 |    95 |    45 |    -5 |
| 1204 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        195 |        -35 |        230 |   195 |    40 |   -35 |    -0 |   120 |    65 |   105 |    95 |
| 1205 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -395 |        395 |  -355 |  -340 |  -345 |  -335 |  -355 |  -330 |  -385 |  -395 |
| 1206 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1055 |       1055 |  -765 |  -860 |  -835 | -1055 | -1050 |  -950 | -1000 | -1020 |
| 1207 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        330 |          0 |        330 |   285 |   330 |   275 |   325 |   275 |   285 |   110 |   155 |
| 1208 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -965 |        965 |  -735 |  -690 |  -775 |  -925 |  -925 |  -860 |  -950 |  -965 |
| 1209 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -955 |        955 |  -760 |  -745 |  -590 |  -795 |  -790 |  -860 |  -815 |  -955 |
| 1210 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        370 |          0 |        370 |   310 |   370 |   230 |   280 |   335 |   305 |   340 |   370 |
| 1211 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -445 |        445 |  -300 |  -305 |  -290 |  -255 |  -440 |  -265 |  -445 |  -435 |
| 1212 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -895 |        895 |  -645 |  -640 |  -660 |  -510 |  -640 |  -715 |  -665 |  -895 |
| 1213 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         75 |       -130 |        205 |    40 |     5 |    60 |   -10 |    75 |    -5 |   -60 |  -130 |
| 1214 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        225 |          0 |        225 |   225 |   215 |   225 |   210 |   200 |   185 |    70 |    10 |
| 1215 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        235 |        -40 |        275 |   195 |   195 |   155 |   235 |   105 |   -40 |    10 |   -30 |
| 1216 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        135 |        -25 |        160 |    90 |   130 |    15 |   -25 |    45 |    10 |   105 |   135 |
| 1217 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |        -95 |        375 |   190 |   220 |   280 |   -95 |   -60 |    50 |    45 |     5 |
| 1218 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        205 |       -135 |        340 |   200 |   110 |   205 |  -115 |   -15 |    -5 |  -135 |   -85 |
| 1219 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        225 |        -65 |        290 |   225 |   110 |   -45 |   -65 |   -10 |   -25 |   -35 |   -60 |
| 1220 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1000 |       1000 |  -945 |  -985 |  -965 | -1000 |  -920 |  -915 |  -940 |  -835 |
| 1221 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        280 |          0 |        280 |   280 |   240 |   240 |   215 |    95 |   245 |   140 |   155 |
| 1222 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -390 |        390 |   -55 |  -155 |  -350 |  -255 |  -220 |  -305 |  -360 |  -390 |
| 1223 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        175 |       -130 |        305 |   175 |   145 |   125 |   100 |   115 |   -30 |  -100 |  -130 |
| 1224 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -690 |        690 |  -685 |  -600 |  -480 |  -555 |  -570 |  -665 |  -670 |  -690 |
| 1225 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        270 |        -85 |        355 |    75 |   270 |   145 |   175 |   200 |   -25 |   -85 |    35 |
| 1226 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        390 |          0 |        390 |   390 |   285 |    50 |   165 |   155 |    95 |   145 |    85 |
| 1227 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        100 |       -145 |        245 |   100 |    40 |    20 |    45 |  -145 |  -130 |  -110 |  -120 |
| 1228 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -540 |        540 |  -400 |  -340 |  -245 |  -495 |  -480 |  -520 |  -540 |  -525 |
| 1229 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        350 |          0 |        350 |   350 |   270 |   160 |   215 |   120 |   180 |   250 |   180 |
| 1230 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -395 |        395 |  -255 |  -160 |  -145 |  -175 |  -265 |  -115 |   -95 |  -395 |
| 1231 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        225 |       -125 |        350 |   170 |   185 |   225 |   145 |   165 |  -125 |  -120 |   -50 |
| 1232 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1040 |       1040 |  -985 |  -920 | -1015 |  -965 |  -895 | -1040 |  -990 | -1035 |
| 1233 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         60 |       -115 |        175 |    60 |   -35 |   -35 |    10 |  -115 |  -105 |    30 |   -80 |
| 1234 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -730 |        730 |  -730 |  -635 |  -730 |  -545 |  -615 |  -600 |  -605 |  -605 |
| 1235 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1110 |       1110 |  -930 | -1080 | -1030 | -1020 | -1110 | -1110 | -1005 | -1060 |
| 1236 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -425 |        425 |  -355 |  -355 |  -345 |  -325 |  -425 |  -340 |  -420 |  -285 |
| 1237 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -605 |        605 |  -410 |  -290 |  -390 |  -415 |  -555 |  -550 |  -585 |  -605 |
| 1238 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1180 |       1180 | -1075 | -1075 | -1155 |  -995 | -1050 | -1095 | -1065 | -1180 |
| 1239 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |          0 |        275 |   175 |   200 |   105 |   150 |   275 |   185 |   155 |    50 |
| 1240 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -740 |        740 |  -445 |  -515 |  -520 |  -740 |  -685 |  -680 |  -640 |  -540 |
| 1241 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        270 |        -95 |        365 |   145 |   270 |   180 |    -5 |   -40 |    35 |   -95 |   -75 |
| 1242 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        210 |        -80 |        290 |   210 |   -20 |    40 |   -15 |   -80 |   -75 |    30 |    70 |
| 1243 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        290 |          0 |        290 |   200 |   290 |   280 |   265 |   195 |   195 |    90 |    15 |
| 1244 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        315 |          0 |        315 |   215 |   275 |   275 |   315 |   315 |   220 |   305 |    80 |
| 1245 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        245 |        -15 |        260 |   245 |   235 |   195 |   160 |   160 |   -15 |    20 |    70 |
| 1246 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        320 |        -10 |        330 |   320 |   240 |   190 |   305 |   115 |    25 |   -10 |   115 |
| 1247 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -990 |        990 |  -780 |  -790 |  -700 |  -900 |  -795 |  -750 |  -990 |  -990 |
| 1248 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        125 |       -215 |        340 |   -25 |    25 |   125 |   -80 |   -10 |   -50 |   -65 |  -215 |
| 1249 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -990 |        990 |  -725 |  -630 |  -815 |  -790 |  -710 |  -750 |  -990 |  -905 |
| 1250 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -415 |        415 |  -155 |  -365 |  -330 |  -355 |  -305 |  -415 |  -290 |  -365 |
| 1251 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        335 |        -65 |        400 |   255 |   335 |   280 |   240 |    65 |    75 |    10 |   -65 |
| 1252 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        330 |          0 |        330 |   330 |    60 |    75 |   155 |   235 |    20 |    75 |    65 |
| 1253 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        300 |        -70 |        370 |   300 |   280 |   295 |   110 |    25 |   -30 |   -35 |   -70 |
| 1254 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        150 |       -115 |        265 |    95 |    75 |    45 |   -30 |    20 |   150 |    55 |  -115 |
| 1255 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -690 |        690 |  -450 |  -465 |  -470 |  -640 |  -570 |  -690 |  -595 |  -575 |
| 1256 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        230 |          0 |        230 |   175 |   180 |   115 |   220 |   230 |   145 |   130 |   160 |
| 1257 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -830 |        830 |  -480 |  -540 |  -520 |  -610 |  -620 |  -550 |  -695 |  -830 |
| 1258 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        285 |          0 |        285 |   285 |   100 |    85 |   105 |   145 |   155 |   190 |    90 |
| 1259 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   295 |   330 |   375 |   290 |   295 |   275 |   400 |   200 |
| 1260 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        310 |        -75 |        385 |   245 |   310 |   240 |   220 |   210 |   -25 |    25 |   -75 |
| 1261 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -580 |        580 |  -580 |  -475 |  -495 |  -420 |  -430 |  -445 |  -490 |  -465 |
| 1262 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1020 |       1020 |  -755 |  -805 |  -740 |  -795 |  -925 |  -860 |  -765 | -1020 |
| 1263 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        305 |        -55 |        360 |   215 |   305 |   115 |   -10 |    45 |   -55 |   140 |   -35 |
| 1264 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        300 |          0 |        300 |   215 |   260 |   270 |   225 |   205 |   275 |   300 |   235 |
| 1265 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |          0 |        240 |   240 |    95 |   145 |   130 |   115 |   160 |   125 |    75 |
| 1266 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -720 |        720 |  -480 |  -500 |  -530 |  -480 |  -600 |  -685 |  -720 |  -715 |
| 1267 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        280 |        -70 |        350 |   280 |   270 |    75 |   -20 |   -20 |    75 |    45 |   -70 |
| 1268 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -430 |        430 |  -365 |  -335 |  -340 |  -430 |  -385 |  -335 |  -380 |  -290 |
| 1269 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -965 |        965 |  -790 |  -830 |  -690 |  -730 |  -800 |  -910 |  -955 |  -965 |
| 1270 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -275 |        275 |  -115 |  -145 |  -145 |  -155 |   -55 |   -55 |  -100 |  -275 |
| 1271 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -835 |        835 |  -595 |  -595 |  -565 |  -645 |  -835 |  -765 |  -800 |  -805 |
| 1272 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        285 |        -40 |        325 |   285 |   220 |    75 |   110 |   -10 |   115 |   -40 |    10 |
| 1273 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -255 |        255 |   -50 |   -55 |  -230 |  -255 |  -230 |  -220 |  -205 |  -145 |
| 1274 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        245 |         -5 |        250 |   220 |   150 |   200 |   230 |   125 |   145 |   245 |    -5 |
| 1275 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -435 |        435 |  -220 |  -175 |  -285 |  -175 |  -180 |  -135 |  -305 |  -435 |
| 1276 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        385 |          0 |        385 |   350 |   295 |   275 |   385 |    50 |   215 |   225 |    75 |
| 1277 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        170 |       -135 |        305 |   110 |   120 |   125 |   170 |    90 |   135 |  -125 |  -135 |
| 1278 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -650 |        650 |  -505 |  -410 |  -325 |  -490 |  -570 |  -515 |  -425 |  -650 |
| 1279 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        275 |          0 |        275 |   240 |   225 |   130 |   235 |   210 |   275 |    35 |    75 |
| 1280 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        235 |        -60 |        295 |   170 |   220 |   185 |   235 |   220 |   -60 |   100 |    50 |
| 1281 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        290 |        -55 |        345 |   205 |   290 |   285 |   255 |   280 |   205 |   100 |   -55 |
| 1282 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -775 |        775 |  -540 |  -620 |  -540 |  -505 |  -585 |  -490 |  -540 |  -775 |
| 1283 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -380 |        380 |  -245 |  -190 |  -310 |  -275 |  -130 |  -220 |  -185 |  -380 |
| 1284 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1140 |       1140 |  -895 |  -765 |  -920 |  -800 |  -900 | -1140 | -1100 | -1040 |
| 1285 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        165 |        -95 |        260 |   165 |   145 |    20 |   -25 |   -25 |    10 |   -60 |   -95 |
| 1286 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          5 |       -340 |        345 |   -40 |   -15 |     5 |   -40 |  -270 |  -190 |  -340 |  -205 |
| 1287 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -730 |        730 |  -425 |  -405 |  -600 |  -730 |  -675 |  -630 |  -690 |  -655 |
| 1288 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        145 |        -55 |        200 |    55 |   105 |    10 |    10 |   -55 |    35 |   145 |   105 |
| 1289 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -795 |  -855 |  -875 |  -815 |  -790 |  -795 |  -865 |  -990 |
| 1290 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1035 |       1035 |  -795 |  -750 | -1005 |  -970 |  -955 |  -960 | -1015 | -1035 |
| 1291 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -610 |        610 |  -370 |  -510 |  -575 |  -590 |  -570 |  -610 |  -570 |  -575 |
| 1292 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -995 |        995 |  -730 |  -720 |  -665 |  -770 |  -830 |  -950 |  -855 |  -995 |
| 1293 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        280 |        -55 |        335 |   210 |   280 |   205 |   250 |   215 |   175 |   -55 |   -40 |
| 1294 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -550 |        550 |  -330 |  -550 |  -455 |  -455 |  -400 |  -465 |  -530 |  -535 |
| 1295 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -260 |        260 |  -185 |  -195 |  -165 |  -260 |  -165 |  -260 |  -180 |  -220 |
| 1296 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |          0 |        275 |   185 |   225 |   175 |   255 |   180 |   275 |   200 |    80 |
| 1297 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1015 |       1015 |  -805 |  -790 |  -860 |  -900 |  -830 |  -800 |  -830 | -1015 |
| 1298 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -580 |        580 |  -280 |  -340 |  -325 |  -360 |  -315 |  -350 |  -360 |  -580 |
| 1299 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -940 |        940 |  -710 |  -715 |  -705 |  -710 |  -595 |  -735 |  -940 |  -825 |
| 1300 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        290 |          0 |        290 |   290 |   185 |   240 |   185 |    25 |    40 |   105 |   100 |
| 1301 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        320 |          0 |        320 |   255 |   200 |   275 |   215 |   320 |    40 |   145 |    50 |
| 1302 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -590 |        590 |  -325 |  -265 |  -590 |  -520 |  -510 |  -575 |  -470 |  -525 |
| 1303 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -525 |        525 |  -380 |  -350 |  -420 |  -350 |  -305 |  -385 |  -340 |  -525 |
| 1304 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        390 |          0 |        390 |   345 |   245 |   185 |   390 |   265 |   300 |   210 |   230 |
| 1305 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        220 |        -45 |        265 |   150 |   115 |   220 |   170 |   -15 |   -45 |    55 |    25 |
| 1306 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        255 |        -30 |        285 |   175 |   115 |    75 |   155 |   170 |   255 |   135 |   -30 |
| 1307 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -435 |        435 |  -230 |  -170 |  -210 |  -170 |  -160 |  -435 |  -390 |  -290 |
| 1308 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -750 |        750 |  -405 |  -710 |  -665 |  -660 |  -670 |  -605 |  -750 |  -725 |
| 1309 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |       -200 |        425 |   105 |   140 |   225 |    20 |  -120 |  -200 |   -35 |   -70 |
| 1310 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        305 |          0 |        305 |   305 |   290 |   305 |    80 |   120 |    70 |   215 |    35 |
| 1311 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -690 |        690 |  -415 |  -430 |  -310 |  -490 |  -470 |  -440 |  -440 |  -690 |
| 1312 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1025 |       1025 |  -695 |  -730 |  -820 |  -760 |  -755 |  -765 |  -760 | -1025 |
| 1313 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        205 |       -115 |        320 |   140 |   205 |   145 |   155 |    15 |   -55 |  -115 |   -30 |
| 1314 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        240 |       -135 |        375 |   200 |   125 |   240 |   110 |   100 |   220 |  -135 |   -75 |
| 1315 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -555 |        555 |  -330 |  -310 |  -555 |  -490 |  -500 |  -490 |  -420 |  -445 |
| 1316 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -755 |        755 |  -485 |  -500 |  -455 |  -715 |  -600 |  -755 |  -680 |  -705 |
| 1317 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -520 |        520 |  -195 |  -390 |  -420 |  -415 |  -335 |  -370 |  -320 |  -520 |
| 1318 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -810 |        810 |  -520 |  -490 |  -460 |  -810 |  -750 |  -675 |  -695 |  -650 |
| 1319 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        230 |        -70 |        300 |   140 |   230 |   155 |    -5 |   -55 |   -70 |   -25 |    -0 |
| 1320 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -550 |        550 |  -250 |  -280 |  -420 |  -275 |  -275 |  -500 |  -475 |  -550 |
| 1321 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -260 |        260 |   -95 |   -55 |  -115 |   -10 |  -215 |  -235 |  -180 |  -260 |
| 1322 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -465 |        465 |  -270 |  -335 |  -270 |  -280 |  -235 |  -310 |  -290 |  -465 |
| 1323 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1110 |       1110 |  -905 |  -890 |  -840 |  -835 | -1110 | -1105 | -1065 | -1065 |
| 1324 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -980 |        980 |  -780 |  -700 |  -805 |  -725 |  -775 |  -785 |  -980 |  -970 |
| 1325 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         60 |       -205 |        265 |    45 |   -55 |    10 |    10 |    60 |    -5 |   -60 |  -205 |
| 1326 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         65 |       -115 |        180 |   -50 |     5 |    65 |   -70 |   -50 |   -30 |   -15 |  -115 |
| 1327 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -595 |        595 |  -565 |  -545 |  -545 |  -550 |  -530 |  -560 |  -595 |  -435 |
| 1328 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1015 |       1015 |  -790 |  -795 |  -725 |  -830 |  -695 |  -925 | -1015 |  -985 |
| 1329 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         75 |       -175 |        250 |    75 |   -25 |   -95 |  -175 |  -165 |   -85 |  -110 |  -165 |
| 1330 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -830 |        830 |  -585 |  -690 |  -570 |  -785 |  -830 |  -830 |  -775 |  -825 |
| 1331 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -835 |        835 |  -545 |  -635 |  -570 |  -750 |  -735 |  -835 |  -750 |  -755 |
| 1332 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        260 |          0 |        260 |   210 |   260 |    75 |    50 |   230 |   135 |    90 |   110 |
| 1333 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        325 |          0 |        325 |   280 |   275 |   325 |   130 |   155 |   135 |    30 |   100 |
| 1334 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -810 |        810 |  -545 |  -615 |  -595 |  -595 |  -810 |  -670 |  -750 |  -735 |
| 1335 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |          0 |        240 |   145 |   240 |   225 |   210 |   180 |   115 |   150 |    40 |
| 1336 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        340 |          0 |        340 |   185 |   275 |   255 |   240 |   340 |   265 |   140 |    90 |
| 1337 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -805 |        805 |  -645 |  -645 |  -695 |  -775 |  -790 |  -805 |  -790 |  -805 |
| 1338 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        115 |       -120 |        235 |    30 |    40 |   115 |    45 |    40 |    40 |   100 |  -120 |
| 1339 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -965 |  -845 | -1050 |  -910 |  -950 |  -970 |  -835 | -1080 |
| 1340 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        105 |       -135 |        240 |   105 |    75 |   100 |    50 |    65 |    75 |    90 |  -135 |
| 1341 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -545 |        545 |  -205 |  -235 |  -235 |  -260 |  -270 |  -280 |  -545 |  -440 |
| 1342 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        370 |          0 |        370 |   245 |   315 |   330 |   370 |    95 |    70 |    80 |   175 |
| 1343 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         25 |       -210 |        235 |    10 |    -5 |   -50 |    25 |  -210 |  -150 |  -155 |  -165 |
| 1344 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        315 |          0 |        315 |   155 |   180 |   260 |   280 |   165 |   315 |   220 |    40 |
| 1345 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -700 |        700 |  -470 |  -340 |  -675 |  -565 |  -700 |  -575 |  -585 |  -700 |
| 1346 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        290 |          0 |        290 |   290 |   160 |   225 |   275 |   205 |    40 |    15 |    50 |
| 1347 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -885 |        885 |  -730 |  -615 |  -685 |  -710 |  -875 |  -885 |  -780 |  -885 |
| 1348 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -640 |        640 |  -445 |  -480 |  -335 |  -370 |  -400 |  -480 |  -640 |  -590 |
| 1349 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -455 |        455 |  -145 |  -155 |  -240 |  -210 |  -380 |  -360 |  -455 |  -420 |
| 1350 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        380 |          0 |        380 |   340 |   305 |   380 |   115 |   175 |   150 |   250 |    55 |
| 1351 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -725 |        725 |  -495 |  -460 |  -685 |  -725 |  -655 |  -640 |  -705 |  -605 |
| 1352 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        330 |          0 |        330 |   305 |   330 |   315 |   325 |   220 |   100 |    70 |    80 |
| 1353 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        225 |       -105 |        330 |   200 |   110 |   190 |    35 |   225 |   180 |   125 |  -105 |
| 1354 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1000 |       1000 |  -515 |  -675 | -1000 |  -925 |  -855 |  -795 |  -870 |  -900 |
| 1355 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        265 |        -95 |        360 |   105 |   200 |   265 |   130 |   215 |    35 |    75 |   -95 |
| 1356 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         30 |       -235 |        265 |    30 |    -0 |  -205 |  -235 |  -185 |  -140 |  -195 |  -195 |
| 1357 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -895 |        895 |  -590 |  -570 |  -620 |  -805 |  -655 |  -810 |  -895 |  -835 |
| 1358 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        255 |          0 |        255 |   135 |   150 |   155 |   100 |   105 |   255 |    95 |   140 |
| 1359 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -625 |        625 |  -410 |  -535 |  -480 |  -430 |  -415 |  -500 |  -605 |  -625 |
| 1360 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        170 |       -100 |        270 |   105 |    80 |    65 |   155 |    70 |   170 |   -75 |  -100 |
| 1361 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1135 |       1135 |  -830 |  -815 | -1135 | -1020 | -1020 | -1080 | -1000 |  -975 |
| 1362 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -600 |        600 |  -370 |  -430 |  -340 |  -365 |  -395 |  -525 |  -280 |  -600 |
| 1363 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        275 |          0 |        275 |   275 |   110 |   100 |    70 |    80 |    65 |   130 |    30 |
| 1364 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        100 |       -175 |        275 |   100 |    75 |    10 |    45 |    80 |   -70 |  -175 |   -50 |
| 1365 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        220 |       -210 |        430 |   120 |   105 |   220 |   195 |  -210 |    -0 |   -95 |  -100 |
| 1366 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        400 |          0 |        400 |   400 |   335 |   350 |   385 |   360 |   295 |   190 |   195 |
| 1367 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        355 |        -10 |        365 |   355 |   300 |   355 |   110 |   185 |    85 |   110 |   -10 |
| 1368 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -955 |        955 |  -955 |  -910 |  -890 |  -870 |  -830 |  -935 |  -840 |  -860 |
| 1369 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        335 |          0 |        335 |   215 |   170 |   335 |   285 |   240 |    30 |    10 |   125 |
| 1370 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -830 |        830 |  -580 |  -580 |  -605 |  -810 |  -805 |  -730 |  -830 |  -800 |
| 1371 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         65 |       -265 |        330 |    60 |    45 |    -5 |    -5 |    25 |   -70 |    65 |  -265 |
| 1372 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         40 |       -340 |        380 |   -85 |   -10 |   -95 |  -130 |  -100 |    40 |  -270 |  -340 |
| 1373 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -315 |        315 |   -45 |   -65 |   -70 |   -15 |   -65 |   -40 |  -315 |  -225 |
| 1374 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -560 |        560 |  -520 |  -460 |  -560 |  -350 |  -465 |  -355 |  -425 |  -515 |
| 1375 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        240 |          0 |        240 |   125 |   150 |   220 |   100 |   190 |   160 |   180 |   240 |
| 1376 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -750 |        750 |  -605 |  -590 |  -460 |  -585 |  -570 |  -720 |  -680 |  -750 |
| 1377 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -345 |        345 |   -30 |  -330 |  -320 |  -315 |  -345 |  -255 |  -330 |  -290 |
| 1378 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        310 |          0 |        310 |   220 |   310 |   205 |   270 |   250 |    35 |    45 |    20 |
| 1379 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        220 |       -135 |        355 |   160 |    70 |   120 |    40 |   220 |   100 |   -30 |  -135 |
| 1380 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        215 |          0 |        215 |   115 |    85 |   150 |    95 |   150 |   165 |   215 |    95 |
| 1381 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |          0 |        205 |    45 |   100 |   175 |   135 |   205 |   120 |    65 |   130 |
| 1382 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        260 |        -65 |        325 |   260 |   105 |   165 |   235 |   165 |   180 |    45 |   -65 |
| 1383 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -665 |        665 |  -370 |  -435 |  -335 |  -450 |  -365 |  -405 |  -330 |  -665 |
| 1384 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          5 |       -140 |        145 |     5 |  -120 |   -40 |  -135 |   -40 |   -65 |  -140 |  -105 |
| 1385 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |          0 |        285 |   185 |   285 |   265 |   235 |   195 |   120 |   105 |    35 |
| 1386 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |        -55 |        350 |   125 |   295 |   155 |    55 |    40 |    -0 |    -0 |   -55 |
| 1387 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |          0 |        265 |   265 |    80 |    70 |    55 |   120 |    15 |   180 |    40 |
| 1388 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -660 |        660 |  -545 |  -390 |  -405 |  -510 |  -420 |  -480 |  -660 |  -640 |
| 1389 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          5 |       -340 |        345 |   -60 |   -10 |   -35 |     5 |  -340 |  -155 |  -310 |  -235 |
| 1390 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -655 |        655 |  -350 |  -605 |  -605 |  -640 |  -650 |  -655 |  -570 |  -595 |
| 1391 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -640 |        640 |  -360 |  -470 |  -445 |  -475 |  -300 |  -415 |  -430 |  -640 |
| 1392 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -605 |        605 |  -335 |  -365 |  -300 |  -430 |  -300 |  -280 |  -395 |  -605 |
| 1393 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -660 |        660 |  -585 |  -515 |  -540 |  -570 |  -555 |  -660 |  -550 |  -575 |
| 1394 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -565 |        565 |  -560 |  -515 |  -545 |  -565 |  -555 |  -430 |  -510 |  -490 |
| 1395 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -850 |        850 |  -450 |  -755 |  -670 |  -735 |  -715 |  -850 |  -745 |  -705 |
| 1396 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -705 |        705 |  -375 |  -300 |  -550 |  -705 |  -570 |  -625 |  -510 |  -645 |
| 1397 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        345 |          0 |        345 |   335 |   275 |   345 |   135 |   125 |   125 |   125 |   125 |
| 1398 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -865 |        865 |  -610 |  -635 |  -585 |  -670 |  -670 |  -795 |  -865 |  -770 |
| 1399 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |        -95 |        335 |   240 |   -55 |   -35 |    30 |   -95 |   -70 |    -0 |   -75 |
| 1400 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -325 |        325 |  -100 |   -45 |  -110 |  -230 |  -275 |  -240 |  -325 |  -205 |
| 1401 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        120 |       -275 |        395 |   -65 |    75 |   120 |   -80 |   -35 |   -30 |  -145 |  -275 |
| 1402 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1000 |       1000 |  -735 |  -815 |  -755 |  -795 |  -940 |  -955 |  -950 | -1000 |
| 1403 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -720 |        720 |  -620 |  -475 |  -570 |  -490 |  -555 |  -600 |  -650 |  -720 |
| 1404 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         60 |       -250 |        310 |   -30 |   -55 |    35 |    60 |   -45 |    10 |  -235 |  -250 |
| 1405 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1080 |       1080 |  -860 | -1040 |  -950 | -1045 | -1020 |  -975 | -1080 | -1020 |
| 1406 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |        -45 |        330 |   170 |   285 |   275 |    95 |   170 |   -15 |    35 |   -45 |
| 1407 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         65 |       -200 |        265 |    60 |    -5 |    20 |    20 |    35 |    65 |    -5 |  -200 |
| 1408 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -715 |        715 |  -500 |  -445 |  -650 |  -575 |  -570 |  -545 |  -575 |  -715 |
| 1409 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        295 |          0 |        295 |   135 |   165 |   295 |   150 |   265 |    80 |   165 |   120 |
| 1410 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -640 |        640 |  -430 |  -405 |  -395 |  -395 |  -460 |  -350 |  -600 |  -640 |
| 1411 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -815 |        815 |  -630 |  -645 |  -720 |  -745 |  -760 |  -765 |  -815 |  -780 |
| 1412 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        290 |        -30 |        320 |   290 |   260 |   235 |   180 |   -30 |   -10 |    70 |    50 |
| 1413 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -565 |        565 |  -345 |  -305 |  -355 |  -310 |  -355 |  -335 |  -530 |  -565 |
| 1414 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        175 |          0 |        175 |    95 |   145 |   155 |    65 |    85 |   145 |   175 |    20 |
| 1415 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        235 |         -5 |        240 |   145 |   190 |   175 |   235 |   115 |   225 |   155 |    -5 |
| 1416 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        385 |        -30 |        415 |   385 |   185 |   240 |   285 |   135 |   220 |   155 |   -30 |
| 1417 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        295 |        -70 |        365 |   295 |   275 |   -70 |    25 |    10 |    15 |   -40 |    -5 |
| 1418 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        195 |       -170 |        365 |   160 |   195 |   185 |   100 |   155 |  -170 |  -100 |   -85 |
| 1419 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -650 |        650 |  -515 |  -430 |  -355 |  -415 |  -615 |  -535 |  -640 |  -650 |
| 1420 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        205 |        -25 |        230 |   195 |   115 |   205 |   135 |    60 |   105 |   125 |   -25 |
| 1421 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1095 |       1095 |  -885 |  -985 |  -980 | -1075 | -1095 | -1045 | -1060 | -1080 |
| 1422 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -990 |        990 |  -745 |  -790 |  -935 |  -975 |  -815 |  -990 |  -945 |  -865 |
| 1423 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -505 |        505 |  -250 |  -290 |  -470 |  -505 |  -470 |  -365 |  -485 |  -430 |
| 1424 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        310 |         -0 |        310 |   175 |   310 |   285 |   155 |   165 |    30 |    -0 |   135 |
| 1425 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        260 |        -25 |        285 |   235 |   135 |   140 |   225 |   260 |   145 |   165 |   -25 |
| 1426 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        100 |        -90 |        190 |    50 |   100 |    -0 |    -5 |    -5 |    50 |   -90 |   -75 |
| 1427 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -535 |        535 |  -250 |  -295 |  -295 |  -280 |  -245 |  -535 |  -470 |  -385 |
| 1428 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1075 |       1075 |  -815 |  -735 |  -865 |  -795 | -1075 |  -980 |  -940 |  -925 |
| 1429 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -635 |        635 |  -560 |  -510 |  -505 |  -350 |  -485 |  -480 |  -435 |  -635 |
| 1430 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -790 |        790 |  -595 |  -510 |  -610 |  -770 |  -790 |  -685 |  -695 |  -700 |
| 1431 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        140 |          0 |        140 |     5 |    50 |    30 |    95 |    45 |    60 |    30 |   140 |
| 1432 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -685 |        685 |  -415 |  -350 |  -415 |  -640 |  -685 |  -595 |  -580 |  -670 |
| 1433 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -430 |        430 |  -385 |  -215 |  -305 |  -365 |  -385 |  -360 |  -230 |  -430 |
| 1434 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        290 |          0 |        290 |   240 |   290 |   180 |   190 |   270 |   170 |   170 |    40 |
| 1435 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -875 |  -955 |  -955 |  -915 |  -990 | -1000 |  -975 | -1080 |
| 1436 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        400 |      -1200 |       1600 | -1200 |   305 |   360 |   300 |   400 |   335 |   370 |   325 |
| 1437 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        365 |          0 |        365 |   360 |   365 |   205 |   235 |   215 |   235 |   270 |   285 |
| 1438 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        250 |        -50 |        300 |   250 |   250 |    15 |   -40 |    20 |   -50 |   -40 |   -40 |
| 1439 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -875 |        875 |  -690 |  -745 |  -750 |  -640 |  -695 |  -615 |  -610 |  -875 |
| 1440 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        150 |        -95 |        245 |   150 |   -70 |   -90 |    20 |   -10 |   -15 |   -25 |   -95 |
| 1441 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -995 |        995 |  -655 |  -650 |  -735 |  -660 |  -790 |  -925 |  -995 |  -955 |
| 1442 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -405 |        405 |  -265 |  -200 |  -245 |  -295 |  -205 |  -280 |  -350 |  -405 |
| 1443 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        230 |        -50 |        280 |   135 |   225 |   145 |   230 |   155 |   -50 |    40 |    -0 |
| 1444 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -970 |        970 |  -815 |  -835 |  -775 |  -755 |  -840 |  -830 |  -970 |  -960 |
| 1445 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -595 |        595 |  -345 |  -355 |  -455 |  -250 |  -530 |  -595 |  -535 |  -570 |
| 1446 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1165 |       1165 |  -985 | -1040 | -1020 | -1030 |  -985 |  -940 | -1055 | -1165 |
| 1447 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -350 |        350 |    -5 |  -115 |   -75 |   -35 |   -40 |  -225 |  -350 |  -250 |
| 1448 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1015 |       1015 |  -660 |  -660 |  -670 |  -650 |  -640 |  -610 | -1015 |  -965 |
| 1449 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -765 |        765 |  -515 |  -540 |  -555 |  -590 |  -440 |  -580 |  -765 |  -660 |
| 1450 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        135 |       -270 |        405 |    20 |    10 |    20 |   -30 |  -105 |   135 |   -30 |  -270 |
| 1451 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -255 |        255 |   -45 |  -190 |   -90 |  -210 |  -255 |  -180 |  -115 |  -205 |
| 1452 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -855 |        855 |  -760 |  -855 |  -805 |  -805 |  -830 |  -815 |  -790 |  -770 |
| 1453 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1035 |       1035 | -1035 |  -900 | -1000 |  -995 |  -980 |  -895 |  -940 |  -985 |
| 1454 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -990 |        990 |  -795 |  -950 |  -910 |  -970 |  -875 |  -880 |  -990 |  -885 |
| 1455 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -980 |        980 |  -805 |  -675 |  -675 |  -740 |  -675 |  -730 |  -730 |  -980 |
| 1456 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        300 |          0 |        300 |   270 |   120 |   110 |   115 |   190 |   300 |   225 |   165 |
| 1457 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        295 |          0 |        295 |   190 |   295 |   220 |   210 |   210 |   235 |    90 |    10 |
| 1458 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         65 |        -75 |        140 |    65 |   -25 |    30 |   -35 |   -25 |   -40 |   -75 |    -0 |
| 1459 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1175 |       1175 |  -860 |  -975 | -1070 | -1175 | -1170 | -1115 | -1115 | -1050 |
| 1460 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -770 |        770 |  -570 |  -545 |  -485 |  -590 |  -500 |  -565 |  -460 |  -770 |
| 1461 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         80 |        -60 |        140 |    50 |    20 |    25 |    80 |    35 |   -60 |    50 |    75 |
| 1462 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        255 |          0 |        255 |   225 |   255 |   170 |   250 |    90 |   140 |   250 |   200 |
| 1463 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -780 |        780 |  -630 |  -680 |  -710 |  -725 |  -655 |  -650 |  -735 |  -780 |
| 1464 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        260 |          0 |        260 |   200 |   180 |   250 |   185 |   230 |   215 |   260 |    50 |
| 1465 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -915 |        915 |  -625 |  -640 |  -645 |  -885 |  -805 |  -870 |  -820 |  -915 |
| 1466 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        135 |        -75 |        210 |   125 |   135 |    50 |    25 |    20 |   -10 |    -5 |   -75 |
| 1467 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -500 |        500 |  -190 |  -280 |  -240 |  -280 |  -360 |  -500 |  -400 |  -500 |
| 1468 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -385 |        385 |  -325 |  -340 |  -260 |  -305 |  -385 |  -380 |  -365 |  -350 |
| 1469 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -290 |        290 |   -70 |   -95 |   -10 |   -15 |   -45 |  -100 |  -250 |  -290 |
| 1470 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         45 |       -235 |        280 |   -80 |    30 |   -30 |    45 |    -5 |  -200 |  -215 |  -235 |
| 1471 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        140 |        -50 |        190 |   140 |    70 |   100 |   140 |   120 |   -50 |    30 |   -50 |
| 1472 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        320 |          0 |        320 |   240 |   320 |   285 |   200 |   185 |    35 |    40 |   100 |
| 1473 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        180 |       -200 |        380 |   180 |    60 |   110 |    50 |   -45 |  -200 |   -70 |   -45 |
| 1474 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |       -165 |        365 |   200 |   115 |   100 |    85 |   135 |   120 |   125 |  -165 |
| 1475 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1070 |       1070 |  -900 |  -910 | -1070 |  -950 | -1005 | -1025 | -1025 | -1010 |
| 1476 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |          0 |        265 |   265 |   110 |   135 |   130 |    85 |    95 |    85 |   105 |
| 1477 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -600 |        600 |  -445 |  -500 |  -385 |  -560 |  -350 |  -435 |  -400 |  -600 |
| 1478 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |          0 |        245 |   225 |   185 |   200 |   220 |   245 |   185 |    45 |    30 |
| 1479 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        300 |         -0 |        300 |   230 |   265 |   290 |   300 |   225 |   280 |    -0 |     5 |
| 1480 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        200 |          0 |        200 |   135 |   110 |   110 |   125 |   120 |   145 |   200 |   130 |
| 1481 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        185 |       -130 |        315 |   185 |    55 |   135 |   165 |  -130 |   -85 |   -80 |   -45 |
| 1482 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -905 |        905 |  -640 |  -645 |  -685 |  -685 |  -610 |  -685 |  -905 |  -845 |
| 1483 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -450 |        450 |  -220 |  -205 |   -85 |  -420 |  -450 |  -375 |  -350 |  -390 |
| 1484 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        400 |          0 |        400 |   375 |   345 |   240 |   345 |   400 |   165 |   170 |   225 |
| 1485 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        120 |       -150 |        270 |   120 |   -80 |   -85 |   -40 |  -150 |   -70 |  -110 |   -55 |
| 1486 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         35 |       -225 |        260 |    25 |  -125 |    35 |    -5 |  -165 |   -50 |  -225 |   -15 |
| 1487 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        125 |          0 |        125 |    35 |    10 |    20 |   125 |    30 |    40 |   105 |    25 |
| 1488 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        275 |        -35 |        310 |   275 |   245 |    45 |    60 |   -35 |    65 |    45 |    45 |
| 1489 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -460 |        460 |  -345 |  -345 |  -220 |  -300 |  -220 |  -370 |  -435 |  -460 |
| 1490 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         40 |       -235 |        275 |   -25 |  -175 |   -15 |    40 |   -60 |  -115 |    -5 |  -235 |
| 1491 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -630 |        630 |  -450 |  -375 |  -515 |  -430 |  -620 |  -610 |  -630 |  -625 |
| 1492 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -545 |        545 |  -545 |  -530 |  -495 |  -430 |  -415 |  -485 |  -505 |  -535 |
| 1493 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -905 |        905 |  -760 |  -650 |  -640 |  -680 |  -635 |  -595 |  -770 |  -905 |
| 1494 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          5 |       -195 |        200 |     5 |  -180 |  -120 |  -175 |  -195 |   -95 |  -110 |   -95 |
| 1495 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -165 |        165 |  -125 |  -145 |   -40 |   -60 |  -165 |   -30 |   -70 |   -75 |
| 1496 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        255 |          0 |        255 |   150 |   155 |   115 |   255 |    95 |   200 |    25 |    35 |
| 1497 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -545 |        545 |  -305 |  -310 |  -300 |  -345 |  -310 |  -275 |  -305 |  -545 |
| 1498 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        245 |       -155 |        400 |   245 |   120 |  -155 |   -80 |   -60 |    -5 |   -80 |  -120 |
| 1499 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -570 |        570 |  -300 |  -490 |  -570 |  -535 |  -405 |  -515 |  -485 |  -445 |
| 1500 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        160 |         -5 |        165 |    45 |    90 |    50 |   115 |    -5 |    95 |   160 |    55 |
| 1501 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        225 |        -85 |        310 |   225 |    35 |   155 |    75 |   105 |   130 |   -85 |   -65 |
| 1502 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |          0 |        280 |   255 |   280 |   265 |    25 |    30 |    90 |   160 |    40 |
| 1503 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -800 |        800 |  -545 |  -615 |  -785 |  -760 |  -730 |  -765 |  -755 |  -800 |
| 1504 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -870 |        870 |  -670 |  -730 |  -740 |  -695 |  -610 |  -685 |  -680 |  -870 |
| 1505 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        340 |        -50 |        390 |   340 |   220 |   190 |   -50 |    50 |   -15 |    65 |   -25 |
| 1506 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        250 |        -35 |        285 |   235 |   220 |   250 |   135 |   -35 |   110 |    75 |    65 |
| 1507 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        220 |        -85 |        305 |   185 |   115 |   220 |   -70 |    15 |   -40 |   -85 |   -35 |
| 1508 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -960 |        960 |  -670 |  -670 |  -685 |  -640 |  -705 |  -590 |  -960 |  -825 |
| 1509 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -140 |        140 |   -65 |   -55 |   -20 |   -65 |   -70 |  -140 |   -75 |   -85 |
| 1510 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -625 |        625 |  -420 |  -420 |  -570 |  -555 |  -485 |  -625 |  -535 |  -535 |
| 1511 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        295 |          0 |        295 |   210 |   295 |   235 |   285 |   290 |   290 |   280 |   135 |
| 1512 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -370 |        370 |  -320 |  -370 |  -280 |  -295 |  -355 |  -320 |  -285 |  -215 |
| 1513 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        235 |       -115 |        350 |    75 |   130 |   170 |   235 |   220 |    30 |  -115 |    -5 |
| 1514 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          5 |       -275 |        280 |   -90 |     5 |   -70 |   -55 |   -55 |  -270 |  -275 |  -260 |
| 1515 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |        -20 |        290 |   195 |   270 |    90 |    10 |   105 |    45 |    25 |   -20 |
| 1516 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -875 |        875 |  -875 |  -840 |  -795 |  -865 |  -790 |  -625 |  -845 |  -795 |
| 1517 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         85 |        -30 |        115 |    45 |    70 |    85 |   -30 |    70 |    10 |    65 |    10 |
| 1518 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -555 |        555 |  -235 |  -325 |  -225 |  -310 |  -245 |  -555 |  -420 |  -485 |
| 1519 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1190 |       1190 |  -870 |  -880 |  -875 |  -985 |  -955 | -1190 | -1135 | -1130 |
| 1520 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         95 |       -215 |        310 |    30 |    45 |    95 |  -140 |   -50 |  -215 |  -180 |  -110 |
| 1521 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         -0 |       -255 |        255 |   -65 |   -45 |   -80 |   -80 |   -10 |    -0 |   -10 |  -255 |
| 1522 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        315 |          0 |        315 |   230 |   200 |   315 |   245 |   205 |   220 |   260 |    85 |
| 1523 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -485 |        485 |  -215 |  -170 |  -105 |  -310 |  -355 |  -425 |  -485 |  -375 |
| 1524 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         15 |       -265 |        280 |  -130 |   -80 |   -65 |  -125 |    10 |  -135 |    15 |  -265 |
| 1525 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -585 |        585 |  -375 |  -445 |  -350 |  -360 |  -410 |  -450 |  -405 |  -585 |
| 1526 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -515 |        515 |  -300 |  -245 |  -320 |  -265 |  -440 |  -495 |  -455 |  -515 |
| 1527 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        300 |          0 |        300 |   110 |   300 |   195 |   190 |   190 |   145 |   200 |   185 |
| 1528 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -730 |        730 |  -545 |  -515 |  -555 |  -645 |  -510 |  -690 |  -670 |  -730 |
| 1529 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |       -115 |        390 |   270 |   245 |   210 |   245 |   275 |    40 |  -115 |    75 |
| 1530 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -595 |        595 |  -510 |  -560 |  -545 |  -535 |  -595 |  -540 |  -530 |  -525 |
| 1531 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1060 |       1060 |  -825 |  -760 |  -730 | -1040 | -1060 |  -965 | -1010 |  -905 |
| 1532 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        355 |          0 |        355 |   300 |   340 |   310 |   355 |   170 |   265 |   260 |   185 |
| 1533 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        240 |        -75 |        315 |   215 |   235 |   240 |   225 |   200 |   200 |    20 |   -75 |
| 1534 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |          0 |        240 |   240 |    90 |    35 |    75 |    75 |    95 |    70 |   115 |
| 1535 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -895 |        895 |  -730 |  -760 |  -670 |  -580 |  -635 |  -690 |  -560 |  -895 |
| 1536 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -560 |        560 |  -315 |  -290 |  -235 |  -280 |  -260 |  -230 |  -225 |  -560 |
| 1537 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        125 |        -70 |        195 |    65 |    95 |    55 |   125 |   105 |    65 |   110 |   -70 |
| 1538 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        170 |        -50 |        220 |   115 |   130 |   170 |   170 |   130 |   155 |     5 |   -50 |
| 1539 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -915 |        915 |  -740 |  -915 |  -825 |  -875 |  -845 |  -770 |  -895 |  -780 |
| 1540 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        170 |        -15 |        185 |   170 |   100 |    95 |    25 |   105 |   -15 |   100 |   100 |
| 1541 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -165 |        165 |   -50 |   -20 |   -35 |    -5 |   -90 |   -20 |   -15 |  -165 |
| 1542 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |          0 |        205 |   155 |   205 |   170 |   170 |   120 |   145 |   155 |   200 |
| 1543 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        325 |        -70 |        395 |   235 |   325 |   240 |   275 |    80 |   -70 |    85 |   -35 |
| 1544 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -920 |        920 |  -640 |  -735 |  -675 |  -685 |  -620 |  -895 |  -895 |  -920 |
| 1545 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1165 |       1165 |  -995 |  -990 |  -940 |  -910 | -1110 | -1085 |  -970 | -1165 |
| 1546 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -845 |        845 |  -575 |  -575 |  -505 |  -490 |  -625 |  -540 |  -495 |  -845 |
| 1547 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -250 |        250 |  -250 |  -170 |  -245 |  -205 |  -215 |  -190 |   -70 |  -220 |
| 1548 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         20 |       -315 |        335 |    20 |   -95 |   -45 |   -55 |  -110 |  -300 |  -315 |  -170 |
| 1549 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -530 |        530 |  -450 |  -395 |  -370 |  -420 |  -380 |  -420 |  -315 |  -530 |
| 1550 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        315 |          0 |        315 |   315 |   265 |   145 |    40 |    95 |    70 |    45 |   175 |
| 1551 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -790 |        790 |  -575 |  -580 |  -575 |  -670 |  -740 |  -680 |  -790 |  -700 |
| 1552 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        275 |          0 |        275 |   165 |   275 |   145 |   190 |   220 |   195 |   140 |   185 |
| 1553 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         60 |       -125 |        185 |    60 |   -75 |   -60 |  -125 |   -70 |   -90 |   -95 |   -55 |
| 1554 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -505 |        505 |  -255 |  -310 |  -395 |  -435 |  -500 |  -505 |  -465 |  -480 |
| 1555 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -580 |        580 |  -320 |  -555 |  -530 |  -570 |  -510 |  -535 |  -580 |  -575 |
| 1556 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         50 |       -215 |        265 |   -75 |    15 |    50 |   -25 |    10 |    45 |   -40 |  -215 |
| 1557 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        130 |        -55 |        185 |     5 |    20 |    80 |   -55 |   130 |    75 |    80 |    85 |
| 1558 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        355 |          0 |        355 |   245 |   355 |   330 |   320 |    90 |   150 |   130 |    75 |
| 1559 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |       -150 |        375 |   205 |   120 |   135 |   225 |  -135 |    30 |  -150 |   -10 |
| 1560 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |       -115 |        400 |   220 |   160 |   135 |   285 |   205 |  -110 |  -115 |   -85 |
| 1561 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -990 |        990 |  -745 |  -850 |  -780 |  -800 |  -755 |  -990 |  -990 |  -990 |
| 1562 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        295 |        -40 |        335 |   255 |   295 |    25 |   -15 |   110 |   100 |    30 |   -40 |
| 1563 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         30 |       -210 |        240 |    -0 |    30 |    15 |   -20 |    15 |    25 |  -185 |  -210 |
| 1564 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        260 |          0 |        260 |   205 |   260 |   215 |   260 |   250 |   225 |     5 |    30 |
| 1565 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -305 |        305 |  -250 |  -190 |  -150 |  -190 |  -255 |  -190 |  -305 |  -200 |
| 1566 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        245 |        -95 |        340 |   130 |   130 |   245 |   240 |   190 |   155 |   120 |   -95 |
| 1567 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1195 |       1195 | -1085 | -1195 | -1120 | -1130 | -1065 | -1165 | -1005 | -1130 |
| 1568 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        210 |          0 |        210 |    90 |   110 |    65 |    65 |   140 |   210 |   125 |   160 |
| 1569 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         45 |       -135 |        180 |    15 |    10 |   -15 |    45 |   -45 |  -125 |  -130 |  -135 |
| 1570 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -405 |        405 |  -220 |  -255 |  -105 |  -180 |  -200 |  -155 |  -255 |  -405 |
| 1571 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -985 |        985 |  -825 |  -970 |  -920 |  -940 |  -960 |  -880 |  -985 |  -950 |
| 1572 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        305 |          0 |        305 |   280 |   305 |   250 |   235 |   195 |   125 |    90 |   195 |
| 1573 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        390 |          0 |        390 |   330 |   390 |   370 |   155 |    80 |    75 |    85 |   105 |
| 1574 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |        -35 |        305 |   150 |   270 |   -35 |    30 |    25 |   -15 |   -15 |   -30 |
| 1575 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         40 |        -60 |        100 |    40 |   -50 |   -60 |   -25 |    10 |    30 |   -60 |    -5 |
| 1576 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -355 |        355 |   -55 |  -130 |   -65 |  -150 |   -75 |  -220 |  -355 |  -315 |
| 1577 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -445 |        445 |  -170 |  -190 |  -190 |  -115 |  -240 |  -395 |  -445 |  -350 |
| 1578 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        130 |       -265 |        395 |    25 |   -60 |   130 |    25 |    40 |    70 |    -0 |  -265 |
| 1579 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        310 |          0 |        310 |   250 |   295 |   310 |   240 |    95 |    60 |    45 |    70 |
| 1580 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -770 |  -880 |  -685 |  -810 |  -645 |  -850 |  -790 |  -990 |
| 1581 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         30 |       -290 |        320 |    30 |     5 |  -215 |  -200 |  -255 |  -290 |  -260 |  -245 |
| 1582 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1165 |       1165 |  -965 |  -920 |  -930 |  -930 | -1165 | -1095 | -1045 | -1125 |
| 1583 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -655 |        655 |  -445 |  -420 |  -550 |  -305 |  -415 |  -655 |  -605 |  -550 |
| 1584 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -885 |        885 |  -705 |  -740 |  -780 |  -840 |  -865 |  -885 |  -820 |  -885 |
| 1585 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -785 |  -745 |  -715 |  -675 |  -750 |  -750 |  -720 |  -990 |
| 1586 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        280 |          0 |        280 |   215 |   280 |   265 |   185 |   275 |   260 |   210 |    30 |
| 1587 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        345 |          0 |        345 |   345 |   205 |   285 |   185 |   250 |   260 |   180 |    60 |
| 1588 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        395 |          0 |        395 |   300 |   310 |   355 |   340 |   300 |   395 |   165 |    70 |
| 1589 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        150 |       -105 |        255 |    85 |    50 |   150 |   -35 |   -80 |  -105 |   -25 |   -30 |
| 1590 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -735 |        735 |  -490 |  -535 |  -465 |  -680 |  -645 |  -695 |  -700 |  -735 |
| 1591 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        130 |        -65 |        195 |    95 |   100 |    70 |   115 |    30 |   130 |   130 |   -65 |
| 1592 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1045 |       1045 |  -910 |  -785 |  -890 |  -935 |  -905 | -1045 |  -985 | -1045 |
| 1593 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1050 |       1050 |  -970 | -1050 |  -955 | -1020 |  -960 |  -965 | -1035 |  -975 |
| 1594 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         10 |       -220 |        230 |    10 |   -80 |  -165 |   -10 |   -60 |  -120 |   -10 |  -220 |
| 1595 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -460 |        460 |   -70 |  -360 |  -300 |  -405 |  -460 |  -320 |  -340 |  -400 |
| 1596 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -875 |        875 |  -675 |  -680 |  -735 |  -755 |  -650 |  -650 |  -720 |  -875 |
| 1597 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        315 |          0 |        315 |   200 |   315 |   140 |   140 |   185 |    85 |   160 |   100 |
| 1598 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1095 |       1095 |  -830 |  -855 |  -830 |  -740 |  -845 | -1070 | -1025 | -1095 |
| 1599 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -340 |        340 |  -200 |  -180 |  -205 |  -135 |  -240 |  -200 |  -215 |  -340 |
| 1600 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         70 |       -120 |        190 |    70 |   -55 |   -35 |   -75 |   -40 |  -120 |  -115 |   -70 |
| 1601 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        210 |          0 |        210 |   170 |    85 |    55 |   125 |    40 |   210 |    85 |   100 |
| 1602 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1180 |       1180 |  -900 |  -990 |  -915 |  -900 | -1130 | -1070 | -1055 | -1180 |
| 1603 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -315 |        315 |   -80 |  -190 |  -145 |   -35 |   -70 |  -145 |   -55 |  -315 |
| 1604 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -715 |        715 |  -560 |  -645 |  -715 |  -715 |  -700 |  -625 |  -635 |  -670 |
| 1605 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -775 |        775 |  -580 |  -565 |  -640 |  -570 |  -550 |  -650 |  -685 |  -775 |
| 1606 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -400 |        400 |  -170 |  -190 |  -400 |  -340 |  -400 |  -270 |  -335 |  -360 |
| 1607 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -985 |        985 |  -790 |  -910 |  -870 |  -905 |  -970 |  -890 |  -985 |  -875 |
| 1608 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -675 |  -650 |  -710 |  -600 |  -675 |  -595 |  -715 |  -860 |
| 1609 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        270 |       -105 |        375 |   270 |   215 |   165 |   180 |  -105 |    60 |   -35 |    10 |
| 1610 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        230 |        -25 |        255 |    85 |   225 |   160 |   230 |   220 |   120 |   215 |   -25 |
| 1611 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        115 |       -165 |        280 |    15 |    55 |   115 |    55 |  -160 |  -165 |   -45 |   -35 |
| 1612 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        295 |          0 |        295 |   145 |   185 |   235 |   295 |   210 |   180 |    20 |    55 |
| 1613 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -705 |        705 |  -590 |  -705 |  -525 |  -615 |  -580 |  -640 |  -630 |  -635 |
| 1614 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -635 |        635 |  -335 |  -290 |  -635 |  -450 |  -495 |  -630 |  -530 |  -610 |
| 1615 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        140 |       -150 |        290 |   140 |   -80 |   -80 |  -120 |    40 |   -70 |  -150 |   -10 |
| 1616 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -635 |        635 |  -385 |  -390 |  -530 |  -415 |  -395 |  -420 |  -450 |  -635 |
| 1617 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         20 |       -270 |        290 |   -10 |  -130 |    20 |  -145 |  -115 |  -235 |  -255 |  -270 |
| 1618 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -780 |        780 |  -620 |  -655 |  -600 |  -660 |  -625 |  -585 |  -645 |  -780 |
| 1619 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        135 |       -140 |        275 |    95 |   135 |  -100 |   -80 |  -140 |  -100 |   -95 |    -5 |
| 1620 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        190 |          0 |        190 |   130 |   130 |   185 |   155 |   185 |   120 |   190 |    35 |
| 1621 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1145 |       1145 |  -890 |  -845 |  -900 |  -915 |  -825 |  -920 | -1145 | -1015 |
| 1622 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -810 |        810 |  -800 |  -785 |  -730 |  -780 |  -745 |  -630 |  -720 |  -810 |
| 1623 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        220 |        -30 |        250 |   170 |   170 |   220 |    35 |    35 |     5 |     5 |   -30 |
| 1624 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        135 |       -165 |        300 |   135 |    10 |    95 |    65 |    20 |    55 |    65 |  -165 |
| 1625 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -805 |        805 |  -805 |  -740 |  -715 |  -705 |  -695 |  -765 |  -685 |  -730 |
| 1626 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1155 |       1155 |  -850 |  -830 |  -900 |  -910 |  -850 |  -945 |  -840 | -1155 |
| 1627 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -620 |        620 |  -410 |  -425 |  -295 |  -440 |  -480 |  -390 |  -470 |  -620 |
| 1628 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        390 |          0 |        390 |   275 |   295 |   390 |   290 |   315 |   355 |   285 |   340 |
| 1629 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        100 |       -245 |        345 |  -115 |   -10 |   100 |  -225 |  -185 |  -140 |  -135 |  -245 |
| 1630 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -415 |        415 |  -135 |  -110 |   -80 |  -335 |  -270 |  -310 |  -325 |  -415 |
| 1631 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1005 |       1005 |  -645 |  -725 |  -765 |  -740 |  -685 |  -805 |  -870 | -1005 |
| 1632 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1165 |       1165 |  -935 |  -930 | -1155 | -1115 | -1040 | -1045 | -1165 | -1110 |
| 1633 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        315 |        -50 |        365 |   255 |   185 |   225 |   245 |   315 |   -50 |   115 |    20 |
| 1634 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -655 |        655 |  -575 |  -545 |  -540 |  -640 |  -575 |  -545 |  -655 |  -580 |
| 1635 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -725 |        725 |  -645 |  -575 |  -625 |  -650 |  -600 |  -725 |  -640 |  -725 |
| 1636 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         80 |       -250 |        330 |   -35 |    80 |    75 |    50 |    45 |    65 |  -155 |  -250 |
| 1637 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -335 |        335 |  -220 |   -50 |  -145 |  -210 |  -165 |   -65 |  -155 |  -335 |
| 1638 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -605 |        605 |  -390 |  -330 |  -520 |  -475 |  -440 |  -375 |  -385 |  -605 |
| 1639 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        380 |          0 |        380 |   380 |   375 |   330 |   365 |   335 |   125 |   170 |   195 |
| 1640 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        165 |        -90 |        255 |   125 |    10 |   165 |    95 |    85 |   -90 |   -45 |    40 |
| 1641 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |        -45 |        320 |   140 |   185 |   140 |   220 |   275 |   -45 |   -35 |   -10 |
| 1642 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -940 |        940 |  -775 |  -675 |  -775 |  -740 |  -805 |  -845 |  -735 |  -940 |
| 1643 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -885 |        885 |  -695 |  -815 |  -730 |  -850 |  -885 |  -775 |  -705 |  -860 |
| 1644 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        320 |          0 |        320 |   295 |   320 |   260 |   240 |   215 |   215 |   145 |     5 |
| 1645 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |        -60 |        340 |   280 |   215 |   205 |   -60 |    45 |   -40 |   -25 |   105 |
| 1646 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        280 |        -65 |        345 |   195 |   245 |   175 |   280 |   240 |    35 |   -65 |    60 |
| 1647 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -830 |        830 |  -450 |  -830 |  -660 |  -760 |  -655 |  -605 |  -640 |  -680 |
| 1648 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        365 |          0 |        365 |   305 |   200 |   365 |   335 |   270 |   340 |   310 |   100 |
| 1649 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -415 |        415 |  -155 |  -200 |  -315 |  -285 |  -345 |  -325 |  -365 |  -415 |
| 1650 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -405 |        405 |  -355 |  -375 |  -360 |  -390 |  -405 |  -400 |  -365 |  -305 |
| 1651 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1095 |       1095 |  -850 |  -860 |  -755 | -1065 | -1025 | -1095 | -1005 | -1055 |
| 1652 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        220 |       -110 |        330 |   220 |   180 |  -110 |    50 |     5 |    55 |    25 |    70 |
| 1653 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -380 |        380 |  -260 |  -170 |  -185 |  -165 |   -45 |   -85 |  -380 |  -320 |
| 1654 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -495 |        495 |  -430 |  -490 |  -495 |  -480 |  -455 |  -475 |  -455 |  -395 |
| 1655 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -710 |        710 |  -380 |  -470 |  -380 |  -385 |  -410 |  -530 |  -710 |  -605 |
| 1656 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        395 |          0 |        395 |   210 |   255 |   395 |   310 |   300 |   240 |    60 |    95 |
| 1657 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1115 |       1115 |  -850 |  -880 |  -965 |  -960 |  -815 | -1035 | -1115 | -1070 |
| 1658 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -920 |        920 |  -665 |  -640 |  -650 |  -625 |  -740 |  -820 |  -920 |  -825 |
| 1659 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -680 |        680 |  -290 |  -325 |  -485 |  -400 |  -605 |  -540 |  -665 |  -680 |
| 1660 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        205 |          0 |        205 |   205 |    20 |    30 |    60 |   110 |   100 |    85 |   145 |
| 1661 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -580 |        580 |  -315 |  -345 |  -305 |  -395 |  -400 |  -345 |  -245 |  -580 |
| 1662 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        225 |        -50 |        275 |   225 |   190 |   155 |    95 |   220 |   135 |   -50 |     5 |
| 1663 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        280 |          0 |        280 |   235 |   265 |   280 |   245 |   205 |   175 |   190 |    90 |
| 1664 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -320 |        320 |   -50 |   -80 |  -175 |   -60 |  -320 |  -320 |  -290 |  -235 |
| 1665 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -890 |        890 |  -720 |  -685 |  -620 |  -680 |  -795 |  -700 |  -800 |  -890 |
| 1666 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1085 |       1085 |  -775 |  -815 |  -830 |  -790 |  -855 |  -765 | -1085 |  -930 |
| 1667 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -905 |        905 |  -665 |  -600 |  -645 |  -690 |  -630 |  -885 |  -890 |  -905 |
| 1668 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        400 |          0 |        400 |   305 |   300 |   340 |   355 |   400 |   135 |    85 |    80 |
| 1669 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        230 |        -40 |        270 |   230 |   125 |   110 |   150 |    10 |   -40 |    35 |   -35 |
| 1670 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -610 |        610 |  -410 |  -370 |  -420 |  -275 |  -360 |  -460 |  -610 |  -560 |
| 1671 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        210 |       -150 |        360 |   135 |   185 |   210 |   150 |   120 |   210 |   110 |  -150 |
| 1672 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        305 |          0 |        305 |   305 |   165 |   160 |    90 |   165 |   100 |    60 |   135 |
| 1673 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -685 |        685 |  -585 |  -685 |  -560 |  -550 |  -580 |  -550 |  -625 |  -610 |
| 1674 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -960 |        960 |  -835 |  -885 |  -760 |  -865 |  -845 |  -805 |  -720 |  -960 |
| 1675 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -415 |        415 |  -155 |  -415 |  -345 |  -355 |  -330 |  -355 |  -340 |  -365 |
| 1676 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |          0 |        265 |   265 |   260 |   115 |    30 |    65 |    40 |    50 |    60 |
| 1677 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        320 |          0 |        320 |   270 |   215 |   240 |   120 |   150 |   220 |   180 |   320 |
| 1678 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        285 |          0 |        285 |   285 |   265 |    90 |   155 |    75 |   120 |    80 |    55 |
| 1679 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1175 |       1175 |  -940 |  -795 |  -850 | -1175 |  -990 | -1005 | -1020 | -1060 |
| 1680 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -555 |        555 |  -435 |  -430 |  -425 |  -530 |  -485 |  -545 |  -380 |  -555 |
| 1681 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        230 |       -105 |        335 |   135 |   230 |   -70 |    -5 |   -45 |  -105 |   -65 |   -80 |
| 1682 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         80 |       -225 |        305 |    80 |    55 |    40 |    15 |  -200 |  -225 |  -165 |  -165 |
| 1683 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -505 |        505 |  -330 |  -255 |  -270 |  -200 |  -380 |  -370 |  -505 |  -480 |
| 1684 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -625 |        625 |  -430 |  -285 |  -435 |  -325 |  -355 |  -420 |  -320 |  -625 |
| 1685 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -430 |        430 |  -395 |  -385 |  -270 |  -395 |  -260 |  -430 |  -350 |  -350 |
| 1686 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        225 |        -60 |        285 |   115 |   125 |   225 |    25 |   -60 |     5 |   -55 |    15 |
| 1687 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        375 |          0 |        375 |   220 |   320 |   375 |    35 |   100 |   105 |   185 |   180 |
| 1688 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -295 |        295 |   -55 |   -70 |   -30 |   -20 |   -35 |  -220 |  -295 |  -180 |
| 1689 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -295 |        295 |  -160 |  -190 |  -225 |  -160 |  -295 |  -260 |  -260 |  -215 |
| 1690 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1110 |       1110 | -1040 |  -995 |  -950 | -1110 | -1010 | -1085 | -1050 | -1075 |
| 1691 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        155 |       -190 |        345 |   110 |    80 |   155 |    70 |    95 |    25 |    45 |  -190 |
| 1692 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1010 |       1010 |  -755 |  -945 |  -975 |  -980 |  -935 | -1000 | -1010 |  -870 |
| 1693 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -380 |        380 |  -190 |  -150 |   -75 |  -295 |  -370 |  -270 |  -380 |  -310 |
| 1694 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        205 |        -40 |        245 |   145 |   150 |   205 |    15 |    55 |    35 |   -40 |    -5 |
| 1695 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -390 |        390 |  -230 |  -165 |  -245 |   -85 |  -135 |  -135 |  -390 |  -305 |
| 1696 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        180 |       -100 |        280 |   180 |    15 |  -100 |   -40 |   -20 |   -20 |    -0 |    45 |
| 1697 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -845 |        845 |  -770 |  -720 |  -750 |  -695 |  -745 |  -825 |  -845 |  -735 |
| 1698 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        160 |       -165 |        325 |    70 |   160 |    50 |   -50 |   -80 |   -30 |  -165 |  -100 |
| 1699 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        365 |          0 |        365 |   330 |   365 |   175 |   255 |   305 |   280 |   290 |   190 |
| 1700 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        185 |        -75 |        260 |    75 |    90 |   185 |    95 |   -75 |    30 |   -30 |   -35 |
| 1701 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -395 |        395 |   -40 |  -330 |  -395 |  -355 |  -325 |  -385 |  -335 |  -340 |
| 1702 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -480 |        480 |  -250 |  -265 |  -375 |  -295 |  -160 |  -295 |  -285 |  -480 |
| 1703 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        290 |          0 |        290 |   290 |   260 |    70 |    85 |    80 |   135 |   170 |   100 |
| 1704 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -530 |        530 |  -305 |  -330 |  -255 |  -290 |  -325 |  -470 |  -475 |  -530 |
| 1705 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1120 |       1120 | -1045 | -1075 | -1020 | -1110 | -1120 | -1105 | -1005 | -1100 |
| 1706 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -735 |        735 |  -590 |  -510 |  -525 |  -625 |  -680 |  -670 |  -735 |  -620 |
| 1707 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        280 |        -10 |        290 |   170 |   205 |   220 |   215 |   185 |   280 |    45 |   -10 |
| 1708 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -635 |        635 |  -455 |  -580 |  -585 |  -585 |  -560 |  -605 |  -635 |  -520 |
| 1709 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -635 |        635 |  -340 |  -375 |  -345 |  -305 |  -635 |  -500 |  -550 |  -495 |
| 1710 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        275 |        -75 |        350 |   275 |   160 |   120 |   -15 |   -75 |    -0 |   -45 |    25 |
| 1711 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -250 |        250 |   -70 |   -60 |   -70 |  -100 |  -230 |  -230 |  -210 |  -250 |
| 1712 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1005 |       1005 |  -740 |  -775 |  -795 |  -780 |  -660 |  -795 |  -975 | -1005 |
| 1713 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        165 |        -60 |        225 |   120 |   165 |    40 |   -25 |    25 |   -60 |    35 |    85 |
| 1714 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1045 |       1045 |  -820 |  -645 |  -840 | -1010 | -1045 | -1000 | -1035 |  -880 |
| 1715 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        125 |        -80 |        205 |     5 |   -10 |   -80 |    -5 |   105 |   -55 |   125 |     5 |
| 1716 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        185 |          0 |        185 |   185 |   150 |    55 |    45 |    10 |    90 |    55 |   135 |
| 1717 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        245 |       -130 |        375 |   245 |   235 |   -80 |   -35 |   -25 |   -60 |  -130 |   -50 |
| 1718 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        285 |        -80 |        365 |   205 |    30 |   185 |   285 |   120 |   -80 |    50 |   -50 |
| 1719 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         50 |       -255 |        305 |   -30 |   -20 |    -0 |    30 |    50 |  -195 |  -220 |  -255 |
| 1720 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -845 |        845 |  -465 |  -620 |  -515 |  -550 |  -590 |  -535 |  -845 |  -795 |
| 1721 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -500 |        500 |  -280 |  -290 |  -225 |  -495 |  -425 |  -500 |  -455 |  -425 |
| 1722 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        310 |          0 |        310 |   290 |   310 |   225 |    50 |    75 |    35 |    90 |   130 |
| 1723 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1015 |       1015 |  -790 |  -780 |  -795 |  -700 |  -920 |  -990 |  -920 | -1015 |
| 1724 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -955 |        955 |  -725 |  -705 |  -800 |  -695 |  -750 |  -725 |  -810 |  -955 |
| 1725 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        305 |          0 |        305 |   220 |   250 |   185 |   305 |   200 |   285 |   200 |   265 |
| 1726 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -950 |        950 |  -695 |  -680 |  -665 |  -695 |  -705 |  -710 |  -715 |  -950 |
| 1727 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -725 |        725 |  -485 |  -420 |  -495 |  -445 |  -605 |  -480 |  -470 |  -725 |
| 1728 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         70 |       -275 |        345 |    -5 |    70 |  -180 |  -230 |  -275 |  -175 |  -225 |  -175 |
| 1729 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1000 |       1000 |  -795 |  -745 |  -745 | -1000 |  -875 |  -910 |  -970 |  -905 |
| 1730 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -535 |        535 |  -395 |  -260 |  -255 |  -305 |  -280 |  -330 |  -535 |  -470 |
| 1731 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         80 |       -255 |        335 |   -30 |   -50 |   -60 |    80 |  -135 |  -180 |  -215 |  -255 |
| 1732 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1005 |       1005 |  -800 |  -825 |  -675 |  -760 |  -890 | -1000 |  -940 | -1005 |
| 1733 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -340 |        340 |  -245 |  -285 |  -305 |  -325 |  -290 |  -210 |  -340 |  -320 |
| 1734 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -865 |  -810 |  -805 |  -820 |  -865 |  -850 |  -840 | -1080 |
| 1735 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        270 |        -25 |        295 |   270 |    50 |   120 |    35 |   115 |    55 |   -25 |   110 |
| 1736 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -885 |        885 |  -665 |  -705 |  -625 |  -630 |  -585 |  -885 |  -830 |  -835 |
| 1737 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -650 |        650 |  -415 |  -395 |  -475 |  -330 |  -500 |  -530 |  -500 |  -650 |
| 1738 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -720 |        720 |  -720 |  -620 |  -620 |  -645 |  -645 |  -675 |  -670 |  -600 |
| 1739 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1080 |       1080 |  -795 |  -845 |  -830 |  -905 |  -890 | -1060 | -1040 | -1080 |
| 1740 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1175 |       1175 | -1075 | -1020 |  -985 |  -960 |  -940 | -1010 | -1155 | -1175 |
| 1741 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1010 |       1010 |  -780 |  -720 |  -795 |  -805 |  -735 |  -790 |  -885 | -1010 |
| 1742 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -320 |        320 |  -140 |  -250 |  -275 |  -190 |  -320 |  -235 |  -245 |  -160 |
| 1743 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -300 |        300 |   -85 |   -25 |  -275 |  -205 |  -285 |  -300 |  -280 |  -300 |
| 1744 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        385 |          0 |        385 |   315 |   385 |   305 |   305 |   280 |   350 |   160 |   135 |
| 1745 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        210 |          0 |        210 |   210 |   190 |   140 |   105 |    85 |   180 |     5 |    30 |
| 1746 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -285 |        285 |  -110 |   -50 |   -25 |   -70 |   -45 |  -170 |  -285 |  -190 |
| 1747 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -460 |        460 |  -205 |  -195 |  -250 |  -305 |  -255 |  -190 |  -245 |  -460 |
| 1748 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        385 |          0 |        385 |   265 |   195 |   240 |   385 |   325 |   275 |   145 |   105 |
| 1749 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -815 |        815 |  -525 |  -540 |  -430 |  -585 |  -815 |  -750 |  -770 |  -775 |
| 1750 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        380 |          0 |        380 |   355 |   350 |   380 |   365 |   305 |   360 |   255 |   360 |
| 1751 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        200 |          0 |        200 |   190 |   200 |   200 |   130 |   175 |   170 |   140 |   150 |
| 1752 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        140 |       -200 |        340 |    40 |   140 |  -110 |   -65 |  -145 |  -200 |  -145 |  -140 |
| 1753 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        135 |          0 |        135 |   135 |   135 |   110 |    85 |    95 |    60 |    85 |    55 |
| 1754 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        400 |          0 |        400 |   400 |   395 |   385 |   380 |   320 |   190 |   110 |    50 |
| 1755 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        290 |          0 |        290 |   290 |   285 |   210 |   175 |   200 |   135 |   190 |   115 |
| 1756 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -585 |        585 |  -315 |  -400 |  -560 |  -510 |  -585 |  -505 |  -540 |  -570 |
| 1757 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -670 |        670 |  -310 |  -315 |  -330 |  -490 |  -525 |  -610 |  -670 |  -575 |
| 1758 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        285 |        -35 |        320 |   285 |   270 |    20 |   100 |    50 |    45 |    -5 |   -35 |
| 1759 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -890 |        890 |  -890 |  -825 |  -855 |  -850 |  -840 |  -860 |  -875 |  -890 |
| 1760 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        205 |        -55 |        260 |   175 |   165 |   155 |   205 |    -0 |   -15 |   -55 |   -30 |
| 1761 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1025 |       1025 |  -990 | -1025 |  -890 | -1015 |  -980 | -1015 | -1015 |  -960 |
| 1762 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1050 |       1050 |  -945 |  -945 |  -900 |  -895 |  -880 |  -905 | -1005 | -1050 |
| 1763 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -275 |        275 |  -170 |   -85 |  -180 |   -65 |   -80 |  -205 |   -15 |  -275 |
| 1764 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        280 |        -50 |        330 |   210 |   215 |   280 |   235 |   125 |   -50 |   -45 |    65 |
| 1765 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        140 |        -90 |        230 |   140 |   125 |   115 |    95 |    90 |   100 |   -90 |   -25 |
| 1766 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -970 |        970 |  -760 |  -770 |  -840 |  -725 |  -770 |  -690 |  -800 |  -970 |
| 1767 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -595 |        595 |  -420 |  -525 |  -595 |  -575 |  -570 |  -585 |  -535 |  -575 |
| 1768 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        265 |         -0 |        265 |   260 |   265 |   180 |   225 |   255 |    -0 |    40 |     5 |
| 1769 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -415 |        415 |  -130 |  -150 |  -325 |  -415 |  -355 |  -270 |  -245 |  -360 |
| 1770 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -495 |        495 |  -280 |  -245 |  -225 |  -200 |  -495 |  -360 |  -465 |  -330 |
| 1771 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        200 |       -130 |        330 |   110 |   200 |  -130 |   -70 |   -60 |   -95 |  -115 |  -115 |
| 1772 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -370 |        370 |  -180 |   -75 |  -125 |  -120 |   -50 |  -340 |  -245 |  -370 |
| 1773 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -730 |        730 |  -475 |  -455 |  -445 |  -665 |  -730 |  -650 |  -680 |  -715 |
| 1774 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -730 |        730 |  -530 |  -570 |  -545 |  -510 |  -565 |  -520 |  -520 |  -730 |
| 1775 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         70 |       -280 |        350 |   -70 |   -80 |   -10 |    70 |  -280 |  -155 |  -225 |  -200 |
| 1776 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -495 |        495 |  -245 |  -225 |  -210 |  -325 |  -305 |  -495 |  -465 |  -485 |
| 1777 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -940 |        940 |  -735 |  -835 |  -730 |  -755 |  -710 |  -740 |  -745 |  -940 |
| 1778 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         35 |       -160 |        195 |   -30 |   -90 |   -95 |     5 |  -160 |    35 |     5 |    20 |
| 1779 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -435 |        435 |  -430 |  -330 |  -435 |  -395 |  -260 |  -320 |  -380 |  -405 |
| 1780 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -905 |        905 |  -740 |  -720 |  -830 |  -700 |  -720 |  -695 |  -905 |  -900 |
| 1781 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1020 |       1020 |  -785 |  -825 |  -775 |  -795 |  -980 | -1020 |  -990 |  -965 |
| 1782 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        165 |       -125 |        290 |    15 |   -15 |   165 |    10 |    75 |    10 |   110 |  -125 |
| 1783 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1045 |       1045 |  -795 |  -840 |  -740 | -1020 |  -990 |  -995 | -1000 | -1045 |
| 1784 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        195 |        -90 |        285 |   120 |   195 |   -90 |   -30 |    35 |   -40 |    25 |   -15 |
| 1785 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -460 |        460 |  -180 |  -185 |  -285 |  -265 |  -170 |  -260 |  -460 |  -405 |
| 1786 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        235 |        -10 |        245 |   195 |   180 |   200 |   145 |   235 |    60 |   160 |   -10 |
| 1787 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        380 |          0 |        380 |   295 |   360 |   375 |   380 |   370 |   275 |    50 |    50 |
| 1788 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        365 |          0 |        365 |   365 |   320 |   340 |   280 |   290 |   295 |   220 |    55 |
| 1789 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         10 |       -375 |        385 |  -110 |    10 |   -20 |  -120 |  -155 |  -240 |  -370 |  -375 |
| 1790 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        365 |          0 |        365 |   330 |   365 |   275 |   255 |   315 |    85 |    95 |   165 |
| 1791 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        275 |          0 |        275 |   245 |   235 |   275 |   260 |   100 |    55 |   135 |   115 |
| 1792 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1005 |       1005 |  -805 |  -790 |  -645 |  -780 | -1005 |  -880 |  -900 |  -855 |
| 1793 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        300 |        -20 |        320 |   150 |   270 |   175 |   100 |   300 |    55 |   -20 |    -0 |
| 1794 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -720 |        720 |  -555 |  -495 |  -530 |  -495 |  -480 |  -565 |  -490 |  -720 |
| 1795 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        375 |          0 |        375 |   375 |   315 |   360 |   110 |   140 |   230 |   140 |   130 |
| 1796 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -710 |        710 |  -325 |  -460 |  -310 |  -500 |  -625 |  -635 |  -710 |  -690 |
| 1797 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        215 |       -150 |        365 |    90 |   120 |   150 |   115 |   215 |  -150 |  -145 |   -80 |
| 1798 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        195 |       -195 |        390 |    85 |   195 |   110 |   -40 |  -115 |    15 |   -40 |  -195 |
| 1799 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        285 |          0 |        285 |   245 |   175 |   110 |   285 |   255 |   125 |   180 |   170 |
| 1800 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        325 |         -5 |        330 |   250 |   325 |   200 |   270 |   290 |   280 |   175 |    -5 |
| 1801 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        205 |       -195 |        400 |   190 |   205 |   125 |  -195 |   -75 |   -30 |   -50 |   -45 |
| 1802 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        380 |          0 |        380 |   380 |   100 |   160 |   115 |   125 |   130 |   210 |   135 |
| 1803 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         90 |       -190 |        280 |   -25 |    90 |    10 |  -190 |  -140 |  -110 |   -70 |   -85 |
| 1804 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        120 |       -240 |        360 |   120 |    95 |   115 |   110 |    95 |   120 |  -170 |  -240 |
| 1805 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -865 |        865 |  -545 |  -600 |  -820 |  -765 |  -865 |  -830 |  -795 |  -745 |
| 1806 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -815 |        815 |  -645 |  -570 |  -550 |  -580 |  -625 |  -765 |  -815 |  -750 |
| 1807 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        195 |       -120 |        315 |   195 |   165 |   -20 |    30 |  -105 |   -90 |   -50 |  -120 |
| 1808 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -950 |        950 |  -665 |  -610 |  -820 |  -825 |  -855 |  -870 |  -870 |  -950 |
| 1809 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -310 |        310 |  -180 |   -95 |   -70 |   -55 |   -25 |  -140 |  -235 |  -310 |
| 1810 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -620 |        620 |  -290 |  -620 |  -590 |  -475 |  -430 |  -610 |  -475 |  -580 |
| 1811 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        165 |          0 |        165 |    30 |   120 |    45 |   120 |    85 |    65 |   165 |     5 |
| 1812 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -875 |        875 |  -705 |  -490 |  -735 |  -830 |  -765 |  -755 |  -875 |  -815 |
| 1813 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        210 |        -55 |        265 |   210 |   125 |   125 |   135 |   135 |   210 |   -55 |    20 |
| 1814 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -115 |        115 |    -5 |   -90 |   -60 |  -110 |   -10 |   -40 |  -115 |   -30 |
| 1815 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1125 |       1125 |  -860 |  -795 |  -925 |  -875 |  -940 |  -870 | -1125 | -1105 |
| 1816 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -650 |        650 |  -395 |  -430 |  -425 |  -360 |  -395 |  -530 |  -650 |  -515 |
| 1817 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        140 |       -105 |        245 |    60 |    80 |    40 |   100 |    35 |   140 |  -105 |   -85 |
| 1818 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        260 |          0 |        260 |   210 |   220 |   225 |   260 |   190 |   155 |   205 |   100 |
| 1819 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -930 |        930 |  -930 |  -805 |  -875 |  -770 |  -845 |  -715 |  -845 |  -910 |
| 1820 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -595 |        595 |  -205 |  -255 |  -475 |  -475 |  -595 |  -385 |  -535 |  -480 |
| 1821 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -850 |        850 |  -525 |  -550 |  -590 |  -560 |  -665 |  -535 |  -755 |  -850 |
| 1822 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        400 |          0 |        400 |   290 |   400 |   280 |   350 |   220 |   105 |    45 |    40 |
| 1823 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        380 |          0 |        380 |   350 |   340 |   255 |   380 |   140 |    95 |   220 |   185 |
| 1824 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -600 |        600 |  -600 |  -455 |  -485 |  -525 |  -580 |  -520 |  -545 |  -555 |
| 1825 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        225 |        -25 |        250 |   225 |    25 |    40 |    95 |    70 |    70 |    15 |   -25 |
| 1826 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -595 |        595 |  -330 |  -595 |  -595 |  -540 |  -510 |  -545 |  -585 |  -570 |
| 1827 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1185 |       1185 |  -925 | -1185 | -1070 | -1095 | -1040 | -1080 | -1140 | -1035 |
| 1828 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        275 |        -40 |        315 |   220 |   270 |   140 |   270 |   190 |   275 |   -40 |   -10 |
| 1829 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        400 |          0 |        400 |   400 |   395 |   370 |   305 |   180 |   140 |   230 |   335 |
| 1830 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        255 |          0 |        255 |   255 |    85 |   180 |   165 |   200 |    65 |    10 |     5 |
| 1831 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -725 |        725 |  -725 |  -590 |  -685 |  -690 |  -700 |  -645 |  -600 |  -710 |
| 1832 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -845 |        845 |  -665 |  -620 |  -570 |  -655 |  -820 |  -780 |  -845 |  -790 |
| 1833 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        235 |        -70 |        305 |   190 |   235 |   200 |    20 |   -25 |    65 |   -10 |   -70 |
| 1834 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -525 |        525 |  -265 |  -240 |  -280 |  -340 |  -295 |  -525 |  -520 |  -480 |
| 1835 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         50 |       -250 |        300 |   -55 |   -60 |    50 |  -160 |  -240 |  -250 |  -245 |  -200 |
| 1836 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        170 |       -100 |        270 |   170 |   130 |   160 |    40 |   -10 |  -100 |    25 |    20 |
| 1837 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -700 |        700 |  -420 |  -700 |  -625 |  -685 |  -620 |  -640 |  -700 |  -615 |
| 1838 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1115 |       1115 | -1070 | -1080 | -1070 | -1040 | -1115 |  -995 | -1080 | -1090 |
| 1839 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         45 |       -175 |        220 |   -40 |    35 |    20 |    45 |   -25 |   -95 |  -150 |  -175 |
| 1840 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        160 |        -95 |        255 |   160 |    75 |   130 |    85 |    20 |   160 |   -95 |    55 |
| 1841 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         10 |       -245 |        255 |   -75 |   -65 |    10 |  -240 |  -100 |  -180 |  -245 |  -235 |
| 1842 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -450 |        450 |  -215 |  -420 |  -375 |  -390 |  -320 |  -450 |  -360 |  -410 |
| 1843 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -445 |        445 |  -285 |  -200 |  -180 |  -180 |  -135 |  -395 |  -445 |  -365 |
| 1844 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         90 |       -170 |        260 |    90 |  -160 |   -30 |  -170 |  -110 |   -80 |   -95 |    -0 |
| 1845 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1110 |       1110 |  -830 |  -840 | -1105 | -1050 | -1065 | -1110 | -1000 | -1095 |
| 1846 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |          0 |        350 |   350 |   240 |   305 |   260 |   340 |   315 |    95 |    40 |
| 1847 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -515 |        515 |  -255 |  -135 |  -180 |  -180 |  -285 |  -360 |  -515 |  -425 |
| 1848 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -675 |        675 |  -500 |  -320 |  -605 |  -665 |  -635 |  -655 |  -675 |  -675 |
| 1849 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        295 |          0 |        295 |   145 |   250 |   295 |   255 |   175 |    65 |    65 |   185 |
| 1850 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -480 |        480 |  -260 |  -260 |  -270 |  -440 |  -210 |  -270 |  -415 |  -480 |
| 1851 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        205 |        -65 |        270 |   205 |   150 |   100 |   165 |   170 |   170 |   120 |   -65 |
| 1852 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        100 |          0 |        100 |    95 |    65 |    60 |   100 |     5 |    15 |    75 |    85 |
| 1853 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         45 |       -270 |        315 |     5 |    45 |    -5 |    15 |  -270 |   -85 |  -165 |  -250 |
| 1854 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1020 |       1020 |  -765 |  -820 |  -795 |  -710 |  -765 |  -850 | -1020 | -1015 |
| 1855 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -900 |        900 |  -570 |  -655 |  -900 |  -840 |  -865 |  -830 |  -800 |  -865 |
| 1856 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -575 |        575 |  -360 |  -355 |  -360 |  -355 |  -430 |  -545 |  -450 |  -575 |
| 1857 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        210 |       -105 |        315 |   210 |   130 |   145 |   -70 |  -105 |   -65 |   -15 |  -105 |
| 1858 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        385 |          0 |        385 |   365 |   280 |   305 |   385 |   105 |   170 |   135 |   100 |
| 1859 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -540 |        540 |  -205 |  -250 |  -420 |  -385 |  -540 |  -415 |  -395 |  -450 |
| 1860 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -810 |        810 |  -595 |  -640 |  -630 |  -640 |  -690 |  -600 |  -555 |  -810 |
| 1861 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        135 |       -210 |        345 |    35 |   -95 |   -20 |    85 |   135 |    25 |   -25 |  -210 |
| 1862 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |        -65 |        305 |   240 |   125 |   175 |   185 |   155 |   185 |    85 |   -65 |
| 1863 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        270 |        -30 |        300 |   270 |    90 |   110 |   125 |   -30 |    75 |    30 |   150 |
| 1864 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -795 |        795 |  -505 |  -630 |  -725 |  -795 |  -755 |  -735 |  -735 |  -750 |
| 1865 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1070 |       1070 |  -810 |  -890 |  -780 |  -875 |  -905 |  -850 |  -865 | -1070 |
| 1866 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        240 |          0 |        240 |   220 |   120 |   225 |   190 |   225 |   240 |   120 |   135 |
| 1867 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -860 |        860 |  -540 |  -590 |  -605 |  -680 |  -830 |  -785 |  -765 |  -860 |
| 1868 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1000 |       1000 |  -825 |  -835 |  -805 |  -750 |  -790 |  -870 |  -765 | -1000 |
| 1869 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -425 |        425 |  -100 |  -410 |  -360 |  -310 |  -350 |  -425 |  -235 |  -340 |
| 1870 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        190 |       -195 |        385 |    25 |    70 |   190 |    20 |  -155 |  -195 |  -190 |  -110 |
| 1871 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -675 |        675 |  -440 |  -540 |  -490 |  -415 |  -675 |  -660 |  -635 |  -675 |
| 1872 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1140 |       1140 |  -950 |  -875 |  -905 | -1070 | -1140 | -1000 | -1060 | -1115 |
| 1873 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        340 |          0 |        340 |   340 |   220 |   260 |   230 |   290 |   220 |   235 |   220 |
| 1874 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |       -110 |        325 |    20 |   215 |   160 |   180 |   115 |    80 |  -100 |  -110 |
| 1875 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        335 |          0 |        335 |   330 |   285 |   290 |   335 |   210 |   300 |   185 |   250 |
| 1876 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1135 |       1135 |  -870 |  -815 |  -855 |  -915 |  -830 |  -810 | -1060 | -1135 |
| 1877 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        205 |       -130 |        335 |   175 |   140 |   185 |   205 |    50 |    -5 |  -130 |   -55 |
| 1878 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -450 |        450 |  -210 |  -375 |  -400 |  -375 |  -280 |  -350 |  -450 |  -370 |
| 1879 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        325 |          0 |        325 |   180 |   320 |   235 |   175 |   325 |   185 |   250 |     5 |
| 1880 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -935 |        935 |  -730 |  -790 |  -700 |  -725 |  -935 |  -900 |  -915 |  -900 |
| 1881 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        105 |       -215 |        320 |   105 |  -150 |  -215 |   -70 |  -110 |   -85 |   -70 |  -105 |
| 1882 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -370 |        370 |  -180 |  -110 |  -260 |  -345 |  -370 |  -325 |  -285 |  -355 |
| 1883 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -445 |        445 |  -280 |  -255 |  -235 |  -310 |  -305 |  -165 |  -290 |  -445 |
| 1884 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        370 |          0 |        370 |   370 |   285 |    35 |   115 |    55 |    70 |   105 |   190 |
| 1885 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        220 |        -70 |        290 |   220 |   125 |   130 |   215 |   -10 |    40 |   -70 |   -45 |
| 1886 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1075 |       1075 |  -925 | -1055 |  -990 | -1025 | -1000 | -1075 |  -980 |  -905 |
| 1887 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -655 |        655 |  -395 |  -430 |  -415 |  -435 |  -355 |  -390 |  -655 |  -605 |
| 1888 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |       -145 |        360 |   190 |   180 |   175 |   215 |   170 |   125 |  -145 |   -10 |
| 1889 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -920 |        920 |  -615 |  -695 |  -650 |  -780 |  -630 |  -690 |  -920 |  -905 |
| 1890 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        305 |          0 |        305 |   305 |   200 |   110 |    30 |    55 |   100 |   130 |    70 |
| 1891 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        200 |       -105 |        305 |   165 |   120 |   200 |   -30 |  -105 |   -40 |    40 |   -85 |
| 1892 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        395 |          0 |        395 |   335 |   350 |   390 |   300 |   395 |   335 |   325 |   135 |
| 1893 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        205 |        -55 |        260 |   170 |   205 |   110 |   -55 |    45 |    90 |    60 |    45 |
| 1894 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1050 |       1050 |  -850 |  -790 |  -820 |  -765 |  -655 |  -930 | -1050 |  -975 |
| 1895 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        280 |        -20 |        300 |   280 |   245 |   125 |   135 |   -20 |   115 |    60 |    -0 |
| 1896 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        185 |        -40 |        225 |   160 |   185 |   140 |   160 |    60 |   -40 |    80 |    55 |
| 1897 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        205 |          0 |        205 |   205 |    15 |     5 |    90 |    20 |    75 |    15 |    10 |
| 1898 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -655 |        655 |  -365 |  -430 |  -390 |  -580 |  -540 |  -655 |  -650 |  -605 |
| 1899 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -475 |        475 |  -190 |  -395 |  -380 |  -435 |  -385 |  -350 |  -295 |  -475 |
| 1900 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        350 |          0 |        350 |   295 |   350 |   110 |   235 |   180 |   140 |   195 |    90 |
| 1901 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -395 |        395 |  -105 |   -80 |  -175 |  -340 |  -295 |  -330 |  -275 |  -395 |
| 1902 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -840 |        840 |  -570 |  -625 |  -575 |  -810 |  -835 |  -840 |  -745 |  -765 |
| 1903 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -855 |        855 |  -645 |  -545 |  -585 |  -695 |  -575 |  -680 |  -610 |  -855 |
| 1904 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -340 |        340 |   -45 |  -340 |  -230 |  -145 |  -180 |  -250 |  -280 |  -190 |
| 1905 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -520 |        520 |  -260 |  -275 |  -210 |  -190 |  -265 |  -430 |  -520 |  -460 |
| 1906 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |       -115 |        380 |   100 |   120 |   265 |   -30 |   -30 |  -115 |   -85 |    60 |
| 1907 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        270 |        -65 |        335 |   120 |   270 |    30 |   185 |   -15 |    20 |   -30 |   -65 |
| 1908 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -575 |        575 |  -360 |  -545 |  -575 |  -515 |  -500 |  -550 |  -525 |  -555 |
| 1909 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -935 |        935 |  -735 |  -750 |  -775 |  -790 |  -770 |  -660 |  -775 |  -935 |
| 1910 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -700 |        700 |  -600 |  -550 |  -530 |  -545 |  -700 |  -670 |  -630 |  -590 |
| 1911 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        380 |          0 |        380 |   380 |   300 |   330 |    90 |   170 |   120 |    35 |    10 |
| 1912 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1035 |       1035 |  -990 | -1000 |  -875 |  -875 |  -900 | -1035 |  -895 |  -985 |
| 1913 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -600 |        600 |  -325 |  -490 |  -460 |  -425 |  -580 |  -510 |  -440 |  -600 |
| 1914 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -810 |        810 |  -520 |  -765 |  -675 |  -775 |  -800 |  -755 |  -810 |  -645 |
| 1915 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        325 |        -35 |        360 |   185 |   325 |   245 |   235 |   105 |    30 |    30 |   -35 |
| 1916 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -765 |        765 |  -555 |  -475 |  -470 |  -540 |  -540 |  -435 |  -515 |  -765 |
| 1917 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -565 |        565 |  -365 |  -330 |  -435 |  -340 |  -440 |  -310 |  -565 |  -505 |
| 1918 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -510 |        510 |  -175 |  -265 |  -255 |  -250 |  -225 |  -135 |  -420 |  -510 |
| 1919 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        235 |       -135 |        370 |   235 |   200 |  -135 |   -25 |    -0 |   -25 |   -20 |    -0 |
| 1920 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        180 |       -105 |        285 |   140 |   180 |    55 |   -30 |    35 |   -55 |  -105 |    20 |
| 1921 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |          0 |        200 |   200 |   150 |   115 |   120 |   170 |   165 |   185 |    70 |
| 1922 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        335 |          0 |        335 |   330 |   185 |   250 |   225 |   175 |   335 |   240 |   245 |
| 1923 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        220 |        -30 |        250 |   165 |   220 |   155 |   200 |   195 |   170 |   -30 |    55 |
| 1924 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -900 |        900 |  -705 |  -685 |  -720 |  -870 |  -765 |  -820 |  -900 |  -800 |
| 1925 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -920 |        920 |  -715 |  -690 |  -695 |  -665 |  -650 |  -730 |  -595 |  -920 |
| 1926 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        300 |        -25 |        325 |   205 |   175 |   300 |   300 |   215 |   195 |    30 |   -25 |
| 1927 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1035 |       1035 |  -860 |  -890 |  -825 |  -790 |  -780 |  -850 |  -930 | -1035 |
| 1928 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -205 |        205 |   -85 |  -105 |  -150 |  -155 |   -70 |   -60 |  -205 |   -90 |
| 1929 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        175 |       -150 |        325 |   175 |   165 |    -0 |   -60 |   -50 |   -30 |    10 |  -150 |
| 1930 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1135 |       1135 |  -845 |  -865 | -1135 | -1075 | -1040 | -1130 | -1040 | -1080 |
| 1931 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        280 |        -10 |        290 |   280 |    20 |    40 |   -10 |   155 |    15 |    90 |    80 |
| 1932 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        170 |       -135 |        305 |    90 |   120 |   170 |   120 |   110 |  -135 |  -135 |   -65 |
| 1933 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -455 |        455 |   -85 |  -245 |  -175 |  -180 |  -170 |  -435 |  -430 |  -455 |
| 1934 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -570 |        570 |  -265 |  -220 |  -420 |  -335 |  -340 |  -270 |  -570 |  -520 |
| 1935 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -545 |        545 |  -335 |  -285 |  -315 |  -280 |  -320 |  -305 |  -180 |  -545 |
| 1936 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        185 |          0 |        185 |   130 |   175 |   135 |   185 |   185 |   140 |   155 |   130 |
| 1937 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -270 |        270 |  -215 |  -215 |  -220 |  -270 |  -185 |  -175 |  -180 |  -195 |
| 1938 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        390 |          0 |        390 |   260 |   240 |   390 |   325 |   305 |   100 |   160 |   115 |
| 1939 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        240 |          0 |        240 |   155 |   100 |   170 |   180 |   215 |   115 |   235 |   240 |
| 1940 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        140 |       -200 |        340 |    65 |   135 |   140 |  -150 |  -155 |   -55 |  -185 |  -200 |
| 1941 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -360 |        360 |   -90 |  -130 |  -160 |   -60 |   -60 |  -240 |  -360 |  -245 |
| 1942 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         30 |       -120 |        150 |  -100 |  -120 |  -100 |    15 |    30 |   -20 |  -105 |   -45 |
| 1943 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1190 |       1190 | -1090 |  -970 | -1085 |  -970 | -1150 | -1135 | -1190 | -1030 |
| 1944 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |          0 |        280 |   280 |   205 |   275 |   125 |    55 |   105 |    50 |   110 |
| 1945 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -375 |        375 |  -120 |  -130 |  -185 |  -305 |  -340 |  -300 |  -355 |  -375 |
| 1946 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         70 |       -135 |        205 |   -30 |    40 |    70 |    65 |    30 |    40 |   -60 |  -135 |
| 1947 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1025 |       1025 |  -880 |  -830 |  -850 |  -910 |  -895 |  -890 |  -910 | -1025 |
| 1948 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        200 |          0 |        200 |   155 |   180 |    95 |   165 |   200 |   120 |   135 |   200 |
| 1949 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        250 |        -75 |        325 |   200 |   235 |   240 |   250 |   -75 |     5 |   -40 |    45 |
| 1950 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -585 |        585 |  -390 |  -370 |  -455 |  -340 |  -405 |  -515 |  -460 |  -585 |
| 1951 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -280 |        280 |  -220 |  -195 |  -215 |  -225 |  -280 |  -160 |  -255 |  -195 |
| 1952 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1035 |       1035 |  -880 |  -805 |  -840 |  -915 |  -880 |  -895 | -1025 | -1035 |
| 1953 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         45 |       -125 |        170 |    -5 |   -55 |   -75 |    45 |  -125 |   -40 |   -70 |  -105 |
| 1954 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -700 |        700 |  -470 |  -475 |  -535 |  -485 |  -455 |  -665 |  -700 |  -675 |
| 1955 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         80 |        -75 |        155 |    -0 |    80 |   -25 |   -10 |    50 |   -75 |    50 |    10 |
| 1956 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        200 |          0 |        200 |   175 |   200 |   160 |   200 |   190 |   190 |   180 |   200 |
| 1957 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -420 |        420 |  -170 |  -175 |  -420 |  -305 |  -350 |  -380 |  -410 |  -350 |
| 1958 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        320 |        -90 |        410 |   205 |   320 |   220 |   180 |    95 |    10 |   -90 |    40 |
| 1959 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -755 |        755 |  -600 |  -550 |  -545 |  -515 |  -425 |  -500 |  -755 |  -745 |
| 1960 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -530 |        530 |  -325 |  -505 |  -500 |  -510 |  -515 |  -530 |  -485 |  -515 |
| 1961 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        350 |          0 |        350 |   350 |   235 |   195 |   250 |    35 |   120 |    30 |    75 |
| 1962 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -950 |        950 |  -560 |  -660 |  -845 |  -725 |  -950 |  -835 |  -925 |  -875 |
| 1963 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -710 |        710 |  -400 |  -585 |  -600 |  -620 |  -635 |  -690 |  -630 |  -710 |
| 1964 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1115 |       1115 | -1025 |  -990 |  -980 |  -975 |  -970 |  -965 |  -970 | -1115 |
| 1965 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -755 |        755 |  -530 |  -535 |  -665 |  -595 |  -625 |  -735 |  -615 |  -755 |
| 1966 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -755 |        755 |  -520 |  -580 |  -490 |  -620 |  -505 |  -530 |  -720 |  -755 |
| 1967 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        320 |          0 |        320 |   320 |   250 |   205 |   280 |   135 |   230 |   225 |   265 |
| 1968 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        245 |        -30 |        275 |   135 |   195 |   180 |   100 |   140 |   195 |   245 |   -30 |
| 1969 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -910 |        910 |  -740 |  -610 |  -600 |  -645 |  -595 |  -630 |  -690 |  -910 |
| 1970 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        255 |          0 |        255 |   195 |   255 |   195 |   245 |   240 |   235 |   220 |     5 |
| 1971 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -340 |        340 |  -205 |  -110 |  -135 |  -100 |  -220 |  -140 |  -110 |  -340 |
| 1972 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -375 |        375 |  -240 |  -375 |  -360 |  -315 |  -230 |  -320 |  -245 |  -335 |
| 1973 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        125 |        -65 |        190 |    65 |   125 |   -65 |   -45 |   -25 |   -30 |    55 |    55 |
| 1974 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -465 |        465 |  -340 |  -335 |  -270 |  -255 |  -330 |  -245 |  -355 |  -465 |
| 1975 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         85 |       -210 |        295 |    85 |    60 |  -210 |  -150 |  -185 |  -185 |   -85 |  -105 |
| 1976 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        145 |       -140 |        285 |   130 |   145 |    -5 |   -35 |   -65 |  -140 |    30 |   -30 |
| 1977 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |        -75 |        315 |   240 |    35 |    35 |    30 |    15 |   -45 |   -75 |   105 |
| 1978 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        130 |         -5 |        135 |   110 |    25 |    75 |    20 |    15 |   130 |    -5 |    20 |
| 1979 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -645 |        645 |  -605 |  -625 |  -555 |  -595 |  -605 |  -645 |  -545 |  -525 |
| 1980 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1070 |       1070 |  -770 |  -870 |  -815 |  -915 | -1065 | -1070 | -1060 | -1035 |
| 1981 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1145 |       1145 |  -820 | -1075 | -1015 | -1105 | -1145 |  -975 | -1030 | -1030 |
| 1982 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1105 |       1105 |  -790 |  -855 |  -835 |  -860 |  -885 |  -910 | -1090 | -1105 |
| 1983 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -795 |        795 |  -480 |  -510 |  -570 |  -565 |  -555 |  -570 |  -560 |  -795 |
| 1984 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        230 |        -75 |        305 |   230 |   160 |   155 |   -50 |   -75 |   105 |   -30 |    30 |
| 1985 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        195 |        -60 |        255 |   195 |   175 |   140 |    15 |   -60 |    -0 |   -35 |   -25 |
| 1986 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -290 |        290 |   -70 |  -185 |  -180 |  -230 |   -80 |  -140 |  -155 |  -290 |
| 1987 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        290 |        -15 |        305 |   290 |   290 |   260 |   140 |   -15 |   180 |    85 |    45 |
| 1988 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |        -20 |        280 |   160 |   150 |   260 |    -5 |   -20 |    -0 |    90 |    15 |
| 1989 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1015 |       1015 |  -800 |  -735 |  -760 |  -750 |  -675 |  -885 |  -920 | -1015 |
| 1990 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         75 |       -145 |        220 |    70 |    75 |   -35 |    45 |    70 |    30 |  -145 |  -135 |
| 1991 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        335 |          0 |        335 |   275 |   240 |   310 |   335 |   240 |   280 |   210 |    80 |
| 1992 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -780 |        780 |  -660 |  -720 |  -700 |  -770 |  -705 |  -750 |  -705 |  -780 |
| 1993 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -605 |        605 |  -400 |  -395 |  -465 |  -490 |  -470 |  -415 |  -605 |  -570 |
| 1994 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        365 |          0 |        365 |   350 |   365 |   180 |   115 |   155 |   185 |   130 |   200 |
| 1995 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1020 |       1020 |  -875 |  -840 |  -750 |  -850 |  -825 |  -805 |  -760 | -1020 |
| 1996 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        275 |          0 |        275 |   275 |   260 |   275 |   230 |   245 |   255 |    40 |   100 |
| 1997 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        365 |        -20 |        385 |   365 |   135 |    70 |   -20 |    50 |   110 |    80 |    45 |
| 1998 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        210 |          0 |        210 |   210 |   155 |   150 |   155 |   175 |    70 |   110 |   125 |
| 1999 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -885 |        885 |  -625 |  -710 |  -675 |  -665 |  -650 |  -620 |  -885 |  -860 |
| 2000 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -565 |        565 |  -205 |  -265 |  -250 |  -240 |  -470 |  -545 |  -540 |  -565 |